In [1]:
import pandas as pd
from datetime import datetime, date
import numpy as np

In [2]:
def make_df(df, year=[2021],bins=[20,150],
            x='Category', quantile=0.1, topcut=20,
            aggregate_by='average'):
    # load/prepare datafile
    
    df = df.loc[df['Date'].dt.year.isin(year)]
    df = df.set_index('Date')

    # choose x, bins
    if bins:
        if bins == 'Time Comparison':
            grabs = df.resample('M').count()
            most_recent = grabs.iloc[-int(grabs.shape[0]/2):].index
            previous = grabs.loc[~grabs.index.isin(most_recent)].index
            labels = [f'before {previous[-1].date()}', f'after {previous[-1].date()}']
            
            if x != 'Date':
                dff = df.groupby(x).resample('M')[['Amount']].mean()
            else:
                dff = df.resample('M')[['Amount']].sum()
            dff = dff.reset_index()
            dff.loc[dff.Date.isin(most_recent), 'Bin'] = labels[1]
            dff.loc[dff.Date.isin(previous), 'Bin'] = labels[0]
            
        else:
            labels = [f'less than {bins[0]}', f'between {bins[0]} and {bins[1]}', f'greater than {bins[1]}']
            if x != 'Date':
                dff = df.loc[(df.Amount < bins[0])].groupby(x).resample('M')[['Amount']].sum()
                dff.columns = [labels[0]]
                dff[labels[1]] = df.loc[(df.Amount >= bins[0]) &
                                        (df.Amount < bins[1])].groupby(x).resample('M')[['Amount']].sum()
                dff[labels[2]] = df.loc[(df.Amount > bins[1])].groupby(x).resample('M')[['Amount']].sum()
                dff = dff.reset_index()
                dff = dff.melt(id_vars=[x, 'Date'], value_name='Amount', var_name='Bin')
            else:
                dff = df.loc[(df.Amount < bins[0])].resample('M')[['Amount']].sum()
                dff.columns = [labels[0]]
                dff[labels[1]] = df.loc[(df.Amount >= bins[0]) &
                                        (df.Amount < bins[1])].resample('M')[['Amount']].sum()
                dff[labels[2]] = df.loc[(df.Amount > bins[1])].resample('M')[['Amount']].sum()
                dff = dff.reset_index()
                dff = dff.melt(id_vars=['Date'], value_name='Amount', var_name='Bin')
    else:
        
        if x != 'Date':
            dff = df.groupby(x).resample('M')[['Amount']].sum()
            dff = dff.reset_index()
        else:
            dff = df.resample('M')[['Amount']].sum()
            dff = dff.reset_index()
            
    if quantile:
        mymap1 = (dff.resample(on='Date', rule='M').sum()['Amount'] <= dff.resample(on='Date', rule='M').sum().quantile(quantile)[0]).to_dict()
        mymap2 = (dff.resample(on='Date', rule='M').sum()['Amount'] >= dff.resample(on='Date', rule='M').sum().quantile(1-quantile)[0]).to_dict()

        mymap = {}
        for key, val in mymap1.items():
            if val == True:
                mymap[key] = 'Bottom'
            elif mymap2[key] == True:
                mymap[key] = 'Top'
            else:
                mymap[key] = 'Middle'
        dff['Quantile'] = dff['Date'].map(mymap)

    # cut off small - value categories (x)
    if x != 'Date':
        dfftop = dff.groupby(x)[['Amount']].sum().sort_values('Amount', ascending=False)[:topcut]
        dfftop = dfftop.reset_index()
        dff1 = dff.loc[dff[x].isin(dfftop[x])]
        dff2 = dff.loc[~dff[x].isin(dfftop[x])]
        if bins and quantile:
            dffbot = pd.DataFrame(dff2.groupby(['Date', 'Bin', 'Quantile'])['Amount'].sum()).reset_index()
        elif quantile:
            dffbot = pd.DataFrame(dff2.groupby(['Date', 'Quantile'])['Amount'].sum()).reset_index()
        elif bins:
            dffbot = pd.DataFrame(dff2.groupby(['Date', 'Bin'])['Amount'].sum()).reset_index()
        else:
            dffbot = pd.DataFrame(dff2.groupby(['Date'])['Amount'].sum()).reset_index()
        dffbot[x] = 'Other'
        dff = pd.concat([dff1, dffbot])

    # collapse onto groupby
    if aggregate_by == 'average':
        if bins and quantile:
            dff = dff.groupby([x, 'Quantile', 'Bin'])[['Amount']].mean().reset_index()
        elif quantile:
            dff = dff.groupby([x, 'Quantile'])[['Amount']].mean().reset_index()
        elif bins:
            dff = dff.groupby([x, 'Bin'])[['Amount']].mean().reset_index()
        else:
            dff = dff.groupby([x])[['Amount']].mean().reset_index()
    else:
        if bins and quantile:
            dff = dff.groupby([x, 'Quantile', 'Bin'])[['Amount']].sum().reset_index()
        elif quantile:
            dff = dff.groupby([x, 'Quantile'])[['Amount']].sum().reset_index()
        elif bins:
            dff = dff.groupby([x, 'Bin'])[['Amount']].sum().reset_index()
        else:
            dff = dff.groupby([x])[['Amount']].sum().reset_index()

    # sort results, up to 3 sort categories
    if x != 'Date':
        sorter1 = dff.groupby(x)['Amount'].sum().sort_values(ascending=False).index
        sorterIndex1 = dict(zip(sorter1, range(len(sorter1))))
    if quantile:
        sorter2 = ['Top', 'Middle', 'Bottom']
        sorterIndex2 = dict(zip(sorter2, range(len(sorter2))))
    if bins:
        sorter3 = labels
        sorterIndex3 = dict(zip(sorter3, range(len(sorter3))))
        
    if bins and x != 'Date' and quantile:
        sorters = [sorterIndex1, sorterIndex2, sorterIndex3]
    elif bins and quantile:
        sorters = [sorterIndex2, sorterIndex3]
    elif quantile:
        sorters = [sorterIndex2]
    elif bins:
        sorters = [sorterIndex3]
        
    if x != 'Date':
        dff['sort1'] = dff[x].map(sorterIndex1)
    if quantile:
        dff['sort2'] = dff.Quantile.map(sorterIndex2)
    if bins:
        dff['sort3'] = dff.Bin.map(sorterIndex3)
        if x != 'Date' and quantile:
            dff = dff.sort_values(['sort1', 'sort2', 'sort3']).reset_index(drop=True)
            dff = dff.drop(['sort1', 'sort2', 'sort3'], axis=1)
        elif quantile:
            dff = dff.sort_values(['sort2', 'sort3']).reset_index(drop=True)
            dff = dff.drop(['sort2', 'sort3'], axis=1)
        elif x != 'Date':
            dff = dff.sort_values(['sort1', 'sort3']).reset_index(drop=True)
            dff = dff.drop(['sort1', 'sort3'], axis=1)
        else:
            dff = dff.sort_values(['sort3']).reset_index(drop=True)
            dff = dff.drop(['sort3'], axis=1)
    else:
        if x != 'Date' and quantile:
            dff = dff.sort_values(['sort1', 'sort2']).reset_index(drop=True)
            dff = dff.drop(['sort1', 'sort2'], axis=1)
            sorters = [sorterIndex1, sorterIndex2]
            labels = None
        elif quantile:
            dff = dff.sort_values(['sort2']).reset_index(drop=True)
            dff = dff.drop(['sort2'], axis=1)
            sorters = [sorterIndex2]
            labels = None
        elif x != 'Date':
            dff = dff.sort_values(['sort1']).reset_index(drop=True)
            dff = dff.drop(['sort1'], axis=1)
            sorters = [sorterIndex1]
            labels = None
        else:
            sorters = None
            labels = None
    return dff, sorters, labels

In [3]:
df = pd.read_csv('../data/transactions.csv', parse_dates=['Date'])

# df = process_data(df, pivot_dates=False)

categories = list(df.Category.unique())
income = ['Income', 'Paycheck', 'Transfer', 'Federal Tax', 'Taxes', 'Rental Income', 'Interest Income']
taxes = ['Federal Tax', 'Taxes']
internal_acc = ['Credit Card Payment', 'Transfer', 'Financial']
non_expense =  income + taxes + internal_acc
expenses = [i for i in categories if i not in non_expense]

df = df.loc[df['Category'].isin(expenses)]
df = df.loc[df['Amount'] < 8000]

# Primary Chart Data

In [4]:
dff, _, _ = make_df(df, x='Date', year=[2021], quantile=0.1, bins=None)
monthly = dff.set_index("Date")
monthly.columns = ['Decile', 'Amount']


for quantile, label in zip([.25,.5], ['Quartile', 'Median']):
    dff, _, _ = make_df(df, x='Date', year=[2021], quantile=quantile, bins=None)
    dff = dff.set_index("Date")
    monthly[label] = dff['Quantile']

monthly.index = monthly.index.month
monthly = monthly.reset_index()
monthly = monthly.sort_values('Date')

monthly = monthly.reset_index(drop=True)
monthly = monthly[['Date', 'Amount', 'Decile', 'Quartile', 'Median']]
monthly['Amount'] = round(monthly['Amount'])
# monthly.to_json('static_data/monthly_spending.json')

In [18]:
dff, _, _ = make_df(df, x='Date', year=[2021], quantile=None, bins=[20, 150])
dff = round(dff)
monthly = dff.groupby(['Date', 'Bin'], sort=False)['Amount'].sum().unstack('Bin').reset_index()
monthly = monthly.set_index('Date')
monthly.index = monthly.index.month
monthly = monthly.reset_index()
monthly = monthly.sort_values('Date')

monthly = monthly.reset_index(drop=True)
monthly.to_json('static_data/monthly_spending_threshold.json')

In [24]:
dff, _, _ = make_df(df, x='Date', year=[2021], quantile=None, bins='Time Comparison')
dff = round(dff)
monthly = dff.groupby(['Date', 'Bin'], sort=False)['Amount'].sum().unstack('Bin').reset_index()#.to_json('static_data/category_spending_10.json')
monthly = monthly.set_index('Date')
monthly.index = monthly.index.month
monthly = monthly.reset_index()
monthly = monthly.sort_values('Date')

monthly = monthly.reset_index(drop=True)
monthly.to_json('static_data/monthly_spending_time.json')

# Secondary Chart Data

In [202]:
dff, _, _ = make_df(df, x='Category', year=[2021], quantile=0.1, bins=None)
dff = round(dff)
dff.groupby(['Category', 'Quantile'], sort=False)['Amount'].sum().unstack('Quantile').reset_index().to_json('static_data/category_spending_10.json')
df1 = pd.concat({"Top/Bottom 10%": dff}, axis=1, names=["Quantile", "Data"])

dff, _, _ = make_df(df, x='Category', year=[2021], quantile=0.25, bins=None)
dff = round(dff)
dff.groupby(['Category', 'Quantile'], sort=False)['Amount'].sum().unstack('Quantile').reset_index().to_json('static_data/category_spending_25.json')
df2 = pd.concat({"Top/Bottom Quartile": dff}, axis=1, names=["Quantile", "Data"])

dff, _, _ = make_df(df, x='Category', year=[2021], quantile=0.5, bins=None)
dff = round(dff)
dff.groupby(['Category', 'Quantile'], sort=False)['Amount'].sum().unstack('Quantile').reset_index().to_json('static_data/category_spending_50.json')
df3 = pd.concat({"Above/Bellow Median": dff}, axis=1, names=["Quantile", "Data"])

dff, _, _ = make_df(df, x='Category', year=[2021], quantile=None, bins=None)
dff = round(dff)
dff.to_json('static_data/category_spending_none.json')
df4 = pd.concat({"None": dff}, axis=1, names=["Quantile", "Data"])

In [23]:
dff, _, _ = make_df(df, x='Category', year=[2021], quantile=None, bins=[20, 150])
dff = round(dff)
dff = dff.groupby(['Category', 'Bin'], sort=False)['Amount'].sum().unstack('Bin').reset_index()
dff.to_json('static_data/category_spending_threshold.json')

Bin,Category,less than 20,between 20 and 150,greater than 150
0,Groceries,57.0,254.0,259.0
1,Gym,4.0,56.0,420.0
2,Travel,5.0,21.0,318.0
3,Rental Car & Taxi,23.0,146.0,155.0
4,Shopping,16.0,141.0,121.0


In [29]:
dff, _, _ = make_df(df, x='Category', year=[2021], quantile=None, bins='Time Comparison')
dff = round(dff)
dff = dff.groupby(['Category', 'Bin'], sort=False)['Amount'].sum().unstack('Bin').reset_index()
dff
dff.to_json('static_data/category_spending_time.json')

# API

In [23]:
import pandas as pd
from datetime import datetime, date
import numpy as np

df = pd.read_csv('../data/transactions.csv', parse_dates=['Date'])
categories = list(df.Category.unique())
income = ['Income', 'Paycheck', 'Transfer', 'Federal Tax', 'Taxes', 'Rental Income', 'Interest Income']
taxes = ['Federal Tax', 'Taxes']
internal_acc = ['Credit Card Payment', 'Transfer', 'Financial']
non_expense =  income + taxes + internal_acc
expenses = [i for i in categories if i not in non_expense]
df = df.loc[df['Category'].isin(expenses)]
df = df.loc[df['Amount'] < 8000]
# df[['Date', 'Description', 'Amount', 'Category']].to_json()[143890:143900]

In [28]:
df['Date'] = df['Date'].dt.date

In [42]:
df[['Date', 'Amount', 'Category']].to_json(date_format='iso',date_unit='s')

'{"Date":{"5":"2021-12-06T00:00:00Z","8":"2021-12-05T00:00:00Z","9":"2021-12-05T00:00:00Z","10":"2021-12-05T00:00:00Z","11":"2021-12-05T00:00:00Z","12":"2021-12-05T00:00:00Z","13":"2021-12-04T00:00:00Z","14":"2021-12-04T00:00:00Z","15":"2021-12-04T00:00:00Z","16":"2021-12-04T00:00:00Z","17":"2021-12-03T00:00:00Z","18":"2021-12-03T00:00:00Z","19":"2021-12-03T00:00:00Z","20":"2021-12-03T00:00:00Z","21":"2021-12-03T00:00:00Z","22":"2021-12-02T00:00:00Z","23":"2021-12-02T00:00:00Z","24":"2021-12-01T00:00:00Z","25":"2021-12-01T00:00:00Z","26":"2021-12-01T00:00:00Z","27":"2021-12-01T00:00:00Z","28":"2021-12-01T00:00:00Z","29":"2021-12-01T00:00:00Z","30":"2021-12-01T00:00:00Z","31":"2021-12-01T00:00:00Z","32":"2021-11-30T00:00:00Z","33":"2021-11-30T00:00:00Z","34":"2021-11-30T00:00:00Z","35":"2021-11-30T00:00:00Z","38":"2021-11-29T00:00:00Z","45":"2021-11-28T00:00:00Z","46":"2021-11-27T00:00:00Z","47":"2021-11-27T00:00:00Z","48":"2021-11-27T00:00:00Z","49":"2021-11-25T00:00:00Z","50":"2021-11

In [47]:
url = {"Date":{"5":"2021-12-06T00:00:00Z","8":"2021-12-05T00:00:00Z","9":"2021-12-05T00:00:00Z","10":"2021-12-05T00:00:00Z","11":"2021-12-05T00:00:00Z","12":"2021-12-05T00:00:00Z","13":"2021-12-04T00:00:00Z","14":"2021-12-04T00:00:00Z","15":"2021-12-04T00:00:00Z","16":"2021-12-04T00:00:00Z","17":"2021-12-03T00:00:00Z","18":"2021-12-03T00:00:00Z","19":"2021-12-03T00:00:00Z","20":"2021-12-03T00:00:00Z","21":"2021-12-03T00:00:00Z","22":"2021-12-02T00:00:00Z","23":"2021-12-02T00:00:00Z","24":"2021-12-01T00:00:00Z","25":"2021-12-01T00:00:00Z","26":"2021-12-01T00:00:00Z","27":"2021-12-01T00:00:00Z","28":"2021-12-01T00:00:00Z","29":"2021-12-01T00:00:00Z","30":"2021-12-01T00:00:00Z","31":"2021-12-01T00:00:00Z","32":"2021-11-30T00:00:00Z","33":"2021-11-30T00:00:00Z","34":"2021-11-30T00:00:00Z","35":"2021-11-30T00:00:00Z","38":"2021-11-29T00:00:00Z","45":"2021-11-28T00:00:00Z","46":"2021-11-27T00:00:00Z","47":"2021-11-27T00:00:00Z","48":"2021-11-27T00:00:00Z","49":"2021-11-25T00:00:00Z","50":"2021-11-24T00:00:00Z","52":"2021-11-24T00:00:00Z","53":"2021-11-24T00:00:00Z","54":"2021-11-23T00:00:00Z","55":"2021-11-23T00:00:00Z","56":"2021-11-23T00:00:00Z","61":"2021-11-21T00:00:00Z","63":"2021-11-20T00:00:00Z","64":"2021-11-20T00:00:00Z","65":"2021-11-19T00:00:00Z","66":"2021-11-19T00:00:00Z","67":"2021-11-19T00:00:00Z","68":"2021-11-19T00:00:00Z","69":"2021-11-19T00:00:00Z","70":"2021-11-19T00:00:00Z","71":"2021-11-19T00:00:00Z","72":"2021-11-19T00:00:00Z","73":"2021-11-18T00:00:00Z","74":"2021-11-18T00:00:00Z","77":"2021-11-17T00:00:00Z","78":"2021-11-17T00:00:00Z","79":"2021-11-16T00:00:00Z","80":"2021-11-16T00:00:00Z","83":"2021-11-16T00:00:00Z","84":"2021-11-16T00:00:00Z","85":"2021-11-16T00:00:00Z","88":"2021-11-15T00:00:00Z","89":"2021-11-15T00:00:00Z","90":"2021-11-15T00:00:00Z","91":"2021-11-15T00:00:00Z","92":"2021-11-15T00:00:00Z","95":"2021-11-14T00:00:00Z","96":"2021-11-13T00:00:00Z","97":"2021-11-13T00:00:00Z","98":"2021-11-13T00:00:00Z","99":"2021-11-13T00:00:00Z","100":"2021-11-13T00:00:00Z","101":"2021-11-12T00:00:00Z","102":"2021-11-12T00:00:00Z","103":"2021-11-12T00:00:00Z","104":"2021-11-12T00:00:00Z","105":"2021-11-12T00:00:00Z","107":"2021-11-12T00:00:00Z","108":"2021-11-11T00:00:00Z","110":"2021-11-10T00:00:00Z","113":"2021-11-09T00:00:00Z","114":"2021-11-09T00:00:00Z","116":"2021-11-08T00:00:00Z","117":"2021-11-08T00:00:00Z","118":"2021-11-08T00:00:00Z","119":"2021-11-08T00:00:00Z","120":"2021-11-07T00:00:00Z","121":"2021-11-07T00:00:00Z","122":"2021-11-07T00:00:00Z","123":"2021-11-07T00:00:00Z","124":"2021-11-06T00:00:00Z","125":"2021-11-06T00:00:00Z","126":"2021-11-06T00:00:00Z","127":"2021-11-06T00:00:00Z","128":"2021-11-05T00:00:00Z","129":"2021-11-05T00:00:00Z","130":"2021-11-05T00:00:00Z","131":"2021-11-05T00:00:00Z","132":"2021-11-05T00:00:00Z","134":"2021-11-04T00:00:00Z","135":"2021-11-04T00:00:00Z","136":"2021-11-02T00:00:00Z","137":"2021-11-02T00:00:00Z","138":"2021-11-01T00:00:00Z","141":"2021-11-01T00:00:00Z","142":"2021-11-01T00:00:00Z","143":"2021-10-31T00:00:00Z","144":"2021-10-31T00:00:00Z","145":"2021-10-31T00:00:00Z","146":"2021-10-31T00:00:00Z","147":"2021-10-31T00:00:00Z","148":"2021-10-30T00:00:00Z","149":"2021-10-30T00:00:00Z","150":"2021-10-30T00:00:00Z","151":"2021-10-30T00:00:00Z","152":"2021-10-29T00:00:00Z","153":"2021-10-29T00:00:00Z","154":"2021-10-29T00:00:00Z","155":"2021-10-29T00:00:00Z","156":"2021-10-29T00:00:00Z","157":"2021-10-29T00:00:00Z","158":"2021-10-29T00:00:00Z","161":"2021-10-28T00:00:00Z","162":"2021-10-28T00:00:00Z","163":"2021-10-28T00:00:00Z","164":"2021-10-28T00:00:00Z","165":"2021-10-28T00:00:00Z","166":"2021-10-28T00:00:00Z","167":"2021-10-27T00:00:00Z","168":"2021-10-26T00:00:00Z","169":"2021-10-26T00:00:00Z","171":"2021-10-26T00:00:00Z","172":"2021-10-25T00:00:00Z","173":"2021-10-25T00:00:00Z","174":"2021-10-25T00:00:00Z","175":"2021-10-25T00:00:00Z","176":"2021-10-25T00:00:00Z","177":"2021-10-25T00:00:00Z","184":"2021-10-24T00:00:00Z","185":"2021-10-24T00:00:00Z","186":"2021-10-24T00:00:00Z","187":"2021-10-24T00:00:00Z","188":"2021-10-23T00:00:00Z","189":"2021-10-23T00:00:00Z","190":"2021-10-23T00:00:00Z","191":"2021-10-23T00:00:00Z","192":"2021-10-23T00:00:00Z","193":"2021-10-23T00:00:00Z","194":"2021-10-23T00:00:00Z","195":"2021-10-22T00:00:00Z","196":"2021-10-22T00:00:00Z","197":"2021-10-22T00:00:00Z","198":"2021-10-21T00:00:00Z","199":"2021-10-21T00:00:00Z","200":"2021-10-21T00:00:00Z","201":"2021-10-21T00:00:00Z","202":"2021-10-21T00:00:00Z","203":"2021-10-21T00:00:00Z","204":"2021-10-20T00:00:00Z","205":"2021-10-19T00:00:00Z","206":"2021-10-19T00:00:00Z","207":"2021-10-19T00:00:00Z","208":"2021-10-19T00:00:00Z","209":"2021-10-18T00:00:00Z","210":"2021-10-18T00:00:00Z","211":"2021-10-18T00:00:00Z","212":"2021-10-18T00:00:00Z","213":"2021-10-18T00:00:00Z","214":"2021-10-18T00:00:00Z","216":"2021-10-17T00:00:00Z","217":"2021-10-17T00:00:00Z","218":"2021-10-17T00:00:00Z","219":"2021-10-17T00:00:00Z","221":"2021-10-16T00:00:00Z","222":"2021-10-16T00:00:00Z","223":"2021-10-16T00:00:00Z","224":"2021-10-16T00:00:00Z","225":"2021-10-15T00:00:00Z","226":"2021-10-15T00:00:00Z","227":"2021-10-15T00:00:00Z","228":"2021-10-15T00:00:00Z","229":"2021-10-15T00:00:00Z","230":"2021-10-15T00:00:00Z","232":"2021-10-14T00:00:00Z","233":"2021-10-14T00:00:00Z","234":"2021-10-14T00:00:00Z","236":"2021-10-13T00:00:00Z","237":"2021-10-13T00:00:00Z","248":"2021-10-12T00:00:00Z","249":"2021-10-12T00:00:00Z","252":"2021-10-11T00:00:00Z","253":"2021-10-11T00:00:00Z","254":"2021-10-10T00:00:00Z","255":"2021-10-10T00:00:00Z","256":"2021-10-08T00:00:00Z","257":"2021-10-08T00:00:00Z","260":"2021-10-07T00:00:00Z","261":"2021-10-06T00:00:00Z","262":"2021-10-05T00:00:00Z","263":"2021-10-05T00:00:00Z","264":"2021-10-04T00:00:00Z","265":"2021-10-04T00:00:00Z","266":"2021-10-03T00:00:00Z","267":"2021-10-03T00:00:00Z","268":"2021-10-03T00:00:00Z","269":"2021-10-02T00:00:00Z","270":"2021-10-02T00:00:00Z","271":"2021-10-01T00:00:00Z","272":"2021-10-01T00:00:00Z","273":"2021-10-01T00:00:00Z","274":"2021-10-01T00:00:00Z","276":"2021-09-30T00:00:00Z","277":"2021-09-30T00:00:00Z","278":"2021-09-30T00:00:00Z","279":"2021-09-30T00:00:00Z","280":"2021-09-30T00:00:00Z","282":"2021-09-29T00:00:00Z","283":"2021-09-29T00:00:00Z","284":"2021-09-29T00:00:00Z","287":"2021-09-29T00:00:00Z","288":"2021-09-29T00:00:00Z","289":"2021-09-29T00:00:00Z","290":"2021-09-28T00:00:00Z","291":"2021-09-28T00:00:00Z","292":"2021-09-28T00:00:00Z","293":"2021-09-28T00:00:00Z","294":"2021-09-28T00:00:00Z","295":"2021-09-28T00:00:00Z","296":"2021-09-28T00:00:00Z","297":"2021-09-27T00:00:00Z","298":"2021-09-27T00:00:00Z","299":"2021-09-27T00:00:00Z","300":"2021-09-27T00:00:00Z","301":"2021-09-26T00:00:00Z","302":"2021-09-26T00:00:00Z","303":"2021-09-25T00:00:00Z","304":"2021-09-25T00:00:00Z","305":"2021-09-25T00:00:00Z","306":"2021-09-25T00:00:00Z","307":"2021-09-24T00:00:00Z","309":"2021-09-24T00:00:00Z","310":"2021-09-24T00:00:00Z","311":"2021-09-24T00:00:00Z","313":"2021-09-23T00:00:00Z","314":"2021-09-23T00:00:00Z","315":"2021-09-22T00:00:00Z","316":"2021-09-22T00:00:00Z","317":"2021-09-22T00:00:00Z","318":"2021-09-22T00:00:00Z","323":"2021-09-21T00:00:00Z","324":"2021-09-21T00:00:00Z","333":"2021-09-20T00:00:00Z","334":"2021-09-20T00:00:00Z","335":"2021-09-20T00:00:00Z","336":"2021-09-20T00:00:00Z","337":"2021-09-20T00:00:00Z","338":"2021-09-20T00:00:00Z","339":"2021-09-20T00:00:00Z","340":"2021-09-20T00:00:00Z","342":"2021-09-20T00:00:00Z","343":"2021-09-20T00:00:00Z","344":"2021-09-20T00:00:00Z","345":"2021-09-20T00:00:00Z","346":"2021-09-20T00:00:00Z","347":"2021-09-17T00:00:00Z","348":"2021-09-17T00:00:00Z","349":"2021-09-17T00:00:00Z","350":"2021-09-17T00:00:00Z","351":"2021-09-17T00:00:00Z","352":"2021-09-17T00:00:00Z","353":"2021-09-16T00:00:00Z","354":"2021-09-16T00:00:00Z","355":"2021-09-16T00:00:00Z","356":"2021-09-16T00:00:00Z","357":"2021-09-16T00:00:00Z","358":"2021-09-15T00:00:00Z","359":"2021-09-15T00:00:00Z","360":"2021-09-15T00:00:00Z","361":"2021-09-15T00:00:00Z","362":"2021-09-14T00:00:00Z","363":"2021-09-14T00:00:00Z","364":"2021-09-14T00:00:00Z","365":"2021-09-13T00:00:00Z","366":"2021-09-13T00:00:00Z","367":"2021-09-13T00:00:00Z","369":"2021-09-12T00:00:00Z","370":"2021-09-12T00:00:00Z","371":"2021-09-12T00:00:00Z","373":"2021-09-11T00:00:00Z","374":"2021-09-11T00:00:00Z","375":"2021-09-11T00:00:00Z","376":"2021-09-11T00:00:00Z","377":"2021-09-11T00:00:00Z","378":"2021-09-11T00:00:00Z","379":"2021-09-11T00:00:00Z","380":"2021-09-10T00:00:00Z","381":"2021-09-10T00:00:00Z","382":"2021-09-10T00:00:00Z","384":"2021-09-09T00:00:00Z","385":"2021-09-09T00:00:00Z","386":"2021-09-09T00:00:00Z","387":"2021-09-09T00:00:00Z","389":"2021-09-09T00:00:00Z","390":"2021-09-09T00:00:00Z","391":"2021-09-09T00:00:00Z","392":"2021-09-09T00:00:00Z","393":"2021-09-09T00:00:00Z","394":"2021-09-08T00:00:00Z","395":"2021-09-08T00:00:00Z","396":"2021-09-08T00:00:00Z","397":"2021-09-08T00:00:00Z","398":"2021-09-08T00:00:00Z","399":"2021-09-08T00:00:00Z","400":"2021-09-08T00:00:00Z","401":"2021-09-08T00:00:00Z","402":"2021-09-08T00:00:00Z","403":"2021-09-07T00:00:00Z","404":"2021-09-07T00:00:00Z","405":"2021-09-07T00:00:00Z","406":"2021-09-07T00:00:00Z","407":"2021-09-07T00:00:00Z","408":"2021-09-07T00:00:00Z","409":"2021-09-06T00:00:00Z","410":"2021-09-06T00:00:00Z","411":"2021-09-06T00:00:00Z","412":"2021-09-05T00:00:00Z","413":"2021-09-04T00:00:00Z","414":"2021-09-04T00:00:00Z","415":"2021-09-04T00:00:00Z","416":"2021-09-04T00:00:00Z","417":"2021-09-04T00:00:00Z","418":"2021-09-04T00:00:00Z","419":"2021-09-04T00:00:00Z","420":"2021-09-03T00:00:00Z","421":"2021-09-03T00:00:00Z","422":"2021-09-03T00:00:00Z","423":"2021-09-03T00:00:00Z","425":"2021-09-02T00:00:00Z","426":"2021-09-02T00:00:00Z","427":"2021-09-02T00:00:00Z","428":"2021-09-02T00:00:00Z","429":"2021-09-02T00:00:00Z","430":"2021-09-02T00:00:00Z","431":"2021-09-01T00:00:00Z","432":"2021-09-01T00:00:00Z","433":"2021-09-01T00:00:00Z","434":"2021-09-01T00:00:00Z","435":"2021-09-01T00:00:00Z","436":"2021-09-01T00:00:00Z","437":"2021-09-01T00:00:00Z","438":"2021-09-01T00:00:00Z","439":"2021-08-31T00:00:00Z","440":"2021-08-31T00:00:00Z","441":"2021-08-31T00:00:00Z","442":"2021-08-31T00:00:00Z","443":"2021-08-31T00:00:00Z","444":"2021-08-31T00:00:00Z","445":"2021-08-31T00:00:00Z","446":"2021-08-31T00:00:00Z","451":"2021-08-30T00:00:00Z","452":"2021-08-30T00:00:00Z","453":"2021-08-30T00:00:00Z","455":"2021-08-30T00:00:00Z","460":"2021-08-29T00:00:00Z","461":"2021-08-29T00:00:00Z","462":"2021-08-28T00:00:00Z","463":"2021-08-28T00:00:00Z","464":"2021-08-28T00:00:00Z","465":"2021-08-28T00:00:00Z","466":"2021-08-28T00:00:00Z","467":"2021-08-28T00:00:00Z","468":"2021-08-28T00:00:00Z","469":"2021-08-28T00:00:00Z","470":"2021-08-28T00:00:00Z","471":"2021-08-28T00:00:00Z","472":"2021-08-28T00:00:00Z","473":"2021-08-28T00:00:00Z","474":"2021-08-28T00:00:00Z","475":"2021-08-28T00:00:00Z","476":"2021-08-25T00:00:00Z","477":"2021-08-25T00:00:00Z","478":"2021-08-25T00:00:00Z","479":"2021-08-25T00:00:00Z","481":"2021-08-24T00:00:00Z","482":"2021-08-24T00:00:00Z","483":"2021-08-24T00:00:00Z","484":"2021-08-24T00:00:00Z","485":"2021-08-23T00:00:00Z","486":"2021-08-23T00:00:00Z","487":"2021-08-23T00:00:00Z","490":"2021-08-22T00:00:00Z","491":"2021-08-22T00:00:00Z","492":"2021-08-21T00:00:00Z","493":"2021-08-21T00:00:00Z","494":"2021-08-21T00:00:00Z","495":"2021-08-20T00:00:00Z","496":"2021-08-20T00:00:00Z","497":"2021-08-20T00:00:00Z","500":"2021-08-20T00:00:00Z","501":"2021-08-20T00:00:00Z","504":"2021-08-20T00:00:00Z","507":"2021-08-19T00:00:00Z","508":"2021-08-19T00:00:00Z","509":"2021-08-19T00:00:00Z","510":"2021-08-18T00:00:00Z","511":"2021-08-18T00:00:00Z","512":"2021-08-17T00:00:00Z","514":"2021-08-16T00:00:00Z","517":"2021-08-15T00:00:00Z","518":"2021-08-15T00:00:00Z","519":"2021-08-14T00:00:00Z","520":"2021-08-14T00:00:00Z","521":"2021-08-14T00:00:00Z","522":"2021-08-14T00:00:00Z","523":"2021-08-14T00:00:00Z","524":"2021-08-14T00:00:00Z","525":"2021-08-13T00:00:00Z","526":"2021-08-13T00:00:00Z","527":"2021-08-13T00:00:00Z","528":"2021-08-13T00:00:00Z","529":"2021-08-13T00:00:00Z","530":"2021-08-13T00:00:00Z","531":"2021-08-13T00:00:00Z","532":"2021-08-13T00:00:00Z","533":"2021-08-13T00:00:00Z","540":"2021-08-12T00:00:00Z","541":"2021-08-12T00:00:00Z","542":"2021-08-12T00:00:00Z","545":"2021-08-11T00:00:00Z","546":"2021-08-11T00:00:00Z","547":"2021-08-10T00:00:00Z","548":"2021-08-10T00:00:00Z","549":"2021-08-10T00:00:00Z","551":"2021-08-09T00:00:00Z","552":"2021-08-09T00:00:00Z","553":"2021-08-09T00:00:00Z","554":"2021-08-09T00:00:00Z","556":"2021-08-08T00:00:00Z","557":"2021-08-08T00:00:00Z","558":"2021-08-08T00:00:00Z","559":"2021-08-08T00:00:00Z","560":"2021-08-08T00:00:00Z","561":"2021-08-08T00:00:00Z","562":"2021-08-07T00:00:00Z","563":"2021-08-07T00:00:00Z","564":"2021-08-07T00:00:00Z","565":"2021-08-06T00:00:00Z","566":"2021-08-06T00:00:00Z","567":"2021-08-06T00:00:00Z","568":"2021-08-06T00:00:00Z","569":"2021-08-06T00:00:00Z","570":"2021-08-05T00:00:00Z","571":"2021-08-05T00:00:00Z","572":"2021-08-05T00:00:00Z","575":"2021-08-03T00:00:00Z","576":"2021-08-03T00:00:00Z","577":"2021-08-03T00:00:00Z","578":"2021-08-03T00:00:00Z","579":"2021-08-03T00:00:00Z","580":"2021-08-03T00:00:00Z","582":"2021-08-01T00:00:00Z","583":"2021-08-01T00:00:00Z","584":"2021-08-01T00:00:00Z","585":"2021-07-31T00:00:00Z","586":"2021-07-31T00:00:00Z","587":"2021-07-31T00:00:00Z","588":"2021-07-31T00:00:00Z","589":"2021-07-31T00:00:00Z","590":"2021-07-30T00:00:00Z","591":"2021-07-30T00:00:00Z","592":"2021-07-30T00:00:00Z","593":"2021-07-30T00:00:00Z","594":"2021-07-30T00:00:00Z","595":"2021-07-30T00:00:00Z","596":"2021-07-30T00:00:00Z","600":"2021-07-28T00:00:00Z","601":"2021-07-28T00:00:00Z","602":"2021-07-28T00:00:00Z","603":"2021-07-28T00:00:00Z","604":"2021-07-28T00:00:00Z","605":"2021-07-28T00:00:00Z","606":"2021-07-28T00:00:00Z","607":"2021-07-26T00:00:00Z","610":"2021-07-26T00:00:00Z","611":"2021-07-26T00:00:00Z","612":"2021-07-25T00:00:00Z","613":"2021-07-25T00:00:00Z","614":"2021-07-25T00:00:00Z","615":"2021-07-24T00:00:00Z","616":"2021-07-24T00:00:00Z","617":"2021-07-24T00:00:00Z","618":"2021-07-24T00:00:00Z","619":"2021-07-24T00:00:00Z","620":"2021-07-24T00:00:00Z","621":"2021-07-24T00:00:00Z","622":"2021-07-24T00:00:00Z","623":"2021-07-23T00:00:00Z","624":"2021-07-23T00:00:00Z","625":"2021-07-23T00:00:00Z","626":"2021-07-23T00:00:00Z","627":"2021-07-23T00:00:00Z","629":"2021-07-23T00:00:00Z","632":"2021-07-22T00:00:00Z","633":"2021-07-22T00:00:00Z","637":"2021-07-21T00:00:00Z","638":"2021-07-20T00:00:00Z","639":"2021-07-19T00:00:00Z","640":"2021-07-19T00:00:00Z","641":"2021-07-19T00:00:00Z","643":"2021-07-19T00:00:00Z","645":"2021-07-18T00:00:00Z","647":"2021-07-17T00:00:00Z","648":"2021-07-17T00:00:00Z","649":"2021-07-16T00:00:00Z","650":"2021-07-16T00:00:00Z","651":"2021-07-16T00:00:00Z","652":"2021-07-16T00:00:00Z","654":"2021-07-15T00:00:00Z","655":"2021-07-15T00:00:00Z","656":"2021-07-15T00:00:00Z","657":"2021-07-14T00:00:00Z","658":"2021-07-14T00:00:00Z","659":"2021-07-14T00:00:00Z","660":"2021-07-14T00:00:00Z","661":"2021-07-14T00:00:00Z","662":"2021-07-13T00:00:00Z","663":"2021-07-13T00:00:00Z","664":"2021-07-13T00:00:00Z","665":"2021-07-12T00:00:00Z","666":"2021-07-12T00:00:00Z","667":"2021-07-12T00:00:00Z","668":"2021-07-12T00:00:00Z","669":"2021-07-12T00:00:00Z","670":"2021-07-12T00:00:00Z","674":"2021-07-11T00:00:00Z","675":"2021-07-10T00:00:00Z","676":"2021-07-10T00:00:00Z","677":"2021-07-10T00:00:00Z","678":"2021-07-09T00:00:00Z","679":"2021-07-09T00:00:00Z","681":"2021-07-09T00:00:00Z","687":"2021-07-08T00:00:00Z","688":"2021-07-08T00:00:00Z","689":"2021-07-08T00:00:00Z","691":"2021-07-07T00:00:00Z","692":"2021-07-07T00:00:00Z","693":"2021-07-07T00:00:00Z","694":"2021-07-06T00:00:00Z","695":"2021-07-06T00:00:00Z","696":"2021-07-06T00:00:00Z","697":"2021-07-06T00:00:00Z","698":"2021-07-06T00:00:00Z","700":"2021-07-05T00:00:00Z","701":"2021-07-05T00:00:00Z","702":"2021-07-04T00:00:00Z","703":"2021-07-03T00:00:00Z","704":"2021-07-03T00:00:00Z","705":"2021-07-02T00:00:00Z","706":"2021-07-02T00:00:00Z","708":"2021-07-02T00:00:00Z","709":"2021-07-02T00:00:00Z","714":"2021-07-01T00:00:00Z","715":"2021-07-01T00:00:00Z","716":"2021-07-01T00:00:00Z","721":"2021-06-30T00:00:00Z","722":"2021-06-30T00:00:00Z","724":"2021-06-30T00:00:00Z","725":"2021-06-29T00:00:00Z","726":"2021-06-29T00:00:00Z","727":"2021-06-29T00:00:00Z","728":"2021-06-29T00:00:00Z","729":"2021-06-29T00:00:00Z","730":"2021-06-29T00:00:00Z","731":"2021-06-29T00:00:00Z","732":"2021-06-29T00:00:00Z","733":"2021-06-29T00:00:00Z","734":"2021-06-28T00:00:00Z","735":"2021-06-28T00:00:00Z","737":"2021-06-28T00:00:00Z","738":"2021-06-28T00:00:00Z","739":"2021-06-27T00:00:00Z","740":"2021-06-26T00:00:00Z","741":"2021-06-26T00:00:00Z","742":"2021-06-26T00:00:00Z","743":"2021-06-26T00:00:00Z","744":"2021-06-26T00:00:00Z","745":"2021-06-26T00:00:00Z","746":"2021-06-26T00:00:00Z","747":"2021-06-26T00:00:00Z","748":"2021-06-26T00:00:00Z","750":"2021-06-24T00:00:00Z","751":"2021-06-24T00:00:00Z","752":"2021-06-24T00:00:00Z","755":"2021-06-23T00:00:00Z","756":"2021-06-23T00:00:00Z","758":"2021-06-22T00:00:00Z","761":"2021-06-21T00:00:00Z","764":"2021-06-20T00:00:00Z","765":"2021-06-20T00:00:00Z","766":"2021-06-20T00:00:00Z","767":"2021-06-20T00:00:00Z","768":"2021-06-19T00:00:00Z","769":"2021-06-18T00:00:00Z","770":"2021-06-18T00:00:00Z","772":"2021-06-17T00:00:00Z","773":"2021-06-17T00:00:00Z","776":"2021-06-16T00:00:00Z","777":"2021-06-16T00:00:00Z","779":"2021-06-14T00:00:00Z","781":"2021-06-13T00:00:00Z","782":"2021-06-13T00:00:00Z","783":"2021-06-13T00:00:00Z","784":"2021-06-13T00:00:00Z","785":"2021-06-13T00:00:00Z","786":"2021-06-12T00:00:00Z","787":"2021-06-12T00:00:00Z","788":"2021-06-12T00:00:00Z","789":"2021-06-12T00:00:00Z","790":"2021-06-12T00:00:00Z","791":"2021-06-12T00:00:00Z","792":"2021-06-12T00:00:00Z","793":"2021-06-11T00:00:00Z","794":"2021-06-11T00:00:00Z","795":"2021-06-11T00:00:00Z","798":"2021-06-09T00:00:00Z","800":"2021-06-09T00:00:00Z","804":"2021-06-08T00:00:00Z","805":"2021-06-06T00:00:00Z","806":"2021-06-05T00:00:00Z","807":"2021-06-05T00:00:00Z","808":"2021-06-05T00:00:00Z","809":"2021-06-05T00:00:00Z","810":"2021-06-05T00:00:00Z","811":"2021-06-05T00:00:00Z","812":"2021-06-05T00:00:00Z","813":"2021-06-04T00:00:00Z","816":"2021-06-02T00:00:00Z","818":"2021-06-01T00:00:00Z","819":"2021-06-01T00:00:00Z","820":"2021-06-01T00:00:00Z","826":"2021-05-30T00:00:00Z","827":"2021-05-30T00:00:00Z","828":"2021-05-30T00:00:00Z","829":"2021-05-30T00:00:00Z","830":"2021-05-30T00:00:00Z","831":"2021-05-29T00:00:00Z","832":"2021-05-29T00:00:00Z","833":"2021-05-29T00:00:00Z","834":"2021-05-28T00:00:00Z","835":"2021-05-28T00:00:00Z","836":"2021-05-28T00:00:00Z","837":"2021-05-28T00:00:00Z","838":"2021-05-28T00:00:00Z","839":"2021-05-28T00:00:00Z","840":"2021-05-28T00:00:00Z","841":"2021-05-28T00:00:00Z","842":"2021-05-28T00:00:00Z","844":"2021-05-26T00:00:00Z","847":"2021-05-25T00:00:00Z","848":"2021-05-25T00:00:00Z","849":"2021-05-25T00:00:00Z","851":"2021-05-24T00:00:00Z","853":"2021-05-24T00:00:00Z","854":"2021-05-24T00:00:00Z","858":"2021-05-24T00:00:00Z","859":"2021-05-23T00:00:00Z","860":"2021-05-23T00:00:00Z","861":"2021-05-23T00:00:00Z","862":"2021-05-22T00:00:00Z","863":"2021-05-22T00:00:00Z","864":"2021-05-22T00:00:00Z","865":"2021-05-21T00:00:00Z","866":"2021-05-21T00:00:00Z","867":"2021-05-21T00:00:00Z","868":"2021-05-19T00:00:00Z","872":"2021-05-19T00:00:00Z","875":"2021-05-18T00:00:00Z","876":"2021-05-17T00:00:00Z","877":"2021-05-16T00:00:00Z","878":"2021-05-16T00:00:00Z","879":"2021-05-15T00:00:00Z","880":"2021-05-15T00:00:00Z","881":"2021-05-15T00:00:00Z","882":"2021-05-14T00:00:00Z","883":"2021-05-14T00:00:00Z","885":"2021-05-13T00:00:00Z","886":"2021-05-13T00:00:00Z","887":"2021-05-13T00:00:00Z","888":"2021-05-13T00:00:00Z","891":"2021-05-11T00:00:00Z","892":"2021-05-11T00:00:00Z","893":"2021-05-10T00:00:00Z","896":"2021-05-09T00:00:00Z","897":"2021-05-09T00:00:00Z","898":"2021-05-09T00:00:00Z","899":"2021-05-08T00:00:00Z","900":"2021-05-08T00:00:00Z","901":"2021-05-08T00:00:00Z","902":"2021-05-08T00:00:00Z","903":"2021-05-07T00:00:00Z","904":"2021-05-06T00:00:00Z","905":"2021-05-05T00:00:00Z","906":"2021-05-05T00:00:00Z","907":"2021-05-05T00:00:00Z","908":"2021-05-05T00:00:00Z","911":"2021-05-04T00:00:00Z","912":"2021-05-04T00:00:00Z","913":"2021-05-04T00:00:00Z","914":"2021-05-04T00:00:00Z","916":"2021-05-03T00:00:00Z","917":"2021-05-03T00:00:00Z","918":"2021-05-03T00:00:00Z","919":"2021-05-02T00:00:00Z","920":"2021-05-02T00:00:00Z","921":"2021-05-02T00:00:00Z","922":"2021-05-01T00:00:00Z","923":"2021-05-01T00:00:00Z","924":"2021-05-01T00:00:00Z","925":"2021-05-01T00:00:00Z","926":"2021-05-01T00:00:00Z","927":"2021-04-30T00:00:00Z","928":"2021-04-30T00:00:00Z","929":"2021-04-30T00:00:00Z","931":"2021-04-29T00:00:00Z","937":"2021-04-28T00:00:00Z","938":"2021-04-27T00:00:00Z","939":"2021-04-27T00:00:00Z","940":"2021-04-27T00:00:00Z","941":"2021-04-27T00:00:00Z","942":"2021-04-27T00:00:00Z","944":"2021-04-25T00:00:00Z","945":"2021-04-25T00:00:00Z","946":"2021-04-25T00:00:00Z","947":"2021-04-24T00:00:00Z","948":"2021-04-24T00:00:00Z","949":"2021-04-24T00:00:00Z","950":"2021-04-24T00:00:00Z","951":"2021-04-23T00:00:00Z","952":"2021-04-22T00:00:00Z","953":"2021-04-21T00:00:00Z","954":"2021-04-21T00:00:00Z","955":"2021-04-21T00:00:00Z","956":"2021-04-21T00:00:00Z","958":"2021-04-19T00:00:00Z","959":"2021-04-19T00:00:00Z","960":"2021-04-19T00:00:00Z","961":"2021-04-18T00:00:00Z","962":"2021-04-17T00:00:00Z","963":"2021-04-16T00:00:00Z","964":"2021-04-16T00:00:00Z","965":"2021-04-16T00:00:00Z","966":"2021-04-16T00:00:00Z","967":"2021-04-16T00:00:00Z","968":"2021-04-15T00:00:00Z","970":"2021-04-13T00:00:00Z","972":"2021-04-12T00:00:00Z","974":"2021-04-11T00:00:00Z","975":"2021-04-10T00:00:00Z","976":"2021-04-09T00:00:00Z","978":"2021-04-09T00:00:00Z","980":"2021-04-08T00:00:00Z","986":"2021-04-05T00:00:00Z","987":"2021-04-05T00:00:00Z","989":"2021-04-03T00:00:00Z","990":"2021-04-03T00:00:00Z","992":"2021-04-02T00:00:00Z","993":"2021-04-02T00:00:00Z","994":"2021-04-02T00:00:00Z","995":"2021-04-01T00:00:00Z","996":"2021-04-01T00:00:00Z","997":"2021-04-01T00:00:00Z","1000":"2021-03-30T00:00:00Z","1001":"2021-03-30T00:00:00Z","1003":"2021-03-29T00:00:00Z","1004":"2021-03-28T00:00:00Z","1005":"2021-03-28T00:00:00Z","1006":"2021-03-28T00:00:00Z","1007":"2021-03-27T00:00:00Z","1008":"2021-03-27T00:00:00Z","1009":"2021-03-27T00:00:00Z","1010":"2021-03-27T00:00:00Z","1012":"2021-03-25T00:00:00Z","1013":"2021-03-25T00:00:00Z","1015":"2021-03-24T00:00:00Z","1016":"2021-03-23T00:00:00Z","1017":"2021-03-23T00:00:00Z","1018":"2021-03-23T00:00:00Z","1019":"2021-03-23T00:00:00Z","1022":"2021-03-22T00:00:00Z","1028":"2021-03-21T00:00:00Z","1029":"2021-03-21T00:00:00Z","1030":"2021-03-21T00:00:00Z","1031":"2021-03-19T00:00:00Z","1032":"2021-03-19T00:00:00Z","1033":"2021-03-19T00:00:00Z","1034":"2021-03-19T00:00:00Z","1035":"2021-03-18T00:00:00Z","1036":"2021-03-18T00:00:00Z","1037":"2021-03-18T00:00:00Z","1038":"2021-03-18T00:00:00Z","1039":"2021-03-17T00:00:00Z","1040":"2021-03-17T00:00:00Z","1041":"2021-03-17T00:00:00Z","1042":"2021-03-16T00:00:00Z","1043":"2021-03-16T00:00:00Z","1044":"2021-03-15T00:00:00Z","1045":"2021-03-14T00:00:00Z","1046":"2021-03-14T00:00:00Z","1047":"2021-03-14T00:00:00Z","1048":"2021-03-14T00:00:00Z","1049":"2021-03-13T00:00:00Z","1050":"2021-03-13T00:00:00Z","1051":"2021-03-11T00:00:00Z","1052":"2021-03-11T00:00:00Z","1055":"2021-03-09T00:00:00Z","1056":"2021-03-07T00:00:00Z","1057":"2021-03-05T00:00:00Z","1058":"2021-03-05T00:00:00Z","1059":"2021-03-05T00:00:00Z","1060":"2021-03-05T00:00:00Z","1061":"2021-03-05T00:00:00Z","1062":"2021-03-05T00:00:00Z","1063":"2021-03-05T00:00:00Z","1064":"2021-03-04T00:00:00Z","1065":"2021-03-04T00:00:00Z","1066":"2021-03-04T00:00:00Z","1067":"2021-03-04T00:00:00Z","1068":"2021-03-03T00:00:00Z","1069":"2021-03-03T00:00:00Z","1070":"2021-03-02T00:00:00Z","1071":"2021-03-02T00:00:00Z","1072":"2021-03-02T00:00:00Z","1073":"2021-03-02T00:00:00Z","1074":"2021-03-02T00:00:00Z","1077":"2021-03-01T00:00:00Z","1078":"2021-03-01T00:00:00Z","1086":"2021-03-01T00:00:00Z","1088":"2021-02-28T00:00:00Z","1089":"2021-02-28T00:00:00Z","1090":"2021-02-27T00:00:00Z","1091":"2021-02-26T00:00:00Z","1092":"2021-02-26T00:00:00Z","1096":"2021-02-24T00:00:00Z","1097":"2021-02-24T00:00:00Z","1099":"2021-02-22T00:00:00Z","1100":"2021-02-21T00:00:00Z","1101":"2021-02-21T00:00:00Z","1102":"2021-02-21T00:00:00Z","1103":"2021-02-21T00:00:00Z","1104":"2021-02-20T00:00:00Z","1105":"2021-02-19T00:00:00Z","1106":"2021-02-19T00:00:00Z","1107":"2021-02-19T00:00:00Z","1108":"2021-02-18T00:00:00Z","1109":"2021-02-18T00:00:00Z","1110":"2021-02-18T00:00:00Z","1111":"2021-02-18T00:00:00Z","1112":"2021-02-16T00:00:00Z","1113":"2021-02-15T00:00:00Z","1114":"2021-02-15T00:00:00Z","1115":"2021-02-15T00:00:00Z","1116":"2021-02-13T00:00:00Z","1117":"2021-02-12T00:00:00Z","1118":"2021-02-12T00:00:00Z","1120":"2021-02-10T00:00:00Z","1121":"2021-02-10T00:00:00Z","1126":"2021-02-09T00:00:00Z","1127":"2021-02-08T00:00:00Z","1130":"2021-02-07T00:00:00Z","1131":"2021-02-06T00:00:00Z","1132":"2021-02-06T00:00:00Z","1133":"2021-02-05T00:00:00Z","1134":"2021-02-05T00:00:00Z","1136":"2021-02-05T00:00:00Z","1139":"2021-02-03T00:00:00Z","1140":"2021-02-03T00:00:00Z","1141":"2021-02-02T00:00:00Z","1142":"2021-02-02T00:00:00Z","1145":"2021-02-01T00:00:00Z","1146":"2021-02-01T00:00:00Z","1155":"2021-01-31T00:00:00Z","1156":"2021-01-31T00:00:00Z","1157":"2021-01-30T00:00:00Z","1158":"2021-01-30T00:00:00Z","1159":"2021-01-30T00:00:00Z","1160":"2021-01-30T00:00:00Z","1161":"2021-01-30T00:00:00Z","1162":"2021-01-29T00:00:00Z","1163":"2021-01-29T00:00:00Z","1164":"2021-01-29T00:00:00Z","1166":"2021-01-28T00:00:00Z","1167":"2021-01-28T00:00:00Z","1168":"2021-01-28T00:00:00Z","1169":"2021-01-28T00:00:00Z","1170":"2021-01-28T00:00:00Z","1171":"2021-01-28T00:00:00Z","1172":"2021-01-28T00:00:00Z","1173":"2021-01-28T00:00:00Z","1174":"2021-01-28T00:00:00Z","1175":"2021-01-28T00:00:00Z","1176":"2021-01-25T00:00:00Z","1177":"2021-01-25T00:00:00Z","1179":"2021-01-24T00:00:00Z","1180":"2021-01-24T00:00:00Z","1181":"2021-01-24T00:00:00Z","1182":"2021-01-23T00:00:00Z","1183":"2021-01-23T00:00:00Z","1184":"2021-01-23T00:00:00Z","1185":"2021-01-22T00:00:00Z","1186":"2021-01-20T00:00:00Z","1187":"2021-01-20T00:00:00Z","1188":"2021-01-19T00:00:00Z","1191":"2021-01-17T00:00:00Z","1192":"2021-01-17T00:00:00Z","1193":"2021-01-16T00:00:00Z","1194":"2021-01-16T00:00:00Z","1195":"2021-01-16T00:00:00Z","1196":"2021-01-16T00:00:00Z","1197":"2021-01-15T00:00:00Z","1198":"2021-01-15T00:00:00Z","1199":"2021-01-15T00:00:00Z","1200":"2021-01-15T00:00:00Z","1203":"2021-01-14T00:00:00Z","1205":"2021-01-13T00:00:00Z","1210":"2021-01-09T00:00:00Z","1215":"2021-01-05T00:00:00Z","1216":"2021-01-05T00:00:00Z","1217":"2021-01-05T00:00:00Z","1218":"2021-01-04T00:00:00Z","1222":"2021-01-03T00:00:00Z","1223":"2021-01-03T00:00:00Z","1224":"2021-01-03T00:00:00Z","1225":"2021-01-03T00:00:00Z","1226":"2021-01-03T00:00:00Z","1227":"2021-01-02T00:00:00Z","1228":"2021-01-01T00:00:00Z","1229":"2021-01-01T00:00:00Z","1230":"2020-12-31T00:00:00Z","1231":"2020-12-31T00:00:00Z","1233":"2020-12-30T00:00:00Z","1234":"2020-12-28T00:00:00Z","1235":"2020-12-28T00:00:00Z","1236":"2020-12-28T00:00:00Z","1237":"2020-12-28T00:00:00Z","1238":"2020-12-27T00:00:00Z","1239":"2020-12-25T00:00:00Z","1240":"2020-12-25T00:00:00Z","1242":"2020-12-23T00:00:00Z","1243":"2020-12-23T00:00:00Z","1244":"2020-12-22T00:00:00Z","1245":"2020-12-22T00:00:00Z","1246":"2020-12-22T00:00:00Z","1248":"2020-12-21T00:00:00Z","1250":"2020-12-20T00:00:00Z","1251":"2020-12-20T00:00:00Z","1252":"2020-12-20T00:00:00Z","1253":"2020-12-19T00:00:00Z","1254":"2020-12-18T00:00:00Z","1255":"2020-12-18T00:00:00Z","1256":"2020-12-18T00:00:00Z","1258":"2020-12-17T00:00:00Z","1259":"2020-12-17T00:00:00Z","1260":"2020-12-17T00:00:00Z","1262":"2020-12-16T00:00:00Z","1263":"2020-12-15T00:00:00Z","1264":"2020-12-14T00:00:00Z","1266":"2020-12-12T00:00:00Z","1267":"2020-12-12T00:00:00Z","1272":"2020-12-10T00:00:00Z","1273":"2020-12-10T00:00:00Z","1278":"2020-12-09T00:00:00Z","1279":"2020-12-09T00:00:00Z","1281":"2020-12-08T00:00:00Z","1282":"2020-12-07T00:00:00Z","1283":"2020-12-07T00:00:00Z","1284":"2020-12-06T00:00:00Z","1285":"2020-12-06T00:00:00Z","1286":"2020-12-05T00:00:00Z","1287":"2020-12-05T00:00:00Z","1288":"2020-12-03T00:00:00Z","1289":"2020-12-03T00:00:00Z","1291":"2020-12-02T00:00:00Z","1293":"2020-12-01T00:00:00Z","1294":"2020-12-01T00:00:00Z","1295":"2020-12-01T00:00:00Z","1296":"2020-12-01T00:00:00Z","1297":"2020-12-01T00:00:00Z","1298":"2020-12-01T00:00:00Z","1301":"2020-11-30T00:00:00Z","1302":"2020-11-30T00:00:00Z","1303":"2020-11-30T00:00:00Z","1305":"2020-11-29T00:00:00Z","1306":"2020-11-28T00:00:00Z","1307":"2020-11-28T00:00:00Z","1308":"2020-11-28T00:00:00Z","1309":"2020-11-27T00:00:00Z","1310":"2020-11-27T00:00:00Z","1311":"2020-11-27T00:00:00Z","1312":"2020-11-26T00:00:00Z","1314":"2020-11-24T00:00:00Z","1315":"2020-11-24T00:00:00Z","1316":"2020-11-21T00:00:00Z","1317":"2020-11-21T00:00:00Z","1318":"2020-11-21T00:00:00Z","1319":"2020-11-21T00:00:00Z","1320":"2020-11-21T00:00:00Z","1321":"2020-11-19T00:00:00Z","1322":"2020-11-19T00:00:00Z","1323":"2020-11-19T00:00:00Z","1324":"2020-11-17T00:00:00Z","1325":"2020-11-16T00:00:00Z","1326":"2020-11-16T00:00:00Z","1328":"2020-11-15T00:00:00Z","1329":"2020-11-15T00:00:00Z","1330":"2020-11-14T00:00:00Z","1331":"2020-11-14T00:00:00Z","1332":"2020-11-14T00:00:00Z","1333":"2020-11-14T00:00:00Z","1334":"2020-11-14T00:00:00Z","1338":"2020-11-10T00:00:00Z","1339":"2020-11-10T00:00:00Z","1340":"2020-11-10T00:00:00Z","1345":"2020-11-09T00:00:00Z","1347":"2020-11-08T00:00:00Z","1348":"2020-11-08T00:00:00Z","1349":"2020-11-08T00:00:00Z","1350":"2020-11-05T00:00:00Z","1351":"2020-11-05T00:00:00Z","1352":"2020-11-04T00:00:00Z","1353":"2020-11-04T00:00:00Z","1356":"2020-11-03T00:00:00Z","1357":"2020-11-02T00:00:00Z","1358":"2020-11-02T00:00:00Z","1359":"2020-11-02T00:00:00Z","1361":"2020-11-01T00:00:00Z","1362":"2020-11-01T00:00:00Z","1363":"2020-11-01T00:00:00Z","1364":"2020-11-01T00:00:00Z","1366":"2020-11-01T00:00:00Z","1367":"2020-10-31T00:00:00Z","1368":"2020-10-31T00:00:00Z","1369":"2020-10-30T00:00:00Z","1370":"2020-10-30T00:00:00Z","1371":"2020-10-30T00:00:00Z","1372":"2020-10-30T00:00:00Z","1374":"2020-10-29T00:00:00Z","1375":"2020-10-29T00:00:00Z","1376":"2020-10-28T00:00:00Z","1378":"2020-10-26T00:00:00Z","1380":"2020-10-25T00:00:00Z","1381":"2020-10-24T00:00:00Z","1382":"2020-10-23T00:00:00Z","1383":"2020-10-22T00:00:00Z","1385":"2020-10-22T00:00:00Z","1387":"2020-10-21T00:00:00Z","1388":"2020-10-19T00:00:00Z","1389":"2020-10-19T00:00:00Z","1390":"2020-10-18T00:00:00Z","1391":"2020-10-18T00:00:00Z","1392":"2020-10-18T00:00:00Z","1393":"2020-10-17T00:00:00Z","1394":"2020-10-17T00:00:00Z","1395":"2020-10-16T00:00:00Z","1397":"2020-10-15T00:00:00Z","1398":"2020-10-15T00:00:00Z","1402":"2020-10-14T00:00:00Z","1407":"2020-10-12T00:00:00Z","1408":"2020-10-12T00:00:00Z","1409":"2020-10-12T00:00:00Z","1411":"2020-10-10T00:00:00Z","1412":"2020-10-10T00:00:00Z","1413":"2020-10-10T00:00:00Z","1414":"2020-10-09T00:00:00Z","1419":"2020-10-08T00:00:00Z","1420":"2020-10-08T00:00:00Z","1421":"2020-10-08T00:00:00Z","1422":"2020-10-07T00:00:00Z","1423":"2020-10-06T00:00:00Z","1424":"2020-10-06T00:00:00Z","1426":"2020-10-05T00:00:00Z","1427":"2020-10-04T00:00:00Z","1428":"2020-10-04T00:00:00Z","1429":"2020-10-04T00:00:00Z","1430":"2020-10-04T00:00:00Z","1431":"2020-10-03T00:00:00Z","1432":"2020-10-02T00:00:00Z","1433":"2020-10-02T00:00:00Z","1436":"2020-10-02T00:00:00Z","1437":"2020-10-01T00:00:00Z","1438":"2020-10-01T00:00:00Z","1439":"2020-10-01T00:00:00Z","1441":"2020-09-30T00:00:00Z","1443":"2020-09-29T00:00:00Z","1444":"2020-09-28T00:00:00Z","1447":"2020-09-27T00:00:00Z","1448":"2020-09-27T00:00:00Z","1449":"2020-09-27T00:00:00Z","1452":"2020-09-24T00:00:00Z","1453":"2020-09-23T00:00:00Z","1454":"2020-09-21T00:00:00Z","1456":"2020-09-21T00:00:00Z","1458":"2020-09-20T00:00:00Z","1459":"2020-09-20T00:00:00Z","1460":"2020-09-19T00:00:00Z","1461":"2020-09-17T00:00:00Z","1463":"2020-09-16T00:00:00Z","1464":"2020-09-16T00:00:00Z","1466":"2020-09-15T00:00:00Z","1467":"2020-09-15T00:00:00Z","1471":"2020-09-15T00:00:00Z","1472":"2020-09-14T00:00:00Z","1473":"2020-09-14T00:00:00Z","1475":"2020-09-13T00:00:00Z","1476":"2020-09-13T00:00:00Z","1477":"2020-09-13T00:00:00Z","1479":"2020-09-10T00:00:00Z","1480":"2020-09-10T00:00:00Z","1484":"2020-09-09T00:00:00Z","1486":"2020-09-07T00:00:00Z","1487":"2020-09-07T00:00:00Z","1488":"2020-09-07T00:00:00Z","1489":"2020-09-06T00:00:00Z","1490":"2020-09-06T00:00:00Z","1491":"2020-09-06T00:00:00Z","1492":"2020-09-06T00:00:00Z","1493":"2020-09-05T00:00:00Z","1494":"2020-09-05T00:00:00Z","1495":"2020-09-05T00:00:00Z","1496":"2020-09-05T00:00:00Z","1497":"2020-09-02T00:00:00Z","1501":"2020-09-02T00:00:00Z","1502":"2020-09-01T00:00:00Z","1504":"2020-08-31T00:00:00Z","1507":"2020-08-30T00:00:00Z","1508":"2020-08-30T00:00:00Z","1509":"2020-08-30T00:00:00Z","1510":"2020-08-28T00:00:00Z","1511":"2020-08-28T00:00:00Z","1512":"2020-08-28T00:00:00Z","1514":"2020-08-25T00:00:00Z","1517":"2020-08-24T00:00:00Z","1518":"2020-08-23T00:00:00Z","1519":"2020-08-22T00:00:00Z","1520":"2020-08-21T00:00:00Z","1524":"2020-08-19T00:00:00Z","1525":"2020-08-18T00:00:00Z","1528":"2020-08-15T00:00:00Z","1531":"2020-08-10T00:00:00Z","1536":"2020-08-09T00:00:00Z","1537":"2020-08-07T00:00:00Z","1538":"2020-08-06T00:00:00Z","1540":"2020-08-05T00:00:00Z","1542":"2020-08-01T00:00:00Z","1544":"2020-07-31T00:00:00Z","1546":"2020-07-30T00:00:00Z","1547":"2020-07-28T00:00:00Z","1548":"2020-07-22T00:00:00Z","1550":"2020-07-22T00:00:00Z","1555":"2020-07-18T00:00:00Z","1557":"2020-07-15T00:00:00Z","1559":"2020-07-13T00:00:00Z","1563":"2020-07-05T00:00:00Z","1564":"2020-07-03T00:00:00Z","1565":"2020-07-01T00:00:00Z","1566":"2020-06-30T00:00:00Z","1569":"2020-06-27T00:00:00Z","1574":"2020-06-21T00:00:00Z","1577":"2020-06-19T00:00:00Z","1581":"2020-06-15T00:00:00Z","1584":"2020-06-14T00:00:00Z","1589":"2020-06-05T00:00:00Z","1590":"2020-06-04T00:00:00Z","1593":"2020-05-30T00:00:00Z","1597":"2020-05-28T00:00:00Z","1602":"2020-05-19T00:00:00Z","1606":"2020-05-14T00:00:00Z","1607":"2020-05-12T00:00:00Z","1615":"2020-05-10T00:00:00Z","1617":"2020-05-08T00:00:00Z","1618":"2020-05-07T00:00:00Z","1619":"2020-05-05T00:00:00Z","1620":"2020-05-04T00:00:00Z","1622":"2020-05-04T00:00:00Z","1623":"2020-05-04T00:00:00Z","1625":"2020-05-04T00:00:00Z","1626":"2020-05-03T00:00:00Z","1627":"2020-05-01T00:00:00Z","1630":"2020-04-28T00:00:00Z","1631":"2020-04-28T00:00:00Z","1633":"2020-04-27T00:00:00Z","1634":"2020-04-27T00:00:00Z","1635":"2020-04-25T00:00:00Z","1636":"2020-04-25T00:00:00Z","1637":"2020-04-24T00:00:00Z","1638":"2020-04-24T00:00:00Z","1639":"2020-04-24T00:00:00Z","1640":"2020-04-23T00:00:00Z","1642":"2020-04-21T00:00:00Z","1643":"2020-04-21T00:00:00Z","1647":"2020-04-20T00:00:00Z","1649":"2020-04-19T00:00:00Z","1650":"2020-04-19T00:00:00Z","1651":"2020-04-18T00:00:00Z","1652":"2020-04-18T00:00:00Z","1655":"2020-04-16T00:00:00Z","1658":"2020-04-14T00:00:00Z","1659":"2020-04-14T00:00:00Z","1661":"2020-04-12T00:00:00Z","1664":"2020-04-12T00:00:00Z","1665":"2020-04-12T00:00:00Z","1666":"2020-04-12T00:00:00Z","1667":"2020-04-12T00:00:00Z","1669":"2020-04-10T00:00:00Z","1672":"2020-04-10T00:00:00Z","1673":"2020-04-10T00:00:00Z","1674":"2020-04-09T00:00:00Z","1675":"2020-04-09T00:00:00Z","1676":"2020-04-09T00:00:00Z","1680":"2020-04-07T00:00:00Z","1681":"2020-04-05T00:00:00Z","1682":"2020-04-04T00:00:00Z","1683":"2020-04-03T00:00:00Z","1685":"2020-04-02T00:00:00Z","1686":"2020-04-02T00:00:00Z","1687":"2020-04-01T00:00:00Z","1688":"2020-04-01T00:00:00Z","1689":"2020-04-01T00:00:00Z","1691":"2020-03-31T00:00:00Z","1692":"2020-03-31T00:00:00Z","1695":"2020-03-30T00:00:00Z","1696":"2020-03-29T00:00:00Z","1697":"2020-03-28T00:00:00Z","1698":"2020-03-28T00:00:00Z","1699":"2020-03-28T00:00:00Z","1700":"2020-03-27T00:00:00Z","1701":"2020-03-25T00:00:00Z","1702":"2020-03-24T00:00:00Z","1703":"2020-03-24T00:00:00Z","1704":"2020-03-23T00:00:00Z","1705":"2020-03-23T00:00:00Z","1707":"2020-03-23T00:00:00Z","1708":"2020-03-22T00:00:00Z","1709":"2020-03-22T00:00:00Z","1710":"2020-03-22T00:00:00Z","1711":"2020-03-22T00:00:00Z","1712":"2020-03-21T00:00:00Z","1713":"2020-03-21T00:00:00Z","1714":"2020-03-20T00:00:00Z","1716":"2020-03-19T00:00:00Z","1717":"2020-03-19T00:00:00Z","1718":"2020-03-18T00:00:00Z","1721":"2020-03-16T00:00:00Z","1722":"2020-03-16T00:00:00Z","1723":"2020-03-14T00:00:00Z","1724":"2020-03-13T00:00:00Z","1726":"2020-03-12T00:00:00Z","1727":"2020-03-12T00:00:00Z","1728":"2020-03-12T00:00:00Z","1731":"2020-03-11T00:00:00Z","1732":"2020-03-11T00:00:00Z","1733":"2020-03-11T00:00:00Z","1734":"2020-03-11T00:00:00Z","1735":"2020-03-11T00:00:00Z","1736":"2020-03-10T00:00:00Z","1737":"2020-03-10T00:00:00Z","1738":"2020-03-10T00:00:00Z","1739":"2020-03-10T00:00:00Z","1740":"2020-03-10T00:00:00Z","1741":"2020-03-10T00:00:00Z","1742":"2020-03-10T00:00:00Z","1743":"2020-03-10T00:00:00Z","1744":"2020-03-09T00:00:00Z","1745":"2020-03-09T00:00:00Z","1747":"2020-03-09T00:00:00Z","1748":"2020-03-08T00:00:00Z","1749":"2020-03-08T00:00:00Z","1750":"2020-03-08T00:00:00Z","1751":"2020-03-08T00:00:00Z","1752":"2020-03-08T00:00:00Z","1753":"2020-03-08T00:00:00Z","1754":"2020-03-08T00:00:00Z","1755":"2020-03-08T00:00:00Z","1756":"2020-03-08T00:00:00Z","1757":"2020-03-08T00:00:00Z","1758":"2020-03-08T00:00:00Z","1759":"2020-03-07T00:00:00Z","1760":"2020-03-06T00:00:00Z","1761":"2020-03-06T00:00:00Z","1762":"2020-03-06T00:00:00Z","1763":"2020-03-06T00:00:00Z","1764":"2020-03-06T00:00:00Z","1766":"2020-03-05T00:00:00Z","1767":"2020-03-05T00:00:00Z","1770":"2020-03-05T00:00:00Z","1771":"2020-03-04T00:00:00Z","1772":"2020-03-04T00:00:00Z","1773":"2020-03-04T00:00:00Z","1774":"2020-03-03T00:00:00Z","1776":"2020-03-02T00:00:00Z","1777":"2020-03-01T00:00:00Z","1778":"2020-03-01T00:00:00Z","1779":"2020-03-01T00:00:00Z","1780":"2020-03-01T00:00:00Z","1781":"2020-03-01T00:00:00Z","1782":"2020-02-29T00:00:00Z","1783":"2020-02-29T00:00:00Z","1784":"2020-02-29T00:00:00Z","1785":"2020-02-29T00:00:00Z","1786":"2020-02-28T00:00:00Z","1787":"2020-02-28T00:00:00Z","1790":"2020-02-27T00:00:00Z","1791":"2020-02-27T00:00:00Z","1792":"2020-02-26T00:00:00Z","1793":"2020-02-26T00:00:00Z","1794":"2020-02-26T00:00:00Z","1795":"2020-02-26T00:00:00Z","1796":"2020-02-26T00:00:00Z","1797":"2020-02-26T00:00:00Z","1798":"2020-02-24T00:00:00Z","1799":"2020-02-23T00:00:00Z","1800":"2020-02-23T00:00:00Z","1801":"2020-02-22T00:00:00Z","1802":"2020-02-22T00:00:00Z","1803":"2020-02-22T00:00:00Z","1804":"2020-02-22T00:00:00Z","1806":"2020-02-21T00:00:00Z","1810":"2020-02-19T00:00:00Z","1811":"2020-02-18T00:00:00Z","1812":"2020-02-18T00:00:00Z","1813":"2020-02-18T00:00:00Z","1814":"2020-02-18T00:00:00Z","1815":"2020-02-18T00:00:00Z","1820":"2020-02-17T00:00:00Z","1821":"2020-02-16T00:00:00Z","1822":"2020-02-16T00:00:00Z","1823":"2020-02-16T00:00:00Z","1824":"2020-02-15T00:00:00Z","1825":"2020-02-15T00:00:00Z","1826":"2020-02-14T00:00:00Z","1827":"2020-02-14T00:00:00Z","1828":"2020-02-14T00:00:00Z","1829":"2020-02-14T00:00:00Z","1830":"2020-02-14T00:00:00Z","1831":"2020-02-14T00:00:00Z","1833":"2020-02-14T00:00:00Z","1834":"2020-02-13T00:00:00Z","1835":"2020-02-13T00:00:00Z","1836":"2020-02-13T00:00:00Z","1837":"2020-02-12T00:00:00Z","1840":"2020-02-11T00:00:00Z","1841":"2020-02-11T00:00:00Z","1842":"2020-02-11T00:00:00Z","1843":"2020-02-11T00:00:00Z","1844":"2020-02-11T00:00:00Z","1845":"2020-02-11T00:00:00Z","1846":"2020-02-11T00:00:00Z","1847":"2020-02-10T00:00:00Z","1848":"2020-02-10T00:00:00Z","1849":"2020-02-10T00:00:00Z","1850":"2020-02-09T00:00:00Z","1851":"2020-02-09T00:00:00Z","1852":"2020-02-09T00:00:00Z","1853":"2020-02-09T00:00:00Z","1854":"2020-02-09T00:00:00Z","1856":"2020-02-07T00:00:00Z","1857":"2020-02-07T00:00:00Z","1859":"2020-02-07T00:00:00Z","1860":"2020-02-06T00:00:00Z","1862":"2020-02-05T00:00:00Z","1864":"2020-02-05T00:00:00Z","1865":"2020-02-04T00:00:00Z","1866":"2020-02-04T00:00:00Z","1867":"2020-02-04T00:00:00Z","1868":"2020-02-04T00:00:00Z","1869":"2020-02-02T00:00:00Z","1870":"2020-02-02T00:00:00Z","1871":"2020-02-02T00:00:00Z","1872":"2020-02-02T00:00:00Z","1873":"2020-02-02T00:00:00Z","1874":"2020-02-02T00:00:00Z","1876":"2020-02-02T00:00:00Z","1877":"2020-02-01T00:00:00Z","1878":"2020-02-01T00:00:00Z","1879":"2020-01-31T00:00:00Z","1880":"2020-01-31T00:00:00Z","1881":"2020-01-31T00:00:00Z","1882":"2020-01-31T00:00:00Z","1884":"2020-01-31T00:00:00Z","1885":"2020-01-31T00:00:00Z","1886":"2020-01-29T00:00:00Z","1888":"2020-01-28T00:00:00Z","1889":"2020-01-28T00:00:00Z","1890":"2020-01-28T00:00:00Z","1891":"2020-01-27T00:00:00Z","1892":"2020-01-27T00:00:00Z","1893":"2020-01-27T00:00:00Z","1894":"2020-01-27T00:00:00Z","1895":"2020-01-27T00:00:00Z","1896":"2020-01-27T00:00:00Z","1897":"2020-01-27T00:00:00Z","1898":"2020-01-27T00:00:00Z","1899":"2020-01-27T00:00:00Z","1900":"2020-01-26T00:00:00Z","1901":"2020-01-26T00:00:00Z","1902":"2020-01-26T00:00:00Z","1903":"2020-01-26T00:00:00Z","1904":"2020-01-26T00:00:00Z","1905":"2020-01-25T00:00:00Z","1906":"2020-01-24T00:00:00Z","1907":"2020-01-24T00:00:00Z","1908":"2020-01-24T00:00:00Z","1909":"2020-01-24T00:00:00Z","1910":"2020-01-23T00:00:00Z","1911":"2020-01-23T00:00:00Z","1912":"2020-01-23T00:00:00Z","1913":"2020-01-23T00:00:00Z","1914":"2020-01-22T00:00:00Z","1915":"2020-01-22T00:00:00Z","1916":"2020-01-22T00:00:00Z","1920":"2020-01-21T00:00:00Z","1921":"2020-01-21T00:00:00Z","1922":"2020-01-21T00:00:00Z","1923":"2020-01-21T00:00:00Z","1924":"2020-01-21T00:00:00Z","1925":"2020-01-21T00:00:00Z","1926":"2020-01-21T00:00:00Z","1930":"2020-01-21T00:00:00Z","1931":"2020-01-21T00:00:00Z","1932":"2020-01-20T00:00:00Z","1933":"2020-01-20T00:00:00Z","1934":"2020-01-20T00:00:00Z","1935":"2020-01-20T00:00:00Z","1936":"2020-01-20T00:00:00Z","1937":"2020-01-20T00:00:00Z","1938":"2020-01-20T00:00:00Z","1939":"2020-01-20T00:00:00Z","1940":"2020-01-20T00:00:00Z","1941":"2020-01-20T00:00:00Z","1942":"2020-01-20T00:00:00Z","1943":"2020-01-19T00:00:00Z","1944":"2020-01-19T00:00:00Z","1945":"2020-01-19T00:00:00Z","1946":"2020-01-19T00:00:00Z","1947":"2020-01-19T00:00:00Z","1948":"2020-01-19T00:00:00Z","1949":"2020-01-19T00:00:00Z","1950":"2020-01-19T00:00:00Z","1951":"2020-01-19T00:00:00Z","1952":"2020-01-19T00:00:00Z","1953":"2020-01-19T00:00:00Z","1954":"2020-01-19T00:00:00Z","1955":"2020-01-19T00:00:00Z","1956":"2020-01-19T00:00:00Z","1957":"2020-01-19T00:00:00Z","1958":"2020-01-19T00:00:00Z","1959":"2020-01-19T00:00:00Z","1960":"2020-01-19T00:00:00Z","1961":"2020-01-18T00:00:00Z","1962":"2020-01-18T00:00:00Z","1964":"2020-01-17T00:00:00Z","1965":"2020-01-16T00:00:00Z","1966":"2020-01-16T00:00:00Z","1967":"2020-01-15T00:00:00Z","1968":"2020-01-15T00:00:00Z","1969":"2020-01-15T00:00:00Z","1970":"2020-01-15T00:00:00Z","1972":"2020-01-14T00:00:00Z","1973":"2020-01-14T00:00:00Z","1974":"2020-01-14T00:00:00Z","1975":"2020-01-14T00:00:00Z","1976":"2020-01-13T00:00:00Z","1977":"2020-01-13T00:00:00Z","1978":"2020-01-13T00:00:00Z","1979":"2020-01-13T00:00:00Z","1980":"2020-01-13T00:00:00Z","1982":"2020-01-12T00:00:00Z","1983":"2020-01-12T00:00:00Z","1985":"2020-01-10T00:00:00Z","1987":"2020-01-08T00:00:00Z","1988":"2020-01-08T00:00:00Z","1989":"2020-01-08T00:00:00Z","1990":"2020-01-08T00:00:00Z","1991":"2020-01-08T00:00:00Z","1992":"2020-01-07T00:00:00Z","1993":"2020-01-07T00:00:00Z","1994":"2020-01-07T00:00:00Z","1995":"2020-01-07T00:00:00Z","1996":"2020-01-07T00:00:00Z","1998":"2020-01-06T00:00:00Z","1999":"2020-01-06T00:00:00Z","2003":"2020-01-06T00:00:00Z","2004":"2020-01-05T00:00:00Z","2005":"2020-01-05T00:00:00Z","2006":"2020-01-05T00:00:00Z","2007":"2020-01-05T00:00:00Z","2008":"2020-01-05T00:00:00Z","2009":"2020-01-04T00:00:00Z","2010":"2020-01-04T00:00:00Z","2011":"2020-01-03T00:00:00Z","2012":"2020-01-02T00:00:00Z","2013":"2020-01-02T00:00:00Z","2014":"2020-01-02T00:00:00Z","2015":"2020-01-01T00:00:00Z","2016":"2020-01-01T00:00:00Z","2017":"2020-01-01T00:00:00Z","2018":"2020-01-01T00:00:00Z","2019":"2020-01-01T00:00:00Z","2020":"2019-12-31T00:00:00Z","2021":"2019-12-31T00:00:00Z","2022":"2019-12-31T00:00:00Z","2023":"2019-12-31T00:00:00Z","2025":"2019-12-31T00:00:00Z","2026":"2019-12-30T00:00:00Z","2027":"2019-12-30T00:00:00Z","2028":"2019-12-30T00:00:00Z","2029":"2019-12-30T00:00:00Z","2030":"2019-12-29T00:00:00Z","2031":"2019-12-29T00:00:00Z","2032":"2019-12-29T00:00:00Z","2033":"2019-12-27T00:00:00Z","2034":"2019-12-27T00:00:00Z","2035":"2019-12-25T00:00:00Z","2036":"2019-12-25T00:00:00Z","2037":"2019-12-25T00:00:00Z","2038":"2019-12-25T00:00:00Z","2039":"2019-12-25T00:00:00Z","2040":"2019-12-24T00:00:00Z","2042":"2019-12-24T00:00:00Z","2043":"2019-12-23T00:00:00Z","2044":"2019-12-23T00:00:00Z","2045":"2019-12-23T00:00:00Z","2046":"2019-12-23T00:00:00Z","2047":"2019-12-23T00:00:00Z","2048":"2019-12-23T00:00:00Z","2050":"2019-12-22T00:00:00Z","2051":"2019-12-22T00:00:00Z","2052":"2019-12-22T00:00:00Z","2053":"2019-12-22T00:00:00Z","2054":"2019-12-22T00:00:00Z","2055":"2019-12-22T00:00:00Z","2056":"2019-12-22T00:00:00Z","2057":"2019-12-22T00:00:00Z","2058":"2019-12-21T00:00:00Z","2060":"2019-12-19T00:00:00Z","2061":"2019-12-19T00:00:00Z","2063":"2019-12-16T00:00:00Z","2064":"2019-12-15T00:00:00Z","2065":"2019-12-14T00:00:00Z","2067":"2019-12-13T00:00:00Z","2068":"2019-12-12T00:00:00Z","2071":"2019-12-10T00:00:00Z","2072":"2019-12-10T00:00:00Z","2073":"2019-12-10T00:00:00Z","2074":"2019-12-09T00:00:00Z","2075":"2019-06-13T00:00:00Z","2077":"2019-06-12T00:00:00Z","2079":"2019-06-12T00:00:00Z","2080":"2019-06-12T00:00:00Z","2081":"2019-06-12T00:00:00Z","2082":"2019-06-12T00:00:00Z","2083":"2019-06-12T00:00:00Z","2084":"2019-06-12T00:00:00Z","2085":"2019-06-11T00:00:00Z","2086":"2019-06-11T00:00:00Z","2087":"2019-06-11T00:00:00Z","2088":"2019-06-11T00:00:00Z","2089":"2019-06-11T00:00:00Z","2090":"2019-06-11T00:00:00Z","2091":"2019-06-10T00:00:00Z","2093":"2019-06-10T00:00:00Z","2094":"2019-06-09T00:00:00Z","2095":"2019-06-09T00:00:00Z","2096":"2019-06-09T00:00:00Z","2097":"2019-06-09T00:00:00Z","2098":"2019-06-09T00:00:00Z","2099":"2019-06-09T00:00:00Z","2100":"2019-06-09T00:00:00Z","2101":"2019-06-08T00:00:00Z","2102":"2019-06-08T00:00:00Z","2103":"2019-06-08T00:00:00Z","2104":"2019-06-08T00:00:00Z","2105":"2019-06-08T00:00:00Z","2106":"2019-06-08T00:00:00Z","2107":"2019-06-08T00:00:00Z","2108":"2019-06-08T00:00:00Z","2109":"2019-06-08T00:00:00Z","2110":"2019-06-07T00:00:00Z","2111":"2019-06-07T00:00:00Z","2112":"2019-06-07T00:00:00Z","2113":"2019-06-07T00:00:00Z","2114":"2019-06-07T00:00:00Z","2115":"2019-06-07T00:00:00Z","2116":"2019-06-06T00:00:00Z","2117":"2019-06-06T00:00:00Z","2118":"2019-06-06T00:00:00Z","2119":"2019-06-05T00:00:00Z","2120":"2019-06-05T00:00:00Z","2121":"2019-06-05T00:00:00Z","2122":"2019-06-05T00:00:00Z","2123":"2019-06-04T00:00:00Z","2124":"2019-06-04T00:00:00Z","2125":"2019-06-04T00:00:00Z","2126":"2019-06-04T00:00:00Z","2127":"2019-06-04T00:00:00Z","2128":"2019-06-04T00:00:00Z","2129":"2019-06-03T00:00:00Z","2130":"2019-06-03T00:00:00Z","2131":"2019-06-03T00:00:00Z","2132":"2019-06-03T00:00:00Z","2133":"2019-06-02T00:00:00Z","2134":"2019-06-02T00:00:00Z","2135":"2019-06-02T00:00:00Z","2136":"2019-06-02T00:00:00Z","2137":"2019-06-02T00:00:00Z","2138":"2019-06-02T00:00:00Z","2139":"2019-06-02T00:00:00Z","2140":"2019-06-01T00:00:00Z","2141":"2019-06-01T00:00:00Z","2142":"2019-05-31T00:00:00Z","2143":"2019-05-31T00:00:00Z","2144":"2019-05-31T00:00:00Z","2147":"2019-05-30T00:00:00Z","2148":"2019-05-30T00:00:00Z","2149":"2019-05-30T00:00:00Z","2150":"2019-05-30T00:00:00Z","2151":"2019-05-30T00:00:00Z","2152":"2019-05-29T00:00:00Z","2153":"2019-05-29T00:00:00Z","2154":"2019-05-29T00:00:00Z","2155":"2019-05-29T00:00:00Z","2156":"2019-05-29T00:00:00Z","2157":"2019-05-28T00:00:00Z","2158":"2019-05-28T00:00:00Z","2159":"2019-05-28T00:00:00Z","2160":"2019-05-28T00:00:00Z","2161":"2019-05-28T00:00:00Z","2162":"2019-05-28T00:00:00Z","2163":"2019-05-28T00:00:00Z","2164":"2019-05-28T00:00:00Z","2165":"2019-05-28T00:00:00Z","2166":"2019-05-28T00:00:00Z","2167":"2019-05-28T00:00:00Z","2168":"2019-05-28T00:00:00Z","2169":"2019-05-27T00:00:00Z","2170":"2019-05-27T00:00:00Z","2171":"2019-05-26T00:00:00Z","2172":"2019-05-26T00:00:00Z","2173":"2019-05-25T00:00:00Z","2174":"2019-05-25T00:00:00Z","2175":"2019-05-25T00:00:00Z","2176":"2019-05-25T00:00:00Z","2177":"2019-05-25T00:00:00Z","2178":"2019-05-25T00:00:00Z","2179":"2019-05-25T00:00:00Z","2180":"2019-05-25T00:00:00Z","2181":"2019-05-25T00:00:00Z","2182":"2019-05-25T00:00:00Z","2183":"2019-05-24T00:00:00Z","2184":"2019-05-24T00:00:00Z","2185":"2019-05-24T00:00:00Z","2186":"2019-05-24T00:00:00Z","2188":"2019-05-23T00:00:00Z","2189":"2019-05-23T00:00:00Z","2190":"2019-05-22T00:00:00Z","2192":"2019-05-21T00:00:00Z","2193":"2019-05-21T00:00:00Z","2194":"2019-05-21T00:00:00Z","2195":"2019-05-21T00:00:00Z","2196":"2019-05-21T00:00:00Z","2197":"2019-05-21T00:00:00Z","2199":"2019-05-21T00:00:00Z","2200":"2019-05-21T00:00:00Z","2201":"2019-05-20T00:00:00Z","2202":"2019-05-20T00:00:00Z","2203":"2019-05-19T00:00:00Z","2204":"2019-05-19T00:00:00Z","2205":"2019-05-19T00:00:00Z","2206":"2019-05-19T00:00:00Z","2207":"2019-05-19T00:00:00Z","2208":"2019-05-19T00:00:00Z","2209":"2019-05-19T00:00:00Z","2210":"2019-05-18T00:00:00Z","2211":"2019-05-18T00:00:00Z","2212":"2019-05-18T00:00:00Z","2213":"2019-05-18T00:00:00Z","2214":"2019-05-18T00:00:00Z","2215":"2019-05-17T00:00:00Z","2216":"2019-05-17T00:00:00Z","2217":"2019-05-17T00:00:00Z","2218":"2019-05-17T00:00:00Z","2219":"2019-05-17T00:00:00Z","2220":"2019-05-16T00:00:00Z","2221":"2019-05-16T00:00:00Z","2222":"2019-05-16T00:00:00Z","2223":"2019-05-15T00:00:00Z","2224":"2019-05-15T00:00:00Z","2225":"2019-05-15T00:00:00Z","2226":"2019-05-14T00:00:00Z","2227":"2019-05-14T00:00:00Z","2228":"2019-05-14T00:00:00Z","2229":"2019-05-14T00:00:00Z","2230":"2019-05-14T00:00:00Z","2232":"2019-05-13T00:00:00Z","2233":"2019-05-13T00:00:00Z","2234":"2019-05-13T00:00:00Z","2235":"2019-05-13T00:00:00Z","2236":"2019-05-13T00:00:00Z","2237":"2019-05-13T00:00:00Z","2239":"2019-05-12T00:00:00Z","2240":"2019-05-12T00:00:00Z","2241":"2019-05-12T00:00:00Z","2242":"2019-05-12T00:00:00Z","2243":"2019-05-12T00:00:00Z","2244":"2019-05-11T00:00:00Z","2245":"2019-05-11T00:00:00Z","2246":"2019-05-11T00:00:00Z","2247":"2019-05-11T00:00:00Z","2248":"2019-05-11T00:00:00Z","2249":"2019-05-11T00:00:00Z","2250":"2019-05-11T00:00:00Z","2252":"2019-05-10T00:00:00Z","2253":"2019-05-10T00:00:00Z","2254":"2019-05-10T00:00:00Z","2255":"2019-05-10T00:00:00Z","2256":"2019-05-10T00:00:00Z","2257":"2019-05-10T00:00:00Z","2259":"2019-05-09T00:00:00Z","2260":"2019-05-09T00:00:00Z","2261":"2019-05-09T00:00:00Z","2262":"2019-05-09T00:00:00Z","2263":"2019-05-09T00:00:00Z","2264":"2019-05-09T00:00:00Z","2265":"2019-05-09T00:00:00Z","2266":"2019-05-08T00:00:00Z","2267":"2019-05-08T00:00:00Z","2268":"2019-05-08T00:00:00Z","2269":"2019-05-08T00:00:00Z","2270":"2019-05-08T00:00:00Z","2271":"2019-05-08T00:00:00Z","2272":"2019-05-08T00:00:00Z","2273":"2019-05-07T00:00:00Z","2274":"2019-05-07T00:00:00Z","2275":"2019-05-07T00:00:00Z","2276":"2019-05-07T00:00:00Z","2278":"2019-05-06T00:00:00Z","2279":"2019-05-06T00:00:00Z","2280":"2019-05-05T00:00:00Z","2281":"2019-05-05T00:00:00Z","2282":"2019-05-05T00:00:00Z","2283":"2019-05-05T00:00:00Z","2284":"2019-05-04T00:00:00Z","2285":"2019-05-04T00:00:00Z","2286":"2019-05-04T00:00:00Z","2287":"2019-05-04T00:00:00Z","2288":"2019-05-03T00:00:00Z","2289":"2019-05-03T00:00:00Z","2290":"2019-05-02T00:00:00Z","2291":"2019-05-02T00:00:00Z","2292":"2019-05-01T00:00:00Z","2295":"2019-04-29T00:00:00Z","2296":"2019-04-29T00:00:00Z","2297":"2019-04-28T00:00:00Z","2298":"2019-04-28T00:00:00Z","2299":"2019-04-28T00:00:00Z","2300":"2019-04-27T00:00:00Z","2301":"2019-04-27T00:00:00Z","2302":"2019-04-27T00:00:00Z","2303":"2019-04-27T00:00:00Z","2304":"2019-04-27T00:00:00Z","2305":"2019-04-27T00:00:00Z","2306":"2019-04-27T00:00:00Z","2307":"2019-04-27T00:00:00Z","2309":"2019-04-25T00:00:00Z","2311":"2019-04-25T00:00:00Z","2312":"2019-04-25T00:00:00Z","2313":"2019-04-25T00:00:00Z","2314":"2019-04-24T00:00:00Z","2316":"2019-04-24T00:00:00Z","2317":"2019-04-24T00:00:00Z","2318":"2019-04-22T00:00:00Z","2320":"2019-04-22T00:00:00Z","2321":"2019-04-22T00:00:00Z","2322":"2019-04-22T00:00:00Z","2323":"2019-04-21T00:00:00Z","2324":"2019-04-21T00:00:00Z","2326":"2019-04-21T00:00:00Z","2327":"2019-04-21T00:00:00Z","2328":"2019-04-20T00:00:00Z","2329":"2019-04-20T00:00:00Z","2330":"2019-04-20T00:00:00Z","2332":"2019-04-18T00:00:00Z","2333":"2019-04-17T00:00:00Z","2334":"2019-04-17T00:00:00Z","2335":"2019-04-17T00:00:00Z","2337":"2019-04-16T00:00:00Z","2338":"2019-04-16T00:00:00Z","2341":"2019-04-15T00:00:00Z","2342":"2019-04-15T00:00:00Z","2343":"2019-04-14T00:00:00Z","2344":"2019-04-14T00:00:00Z","2345":"2019-04-13T00:00:00Z","2346":"2019-04-13T00:00:00Z","2347":"2019-04-13T00:00:00Z","2348":"2019-04-13T00:00:00Z","2349":"2019-04-13T00:00:00Z","2350":"2019-04-13T00:00:00Z","2351":"2019-04-13T00:00:00Z","2352":"2019-04-12T00:00:00Z","2353":"2019-04-12T00:00:00Z","2354":"2019-04-12T00:00:00Z","2355":"2019-04-12T00:00:00Z","2356":"2019-04-12T00:00:00Z","2359":"2019-04-11T00:00:00Z","2360":"2019-04-11T00:00:00Z","2361":"2019-04-11T00:00:00Z","2364":"2019-04-10T00:00:00Z","2365":"2019-04-10T00:00:00Z","2366":"2019-04-09T00:00:00Z","2367":"2019-04-09T00:00:00Z","2368":"2019-04-09T00:00:00Z","2369":"2019-04-09T00:00:00Z","2373":"2019-04-08T00:00:00Z","2374":"2019-04-08T00:00:00Z","2375":"2019-04-08T00:00:00Z","2376":"2019-04-07T00:00:00Z","2377":"2019-04-07T00:00:00Z","2378":"2019-04-06T00:00:00Z","2379":"2019-04-06T00:00:00Z","2380":"2019-04-06T00:00:00Z","2381":"2019-04-05T00:00:00Z","2382":"2019-04-05T00:00:00Z","2383":"2019-04-05T00:00:00Z","2384":"2019-04-05T00:00:00Z","2385":"2019-04-05T00:00:00Z","2386":"2019-04-04T00:00:00Z","2387":"2019-04-04T00:00:00Z","2388":"2019-04-04T00:00:00Z","2389":"2019-04-03T00:00:00Z","2390":"2019-04-03T00:00:00Z","2391":"2019-04-02T00:00:00Z","2392":"2019-04-02T00:00:00Z","2393":"2019-04-02T00:00:00Z","2394":"2019-04-02T00:00:00Z","2397":"2019-04-01T00:00:00Z","2398":"2019-04-01T00:00:00Z","2399":"2019-04-01T00:00:00Z","2400":"2019-04-01T00:00:00Z","2401":"2019-03-31T00:00:00Z","2402":"2019-03-31T00:00:00Z","2403":"2019-03-31T00:00:00Z","2404":"2019-03-30T00:00:00Z","2405":"2019-03-30T00:00:00Z","2406":"2019-03-30T00:00:00Z","2407":"2019-03-30T00:00:00Z","2408":"2019-03-30T00:00:00Z","2409":"2019-03-29T00:00:00Z","2410":"2019-03-29T00:00:00Z","2412":"2019-03-28T00:00:00Z","2413":"2019-03-28T00:00:00Z","2414":"2019-03-28T00:00:00Z","2415":"2019-03-28T00:00:00Z","2417":"2019-03-28T00:00:00Z","2418":"2019-03-28T00:00:00Z","2419":"2019-03-28T00:00:00Z","2420":"2019-03-28T00:00:00Z","2421":"2019-03-28T00:00:00Z","2422":"2019-03-28T00:00:00Z","2423":"2019-03-28T00:00:00Z","2424":"2019-03-25T00:00:00Z","2426":"2019-03-25T00:00:00Z","2427":"2019-03-25T00:00:00Z","2428":"2019-03-25T00:00:00Z","2429":"2019-03-25T00:00:00Z","2430":"2019-03-25T00:00:00Z","2431":"2019-03-25T00:00:00Z","2432":"2019-03-25T00:00:00Z","2433":"2019-03-24T00:00:00Z","2434":"2019-03-24T00:00:00Z","2435":"2019-03-24T00:00:00Z","2436":"2019-03-24T00:00:00Z","2437":"2019-03-23T00:00:00Z","2438":"2019-03-23T00:00:00Z","2439":"2019-03-23T00:00:00Z","2440":"2019-03-23T00:00:00Z","2441":"2019-03-23T00:00:00Z","2442":"2019-03-23T00:00:00Z","2443":"2019-03-22T00:00:00Z","2444":"2019-03-22T00:00:00Z","2445":"2019-03-22T00:00:00Z","2446":"2019-03-22T00:00:00Z","2447":"2019-03-21T00:00:00Z","2448":"2019-03-21T00:00:00Z","2449":"2019-03-21T00:00:00Z","2450":"2019-03-21T00:00:00Z","2452":"2019-03-21T00:00:00Z","2454":"2019-03-20T00:00:00Z","2455":"2019-03-20T00:00:00Z","2456":"2019-03-20T00:00:00Z","2457":"2019-03-20T00:00:00Z","2458":"2019-03-20T00:00:00Z","2459":"2019-03-20T00:00:00Z","2460":"2019-03-20T00:00:00Z","2461":"2019-03-19T00:00:00Z","2462":"2019-03-19T00:00:00Z","2463":"2019-03-19T00:00:00Z","2464":"2019-03-19T00:00:00Z","2465":"2019-03-19T00:00:00Z","2466":"2019-03-18T00:00:00Z","2467":"2019-03-18T00:00:00Z","2468":"2019-03-18T00:00:00Z","2469":"2019-03-17T00:00:00Z","2470":"2019-03-16T00:00:00Z","2471":"2019-03-16T00:00:00Z","2472":"2019-03-16T00:00:00Z","2473":"2019-03-16T00:00:00Z","2474":"2019-03-16T00:00:00Z","2475":"2019-03-15T00:00:00Z","2476":"2019-03-15T00:00:00Z","2477":"2019-03-15T00:00:00Z","2478":"2019-03-14T00:00:00Z","2479":"2019-03-14T00:00:00Z","2480":"2019-03-13T00:00:00Z","2481":"2019-03-13T00:00:00Z","2482":"2019-03-13T00:00:00Z","2483":"2019-03-13T00:00:00Z","2484":"2019-03-13T00:00:00Z","2485":"2019-03-12T00:00:00Z","2487":"2019-03-12T00:00:00Z","2488":"2019-03-12T00:00:00Z","2490":"2019-03-12T00:00:00Z","2491":"2019-03-11T00:00:00Z","2492":"2019-03-11T00:00:00Z","2493":"2019-03-11T00:00:00Z","2494":"2019-03-11T00:00:00Z","2496":"2019-03-11T00:00:00Z","2497":"2019-03-11T00:00:00Z","2498":"2019-03-11T00:00:00Z","2499":"2019-03-09T00:00:00Z","2500":"2019-03-09T00:00:00Z","2501":"2019-03-09T00:00:00Z","2502":"2019-03-08T00:00:00Z","2503":"2019-03-08T00:00:00Z","2504":"2019-03-08T00:00:00Z","2505":"2019-03-08T00:00:00Z","2506":"2019-03-08T00:00:00Z","2507":"2019-03-08T00:00:00Z","2508":"2019-03-08T00:00:00Z","2509":"2019-03-07T00:00:00Z","2510":"2019-03-07T00:00:00Z","2511":"2019-03-07T00:00:00Z","2512":"2019-03-07T00:00:00Z","2513":"2019-03-07T00:00:00Z","2514":"2019-03-07T00:00:00Z","2515":"2019-03-07T00:00:00Z","2516":"2019-03-07T00:00:00Z","2517":"2019-03-06T00:00:00Z","2518":"2019-03-05T00:00:00Z","2519":"2019-03-05T00:00:00Z","2520":"2019-03-05T00:00:00Z","2521":"2019-03-05T00:00:00Z","2522":"2019-03-04T00:00:00Z","2523":"2019-03-04T00:00:00Z","2524":"2019-03-04T00:00:00Z","2525":"2019-03-04T00:00:00Z","2526":"2019-03-04T00:00:00Z","2527":"2019-03-03T00:00:00Z","2528":"2019-03-03T00:00:00Z","2529":"2019-03-03T00:00:00Z","2530":"2019-03-02T00:00:00Z","2531":"2019-03-02T00:00:00Z","2532":"2019-03-02T00:00:00Z","2533":"2019-03-01T00:00:00Z","2534":"2019-02-28T00:00:00Z","2536":"2019-02-27T00:00:00Z","2537":"2019-02-27T00:00:00Z","2539":"2019-02-26T00:00:00Z","2540":"2019-02-26T00:00:00Z","2541":"2019-02-25T00:00:00Z","2543":"2019-02-25T00:00:00Z","2544":"2019-02-25T00:00:00Z","2545":"2019-02-25T00:00:00Z","2546":"2019-02-25T00:00:00Z","2547":"2019-02-25T00:00:00Z","2548":"2019-02-25T00:00:00Z","2549":"2019-02-25T00:00:00Z","2550":"2019-02-25T00:00:00Z","2551":"2019-02-25T00:00:00Z","2552":"2019-02-25T00:00:00Z","2553":"2019-02-25T00:00:00Z","2554":"2019-02-25T00:00:00Z","2555":"2019-02-23T00:00:00Z","2556":"2019-02-23T00:00:00Z","2557":"2019-02-22T00:00:00Z","2558":"2019-02-22T00:00:00Z","2559":"2019-02-22T00:00:00Z","2562":"2019-02-21T00:00:00Z","2563":"2019-02-21T00:00:00Z","2564":"2019-02-20T00:00:00Z","2565":"2019-02-20T00:00:00Z","2568":"2019-02-19T00:00:00Z","2569":"2019-02-19T00:00:00Z","2570":"2019-02-19T00:00:00Z","2571":"2019-02-19T00:00:00Z","2572":"2019-02-18T00:00:00Z","2573":"2019-02-18T00:00:00Z","2574":"2019-02-18T00:00:00Z","2575":"2019-02-17T00:00:00Z","2576":"2019-02-17T00:00:00Z","2577":"2019-02-17T00:00:00Z","2578":"2019-02-16T00:00:00Z","2579":"2019-02-16T00:00:00Z","2580":"2019-02-16T00:00:00Z","2581":"2019-02-15T00:00:00Z","2582":"2019-02-15T00:00:00Z","2583":"2019-02-15T00:00:00Z","2584":"2019-02-14T00:00:00Z","2585":"2019-02-14T00:00:00Z","2588":"2019-02-13T00:00:00Z","2589":"2019-02-12T00:00:00Z","2592":"2019-02-12T00:00:00Z","2593":"2019-02-12T00:00:00Z","2594":"2019-02-10T00:00:00Z","2595":"2019-02-10T00:00:00Z","2596":"2019-02-10T00:00:00Z","2597":"2019-02-09T00:00:00Z","2598":"2019-02-09T00:00:00Z","2599":"2019-02-09T00:00:00Z","2600":"2019-02-09T00:00:00Z","2601":"2019-02-09T00:00:00Z","2602":"2019-02-09T00:00:00Z","2603":"2019-02-08T00:00:00Z","2604":"2019-02-08T00:00:00Z","2606":"2019-02-07T00:00:00Z","2607":"2019-02-06T00:00:00Z","2608":"2019-02-06T00:00:00Z","2609":"2019-02-06T00:00:00Z","2610":"2019-02-05T00:00:00Z","2611":"2019-02-05T00:00:00Z","2612":"2019-02-05T00:00:00Z","2613":"2019-02-05T00:00:00Z","2614":"2019-02-05T00:00:00Z","2615":"2019-02-04T00:00:00Z","2616":"2019-02-04T00:00:00Z","2617":"2019-02-04T00:00:00Z","2618":"2019-02-04T00:00:00Z","2619":"2019-02-04T00:00:00Z","2620":"2019-02-04T00:00:00Z","2621":"2019-02-03T00:00:00Z","2622":"2019-02-03T00:00:00Z","2623":"2019-02-03T00:00:00Z","2624":"2019-02-03T00:00:00Z","2625":"2019-02-03T00:00:00Z","2626":"2019-02-03T00:00:00Z","2627":"2019-02-02T00:00:00Z","2628":"2019-02-02T00:00:00Z","2629":"2019-02-02T00:00:00Z","2630":"2019-02-02T00:00:00Z","2631":"2019-02-02T00:00:00Z","2632":"2019-02-02T00:00:00Z","2633":"2019-02-02T00:00:00Z","2634":"2019-02-02T00:00:00Z","2635":"2019-02-02T00:00:00Z","2636":"2019-02-02T00:00:00Z","2637":"2019-02-02T00:00:00Z","2638":"2019-02-01T00:00:00Z","2639":"2019-02-01T00:00:00Z","2640":"2019-02-01T00:00:00Z","2641":"2019-02-01T00:00:00Z","2642":"2019-01-31T00:00:00Z","2643":"2019-01-31T00:00:00Z","2644":"2019-01-31T00:00:00Z","2645":"2019-01-31T00:00:00Z","2646":"2019-01-31T00:00:00Z","2647":"2019-01-30T00:00:00Z","2648":"2019-01-30T00:00:00Z","2649":"2019-01-30T00:00:00Z","2651":"2019-01-29T00:00:00Z","2652":"2019-01-28T00:00:00Z","2654":"2019-01-28T00:00:00Z","2655":"2019-01-28T00:00:00Z","2656":"2019-01-28T00:00:00Z","2657":"2019-01-28T00:00:00Z","2658":"2019-01-28T00:00:00Z","2659":"2019-01-28T00:00:00Z","2660":"2019-01-28T00:00:00Z","2661":"2019-01-28T00:00:00Z","2662":"2019-01-28T00:00:00Z","2663":"2019-01-28T00:00:00Z","2664":"2019-01-28T00:00:00Z","2665":"2019-01-28T00:00:00Z","2666":"2019-01-28T00:00:00Z","2667":"2019-01-28T00:00:00Z","2668":"2019-01-26T00:00:00Z","2669":"2019-01-26T00:00:00Z","2670":"2019-01-25T00:00:00Z","2671":"2019-01-25T00:00:00Z","2672":"2019-01-25T00:00:00Z","2673":"2019-01-25T00:00:00Z","2674":"2019-01-25T00:00:00Z","2676":"2019-01-24T00:00:00Z","2677":"2019-01-24T00:00:00Z","2679":"2019-01-23T00:00:00Z","2680":"2019-01-22T00:00:00Z","2682":"2019-01-22T00:00:00Z","2683":"2019-01-22T00:00:00Z","2684":"2019-01-22T00:00:00Z","2685":"2019-01-22T00:00:00Z","2686":"2019-01-22T00:00:00Z","2688":"2019-01-21T00:00:00Z","2689":"2019-01-20T00:00:00Z","2690":"2019-01-20T00:00:00Z","2691":"2019-01-20T00:00:00Z","2692":"2019-01-20T00:00:00Z","2693":"2019-01-20T00:00:00Z","2694":"2019-01-19T00:00:00Z","2695":"2019-01-19T00:00:00Z","2696":"2019-01-19T00:00:00Z","2697":"2019-01-19T00:00:00Z","2698":"2019-01-19T00:00:00Z","2699":"2019-01-19T00:00:00Z","2700":"2019-01-19T00:00:00Z","2701":"2019-01-19T00:00:00Z","2702":"2019-01-19T00:00:00Z","2703":"2019-01-19T00:00:00Z","2704":"2019-01-18T00:00:00Z","2705":"2019-01-18T00:00:00Z","2706":"2019-01-17T00:00:00Z","2707":"2019-01-17T00:00:00Z","2708":"2019-01-17T00:00:00Z","2712":"2019-01-14T00:00:00Z","2713":"2019-01-13T00:00:00Z","2714":"2019-01-13T00:00:00Z","2715":"2019-01-13T00:00:00Z","2717":"2019-01-12T00:00:00Z","2718":"2019-01-12T00:00:00Z","2719":"2019-01-12T00:00:00Z","2720":"2019-01-12T00:00:00Z","2721":"2019-01-12T00:00:00Z","2722":"2019-01-11T00:00:00Z","2723":"2019-01-11T00:00:00Z","2724":"2019-01-11T00:00:00Z","2725":"2019-01-11T00:00:00Z","2726":"2019-01-11T00:00:00Z","2727":"2019-01-11T00:00:00Z","2728":"2019-01-11T00:00:00Z","2729":"2019-01-11T00:00:00Z","2731":"2019-01-10T00:00:00Z","2732":"2019-01-10T00:00:00Z","2733":"2019-01-10T00:00:00Z","2734":"2019-01-09T00:00:00Z","2735":"2019-01-08T00:00:00Z","2736":"2019-01-07T00:00:00Z","2737":"2019-01-06T00:00:00Z","2738":"2019-01-06T00:00:00Z","2739":"2019-01-06T00:00:00Z","2740":"2019-01-05T00:00:00Z","2741":"2019-01-05T00:00:00Z","2742":"2019-01-05T00:00:00Z","2743":"2019-01-05T00:00:00Z","2744":"2019-01-04T00:00:00Z","2745":"2019-01-04T00:00:00Z","2746":"2019-01-04T00:00:00Z","2747":"2019-01-04T00:00:00Z","2748":"2019-01-04T00:00:00Z","2750":"2019-01-03T00:00:00Z","2751":"2019-01-03T00:00:00Z","2752":"2019-01-03T00:00:00Z","2753":"2019-01-02T00:00:00Z","2754":"2019-01-02T00:00:00Z","2755":"2019-01-02T00:00:00Z","2757":"2018-12-30T00:00:00Z","2758":"2018-12-29T00:00:00Z","2759":"2018-12-29T00:00:00Z","2760":"2018-12-28T00:00:00Z","2761":"2018-12-28T00:00:00Z","2762":"2018-12-28T00:00:00Z","2763":"2018-12-25T00:00:00Z","2764":"2018-12-25T00:00:00Z","2775":"2018-12-24T00:00:00Z","2776":"2018-12-23T00:00:00Z","2777":"2018-12-22T00:00:00Z","2778":"2018-12-22T00:00:00Z","2779":"2018-12-22T00:00:00Z","2781":"2018-12-21T00:00:00Z","2783":"2018-12-20T00:00:00Z","2784":"2018-12-20T00:00:00Z","2785":"2018-12-20T00:00:00Z","2786":"2018-12-20T00:00:00Z","2787":"2018-12-19T00:00:00Z","2788":"2018-12-19T00:00:00Z","2789":"2018-12-18T00:00:00Z","2790":"2018-12-17T00:00:00Z","2791":"2018-12-16T00:00:00Z","2792":"2018-12-15T00:00:00Z","2793":"2018-12-14T00:00:00Z","2794":"2018-12-12T00:00:00Z","2797":"2018-12-12T00:00:00Z","2798":"2018-12-11T00:00:00Z","2799":"2018-12-10T00:00:00Z","2801":"2018-12-10T00:00:00Z","2802":"2018-12-09T00:00:00Z","2803":"2018-12-09T00:00:00Z","2804":"2018-12-08T00:00:00Z","2805":"2018-12-08T00:00:00Z","2806":"2018-12-08T00:00:00Z","2807":"2018-12-08T00:00:00Z","2808":"2018-12-08T00:00:00Z","2809":"2018-12-08T00:00:00Z","2810":"2018-12-08T00:00:00Z","2811":"2018-12-07T00:00:00Z","2812":"2018-12-07T00:00:00Z","2813":"2018-12-07T00:00:00Z","2814":"2018-12-06T00:00:00Z","2815":"2018-12-06T00:00:00Z","2816":"2018-12-05T00:00:00Z","2817":"2018-12-05T00:00:00Z","2818":"2018-12-05T00:00:00Z","2819":"2018-12-05T00:00:00Z","2820":"2018-12-04T00:00:00Z","2821":"2018-12-04T00:00:00Z","2822":"2018-12-04T00:00:00Z","2823":"2018-12-04T00:00:00Z","2825":"2018-12-03T00:00:00Z","2826":"2018-12-03T00:00:00Z","2827":"2018-12-02T00:00:00Z","2828":"2018-12-02T00:00:00Z","2829":"2018-11-30T00:00:00Z","2830":"2018-11-30T00:00:00Z","2831":"2018-11-29T00:00:00Z","2832":"2018-11-29T00:00:00Z","2833":"2018-11-29T00:00:00Z","2835":"2018-11-28T00:00:00Z","2836":"2018-11-28T00:00:00Z","2837":"2018-11-28T00:00:00Z","2838":"2018-11-28T00:00:00Z","2839":"2018-11-27T00:00:00Z","2840":"2018-11-27T00:00:00Z","2841":"2018-11-27T00:00:00Z","2842":"2018-11-27T00:00:00Z","2843":"2018-11-27T00:00:00Z","2844":"2018-11-26T00:00:00Z","2846":"2018-11-25T00:00:00Z","2847":"2018-11-25T00:00:00Z","2848":"2018-11-25T00:00:00Z","2849":"2018-11-25T00:00:00Z","2850":"2018-11-25T00:00:00Z","2851":"2018-11-25T00:00:00Z","2852":"2018-11-24T00:00:00Z","2853":"2018-11-24T00:00:00Z","2854":"2018-11-24T00:00:00Z","2855":"2018-11-24T00:00:00Z","2856":"2018-11-24T00:00:00Z","2857":"2018-11-23T00:00:00Z","2858":"2018-11-22T00:00:00Z","2861":"2018-11-21T00:00:00Z","2862":"2018-11-21T00:00:00Z","2863":"2018-11-21T00:00:00Z","2864":"2018-11-21T00:00:00Z","2865":"2018-11-21T00:00:00Z","2866":"2018-11-20T00:00:00Z","2867":"2018-11-20T00:00:00Z","2868":"2018-11-20T00:00:00Z","2869":"2018-11-20T00:00:00Z","2870":"2018-11-20T00:00:00Z","2871":"2018-11-20T00:00:00Z","2872":"2018-11-20T00:00:00Z","2873":"2018-11-20T00:00:00Z","2874":"2018-11-19T00:00:00Z","2875":"2018-11-18T00:00:00Z","2876":"2018-11-18T00:00:00Z","2877":"2018-11-18T00:00:00Z","2878":"2018-11-17T00:00:00Z","2879":"2018-11-17T00:00:00Z","2880":"2018-11-17T00:00:00Z","2881":"2018-11-17T00:00:00Z","2882":"2018-11-17T00:00:00Z","2883":"2018-11-17T00:00:00Z","2884":"2018-11-17T00:00:00Z","2885":"2018-11-16T00:00:00Z","2886":"2018-11-15T00:00:00Z","2887":"2018-11-15T00:00:00Z","2888":"2018-11-15T00:00:00Z","2889":"2018-11-14T00:00:00Z","2890":"2018-11-14T00:00:00Z","2891":"2018-11-14T00:00:00Z","2893":"2018-11-13T00:00:00Z","2894":"2018-11-13T00:00:00Z","2895":"2018-11-13T00:00:00Z","2896":"2018-11-13T00:00:00Z","2898":"2018-11-12T00:00:00Z","2899":"2018-11-12T00:00:00Z","2900":"2018-11-11T00:00:00Z","2901":"2018-11-11T00:00:00Z","2902":"2018-11-11T00:00:00Z","2903":"2018-11-10T00:00:00Z","2904":"2018-11-10T00:00:00Z","2905":"2018-11-10T00:00:00Z","2906":"2018-11-09T00:00:00Z","2907":"2018-11-09T00:00:00Z","2908":"2018-11-09T00:00:00Z","2909":"2018-11-09T00:00:00Z","2910":"2018-11-09T00:00:00Z","2912":"2018-11-09T00:00:00Z","2913":"2018-11-08T00:00:00Z","2914":"2018-11-08T00:00:00Z","2915":"2018-11-08T00:00:00Z","2916":"2018-11-08T00:00:00Z","2917":"2018-11-08T00:00:00Z","2918":"2018-11-07T00:00:00Z","2919":"2018-11-07T00:00:00Z","2920":"2018-11-07T00:00:00Z","2921":"2018-11-07T00:00:00Z","2922":"2018-11-06T00:00:00Z","2923":"2018-11-06T00:00:00Z","2924":"2018-11-05T00:00:00Z","2925":"2018-11-05T00:00:00Z","2926":"2018-11-05T00:00:00Z","2927":"2018-11-05T00:00:00Z","2928":"2018-11-05T00:00:00Z","2929":"2018-11-05T00:00:00Z","2930":"2018-11-04T00:00:00Z","2931":"2018-11-04T00:00:00Z","2932":"2018-11-04T00:00:00Z","2933":"2018-11-04T00:00:00Z","2934":"2018-11-04T00:00:00Z","2935":"2018-11-04T00:00:00Z","2936":"2018-11-04T00:00:00Z","2937":"2018-11-04T00:00:00Z","2938":"2018-11-04T00:00:00Z","2939":"2018-11-03T00:00:00Z","2940":"2018-11-03T00:00:00Z","2941":"2018-11-03T00:00:00Z","2942":"2018-11-03T00:00:00Z","2943":"2018-11-02T00:00:00Z","2944":"2018-11-02T00:00:00Z","2945":"2018-11-02T00:00:00Z","2946":"2018-11-02T00:00:00Z","2947":"2018-11-02T00:00:00Z","2948":"2018-11-01T00:00:00Z","2949":"2018-10-31T00:00:00Z","2950":"2018-10-31T00:00:00Z","2951":"2018-10-31T00:00:00Z","2955":"2018-10-29T00:00:00Z","2956":"2018-10-29T00:00:00Z","2957":"2018-10-29T00:00:00Z","2958":"2018-10-29T00:00:00Z","2959":"2018-10-28T00:00:00Z","2960":"2018-10-28T00:00:00Z","2961":"2018-10-28T00:00:00Z","2962":"2018-10-27T00:00:00Z","2963":"2018-10-27T00:00:00Z","2964":"2018-10-27T00:00:00Z","2965":"2018-10-27T00:00:00Z","2966":"2018-10-27T00:00:00Z","2967":"2018-10-27T00:00:00Z","2968":"2018-10-27T00:00:00Z","2969":"2018-10-27T00:00:00Z","2970":"2018-10-27T00:00:00Z","2972":"2018-10-25T00:00:00Z","2973":"2018-10-25T00:00:00Z","2974":"2018-10-25T00:00:00Z","2975":"2018-10-23T00:00:00Z","2976":"2018-10-23T00:00:00Z","2977":"2018-10-23T00:00:00Z","2978":"2018-10-23T00:00:00Z","2980":"2018-10-22T00:00:00Z","2981":"2018-10-21T00:00:00Z","2983":"2018-10-21T00:00:00Z","2984":"2018-10-21T00:00:00Z","2985":"2018-10-21T00:00:00Z","2986":"2018-10-19T00:00:00Z","2988":"2018-10-17T00:00:00Z","2989":"2018-10-14T00:00:00Z","2990":"2018-10-14T00:00:00Z","2991":"2018-10-14T00:00:00Z","2992":"2018-10-14T00:00:00Z","2993":"2018-10-14T00:00:00Z","2994":"2018-10-14T00:00:00Z","2995":"2018-10-14T00:00:00Z","2996":"2018-10-13T00:00:00Z","2997":"2018-10-13T00:00:00Z","2998":"2018-10-13T00:00:00Z","2999":"2018-10-13T00:00:00Z","3000":"2018-10-13T00:00:00Z","3001":"2018-10-13T00:00:00Z","3002":"2018-10-13T00:00:00Z","3003":"2018-10-13T00:00:00Z","3004":"2018-10-13T00:00:00Z","3006":"2018-10-12T00:00:00Z","3008":"2018-10-11T00:00:00Z","3009":"2018-10-11T00:00:00Z","3010":"2018-10-11T00:00:00Z","3011":"2018-10-11T00:00:00Z","3012":"2018-10-11T00:00:00Z","3015":"2018-10-10T00:00:00Z","3016":"2018-10-10T00:00:00Z","3017":"2018-10-10T00:00:00Z","3018":"2018-10-10T00:00:00Z","3020":"2018-10-09T00:00:00Z","3021":"2018-10-09T00:00:00Z","3022":"2018-10-09T00:00:00Z","3023":"2018-10-08T00:00:00Z","3024":"2018-10-08T00:00:00Z","3025":"2018-10-08T00:00:00Z","3026":"2018-10-08T00:00:00Z","3027":"2018-10-06T00:00:00Z","3028":"2018-10-05T00:00:00Z","3029":"2018-10-05T00:00:00Z","3030":"2018-10-05T00:00:00Z","3031":"2018-10-05T00:00:00Z","3032":"2018-10-05T00:00:00Z","3033":"2018-10-04T00:00:00Z","3034":"2018-10-04T00:00:00Z","3035":"2018-10-03T00:00:00Z","3036":"2018-10-02T00:00:00Z","3037":"2018-10-02T00:00:00Z","3038":"2018-10-02T00:00:00Z","3039":"2018-10-02T00:00:00Z","3040":"2018-10-02T00:00:00Z","3043":"2018-10-01T00:00:00Z","3044":"2018-09-30T00:00:00Z","3045":"2018-09-30T00:00:00Z","3046":"2018-09-29T00:00:00Z","3047":"2018-09-29T00:00:00Z","3048":"2018-09-29T00:00:00Z","3049":"2018-09-29T00:00:00Z","3050":"2018-09-29T00:00:00Z","3051":"2018-09-29T00:00:00Z","3052":"2018-09-29T00:00:00Z","3054":"2018-09-28T00:00:00Z","3055":"2018-09-28T00:00:00Z","3056":"2018-09-28T00:00:00Z","3060":"2018-09-27T00:00:00Z","3062":"2018-09-27T00:00:00Z","3063":"2018-09-27T00:00:00Z","3064":"2018-09-27T00:00:00Z","3065":"2018-09-27T00:00:00Z","3066":"2018-09-27T00:00:00Z","3067":"2018-09-27T00:00:00Z","3068":"2018-09-27T00:00:00Z","3069":"2018-09-27T00:00:00Z","3070":"2018-09-27T00:00:00Z","3072":"2018-09-25T00:00:00Z","3073":"2018-09-25T00:00:00Z","3074":"2018-09-24T00:00:00Z","3075":"2018-09-24T00:00:00Z","3076":"2018-09-23T00:00:00Z","3077":"2018-09-23T00:00:00Z","3078":"2018-09-23T00:00:00Z","3081":"2018-09-21T00:00:00Z","3083":"2018-09-20T00:00:00Z","3084":"2018-09-19T00:00:00Z","3085":"2018-09-19T00:00:00Z","3086":"2018-09-19T00:00:00Z","3087":"2018-09-19T00:00:00Z","3088":"2018-09-18T00:00:00Z","3089":"2018-09-18T00:00:00Z","3090":"2018-09-17T00:00:00Z","3091":"2018-09-17T00:00:00Z","3092":"2018-09-17T00:00:00Z","3093":"2018-09-17T00:00:00Z","3094":"2018-09-17T00:00:00Z","3095":"2018-09-16T00:00:00Z","3096":"2018-09-16T00:00:00Z","3097":"2018-09-15T00:00:00Z","3098":"2018-09-15T00:00:00Z","3099":"2018-09-14T00:00:00Z","3100":"2018-09-14T00:00:00Z","3101":"2018-09-14T00:00:00Z","3103":"2018-09-14T00:00:00Z","3107":"2018-09-13T00:00:00Z","3108":"2018-09-12T00:00:00Z","3111":"2018-09-11T00:00:00Z","3114":"2018-09-10T00:00:00Z","3115":"2018-09-10T00:00:00Z","3116":"2018-09-10T00:00:00Z","3119":"2018-09-10T00:00:00Z","3120":"2018-09-10T00:00:00Z","3121":"2018-09-10T00:00:00Z","3122":"2018-09-09T00:00:00Z","3123":"2018-09-09T00:00:00Z","3124":"2018-09-09T00:00:00Z","3125":"2018-09-09T00:00:00Z","3126":"2018-09-09T00:00:00Z","3127":"2018-09-08T00:00:00Z","3128":"2018-09-08T00:00:00Z","3129":"2018-09-08T00:00:00Z","3130":"2018-09-05T00:00:00Z","3132":"2018-09-05T00:00:00Z","3133":"2018-09-05T00:00:00Z","3134":"2018-09-05T00:00:00Z","3135":"2018-09-05T00:00:00Z","3136":"2018-09-05T00:00:00Z","3138":"2018-09-04T00:00:00Z","3139":"2018-09-04T00:00:00Z","3140":"2018-09-04T00:00:00Z","3141":"2018-09-03T00:00:00Z","3142":"2018-09-03T00:00:00Z","3143":"2018-09-03T00:00:00Z","3144":"2018-09-03T00:00:00Z","3145":"2018-09-03T00:00:00Z","3146":"2018-09-03T00:00:00Z","3147":"2018-09-02T00:00:00Z","3148":"2018-09-02T00:00:00Z","3149":"2018-09-02T00:00:00Z","3150":"2018-09-02T00:00:00Z","3151":"2018-09-02T00:00:00Z","3152":"2018-09-02T00:00:00Z","3153":"2018-09-02T00:00:00Z","3154":"2018-09-02T00:00:00Z","3155":"2018-09-02T00:00:00Z","3157":"2018-09-02T00:00:00Z","3158":"2018-09-01T00:00:00Z","3159":"2018-09-01T00:00:00Z","3160":"2018-09-01T00:00:00Z","3161":"2018-09-01T00:00:00Z","3162":"2018-08-31T00:00:00Z","3163":"2018-08-31T00:00:00Z","3164":"2018-08-31T00:00:00Z","3165":"2018-08-31T00:00:00Z","3167":"2018-08-30T00:00:00Z","3168":"2018-08-30T00:00:00Z","3169":"2018-08-30T00:00:00Z","3170":"2018-08-30T00:00:00Z","3171":"2018-08-30T00:00:00Z","3172":"2018-08-30T00:00:00Z","3174":"2018-08-29T00:00:00Z","3175":"2018-08-29T00:00:00Z","3177":"2018-08-28T00:00:00Z","3179":"2018-08-28T00:00:00Z","3180":"2018-08-28T00:00:00Z","3181":"2018-08-28T00:00:00Z","3182":"2018-08-28T00:00:00Z","3183":"2018-08-28T00:00:00Z","3184":"2018-08-28T00:00:00Z","3185":"2018-08-28T00:00:00Z","3186":"2018-08-27T00:00:00Z","3187":"2018-08-26T00:00:00Z","3188":"2018-08-26T00:00:00Z","3189":"2018-08-25T00:00:00Z","3190":"2018-08-25T00:00:00Z","3191":"2018-08-25T00:00:00Z","3192":"2018-08-25T00:00:00Z","3193":"2018-08-25T00:00:00Z","3194":"2018-08-24T00:00:00Z","3195":"2018-08-24T00:00:00Z","3197":"2018-08-24T00:00:00Z","3198":"2018-08-23T00:00:00Z","3199":"2018-08-22T00:00:00Z","3200":"2018-08-22T00:00:00Z","3201":"2018-08-22T00:00:00Z","3202":"2018-08-21T00:00:00Z","3203":"2018-08-20T00:00:00Z","3204":"2018-08-20T00:00:00Z","3205":"2018-08-20T00:00:00Z","3208":"2018-08-19T00:00:00Z","3209":"2018-08-19T00:00:00Z","3210":"2018-08-19T00:00:00Z","3211":"2018-08-19T00:00:00Z","3212":"2018-08-19T00:00:00Z","3213":"2018-08-18T00:00:00Z","3214":"2018-08-18T00:00:00Z","3215":"2018-08-18T00:00:00Z","3216":"2018-08-18T00:00:00Z","3217":"2018-08-17T00:00:00Z","3218":"2018-08-17T00:00:00Z","3219":"2018-08-17T00:00:00Z","3220":"2018-08-17T00:00:00Z","3221":"2018-08-17T00:00:00Z","3222":"2018-08-16T00:00:00Z","3223":"2018-08-14T00:00:00Z","3224":"2018-08-14T00:00:00Z","3225":"2018-08-14T00:00:00Z","3226":"2018-08-13T00:00:00Z","3227":"2018-08-13T00:00:00Z","3228":"2018-08-13T00:00:00Z","3229":"2018-08-13T00:00:00Z","3231":"2018-08-13T00:00:00Z","3232":"2018-08-12T00:00:00Z","3233":"2018-08-12T00:00:00Z","3236":"2018-08-09T00:00:00Z","3237":"2018-08-09T00:00:00Z","3238":"2018-08-09T00:00:00Z","3239":"2018-08-09T00:00:00Z","3240":"2018-08-09T00:00:00Z","3241":"2018-08-09T00:00:00Z","3242":"2018-08-09T00:00:00Z","3243":"2018-08-09T00:00:00Z","3244":"2018-08-09T00:00:00Z","3245":"2018-08-08T00:00:00Z","3246":"2018-08-08T00:00:00Z","3247":"2018-08-08T00:00:00Z","3248":"2018-08-07T00:00:00Z","3249":"2018-08-07T00:00:00Z","3250":"2018-08-07T00:00:00Z","3251":"2018-08-06T00:00:00Z","3252":"2018-08-05T00:00:00Z","3253":"2018-08-05T00:00:00Z","3254":"2018-08-04T00:00:00Z","3255":"2018-08-04T00:00:00Z","3256":"2018-08-04T00:00:00Z","3257":"2018-08-04T00:00:00Z","3258":"2018-08-04T00:00:00Z","3259":"2018-08-04T00:00:00Z","3260":"2018-08-03T00:00:00Z","3261":"2018-08-03T00:00:00Z","3262":"2018-08-03T00:00:00Z","3263":"2018-08-03T00:00:00Z","3264":"2018-08-03T00:00:00Z","3265":"2018-08-02T00:00:00Z","3266":"2018-08-01T00:00:00Z","3267":"2018-08-01T00:00:00Z","3268":"2018-08-01T00:00:00Z","3271":"2018-07-31T00:00:00Z","3272":"2018-07-31T00:00:00Z","3273":"2018-07-31T00:00:00Z","3274":"2018-07-31T00:00:00Z","3275":"2018-07-31T00:00:00Z","3276":"2018-07-30T00:00:00Z","3278":"2018-07-28T00:00:00Z","3279":"2018-07-28T00:00:00Z","3280":"2018-07-26T00:00:00Z","3281":"2018-07-26T00:00:00Z","3282":"2018-07-26T00:00:00Z","3283":"2018-07-26T00:00:00Z","3284":"2018-07-25T00:00:00Z","3285":"2018-07-25T00:00:00Z","3286":"2018-07-25T00:00:00Z","3287":"2018-07-25T00:00:00Z","3289":"2018-07-24T00:00:00Z","3290":"2018-07-24T00:00:00Z","3291":"2018-07-23T00:00:00Z","3292":"2018-07-22T00:00:00Z","3293":"2018-07-22T00:00:00Z","3294":"2018-07-22T00:00:00Z","3295":"2018-07-21T00:00:00Z","3296":"2018-07-21T00:00:00Z","3297":"2018-07-21T00:00:00Z","3298":"2018-07-21T00:00:00Z","3299":"2018-07-21T00:00:00Z","3300":"2018-07-20T00:00:00Z","3301":"2018-07-20T00:00:00Z","3302":"2018-07-20T00:00:00Z","3303":"2018-07-19T00:00:00Z","3304":"2018-07-19T00:00:00Z","3305":"2018-07-19T00:00:00Z","3306":"2018-07-19T00:00:00Z","3307":"2018-07-19T00:00:00Z","3308":"2018-07-18T00:00:00Z","3310":"2018-07-18T00:00:00Z","3311":"2018-07-16T00:00:00Z","3312":"2018-07-16T00:00:00Z","3313":"2018-07-16T00:00:00Z","3314":"2018-07-16T00:00:00Z","3315":"2018-07-16T00:00:00Z","3316":"2018-07-16T00:00:00Z","3317":"2018-07-15T00:00:00Z","3318":"2018-07-15T00:00:00Z","3319":"2018-07-15T00:00:00Z","3320":"2018-07-15T00:00:00Z","3321":"2018-07-15T00:00:00Z","3322":"2018-07-14T00:00:00Z","3323":"2018-07-14T00:00:00Z","3324":"2018-07-14T00:00:00Z","3325":"2018-07-14T00:00:00Z","3326":"2018-07-14T00:00:00Z","3327":"2018-07-14T00:00:00Z","3328":"2018-07-14T00:00:00Z","3329":"2018-07-14T00:00:00Z","3330":"2018-07-14T00:00:00Z","3331":"2018-07-13T00:00:00Z","3332":"2018-07-13T00:00:00Z","3334":"2018-07-13T00:00:00Z","3335":"2018-07-13T00:00:00Z","3336":"2018-07-13T00:00:00Z","3338":"2018-07-12T00:00:00Z","3339":"2018-07-12T00:00:00Z","3340":"2018-07-12T00:00:00Z","3341":"2018-07-12T00:00:00Z","3342":"2018-07-12T00:00:00Z","3343":"2018-07-11T00:00:00Z","3344":"2018-07-10T00:00:00Z","3345":"2018-07-10T00:00:00Z","3347":"2018-07-09T00:00:00Z","3348":"2018-07-09T00:00:00Z","3349":"2018-07-08T00:00:00Z","3350":"2018-07-08T00:00:00Z","3351":"2018-07-08T00:00:00Z","3352":"2018-07-07T00:00:00Z","3353":"2018-07-07T00:00:00Z","3354":"2018-07-06T00:00:00Z","3355":"2018-07-06T00:00:00Z","3356":"2018-07-06T00:00:00Z","3357":"2018-07-05T00:00:00Z","3358":"2018-07-05T00:00:00Z","3361":"2018-07-03T00:00:00Z","3362":"2018-07-03T00:00:00Z","3364":"2018-07-02T00:00:00Z","3365":"2018-07-02T00:00:00Z","3367":"2018-07-01T00:00:00Z","3368":"2018-07-01T00:00:00Z","3369":"2018-07-01T00:00:00Z","3370":"2018-07-01T00:00:00Z","3371":"2018-07-01T00:00:00Z","3372":"2018-06-30T00:00:00Z","3373":"2018-06-30T00:00:00Z","3374":"2018-06-30T00:00:00Z","3375":"2018-06-30T00:00:00Z","3376":"2018-06-30T00:00:00Z","3377":"2018-06-30T00:00:00Z","3378":"2018-06-29T00:00:00Z","3379":"2018-06-29T00:00:00Z","3380":"2018-06-29T00:00:00Z","3381":"2018-06-27T00:00:00Z","3382":"2018-06-27T00:00:00Z","3383":"2018-06-27T00:00:00Z","3384":"2018-06-27T00:00:00Z","3387":"2018-06-27T00:00:00Z","3388":"2018-06-26T00:00:00Z","3389":"2018-06-25T00:00:00Z","3390":"2018-06-25T00:00:00Z","3391":"2018-06-25T00:00:00Z","3393":"2018-06-24T00:00:00Z","3394":"2018-06-23T00:00:00Z","3395":"2018-06-23T00:00:00Z","3396":"2018-06-23T00:00:00Z","3397":"2018-06-23T00:00:00Z","3398":"2018-06-22T00:00:00Z","3399":"2018-06-21T00:00:00Z","3400":"2018-06-21T00:00:00Z","3401":"2018-06-21T00:00:00Z","3402":"2018-06-21T00:00:00Z","3403":"2018-06-21T00:00:00Z","3404":"2018-06-20T00:00:00Z","3405":"2018-06-20T00:00:00Z","3406":"2018-06-20T00:00:00Z","3407":"2018-06-20T00:00:00Z","3409":"2018-06-19T00:00:00Z","3410":"2018-06-19T00:00:00Z","3411":"2018-06-19T00:00:00Z","3412":"2018-06-19T00:00:00Z","3413":"2018-06-18T00:00:00Z","3414":"2018-06-18T00:00:00Z","3415":"2018-06-18T00:00:00Z","3416":"2018-06-18T00:00:00Z","3417":"2018-06-18T00:00:00Z","3419":"2018-06-17T00:00:00Z","3420":"2018-06-17T00:00:00Z","3421":"2018-06-17T00:00:00Z","3422":"2018-06-16T00:00:00Z","3423":"2018-06-16T00:00:00Z","3424":"2018-06-16T00:00:00Z","3425":"2018-06-15T00:00:00Z","3426":"2018-06-15T00:00:00Z","3427":"2018-06-14T00:00:00Z","3428":"2018-06-12T00:00:00Z","3431":"2018-06-11T00:00:00Z","3432":"2018-06-11T00:00:00Z","3434":"2018-06-11T00:00:00Z","3439":"2018-06-10T00:00:00Z","3440":"2018-06-10T00:00:00Z","3441":"2018-06-10T00:00:00Z","3442":"2018-06-09T00:00:00Z","3443":"2018-06-09T00:00:00Z","3444":"2018-06-08T00:00:00Z","3445":"2018-06-08T00:00:00Z","3446":"2018-06-08T00:00:00Z","3447":"2018-06-07T00:00:00Z","3448":"2018-06-07T00:00:00Z","3449":"2018-06-06T00:00:00Z","3450":"2018-06-06T00:00:00Z","3452":"2018-06-06T00:00:00Z","3453":"2018-06-06T00:00:00Z","3454":"2018-06-05T00:00:00Z","3455":"2018-06-05T00:00:00Z","3459":"2018-06-04T00:00:00Z","3462":"2018-06-03T00:00:00Z","3463":"2018-06-03T00:00:00Z","3464":"2018-06-02T00:00:00Z","3465":"2018-06-02T00:00:00Z","3466":"2018-06-02T00:00:00Z","3467":"2018-06-01T00:00:00Z","3468":"2018-06-01T00:00:00Z","3469":"2018-06-01T00:00:00Z","3470":"2018-06-01T00:00:00Z","3471":"2018-05-31T00:00:00Z","3472":"2018-05-31T00:00:00Z","3473":"2018-05-31T00:00:00Z","3474":"2018-05-29T00:00:00Z","3475":"2018-05-29T00:00:00Z","3476":"2018-05-29T00:00:00Z","3477":"2018-05-28T00:00:00Z","3478":"2018-05-27T00:00:00Z","3479":"2018-05-26T00:00:00Z","3480":"2018-05-26T00:00:00Z","3481":"2018-05-26T00:00:00Z","3482":"2018-05-25T00:00:00Z","3483":"2018-05-25T00:00:00Z","3484":"2018-05-25T00:00:00Z","3486":"2018-05-25T00:00:00Z","3487":"2018-05-23T00:00:00Z","3488":"2018-05-22T00:00:00Z","3489":"2018-05-22T00:00:00Z","3490":"2018-05-21T00:00:00Z","3491":"2018-05-21T00:00:00Z","3493":"2018-05-21T00:00:00Z","3494":"2018-05-20T00:00:00Z","3495":"2018-05-20T00:00:00Z","3496":"2018-05-20T00:00:00Z","3497":"2018-05-19T00:00:00Z","3498":"2018-05-19T00:00:00Z","3499":"2018-05-19T00:00:00Z","3500":"2018-05-19T00:00:00Z","3501":"2018-05-19T00:00:00Z","3502":"2018-05-19T00:00:00Z","3503":"2018-05-19T00:00:00Z","3504":"2018-05-18T00:00:00Z","3505":"2018-05-17T00:00:00Z","3506":"2018-05-17T00:00:00Z","3507":"2018-05-16T00:00:00Z","3508":"2018-05-16T00:00:00Z","3509":"2018-05-16T00:00:00Z","3510":"2018-05-15T00:00:00Z","3511":"2018-05-15T00:00:00Z","3512":"2018-05-15T00:00:00Z","3513":"2018-05-15T00:00:00Z","3515":"2018-05-14T00:00:00Z","3516":"2018-05-13T00:00:00Z","3517":"2018-05-13T00:00:00Z","3518":"2018-05-13T00:00:00Z","3519":"2018-05-12T00:00:00Z","3520":"2018-05-12T00:00:00Z","3521":"2018-05-12T00:00:00Z","3522":"2018-05-11T00:00:00Z","3523":"2018-05-11T00:00:00Z","3525":"2018-05-11T00:00:00Z","3526":"2018-05-11T00:00:00Z","3528":"2018-05-09T00:00:00Z","3529":"2018-05-09T00:00:00Z","3530":"2018-05-09T00:00:00Z","3531":"2018-05-09T00:00:00Z","3532":"2018-05-08T00:00:00Z","3533":"2018-05-08T00:00:00Z","3534":"2018-05-08T00:00:00Z","3535":"2018-05-08T00:00:00Z","3536":"2018-05-06T00:00:00Z","3537":"2018-05-06T00:00:00Z","3538":"2018-05-05T00:00:00Z","3539":"2018-05-05T00:00:00Z","3540":"2018-05-05T00:00:00Z","3541":"2018-05-05T00:00:00Z","3542":"2018-05-04T00:00:00Z","3543":"2018-05-04T00:00:00Z","3544":"2018-05-04T00:00:00Z","3546":"2018-05-03T00:00:00Z","3547":"2018-05-02T00:00:00Z","3548":"2018-05-02T00:00:00Z","3550":"2018-04-30T00:00:00Z","3552":"2018-04-29T00:00:00Z","3553":"2018-04-28T00:00:00Z","3554":"2018-04-28T00:00:00Z","3555":"2018-04-28T00:00:00Z","3556":"2018-04-28T00:00:00Z","3557":"2018-04-28T00:00:00Z","3558":"2018-04-27T00:00:00Z","3559":"2018-04-27T00:00:00Z","3560":"2018-04-27T00:00:00Z","3561":"2018-04-27T00:00:00Z","3562":"2018-04-26T00:00:00Z","3563":"2018-04-26T00:00:00Z","3564":"2018-04-26T00:00:00Z","3565":"2018-04-26T00:00:00Z","3566":"2018-04-26T00:00:00Z","3567":"2018-04-25T00:00:00Z","3568":"2018-04-25T00:00:00Z","3570":"2018-04-25T00:00:00Z","3571":"2018-04-24T00:00:00Z","3572":"2018-04-24T00:00:00Z","3573":"2018-04-24T00:00:00Z","3574":"2018-04-24T00:00:00Z","3575":"2018-04-24T00:00:00Z","3576":"2018-04-23T00:00:00Z","3577":"2018-04-23T00:00:00Z","3578":"2018-04-23T00:00:00Z","3579":"2018-04-23T00:00:00Z","3580":"2018-04-21T00:00:00Z","3581":"2018-04-21T00:00:00Z","3582":"2018-04-20T00:00:00Z","3583":"2018-04-20T00:00:00Z","3584":"2018-04-19T00:00:00Z","3585":"2018-04-19T00:00:00Z","3586":"2018-04-19T00:00:00Z","3587":"2018-04-19T00:00:00Z","3588":"2018-04-18T00:00:00Z","3589":"2018-04-18T00:00:00Z","3590":"2018-04-18T00:00:00Z","3591":"2018-04-18T00:00:00Z","3592":"2018-04-18T00:00:00Z","3593":"2018-04-18T00:00:00Z","3597":"2018-04-16T00:00:00Z","3601":"2018-04-15T00:00:00Z","3602":"2018-04-15T00:00:00Z","3603":"2018-04-15T00:00:00Z","3604":"2018-04-15T00:00:00Z","3605":"2018-04-15T00:00:00Z","3606":"2018-04-15T00:00:00Z","3607":"2018-04-14T00:00:00Z","3608":"2018-04-14T00:00:00Z","3609":"2018-04-14T00:00:00Z","3610":"2018-04-14T00:00:00Z","3611":"2018-04-14T00:00:00Z","3612":"2018-04-14T00:00:00Z","3613":"2018-04-13T00:00:00Z","3614":"2018-04-13T00:00:00Z","3615":"2018-04-13T00:00:00Z","3616":"2018-04-12T00:00:00Z","3617":"2018-04-12T00:00:00Z","3618":"2018-04-12T00:00:00Z","3621":"2018-04-11T00:00:00Z","3622":"2018-04-10T00:00:00Z","3623":"2018-04-10T00:00:00Z","3624":"2018-04-09T00:00:00Z","3626":"2018-04-09T00:00:00Z","3627":"2018-04-08T00:00:00Z","3628":"2018-04-08T00:00:00Z","3629":"2018-04-08T00:00:00Z","3630":"2018-04-08T00:00:00Z","3631":"2018-04-08T00:00:00Z","3632":"2018-04-07T00:00:00Z","3633":"2018-04-07T00:00:00Z","3634":"2018-04-06T00:00:00Z","3635":"2018-04-05T00:00:00Z","3636":"2018-03-29T00:00:00Z","3637":"2018-03-28T00:00:00Z","3638":"2018-03-27T00:00:00Z","3639":"2018-03-27T00:00:00Z","3640":"2018-03-27T00:00:00Z","3641":"2018-03-26T00:00:00Z","3642":"2018-03-26T00:00:00Z","3643":"2018-03-26T00:00:00Z","3644":"2018-03-26T00:00:00Z","3646":"2018-03-25T00:00:00Z","3647":"2018-03-25T00:00:00Z","3648":"2018-03-25T00:00:00Z","3649":"2018-03-25T00:00:00Z","3650":"2018-03-25T00:00:00Z","3651":"2018-03-24T00:00:00Z","3652":"2018-03-24T00:00:00Z","3653":"2018-03-24T00:00:00Z","3654":"2018-03-24T00:00:00Z","3655":"2018-03-24T00:00:00Z","3656":"2018-03-24T00:00:00Z","3657":"2018-03-23T00:00:00Z","3658":"2018-03-23T00:00:00Z","3659":"2018-03-23T00:00:00Z","3660":"2018-03-22T00:00:00Z","3661":"2018-03-22T00:00:00Z","3662":"2018-03-22T00:00:00Z","3663":"2018-03-21T00:00:00Z","3664":"2018-03-21T00:00:00Z","3665":"2018-03-21T00:00:00Z","3666":"2018-03-21T00:00:00Z","3668":"2018-03-20T00:00:00Z","3669":"2018-03-19T00:00:00Z","3670":"2018-03-19T00:00:00Z","3671":"2018-03-19T00:00:00Z","3672":"2018-03-19T00:00:00Z","3673":"2018-03-17T00:00:00Z","3674":"2018-03-17T00:00:00Z","3675":"2018-03-16T00:00:00Z","3676":"2018-03-16T00:00:00Z","3677":"2018-03-16T00:00:00Z","3678":"2018-03-15T00:00:00Z","3679":"2018-03-15T00:00:00Z","3680":"2018-03-12T00:00:00Z","3681":"2018-03-12T00:00:00Z","3682":"2018-03-12T00:00:00Z","3683":"2018-03-12T00:00:00Z","3689":"2018-03-11T00:00:00Z","3690":"2018-03-10T00:00:00Z","3691":"2018-03-10T00:00:00Z","3692":"2018-03-10T00:00:00Z","3693":"2018-03-10T00:00:00Z","3694":"2018-03-09T00:00:00Z","3695":"2018-03-09T00:00:00Z","3696":"2018-03-09T00:00:00Z","3698":"2018-03-09T00:00:00Z","3699":"2018-03-08T00:00:00Z","3700":"2018-03-07T00:00:00Z","3701":"2018-03-07T00:00:00Z","3702":"2018-03-06T00:00:00Z","3703":"2018-03-06T00:00:00Z","3704":"2018-03-06T00:00:00Z","3705":"2018-03-06T00:00:00Z","3706":"2018-03-05T00:00:00Z","3707":"2018-03-05T00:00:00Z","3708":"2018-03-05T00:00:00Z","3709":"2018-03-05T00:00:00Z","3710":"2018-03-05T00:00:00Z","3712":"2018-03-05T00:00:00Z","3713":"2018-03-04T00:00:00Z","3714":"2018-03-04T00:00:00Z","3715":"2018-03-04T00:00:00Z","3716":"2018-03-03T00:00:00Z","3717":"2018-03-02T00:00:00Z","3718":"2018-03-02T00:00:00Z","3719":"2018-03-02T00:00:00Z","3720":"2018-03-02T00:00:00Z","3721":"2018-03-02T00:00:00Z","3722":"2018-03-01T00:00:00Z","3723":"2018-03-01T00:00:00Z","3724":"2018-02-28T00:00:00Z","3725":"2018-02-28T00:00:00Z","3726":"2018-02-27T00:00:00Z","3727":"2018-02-27T00:00:00Z","3728":"2018-02-27T00:00:00Z","3729":"2018-02-26T00:00:00Z","3730":"2018-02-26T00:00:00Z","3731":"2018-02-26T00:00:00Z","3734":"2018-02-25T00:00:00Z","3735":"2018-02-25T00:00:00Z","3736":"2018-02-25T00:00:00Z","3737":"2018-02-25T00:00:00Z","3738":"2018-02-25T00:00:00Z","3739":"2018-02-24T00:00:00Z","3740":"2018-02-24T00:00:00Z","3741":"2018-02-24T00:00:00Z","3742":"2018-02-23T00:00:00Z","3743":"2018-02-23T00:00:00Z","3744":"2018-02-23T00:00:00Z","3745":"2018-02-22T00:00:00Z","3746":"2018-02-22T00:00:00Z","3747":"2018-02-21T00:00:00Z","3748":"2018-02-20T00:00:00Z","3749":"2018-02-19T00:00:00Z","3750":"2018-02-18T00:00:00Z","3751":"2018-02-18T00:00:00Z","3752":"2018-02-18T00:00:00Z","3753":"2018-02-18T00:00:00Z","3754":"2018-02-18T00:00:00Z","3755":"2018-02-17T00:00:00Z","3756":"2018-02-17T00:00:00Z","3757":"2018-02-17T00:00:00Z","3758":"2018-02-16T00:00:00Z","3759":"2018-02-16T00:00:00Z","3760":"2018-02-16T00:00:00Z","3761":"2018-02-16T00:00:00Z","3762":"2018-02-16T00:00:00Z","3763":"2018-02-16T00:00:00Z","3764":"2018-02-14T00:00:00Z","3766":"2018-02-13T00:00:00Z","3767":"2018-02-13T00:00:00Z","3768":"2018-02-13T00:00:00Z","3769":"2018-02-13T00:00:00Z","3770":"2018-02-13T00:00:00Z","3772":"2018-02-12T00:00:00Z","3774":"2018-02-11T00:00:00Z","3775":"2018-02-11T00:00:00Z","3776":"2018-02-10T00:00:00Z","3777":"2018-02-10T00:00:00Z","3778":"2018-02-10T00:00:00Z","3779":"2018-02-10T00:00:00Z","3780":"2018-02-10T00:00:00Z","3781":"2018-02-09T00:00:00Z","3783":"2018-02-09T00:00:00Z","3784":"2018-02-08T00:00:00Z","3785":"2018-02-08T00:00:00Z","3786":"2018-02-07T00:00:00Z","3787":"2018-02-07T00:00:00Z","3788":"2018-02-06T00:00:00Z","3789":"2018-02-05T00:00:00Z","3793":"2018-02-05T00:00:00Z","3795":"2018-02-05T00:00:00Z","3796":"2018-02-05T00:00:00Z","3797":"2018-02-04T00:00:00Z","3798":"2018-02-04T00:00:00Z","3800":"2018-02-03T00:00:00Z","3801":"2018-02-03T00:00:00Z","3802":"2018-02-03T00:00:00Z","3803":"2018-02-02T00:00:00Z","3804":"2018-02-02T00:00:00Z","3805":"2018-02-01T00:00:00Z","3806":"2018-01-31T00:00:00Z","3807":"2018-01-31T00:00:00Z","3808":"2018-01-30T00:00:00Z","3809":"2018-01-30T00:00:00Z","3810":"2018-01-30T00:00:00Z","3811":"2018-01-28T00:00:00Z","3812":"2018-01-28T00:00:00Z","3813":"2018-01-28T00:00:00Z","3814":"2018-01-27T00:00:00Z","3815":"2018-01-27T00:00:00Z","3816":"2018-01-27T00:00:00Z","3817":"2018-01-27T00:00:00Z","3818":"2018-01-26T00:00:00Z","3819":"2018-01-26T00:00:00Z","3820":"2018-01-26T00:00:00Z","3821":"2018-01-25T00:00:00Z","3822":"2018-01-25T00:00:00Z","3824":"2018-01-25T00:00:00Z","3825":"2018-01-24T00:00:00Z","3826":"2018-01-24T00:00:00Z","3827":"2018-01-24T00:00:00Z","3828":"2018-01-23T00:00:00Z","3829":"2018-01-22T00:00:00Z","3830":"2018-01-22T00:00:00Z","3833":"2018-01-21T00:00:00Z","3834":"2018-01-21T00:00:00Z","3835":"2018-01-21T00:00:00Z","3836":"2018-01-20T00:00:00Z","3837":"2018-01-20T00:00:00Z","3838":"2018-01-19T00:00:00Z","3839":"2018-01-19T00:00:00Z","3840":"2018-01-19T00:00:00Z","3841":"2018-01-19T00:00:00Z","3842":"2018-01-19T00:00:00Z","3843":"2018-01-19T00:00:00Z","3844":"2018-01-19T00:00:00Z","3845":"2018-01-19T00:00:00Z","3846":"2018-01-19T00:00:00Z","3847":"2018-01-19T00:00:00Z","3848":"2018-01-17T00:00:00Z","3849":"2018-01-16T00:00:00Z","3850":"2018-01-15T00:00:00Z","3851":"2018-01-15T00:00:00Z","3852":"2018-01-14T00:00:00Z","3853":"2018-01-14T00:00:00Z","3854":"2018-01-13T00:00:00Z","3855":"2018-01-12T00:00:00Z","3856":"2018-01-12T00:00:00Z","3859":"2018-01-12T00:00:00Z","3860":"2018-01-11T00:00:00Z","3861":"2018-01-10T00:00:00Z","3865":"2018-01-09T00:00:00Z","3866":"2018-01-09T00:00:00Z","3867":"2018-01-09T00:00:00Z","3868":"2018-01-07T00:00:00Z","3869":"2018-01-07T00:00:00Z","3870":"2018-01-07T00:00:00Z","3871":"2018-01-06T00:00:00Z","3872":"2018-01-06T00:00:00Z","3873":"2018-01-05T00:00:00Z","3874":"2018-01-05T00:00:00Z","3875":"2018-01-04T00:00:00Z","3878":"2018-01-03T00:00:00Z","3879":"2018-01-03T00:00:00Z","3880":"2018-01-03T00:00:00Z","3883":"2018-01-02T00:00:00Z","3884":"2018-01-02T00:00:00Z","3885":"2018-01-02T00:00:00Z","3886":"2018-01-02T00:00:00Z","3887":"2018-01-02T00:00:00Z","3888":"2018-01-02T00:00:00Z","3889":"2018-01-01T00:00:00Z","3890":"2017-12-31T00:00:00Z","3891":"2017-12-31T00:00:00Z","3892":"2017-12-31T00:00:00Z","3893":"2017-12-30T00:00:00Z","3894":"2017-12-30T00:00:00Z","3895":"2017-12-30T00:00:00Z","3896":"2017-12-30T00:00:00Z","3897":"2017-12-29T00:00:00Z","3898":"2017-12-28T00:00:00Z","3899":"2017-12-27T00:00:00Z","3900":"2017-12-27T00:00:00Z","3901":"2017-12-25T00:00:00Z","3902":"2017-12-24T00:00:00Z","3903":"2017-12-23T00:00:00Z","3904":"2017-12-23T00:00:00Z","3905":"2017-12-22T00:00:00Z","3906":"2017-12-22T00:00:00Z","3907":"2017-12-22T00:00:00Z","3908":"2017-12-22T00:00:00Z","3909":"2017-12-22T00:00:00Z","3910":"2017-12-22T00:00:00Z","3912":"2017-12-22T00:00:00Z","3913":"2017-12-22T00:00:00Z","3914":"2017-12-21T00:00:00Z","3915":"2017-12-21T00:00:00Z","3917":"2017-12-20T00:00:00Z","3919":"2017-12-20T00:00:00Z","3920":"2017-12-19T00:00:00Z","3921":"2017-12-19T00:00:00Z","3922":"2017-12-19T00:00:00Z","3923":"2017-12-19T00:00:00Z","3924":"2017-12-19T00:00:00Z","3925":"2017-12-19T00:00:00Z","3926":"2017-12-19T00:00:00Z","3927":"2017-12-19T00:00:00Z","3929":"2017-12-18T00:00:00Z","3930":"2017-12-18T00:00:00Z","3931":"2017-12-17T00:00:00Z","3932":"2017-12-17T00:00:00Z","3934":"2017-12-16T00:00:00Z","3935":"2017-12-16T00:00:00Z","3936":"2017-12-16T00:00:00Z","3937":"2017-12-16T00:00:00Z","3938":"2017-12-15T00:00:00Z","3939":"2017-12-15T00:00:00Z","3940":"2017-12-15T00:00:00Z","3941":"2017-12-14T00:00:00Z","3942":"2017-12-14T00:00:00Z","3943":"2017-12-14T00:00:00Z","3944":"2017-12-13T00:00:00Z","3945":"2017-12-13T00:00:00Z","3946":"2017-12-13T00:00:00Z","3947":"2017-12-12T00:00:00Z","3949":"2017-12-12T00:00:00Z","3950":"2017-12-12T00:00:00Z","3951":"2017-12-12T00:00:00Z","3952":"2017-12-12T00:00:00Z","3954":"2017-12-12T00:00:00Z","3955":"2017-12-11T00:00:00Z","3956":"2017-12-11T00:00:00Z","3957":"2017-12-11T00:00:00Z","3958":"2017-12-11T00:00:00Z","3959":"2017-12-11T00:00:00Z","3961":"2017-12-11T00:00:00Z","3962":"2017-12-11T00:00:00Z","3963":"2017-12-10T00:00:00Z","3964":"2017-12-10T00:00:00Z","3965":"2017-12-09T00:00:00Z","3966":"2017-12-09T00:00:00Z","3967":"2017-12-09T00:00:00Z","3968":"2017-12-09T00:00:00Z","3969":"2017-12-09T00:00:00Z","3970":"2017-12-08T00:00:00Z","3971":"2017-12-08T00:00:00Z","3972":"2017-12-08T00:00:00Z","3973":"2017-12-08T00:00:00Z","3974":"2017-12-08T00:00:00Z","3977":"2017-12-07T00:00:00Z","3978":"2017-12-07T00:00:00Z","3979":"2017-12-06T00:00:00Z","3980":"2017-12-06T00:00:00Z","3982":"2017-12-05T00:00:00Z","3983":"2017-12-05T00:00:00Z","3984":"2017-12-05T00:00:00Z","3985":"2017-12-05T00:00:00Z","3986":"2017-12-05T00:00:00Z","3987":"2017-12-05T00:00:00Z","3988":"2017-12-05T00:00:00Z","3989":"2017-12-04T00:00:00Z","3992":"2017-12-04T00:00:00Z","3993":"2017-12-03T00:00:00Z","3994":"2017-12-03T00:00:00Z","3995":"2017-12-03T00:00:00Z","3996":"2017-12-03T00:00:00Z","3998":"2017-12-02T00:00:00Z","3999":"2017-12-02T00:00:00Z","4000":"2017-12-02T00:00:00Z","4001":"2017-12-02T00:00:00Z","4002":"2017-12-01T00:00:00Z","4003":"2017-12-01T00:00:00Z","4004":"2017-12-01T00:00:00Z","4005":"2017-12-01T00:00:00Z","4007":"2017-11-30T00:00:00Z","4008":"2017-11-30T00:00:00Z","4009":"2017-11-29T00:00:00Z","4010":"2017-11-28T00:00:00Z","4011":"2017-11-28T00:00:00Z","4012":"2017-11-28T00:00:00Z","4013":"2017-11-27T00:00:00Z","4014":"2017-11-27T00:00:00Z","4015":"2017-11-27T00:00:00Z","4016":"2017-11-27T00:00:00Z","4017":"2017-11-26T00:00:00Z","4018":"2017-11-26T00:00:00Z","4019":"2017-11-25T00:00:00Z","4020":"2017-11-25T00:00:00Z","4021":"2017-11-25T00:00:00Z","4022":"2017-11-25T00:00:00Z","4023":"2017-11-25T00:00:00Z","4024":"2017-11-24T00:00:00Z","4025":"2017-11-24T00:00:00Z","4026":"2017-11-24T00:00:00Z","4027":"2017-11-24T00:00:00Z","4028":"2017-11-24T00:00:00Z","4029":"2017-11-24T00:00:00Z","4030":"2017-11-22T00:00:00Z","4033":"2017-11-21T00:00:00Z","4034":"2017-11-20T00:00:00Z","4035":"2017-11-19T00:00:00Z","4036":"2017-11-19T00:00:00Z","4037":"2017-11-19T00:00:00Z","4038":"2017-11-19T00:00:00Z","4039":"2017-11-19T00:00:00Z","4040":"2017-11-19T00:00:00Z","4041":"2017-11-19T00:00:00Z","4042":"2017-11-19T00:00:00Z","4043":"2017-11-18T00:00:00Z","4044":"2017-11-18T00:00:00Z","4045":"2017-11-18T00:00:00Z","4046":"2017-11-18T00:00:00Z","4047":"2017-11-18T00:00:00Z","4048":"2017-11-18T00:00:00Z","4049":"2017-11-18T00:00:00Z","4050":"2017-11-18T00:00:00Z","4051":"2017-11-18T00:00:00Z","4052":"2017-11-17T00:00:00Z","4053":"2017-11-16T00:00:00Z","4054":"2017-11-16T00:00:00Z","4055":"2017-11-15T00:00:00Z","4056":"2017-11-15T00:00:00Z","4057":"2017-11-15T00:00:00Z","4058":"2017-11-14T00:00:00Z","4059":"2017-11-14T00:00:00Z","4060":"2017-11-14T00:00:00Z","4063":"2017-11-13T00:00:00Z","4066":"2017-11-12T00:00:00Z","4068":"2017-11-12T00:00:00Z","4069":"2017-11-12T00:00:00Z","4070":"2017-11-12T00:00:00Z","4071":"2017-11-12T00:00:00Z","4072":"2017-11-11T00:00:00Z","4073":"2017-11-11T00:00:00Z","4074":"2017-11-10T00:00:00Z","4075":"2017-11-09T00:00:00Z","4076":"2017-11-09T00:00:00Z","4078":"2017-11-09T00:00:00Z","4079":"2017-11-08T00:00:00Z","4080":"2017-11-08T00:00:00Z","4081":"2017-11-07T00:00:00Z","4082":"2017-11-07T00:00:00Z","4084":"2017-11-06T00:00:00Z","4085":"2017-11-06T00:00:00Z","4086":"2017-11-06T00:00:00Z","4087":"2017-11-05T00:00:00Z","4089":"2017-11-05T00:00:00Z","4090":"2017-11-05T00:00:00Z","4091":"2017-11-05T00:00:00Z","4092":"2017-11-05T00:00:00Z","4093":"2017-11-05T00:00:00Z","4094":"2017-11-04T00:00:00Z","4095":"2017-11-04T00:00:00Z","4096":"2017-11-03T00:00:00Z","4097":"2017-11-03T00:00:00Z","4098":"2017-11-03T00:00:00Z","4099":"2017-11-02T00:00:00Z","4100":"2017-11-02T00:00:00Z","4101":"2017-11-02T00:00:00Z","4102":"2017-11-02T00:00:00Z","4103":"2017-11-02T00:00:00Z","4105":"2017-11-01T00:00:00Z","4107":"2017-11-01T00:00:00Z","4108":"2017-11-01T00:00:00Z","4109":"2017-11-01T00:00:00Z","4110":"2017-11-01T00:00:00Z","4111":"2017-11-01T00:00:00Z","4112":"2017-10-31T00:00:00Z","4113":"2017-10-31T00:00:00Z","4114":"2017-10-30T00:00:00Z","4117":"2017-10-29T00:00:00Z","4118":"2017-10-29T00:00:00Z","4119":"2017-10-29T00:00:00Z","4120":"2017-10-28T00:00:00Z","4121":"2017-10-28T00:00:00Z","4122":"2017-10-28T00:00:00Z","4123":"2017-10-28T00:00:00Z","4124":"2017-10-28T00:00:00Z","4125":"2017-10-28T00:00:00Z","4127":"2017-10-26T00:00:00Z","4128":"2017-10-26T00:00:00Z","4129":"2017-10-26T00:00:00Z","4131":"2017-10-26T00:00:00Z","4134":"2017-10-25T00:00:00Z","4135":"2017-10-25T00:00:00Z","4136":"2017-10-25T00:00:00Z","4138":"2017-10-25T00:00:00Z","4139":"2017-10-25T00:00:00Z","4140":"2017-10-24T00:00:00Z","4141":"2017-10-24T00:00:00Z","4142":"2017-10-24T00:00:00Z","4143":"2017-10-24T00:00:00Z","4144":"2017-10-24T00:00:00Z","4145":"2017-10-24T00:00:00Z","4149":"2017-10-23T00:00:00Z","4150":"2017-10-23T00:00:00Z","4151":"2017-10-23T00:00:00Z","4152":"2017-10-23T00:00:00Z","4153":"2017-10-23T00:00:00Z","4154":"2017-10-22T00:00:00Z","4155":"2017-10-22T00:00:00Z","4156":"2017-10-22T00:00:00Z","4157":"2017-10-22T00:00:00Z","4158":"2017-10-22T00:00:00Z","4160":"2017-10-22T00:00:00Z","4161":"2017-10-22T00:00:00Z","4162":"2017-10-22T00:00:00Z","4163":"2017-10-21T00:00:00Z","4165":"2017-10-21T00:00:00Z","4166":"2017-10-21T00:00:00Z","4167":"2017-10-21T00:00:00Z","4168":"2017-10-20T00:00:00Z","4170":"2017-10-19T00:00:00Z","4171":"2017-10-19T00:00:00Z","4174":"2017-10-18T00:00:00Z","4175":"2017-10-18T00:00:00Z","4176":"2017-10-17T00:00:00Z","4177":"2017-10-17T00:00:00Z","4178":"2017-10-17T00:00:00Z","4179":"2017-10-17T00:00:00Z","4182":"2017-10-16T00:00:00Z","4183":"2017-10-16T00:00:00Z","4184":"2017-10-16T00:00:00Z","4185":"2017-10-16T00:00:00Z","4186":"2017-10-15T00:00:00Z","4187":"2017-10-15T00:00:00Z","4188":"2017-10-15T00:00:00Z","4190":"2017-10-14T00:00:00Z","4191":"2017-10-14T00:00:00Z","4192":"2017-10-14T00:00:00Z","4193":"2017-10-14T00:00:00Z","4194":"2017-10-14T00:00:00Z","4195":"2017-10-14T00:00:00Z","4196":"2017-10-14T00:00:00Z","4197":"2017-10-14T00:00:00Z","4198":"2017-10-14T00:00:00Z","4199":"2017-10-14T00:00:00Z","4200":"2017-10-13T00:00:00Z","4201":"2017-10-13T00:00:00Z","4202":"2017-10-13T00:00:00Z","4203":"2017-10-12T00:00:00Z","4204":"2017-10-12T00:00:00Z","4205":"2017-10-12T00:00:00Z","4207":"2017-10-12T00:00:00Z","4208":"2017-10-12T00:00:00Z","4209":"2017-10-12T00:00:00Z","4211":"2017-10-12T00:00:00Z","4213":"2017-10-11T00:00:00Z","4215":"2017-10-09T00:00:00Z","4216":"2017-10-09T00:00:00Z","4217":"2017-10-09T00:00:00Z","4218":"2017-10-08T00:00:00Z","4219":"2017-10-08T00:00:00Z","4220":"2017-10-08T00:00:00Z","4221":"2017-10-08T00:00:00Z","4222":"2017-10-08T00:00:00Z","4223":"2017-10-07T00:00:00Z","4224":"2017-10-07T00:00:00Z","4225":"2017-10-07T00:00:00Z","4226":"2017-10-07T00:00:00Z","4227":"2017-10-07T00:00:00Z","4228":"2017-10-07T00:00:00Z","4229":"2017-10-07T00:00:00Z","4230":"2017-10-07T00:00:00Z","4231":"2017-10-07T00:00:00Z","4232":"2017-10-07T00:00:00Z","4233":"2017-10-06T00:00:00Z","4234":"2017-10-06T00:00:00Z","4235":"2017-10-06T00:00:00Z","4236":"2017-10-06T00:00:00Z","4237":"2017-10-06T00:00:00Z","4239":"2017-10-06T00:00:00Z","4240":"2017-10-05T00:00:00Z","4241":"2017-10-05T00:00:00Z","4243":"2017-10-05T00:00:00Z","4244":"2017-10-05T00:00:00Z","4245":"2017-10-05T00:00:00Z","4246":"2017-10-05T00:00:00Z","4247":"2017-10-05T00:00:00Z","4248":"2017-10-05T00:00:00Z","4250":"2017-10-04T00:00:00Z","4251":"2017-10-04T00:00:00Z","4252":"2017-10-04T00:00:00Z","4253":"2017-10-04T00:00:00Z","4255":"2017-10-03T00:00:00Z","4256":"2017-10-03T00:00:00Z","4257":"2017-10-03T00:00:00Z","4258":"2017-10-03T00:00:00Z","4259":"2017-10-03T00:00:00Z","4260":"2017-10-03T00:00:00Z","4261":"2017-10-03T00:00:00Z","4262":"2017-10-03T00:00:00Z","4263":"2017-10-02T00:00:00Z","4264":"2017-10-02T00:00:00Z","4265":"2017-10-01T00:00:00Z","4266":"2017-10-01T00:00:00Z","4267":"2017-10-01T00:00:00Z","4268":"2017-10-01T00:00:00Z","4269":"2017-09-30T00:00:00Z","4270":"2017-09-30T00:00:00Z","4271":"2017-09-30T00:00:00Z","4272":"2017-09-30T00:00:00Z","4273":"2017-09-30T00:00:00Z","4274":"2017-09-29T00:00:00Z","4275":"2017-09-29T00:00:00Z","4276":"2017-09-29T00:00:00Z","4277":"2017-09-29T00:00:00Z","4279":"2017-09-28T00:00:00Z","4280":"2017-09-27T00:00:00Z","4281":"2017-09-27T00:00:00Z","4282":"2017-09-26T00:00:00Z","4283":"2017-09-26T00:00:00Z","4284":"2017-09-26T00:00:00Z","4288":"2017-09-25T00:00:00Z","4289":"2017-09-25T00:00:00Z","4290":"2017-09-24T00:00:00Z","4291":"2017-09-24T00:00:00Z","4292":"2017-09-24T00:00:00Z","4293":"2017-09-24T00:00:00Z","4294":"2017-09-23T00:00:00Z","4295":"2017-09-23T00:00:00Z","4296":"2017-09-23T00:00:00Z","4297":"2017-09-23T00:00:00Z","4298":"2017-09-23T00:00:00Z","4299":"2017-09-23T00:00:00Z","4300":"2017-09-23T00:00:00Z","4301":"2017-09-23T00:00:00Z","4302":"2017-09-23T00:00:00Z","4303":"2017-09-22T00:00:00Z","4304":"2017-09-22T00:00:00Z","4305":"2017-09-22T00:00:00Z","4306":"2017-09-22T00:00:00Z","4307":"2017-09-22T00:00:00Z","4308":"2017-09-20T00:00:00Z","4309":"2017-09-20T00:00:00Z","4310":"2017-09-20T00:00:00Z","4311":"2017-09-19T00:00:00Z","4312":"2017-09-19T00:00:00Z","4313":"2017-09-18T00:00:00Z","4314":"2017-09-18T00:00:00Z","4315":"2017-09-18T00:00:00Z","4316":"2017-09-18T00:00:00Z","4318":"2017-09-18T00:00:00Z","4319":"2017-09-18T00:00:00Z","4320":"2017-09-18T00:00:00Z","4321":"2017-09-18T00:00:00Z","4322":"2017-09-18T00:00:00Z","4323":"2017-09-18T00:00:00Z","4324":"2017-09-18T00:00:00Z","4325":"2017-09-17T00:00:00Z","4326":"2017-09-16T00:00:00Z","4327":"2017-09-15T00:00:00Z","4328":"2017-09-14T00:00:00Z","4329":"2017-09-14T00:00:00Z","4331":"2017-09-14T00:00:00Z","4332":"2017-09-14T00:00:00Z","4333":"2017-09-14T00:00:00Z","4334":"2017-09-13T00:00:00Z","4335":"2017-09-13T00:00:00Z","4336":"2017-09-13T00:00:00Z","4338":"2017-09-13T00:00:00Z","4343":"2017-09-12T00:00:00Z","4344":"2017-09-12T00:00:00Z","4345":"2017-09-12T00:00:00Z","4347":"2017-09-12T00:00:00Z","4348":"2017-09-12T00:00:00Z","4349":"2017-09-12T00:00:00Z","4351":"2017-09-11T00:00:00Z","4352":"2017-09-11T00:00:00Z","4353":"2017-09-10T00:00:00Z","4354":"2017-09-10T00:00:00Z","4355":"2017-09-09T00:00:00Z","4356":"2017-09-09T00:00:00Z","4357":"2017-09-09T00:00:00Z","4358":"2017-09-08T00:00:00Z","4359":"2017-09-08T00:00:00Z","4360":"2017-09-08T00:00:00Z","4361":"2017-09-08T00:00:00Z","4362":"2017-09-07T00:00:00Z","4363":"2017-09-07T00:00:00Z","4364":"2017-09-07T00:00:00Z","4366":"2017-09-07T00:00:00Z","4367":"2017-09-07T00:00:00Z","4368":"2017-09-07T00:00:00Z","4369":"2017-09-07T00:00:00Z","4370":"2017-09-06T00:00:00Z","4371":"2017-09-06T00:00:00Z","4372":"2017-09-06T00:00:00Z","4373":"2017-09-06T00:00:00Z","4374":"2017-09-05T00:00:00Z","4375":"2017-09-05T00:00:00Z","4376":"2017-09-05T00:00:00Z","4377":"2017-09-05T00:00:00Z","4379":"2017-09-05T00:00:00Z","4380":"2017-09-05T00:00:00Z","4382":"2017-09-05T00:00:00Z","4383":"2017-09-04T00:00:00Z","4384":"2017-09-04T00:00:00Z","4385":"2017-09-03T00:00:00Z","4386":"2017-09-03T00:00:00Z","4387":"2017-09-03T00:00:00Z","4388":"2017-09-02T00:00:00Z","4389":"2017-09-02T00:00:00Z","4390":"2017-09-01T00:00:00Z","4391":"2017-09-01T00:00:00Z","4392":"2017-09-01T00:00:00Z","4393":"2017-09-01T00:00:00Z","4396":"2017-08-31T00:00:00Z","4397":"2017-08-31T00:00:00Z","4398":"2017-08-31T00:00:00Z","4399":"2017-08-31T00:00:00Z","4400":"2017-08-30T00:00:00Z","4401":"2017-08-30T00:00:00Z","4402":"2017-08-30T00:00:00Z","4403":"2017-08-30T00:00:00Z","4406":"2017-08-29T00:00:00Z","4407":"2017-08-29T00:00:00Z","4408":"2017-08-29T00:00:00Z","4409":"2017-08-29T00:00:00Z","4410":"2017-08-28T00:00:00Z","4411":"2017-08-28T00:00:00Z","4412":"2017-08-28T00:00:00Z","4413":"2017-08-28T00:00:00Z","4414":"2017-08-28T00:00:00Z","4415":"2017-08-28T00:00:00Z","4416":"2017-08-28T00:00:00Z","4417":"2017-08-28T00:00:00Z","4418":"2017-08-26T00:00:00Z","4419":"2017-08-26T00:00:00Z","4422":"2017-08-25T00:00:00Z","4423":"2017-08-25T00:00:00Z","4424":"2017-08-25T00:00:00Z","4425":"2017-08-25T00:00:00Z","4426":"2017-08-25T00:00:00Z","4427":"2017-08-25T00:00:00Z","4428":"2017-08-25T00:00:00Z","4429":"2017-08-24T00:00:00Z","4430":"2017-08-24T00:00:00Z","4431":"2017-08-24T00:00:00Z","4432":"2017-08-23T00:00:00Z","4433":"2017-08-23T00:00:00Z","4434":"2017-08-23T00:00:00Z","4435":"2017-08-23T00:00:00Z","4436":"2017-08-22T00:00:00Z","4437":"2017-08-22T00:00:00Z","4438":"2017-08-22T00:00:00Z","4439":"2017-08-22T00:00:00Z","4440":"2017-08-21T00:00:00Z","4441":"2017-08-21T00:00:00Z","4442":"2017-08-21T00:00:00Z","4443":"2017-08-21T00:00:00Z","4444":"2017-08-21T00:00:00Z","4445":"2017-08-20T00:00:00Z","4446":"2017-08-20T00:00:00Z","4447":"2017-08-20T00:00:00Z","4448":"2017-08-20T00:00:00Z","4449":"2017-08-19T00:00:00Z","4450":"2017-08-19T00:00:00Z","4451":"2017-08-19T00:00:00Z","4452":"2017-08-19T00:00:00Z","4453":"2017-08-19T00:00:00Z","4454":"2017-08-19T00:00:00Z","4455":"2017-08-19T00:00:00Z","4456":"2017-08-18T00:00:00Z","4457":"2017-08-18T00:00:00Z","4458":"2017-08-18T00:00:00Z","4459":"2017-08-18T00:00:00Z","4460":"2017-08-18T00:00:00Z","4461":"2017-08-18T00:00:00Z","4462":"2017-08-17T00:00:00Z","4463":"2017-08-17T00:00:00Z","4464":"2017-08-16T00:00:00Z","4465":"2017-08-16T00:00:00Z","4466":"2017-08-16T00:00:00Z","4467":"2017-08-15T00:00:00Z","4468":"2017-08-15T00:00:00Z","4469":"2017-08-15T00:00:00Z","4470":"2017-08-15T00:00:00Z","4471":"2017-08-15T00:00:00Z","4473":"2017-08-14T00:00:00Z","4474":"2017-08-14T00:00:00Z","4475":"2017-08-13T00:00:00Z","4476":"2017-08-12T00:00:00Z","4477":"2017-08-12T00:00:00Z","4478":"2017-08-12T00:00:00Z","4479":"2017-08-12T00:00:00Z","4480":"2017-08-12T00:00:00Z","4481":"2017-08-12T00:00:00Z","4482":"2017-08-11T00:00:00Z","4483":"2017-08-11T00:00:00Z","4487":"2017-08-10T00:00:00Z","4488":"2017-08-10T00:00:00Z","4489":"2017-08-10T00:00:00Z","4490":"2017-08-10T00:00:00Z","4491":"2017-08-10T00:00:00Z","4492":"2017-08-10T00:00:00Z","4494":"2017-08-09T00:00:00Z","4495":"2017-08-09T00:00:00Z","4496":"2017-08-09T00:00:00Z","4497":"2017-08-09T00:00:00Z","4498":"2017-08-09T00:00:00Z","4499":"2017-08-08T00:00:00Z","4500":"2017-08-08T00:00:00Z","4501":"2017-08-07T00:00:00Z","4502":"2017-08-07T00:00:00Z","4503":"2017-08-07T00:00:00Z","4504":"2017-08-06T00:00:00Z","4505":"2017-08-05T00:00:00Z","4506":"2017-08-05T00:00:00Z","4507":"2017-08-05T00:00:00Z","4508":"2017-08-04T00:00:00Z","4509":"2017-08-04T00:00:00Z","4510":"2017-08-04T00:00:00Z","4511":"2017-08-04T00:00:00Z","4512":"2017-08-04T00:00:00Z","4513":"2017-08-03T00:00:00Z","4514":"2017-08-03T00:00:00Z","4515":"2017-08-02T00:00:00Z","4517":"2017-08-02T00:00:00Z","4518":"2017-08-02T00:00:00Z","4519":"2017-08-01T00:00:00Z","4520":"2017-08-01T00:00:00Z","4523":"2017-08-01T00:00:00Z","4526":"2017-07-31T00:00:00Z","4527":"2017-07-31T00:00:00Z","4528":"2017-07-31T00:00:00Z","4529":"2017-07-31T00:00:00Z","4530":"2017-07-31T00:00:00Z","4531":"2017-07-30T00:00:00Z","4532":"2017-07-30T00:00:00Z","4533":"2017-07-30T00:00:00Z","4534":"2017-07-30T00:00:00Z","4535":"2017-07-30T00:00:00Z","4536":"2017-07-29T00:00:00Z","4537":"2017-07-29T00:00:00Z","4538":"2017-07-29T00:00:00Z","4539":"2017-07-29T00:00:00Z","4540":"2017-07-29T00:00:00Z","4541":"2017-07-28T00:00:00Z","4542":"2017-07-28T00:00:00Z","4543":"2017-07-28T00:00:00Z","4544":"2017-07-28T00:00:00Z","4545":"2017-07-28T00:00:00Z","4546":"2017-07-27T00:00:00Z","4547":"2017-07-27T00:00:00Z","4548":"2017-07-27T00:00:00Z","4549":"2017-07-27T00:00:00Z","4550":"2017-07-26T00:00:00Z","4551":"2017-07-26T00:00:00Z","4552":"2017-07-26T00:00:00Z","4553":"2017-07-26T00:00:00Z","4556":"2017-07-25T00:00:00Z","4557":"2017-07-25T00:00:00Z","4558":"2017-07-25T00:00:00Z","4559":"2017-07-25T00:00:00Z","4560":"2017-07-24T00:00:00Z","4561":"2017-07-24T00:00:00Z","4562":"2017-07-24T00:00:00Z","4563":"2017-07-24T00:00:00Z","4564":"2017-07-24T00:00:00Z","4565":"2017-07-23T00:00:00Z","4566":"2017-07-22T00:00:00Z","4567":"2017-07-22T00:00:00Z","4568":"2017-07-21T00:00:00Z","4569":"2017-07-21T00:00:00Z","4570":"2017-07-21T00:00:00Z","4571":"2017-07-20T00:00:00Z","4572":"2017-07-20T00:00:00Z","4574":"2017-07-20T00:00:00Z","4575":"2017-07-19T00:00:00Z","4576":"2017-07-19T00:00:00Z","4577":"2017-07-19T00:00:00Z","4578":"2017-07-19T00:00:00Z","4579":"2017-07-19T00:00:00Z","4580":"2017-07-19T00:00:00Z","4581":"2017-07-19T00:00:00Z","4582":"2017-07-19T00:00:00Z","4583":"2017-07-19T00:00:00Z","4585":"2017-07-16T00:00:00Z","4587":"2017-07-16T00:00:00Z","4588":"2017-07-16T00:00:00Z","4589":"2017-07-16T00:00:00Z","4590":"2017-07-14T00:00:00Z","4591":"2017-07-14T00:00:00Z","4592":"2017-07-13T00:00:00Z","4593":"2017-07-13T00:00:00Z","4594":"2017-07-13T00:00:00Z","4597":"2017-07-12T00:00:00Z","4598":"2017-07-12T00:00:00Z","4600":"2017-07-12T00:00:00Z","4601":"2017-07-12T00:00:00Z","4602":"2017-07-11T00:00:00Z","4603":"2017-07-11T00:00:00Z","4604":"2017-07-11T00:00:00Z","4605":"2017-07-11T00:00:00Z","4606":"2017-07-11T00:00:00Z","4607":"2017-07-11T00:00:00Z","4608":"2017-07-11T00:00:00Z","4609":"2017-07-11T00:00:00Z","4611":"2017-07-10T00:00:00Z","4612":"2017-07-09T00:00:00Z","4613":"2017-07-09T00:00:00Z","4614":"2017-07-09T00:00:00Z","4615":"2017-07-08T00:00:00Z","4616":"2017-07-08T00:00:00Z","4617":"2017-07-08T00:00:00Z","4618":"2017-07-08T00:00:00Z","4619":"2017-07-08T00:00:00Z","4620":"2017-07-08T00:00:00Z","4621":"2017-07-08T00:00:00Z","4622":"2017-07-07T00:00:00Z","4623":"2017-07-07T00:00:00Z","4624":"2017-07-07T00:00:00Z","4625":"2017-07-06T00:00:00Z","4626":"2017-07-06T00:00:00Z","4627":"2017-07-06T00:00:00Z","4628":"2017-07-06T00:00:00Z","4630":"2017-07-05T00:00:00Z","4631":"2017-07-05T00:00:00Z","4632":"2017-07-05T00:00:00Z","4633":"2017-07-05T00:00:00Z","4634":"2017-07-05T00:00:00Z","4635":"2017-07-05T00:00:00Z","4636":"2017-07-04T00:00:00Z","4637":"2017-07-04T00:00:00Z","4639":"2017-07-03T00:00:00Z","4640":"2017-07-03T00:00:00Z","4641":"2017-07-03T00:00:00Z","4642":"2017-07-03T00:00:00Z","4643":"2017-07-03T00:00:00Z","4645":"2017-07-03T00:00:00Z","4647":"2017-06-30T00:00:00Z","4648":"2017-06-30T00:00:00Z","4649":"2017-06-29T00:00:00Z","4651":"2017-06-28T00:00:00Z","4652":"2017-06-28T00:00:00Z","4653":"2017-06-28T00:00:00Z","4654":"2017-06-28T00:00:00Z","4655":"2017-06-27T00:00:00Z","4656":"2017-06-27T00:00:00Z","4657":"2017-06-27T00:00:00Z","4658":"2017-06-27T00:00:00Z","4659":"2017-06-27T00:00:00Z","4661":"2017-06-26T00:00:00Z","4662":"2017-06-26T00:00:00Z","4663":"2017-06-26T00:00:00Z","4664":"2017-06-26T00:00:00Z","4665":"2017-06-26T00:00:00Z","4666":"2017-06-26T00:00:00Z","4667":"2017-06-26T00:00:00Z","4668":"2017-06-26T00:00:00Z","4669":"2017-06-26T00:00:00Z","4670":"2017-06-25T00:00:00Z","4672":"2017-06-23T00:00:00Z","4673":"2017-06-22T00:00:00Z","4674":"2017-06-22T00:00:00Z","4677":"2017-06-21T00:00:00Z","4678":"2017-06-21T00:00:00Z","4679":"2017-06-21T00:00:00Z","4680":"2017-06-20T00:00:00Z","4684":"2017-06-19T00:00:00Z","4685":"2017-06-18T00:00:00Z","4686":"2017-06-17T00:00:00Z","4687":"2017-06-17T00:00:00Z","4688":"2017-06-17T00:00:00Z","4689":"2017-06-17T00:00:00Z","4690":"2017-06-17T00:00:00Z","4691":"2017-06-17T00:00:00Z","4692":"2017-06-17T00:00:00Z","4693":"2017-06-17T00:00:00Z","4694":"2017-06-17T00:00:00Z","4695":"2017-06-17T00:00:00Z","4696":"2017-06-17T00:00:00Z","4697":"2017-06-16T00:00:00Z","4698":"2017-06-15T00:00:00Z","4699":"2017-06-14T00:00:00Z","4700":"2017-06-14T00:00:00Z","4703":"2017-06-13T00:00:00Z","4704":"2017-06-13T00:00:00Z","4707":"2017-06-12T00:00:00Z","4708":"2017-06-12T00:00:00Z","4709":"2017-06-12T00:00:00Z","4711":"2017-06-12T00:00:00Z","4712":"2017-06-11T00:00:00Z","4713":"2017-06-11T00:00:00Z","4714":"2017-06-11T00:00:00Z","4715":"2017-06-11T00:00:00Z","4716":"2017-06-11T00:00:00Z","4717":"2017-06-11T00:00:00Z","4718":"2017-06-10T00:00:00Z","4719":"2017-06-10T00:00:00Z","4720":"2017-06-10T00:00:00Z","4723":"2017-06-09T00:00:00Z","4724":"2017-06-09T00:00:00Z","4725":"2017-06-09T00:00:00Z","4726":"2017-06-09T00:00:00Z","4727":"2017-06-08T00:00:00Z","4728":"2017-06-08T00:00:00Z","4729":"2017-06-08T00:00:00Z","4730":"2017-06-08T00:00:00Z","4731":"2017-06-07T00:00:00Z","4732":"2017-06-07T00:00:00Z","4733":"2017-06-07T00:00:00Z","4734":"2017-06-07T00:00:00Z","4735":"2017-06-07T00:00:00Z","4736":"2017-06-07T00:00:00Z","4737":"2017-06-07T00:00:00Z","4738":"2017-06-06T00:00:00Z","4739":"2017-06-06T00:00:00Z","4740":"2017-06-06T00:00:00Z","4741":"2017-06-06T00:00:00Z","4742":"2017-06-06T00:00:00Z","4744":"2017-06-05T00:00:00Z","4745":"2017-06-05T00:00:00Z","4746":"2017-06-05T00:00:00Z","4747":"2017-06-04T00:00:00Z","4748":"2017-06-04T00:00:00Z","4749":"2017-06-04T00:00:00Z","4750":"2017-06-04T00:00:00Z","4751":"2017-06-03T00:00:00Z","4752":"2017-06-03T00:00:00Z","4753":"2017-06-03T00:00:00Z","4755":"2017-06-02T00:00:00Z","4756":"2017-06-02T00:00:00Z","4757":"2017-06-02T00:00:00Z","4759":"2017-06-01T00:00:00Z","4760":"2017-06-01T00:00:00Z","4761":"2017-06-01T00:00:00Z","4762":"2017-06-01T00:00:00Z","4763":"2017-06-01T00:00:00Z","4764":"2017-06-01T00:00:00Z","4767":"2017-05-31T00:00:00Z","4768":"2017-05-31T00:00:00Z","4769":"2017-05-31T00:00:00Z","4770":"2017-05-31T00:00:00Z","4771":"2017-05-31T00:00:00Z","4772":"2017-05-31T00:00:00Z","4773":"2017-05-30T00:00:00Z","4774":"2017-05-30T00:00:00Z","4776":"2017-05-30T00:00:00Z","4777":"2017-05-30T00:00:00Z","4778":"2017-05-29T00:00:00Z","4779":"2017-05-29T00:00:00Z","4780":"2017-05-27T00:00:00Z","4781":"2017-05-27T00:00:00Z","4782":"2017-05-27T00:00:00Z","4783":"2017-05-26T00:00:00Z","4784":"2017-05-26T00:00:00Z","4785":"2017-05-26T00:00:00Z","4786":"2017-05-26T00:00:00Z","4788":"2017-05-25T00:00:00Z","4789":"2017-05-25T00:00:00Z","4790":"2017-05-24T00:00:00Z","4791":"2017-05-24T00:00:00Z","4792":"2017-05-24T00:00:00Z","4793":"2017-05-23T00:00:00Z","4794":"2017-05-23T00:00:00Z","4795":"2017-05-23T00:00:00Z","4797":"2017-05-22T00:00:00Z","4798":"2017-05-22T00:00:00Z","4799":"2017-05-22T00:00:00Z","4800":"2017-05-21T00:00:00Z","4801":"2017-05-21T00:00:00Z","4802":"2017-05-20T00:00:00Z","4803":"2017-05-20T00:00:00Z","4804":"2017-05-20T00:00:00Z","4805":"2017-05-20T00:00:00Z","4806":"2017-05-19T00:00:00Z","4807":"2017-05-19T00:00:00Z","4808":"2017-05-19T00:00:00Z","4809":"2017-05-19T00:00:00Z","4810":"2017-05-19T00:00:00Z","4811":"2017-05-19T00:00:00Z","4812":"2017-05-19T00:00:00Z","4813":"2017-05-19T00:00:00Z","4814":"2017-05-19T00:00:00Z","4816":"2017-05-17T00:00:00Z","4817":"2017-05-16T00:00:00Z","4818":"2017-05-16T00:00:00Z","4819":"2017-05-16T00:00:00Z","4820":"2017-05-16T00:00:00Z","4822":"2017-05-15T00:00:00Z","4823":"2017-05-15T00:00:00Z","4824":"2017-05-15T00:00:00Z","4825":"2017-05-14T00:00:00Z","4826":"2017-05-12T00:00:00Z","4831":"2017-05-12T00:00:00Z","4832":"2017-05-11T00:00:00Z","4833":"2017-05-11T00:00:00Z","4834":"2017-05-11T00:00:00Z","4835":"2017-05-11T00:00:00Z","4836":"2017-05-11T00:00:00Z","4837":"2017-05-11T00:00:00Z","4839":"2017-05-10T00:00:00Z","4840":"2017-05-10T00:00:00Z","4842":"2017-05-09T00:00:00Z","4844":"2017-05-08T00:00:00Z","4845":"2017-05-08T00:00:00Z","4846":"2017-05-07T00:00:00Z","4847":"2017-05-06T00:00:00Z","4848":"2017-05-06T00:00:00Z","4849":"2017-05-05T00:00:00Z","4850":"2017-05-05T00:00:00Z","4851":"2017-05-04T00:00:00Z","4852":"2017-05-04T00:00:00Z","4853":"2017-05-04T00:00:00Z","4855":"2017-05-03T00:00:00Z","4856":"2017-05-03T00:00:00Z","4857":"2017-05-02T00:00:00Z","4858":"2017-05-02T00:00:00Z","4861":"2017-05-01T00:00:00Z","4862":"2017-05-01T00:00:00Z","4863":"2017-05-01T00:00:00Z","4864":"2017-04-29T00:00:00Z","4865":"2017-04-29T00:00:00Z","4866":"2017-04-29T00:00:00Z","4867":"2017-04-29T00:00:00Z","4868":"2017-04-28T00:00:00Z","4869":"2017-04-27T00:00:00Z","4870":"2017-04-27T00:00:00Z","4871":"2017-04-27T00:00:00Z","4873":"2017-04-26T00:00:00Z","4874":"2017-04-26T00:00:00Z","4875":"2017-04-26T00:00:00Z","4876":"2017-04-26T00:00:00Z","4877":"2017-04-26T00:00:00Z","4880":"2017-04-25T00:00:00Z","4881":"2017-04-25T00:00:00Z","4882":"2017-04-25T00:00:00Z","4883":"2017-04-25T00:00:00Z","4884":"2017-04-25T00:00:00Z","4886":"2017-04-24T00:00:00Z","4887":"2017-04-24T00:00:00Z","4888":"2017-04-24T00:00:00Z","4889":"2017-04-24T00:00:00Z","4890":"2017-04-24T00:00:00Z","4891":"2017-04-23T00:00:00Z","4892":"2017-04-23T00:00:00Z","4893":"2017-04-23T00:00:00Z","4894":"2017-04-22T00:00:00Z","4896":"2017-04-22T00:00:00Z","4897":"2017-04-22T00:00:00Z","4898":"2017-04-22T00:00:00Z","4899":"2017-04-22T00:00:00Z","4900":"2017-04-21T00:00:00Z","4901":"2017-04-21T00:00:00Z","4902":"2017-04-21T00:00:00Z","4903":"2017-04-20T00:00:00Z","4904":"2017-04-20T00:00:00Z","4905":"2017-04-20T00:00:00Z","4906":"2017-04-20T00:00:00Z","4910":"2017-04-13T00:00:00Z","4911":"2017-04-12T00:00:00Z","4912":"2017-04-12T00:00:00Z","4913":"2017-04-12T00:00:00Z","4914":"2017-04-12T00:00:00Z","4915":"2017-04-12T00:00:00Z","4916":"2017-04-12T00:00:00Z","4918":"2017-04-11T00:00:00Z","4919":"2017-04-11T00:00:00Z","4920":"2017-04-11T00:00:00Z","4921":"2017-04-11T00:00:00Z","4922":"2017-04-10T00:00:00Z","4924":"2017-04-10T00:00:00Z","4925":"2017-04-10T00:00:00Z","4926":"2017-04-10T00:00:00Z","4927":"2017-04-09T00:00:00Z","4928":"2017-04-09T00:00:00Z","4929":"2017-04-09T00:00:00Z","4930":"2017-04-08T00:00:00Z","4931":"2017-04-08T00:00:00Z","4932":"2017-04-08T00:00:00Z","4933":"2017-04-08T00:00:00Z","4935":"2017-04-07T00:00:00Z","4936":"2017-04-07T00:00:00Z","4937":"2017-04-07T00:00:00Z","4938":"2017-04-07T00:00:00Z","4939":"2017-04-06T00:00:00Z","4940":"2017-04-06T00:00:00Z","4941":"2017-04-06T00:00:00Z","4942":"2017-04-06T00:00:00Z","4943":"2017-04-05T00:00:00Z","4944":"2017-04-05T00:00:00Z","4945":"2017-04-05T00:00:00Z","4946":"2017-04-04T00:00:00Z","4947":"2017-04-04T00:00:00Z","4948":"2017-04-04T00:00:00Z","4950":"2017-04-03T00:00:00Z","4951":"2017-04-03T00:00:00Z","4952":"2017-04-03T00:00:00Z","4953":"2017-04-02T00:00:00Z","4954":"2017-04-02T00:00:00Z","4955":"2017-04-02T00:00:00Z","4956":"2017-04-01T00:00:00Z","4957":"2017-04-01T00:00:00Z","4958":"2017-04-01T00:00:00Z","4959":"2017-03-31T00:00:00Z","4960":"2017-03-31T00:00:00Z","4961":"2017-03-31T00:00:00Z","4962":"2017-03-31T00:00:00Z","4964":"2017-03-30T00:00:00Z","4965":"2017-03-30T00:00:00Z","4966":"2017-03-30T00:00:00Z","4969":"2017-03-29T00:00:00Z","4970":"2017-03-29T00:00:00Z","4971":"2017-03-28T00:00:00Z","4973":"2017-03-28T00:00:00Z","4974":"2017-03-28T00:00:00Z","4975":"2017-03-28T00:00:00Z","4976":"2017-03-28T00:00:00Z","4978":"2017-03-27T00:00:00Z","4979":"2017-03-27T00:00:00Z","4980":"2017-03-27T00:00:00Z","4981":"2017-03-27T00:00:00Z","4982":"2017-03-26T00:00:00Z","4983":"2017-03-26T00:00:00Z","4984":"2017-03-25T00:00:00Z","4985":"2017-03-25T00:00:00Z","4986":"2017-03-25T00:00:00Z","4987":"2017-03-25T00:00:00Z","4988":"2017-03-25T00:00:00Z","4989":"2017-03-25T00:00:00Z","4990":"2017-03-24T00:00:00Z","4991":"2017-03-24T00:00:00Z","4993":"2017-03-23T00:00:00Z","4994":"2017-03-23T00:00:00Z","4995":"2017-03-23T00:00:00Z","4996":"2017-03-22T00:00:00Z","4997":"2017-03-22T00:00:00Z","4998":"2017-03-22T00:00:00Z","4999":"2017-03-22T00:00:00Z","5000":"2017-03-22T00:00:00Z","5001":"2017-03-21T00:00:00Z","5002":"2017-03-21T00:00:00Z","5003":"2017-03-21T00:00:00Z","5004":"2017-03-21T00:00:00Z","5006":"2017-03-20T00:00:00Z","5007":"2017-03-20T00:00:00Z","5008":"2017-03-19T00:00:00Z","5009":"2017-03-19T00:00:00Z","5010":"2017-03-19T00:00:00Z","5011":"2017-03-19T00:00:00Z","5012":"2017-03-18T00:00:00Z","5013":"2017-03-18T00:00:00Z","5014":"2017-03-18T00:00:00Z","5015":"2017-03-18T00:00:00Z","5016":"2017-03-18T00:00:00Z","5017":"2017-03-18T00:00:00Z","5018":"2017-03-18T00:00:00Z","5019":"2017-03-18T00:00:00Z","5020":"2017-03-18T00:00:00Z","5021":"2017-03-15T00:00:00Z","5022":"2017-03-15T00:00:00Z","5023":"2017-03-15T00:00:00Z","5026":"2017-03-13T00:00:00Z","5027":"2017-03-12T00:00:00Z","5029":"2017-03-12T00:00:00Z","5030":"2017-03-11T00:00:00Z","5031":"2017-03-11T00:00:00Z","5032":"2017-03-11T00:00:00Z","5033":"2017-03-11T00:00:00Z","5034":"2017-03-11T00:00:00Z","5035":"2017-03-11T00:00:00Z","5036":"2017-03-11T00:00:00Z","5037":"2017-03-11T00:00:00Z","5038":"2017-03-10T00:00:00Z","5039":"2017-03-10T00:00:00Z","5040":"2017-03-10T00:00:00Z","5041":"2017-03-10T00:00:00Z","5043":"2017-03-10T00:00:00Z","5044":"2017-03-09T00:00:00Z","5045":"2017-03-09T00:00:00Z","5046":"2017-03-09T00:00:00Z","5047":"2017-03-09T00:00:00Z","5048":"2017-03-08T00:00:00Z","5049":"2017-03-08T00:00:00Z","5050":"2017-03-08T00:00:00Z","5051":"2017-03-07T00:00:00Z","5052":"2017-03-07T00:00:00Z","5053":"2017-03-06T00:00:00Z","5054":"2017-03-06T00:00:00Z","5055":"2017-03-06T00:00:00Z","5056":"2017-03-06T00:00:00Z","5057":"2017-03-06T00:00:00Z","5058":"2017-03-06T00:00:00Z","5059":"2017-03-05T00:00:00Z","5060":"2017-03-05T00:00:00Z","5061":"2017-03-05T00:00:00Z","5062":"2017-03-05T00:00:00Z","5063":"2017-03-04T00:00:00Z","5064":"2017-03-04T00:00:00Z","5065":"2017-03-04T00:00:00Z","5066":"2017-03-04T00:00:00Z","5067":"2017-03-04T00:00:00Z","5068":"2017-03-04T00:00:00Z","5069":"2017-03-04T00:00:00Z","5070":"2017-03-04T00:00:00Z","5071":"2017-03-04T00:00:00Z","5072":"2017-03-03T00:00:00Z","5073":"2017-03-02T00:00:00Z","5074":"2017-03-02T00:00:00Z","5075":"2017-03-02T00:00:00Z","5076":"2017-03-02T00:00:00Z","5081":"2017-03-01T00:00:00Z","5083":"2017-03-01T00:00:00Z","5084":"2017-02-28T00:00:00Z","5085":"2017-02-28T00:00:00Z","5086":"2017-02-28T00:00:00Z","5087":"2017-02-27T00:00:00Z","5088":"2017-02-27T00:00:00Z","5089":"2017-02-26T00:00:00Z","5090":"2017-02-26T00:00:00Z","5091":"2017-02-26T00:00:00Z","5092":"2017-02-26T00:00:00Z","5093":"2017-02-25T00:00:00Z","5094":"2017-02-25T00:00:00Z","5095":"2017-02-25T00:00:00Z","5096":"2017-02-25T00:00:00Z","5097":"2017-02-25T00:00:00Z","5098":"2017-02-25T00:00:00Z","5099":"2017-02-24T00:00:00Z","5100":"2017-02-24T00:00:00Z","5101":"2017-02-24T00:00:00Z","5103":"2017-02-24T00:00:00Z","5104":"2017-02-23T00:00:00Z","5105":"2017-02-23T00:00:00Z","5106":"2017-02-23T00:00:00Z","5107":"2017-02-23T00:00:00Z","5108":"2017-02-22T00:00:00Z","5109":"2017-02-22T00:00:00Z","5111":"2017-02-21T00:00:00Z","5112":"2017-02-21T00:00:00Z","5113":"2017-02-21T00:00:00Z","5114":"2017-02-21T00:00:00Z","5115":"2017-02-21T00:00:00Z","5116":"2017-02-20T00:00:00Z","5117":"2017-02-20T00:00:00Z","5118":"2017-02-20T00:00:00Z","5119":"2017-02-20T00:00:00Z","5120":"2017-02-19T00:00:00Z","5121":"2017-02-19T00:00:00Z","5122":"2017-02-19T00:00:00Z","5123":"2017-02-19T00:00:00Z","5124":"2017-02-19T00:00:00Z","5125":"2017-02-19T00:00:00Z","5126":"2017-02-19T00:00:00Z","5127":"2017-02-18T00:00:00Z","5128":"2017-02-18T00:00:00Z","5129":"2017-02-18T00:00:00Z","5130":"2017-02-18T00:00:00Z","5131":"2017-02-18T00:00:00Z","5132":"2017-02-18T00:00:00Z","5133":"2017-02-17T00:00:00Z","5134":"2017-02-17T00:00:00Z","5135":"2017-02-17T00:00:00Z","5136":"2017-02-17T00:00:00Z","5137":"2017-02-17T00:00:00Z","5139":"2017-02-16T00:00:00Z","5140":"2017-02-16T00:00:00Z","5141":"2017-02-16T00:00:00Z","5142":"2017-02-16T00:00:00Z","5143":"2017-02-16T00:00:00Z","5144":"2017-02-16T00:00:00Z","5145":"2017-02-16T00:00:00Z","5146":"2017-02-16T00:00:00Z","5147":"2017-02-16T00:00:00Z","5148":"2017-02-16T00:00:00Z","5149":"2017-02-14T00:00:00Z","5151":"2017-02-13T00:00:00Z","5153":"2017-02-12T00:00:00Z","5154":"2017-02-10T00:00:00Z","5155":"2017-02-10T00:00:00Z","5157":"2017-02-10T00:00:00Z","5158":"2017-02-09T00:00:00Z","5159":"2017-02-09T00:00:00Z","5160":"2017-02-09T00:00:00Z","5161":"2017-02-09T00:00:00Z","5162":"2017-02-08T00:00:00Z","5163":"2017-02-08T00:00:00Z","5164":"2017-02-08T00:00:00Z","5165":"2017-02-08T00:00:00Z","5166":"2017-02-08T00:00:00Z","5167":"2017-02-07T00:00:00Z","5168":"2017-02-07T00:00:00Z","5170":"2017-02-06T00:00:00Z","5171":"2017-02-06T00:00:00Z","5172":"2017-02-06T00:00:00Z","5173":"2017-02-06T00:00:00Z","5174":"2017-02-06T00:00:00Z","5175":"2017-02-06T00:00:00Z","5176":"2017-02-06T00:00:00Z","5177":"2017-02-05T00:00:00Z","5178":"2017-02-05T00:00:00Z","5179":"2017-02-05T00:00:00Z","5180":"2017-02-04T00:00:00Z","5181":"2017-02-04T00:00:00Z","5183":"2017-02-04T00:00:00Z","5184":"2017-02-04T00:00:00Z","5185":"2017-02-04T00:00:00Z","5186":"2017-02-04T00:00:00Z","5188":"2017-02-02T00:00:00Z","5189":"2017-02-02T00:00:00Z","5194":"2017-02-01T00:00:00Z","5195":"2017-02-01T00:00:00Z","5196":"2017-02-01T00:00:00Z","5197":"2017-02-01T00:00:00Z","5198":"2017-02-01T00:00:00Z","5199":"2017-01-31T00:00:00Z","5200":"2017-01-31T00:00:00Z","5201":"2017-01-30T00:00:00Z","5203":"2017-01-30T00:00:00Z","5204":"2017-01-30T00:00:00Z","5205":"2017-01-29T00:00:00Z","5206":"2017-01-29T00:00:00Z","5207":"2017-01-29T00:00:00Z","5208":"2017-01-29T00:00:00Z","5209":"2017-01-29T00:00:00Z","5210":"2017-01-29T00:00:00Z","5211":"2017-01-29T00:00:00Z","5212":"2017-01-28T00:00:00Z","5213":"2017-01-28T00:00:00Z","5214":"2017-01-28T00:00:00Z","5215":"2017-01-28T00:00:00Z","5216":"2017-01-28T00:00:00Z","5217":"2017-01-28T00:00:00Z","5218":"2017-01-28T00:00:00Z","5219":"2017-01-28T00:00:00Z","5220":"2017-01-27T00:00:00Z","5221":"2017-01-27T00:00:00Z","5222":"2017-01-27T00:00:00Z","5223":"2017-01-27T00:00:00Z","5224":"2017-01-27T00:00:00Z","5225":"2017-01-26T00:00:00Z","5226":"2017-01-26T00:00:00Z","5227":"2017-01-26T00:00:00Z","5228":"2017-01-26T00:00:00Z","5229":"2017-01-26T00:00:00Z","5231":"2017-01-25T00:00:00Z","5232":"2017-01-25T00:00:00Z","5233":"2017-01-25T00:00:00Z","5234":"2017-01-25T00:00:00Z","5235":"2017-01-24T00:00:00Z","5236":"2017-01-24T00:00:00Z","5237":"2017-01-24T00:00:00Z","5238":"2017-01-24T00:00:00Z","5239":"2017-01-24T00:00:00Z","5240":"2017-01-23T00:00:00Z","5241":"2017-01-23T00:00:00Z","5242":"2017-01-23T00:00:00Z","5243":"2017-01-22T00:00:00Z","5244":"2017-01-22T00:00:00Z","5245":"2017-01-22T00:00:00Z","5246":"2017-01-22T00:00:00Z","5247":"2017-01-22T00:00:00Z","5248":"2017-01-21T00:00:00Z","5249":"2017-01-21T00:00:00Z","5250":"2017-01-21T00:00:00Z","5251":"2017-01-21T00:00:00Z","5252":"2017-01-20T00:00:00Z","5254":"2017-01-19T00:00:00Z","5255":"2017-01-19T00:00:00Z","5256":"2017-01-19T00:00:00Z","5257":"2017-01-19T00:00:00Z","5258":"2017-01-19T00:00:00Z","5259":"2017-01-19T00:00:00Z","5260":"2017-01-19T00:00:00Z","5261":"2017-01-16T00:00:00Z","5262":"2017-01-16T00:00:00Z","5263":"2017-01-15T00:00:00Z","5264":"2017-01-14T00:00:00Z","5265":"2017-01-14T00:00:00Z","5266":"2017-01-13T00:00:00Z","5267":"2017-01-13T00:00:00Z","5268":"2017-01-13T00:00:00Z","5269":"2017-01-12T00:00:00Z","5270":"2017-01-12T00:00:00Z","5271":"2017-01-12T00:00:00Z","5272":"2017-01-12T00:00:00Z","5273":"2017-01-11T00:00:00Z","5274":"2017-01-11T00:00:00Z","5275":"2017-01-11T00:00:00Z","5276":"2017-01-10T00:00:00Z","5277":"2017-01-10T00:00:00Z","5279":"2017-01-10T00:00:00Z","5280":"2017-01-10T00:00:00Z","5281":"2017-01-10T00:00:00Z","5282":"2017-01-10T00:00:00Z","5285":"2017-01-09T00:00:00Z","5286":"2017-01-09T00:00:00Z","5287":"2017-01-09T00:00:00Z","5288":"2017-01-09T00:00:00Z","5289":"2017-01-09T00:00:00Z","5290":"2017-01-09T00:00:00Z","5291":"2017-01-08T00:00:00Z","5292":"2017-01-08T00:00:00Z","5293":"2017-01-08T00:00:00Z","5294":"2017-01-08T00:00:00Z","5295":"2017-01-07T00:00:00Z","5296":"2017-01-07T00:00:00Z","5297":"2017-01-07T00:00:00Z","5298":"2017-01-07T00:00:00Z","5299":"2017-01-07T00:00:00Z","5300":"2017-01-06T00:00:00Z","5301":"2017-01-06T00:00:00Z","5304":"2017-01-05T00:00:00Z","5306":"2017-01-04T00:00:00Z","5307":"2017-01-04T00:00:00Z","5309":"2017-01-03T00:00:00Z","5310":"2017-01-03T00:00:00Z","5311":"2017-01-03T00:00:00Z","5312":"2017-01-03T00:00:00Z","5313":"2017-01-03T00:00:00Z","5314":"2017-01-03T00:00:00Z","5315":"2016-12-31T00:00:00Z","5316":"2016-12-30T00:00:00Z","5318":"2016-12-30T00:00:00Z","5319":"2016-12-29T00:00:00Z","5320":"2016-12-28T00:00:00Z","5321":"2016-12-27T00:00:00Z","5322":"2016-12-26T00:00:00Z","5323":"2016-12-26T00:00:00Z","5324":"2016-12-23T00:00:00Z","5325":"2016-12-23T00:00:00Z","5326":"2016-12-23T00:00:00Z","5327":"2016-12-23T00:00:00Z","5329":"2016-12-22T00:00:00Z","5330":"2016-12-21T00:00:00Z","5331":"2016-12-21T00:00:00Z","5332":"2016-12-21T00:00:00Z","5333":"2016-12-21T00:00:00Z","5334":"2016-12-21T00:00:00Z","5335":"2016-12-21T00:00:00Z","5336":"2016-12-20T00:00:00Z","5337":"2016-12-20T00:00:00Z","5339":"2016-12-20T00:00:00Z","5340":"2016-12-19T00:00:00Z","5341":"2016-12-19T00:00:00Z","5342":"2016-12-19T00:00:00Z","5343":"2016-12-19T00:00:00Z","5344":"2016-12-19T00:00:00Z","5345":"2016-12-15T00:00:00Z","5346":"2016-12-15T00:00:00Z","5347":"2016-12-15T00:00:00Z","5348":"2016-12-14T00:00:00Z","5349":"2016-12-14T00:00:00Z","5350":"2016-12-14T00:00:00Z","5351":"2016-12-14T00:00:00Z","5352":"2016-12-13T00:00:00Z","5353":"2016-12-13T00:00:00Z","5354":"2016-12-12T00:00:00Z","5355":"2016-12-12T00:00:00Z","5357":"2016-12-12T00:00:00Z","5358":"2016-12-12T00:00:00Z","5359":"2016-12-11T00:00:00Z","5360":"2016-12-11T00:00:00Z","5361":"2016-12-10T00:00:00Z","5362":"2016-12-10T00:00:00Z","5363":"2016-12-10T00:00:00Z","5364":"2016-12-10T00:00:00Z","5365":"2016-12-10T00:00:00Z","5367":"2016-12-09T00:00:00Z","5368":"2016-12-09T00:00:00Z","5372":"2016-12-09T00:00:00Z","5373":"2016-12-09T00:00:00Z","5374":"2016-12-08T00:00:00Z","5375":"2016-12-08T00:00:00Z","5376":"2016-12-08T00:00:00Z","5377":"2016-12-08T00:00:00Z","5378":"2016-12-07T00:00:00Z","5379":"2016-12-07T00:00:00Z","5380":"2016-12-07T00:00:00Z","5381":"2016-12-07T00:00:00Z","5382":"2016-12-07T00:00:00Z","5383":"2016-12-06T00:00:00Z","5384":"2016-12-06T00:00:00Z","5385":"2016-12-06T00:00:00Z","5391":"2016-12-05T00:00:00Z","5392":"2016-12-05T00:00:00Z","5393":"2016-12-05T00:00:00Z","5394":"2016-12-04T00:00:00Z","5395":"2016-12-04T00:00:00Z","5396":"2016-12-03T00:00:00Z","5397":"2016-12-03T00:00:00Z","5398":"2016-11-30T00:00:00Z","5399":"2016-11-29T00:00:00Z","5400":"2016-11-29T00:00:00Z","5401":"2016-11-28T00:00:00Z","5402":"2016-11-28T00:00:00Z","5403":"2016-11-27T00:00:00Z","5404":"2016-11-26T00:00:00Z","5405":"2016-11-23T00:00:00Z","5406":"2016-11-23T00:00:00Z","5407":"2016-11-23T00:00:00Z","5408":"2016-11-23T00:00:00Z","5409":"2016-11-23T00:00:00Z","5410":"2016-11-23T00:00:00Z","5412":"2016-11-23T00:00:00Z","5413":"2016-11-22T00:00:00Z","5414":"2016-11-22T00:00:00Z","5415":"2016-11-22T00:00:00Z","5416":"2016-11-22T00:00:00Z","5417":"2016-11-22T00:00:00Z","5420":"2016-11-21T00:00:00Z","5422":"2016-11-21T00:00:00Z","5423":"2016-11-21T00:00:00Z","5424":"2016-11-21T00:00:00Z","5425":"2016-11-21T00:00:00Z","5426":"2016-11-21T00:00:00Z","5427":"2016-11-21T00:00:00Z","5428":"2016-11-21T00:00:00Z","5429":"2016-11-21T00:00:00Z","5430":"2016-11-19T00:00:00Z","5431":"2016-11-18T00:00:00Z","5432":"2016-11-18T00:00:00Z","5433":"2016-11-18T00:00:00Z","5435":"2016-11-18T00:00:00Z","5436":"2016-11-18T00:00:00Z","5437":"2016-11-18T00:00:00Z","5438":"2016-11-18T00:00:00Z","5439":"2016-11-17T00:00:00Z","5440":"2016-11-16T00:00:00Z","5441":"2016-11-16T00:00:00Z","5442":"2016-11-15T00:00:00Z","5443":"2016-11-15T00:00:00Z","5444":"2016-11-15T00:00:00Z","5445":"2016-11-15T00:00:00Z","5446":"2016-11-15T00:00:00Z","5447":"2016-11-14T00:00:00Z","5448":"2016-11-14T00:00:00Z","5449":"2016-11-14T00:00:00Z","5450":"2016-11-14T00:00:00Z","5451":"2016-11-14T00:00:00Z","5452":"2016-11-14T00:00:00Z","5453":"2016-11-14T00:00:00Z","5456":"2016-11-13T00:00:00Z","5457":"2016-11-13T00:00:00Z","5458":"2016-11-13T00:00:00Z","5459":"2016-11-13T00:00:00Z","5460":"2016-11-13T00:00:00Z","5461":"2016-11-13T00:00:00Z","5463":"2016-11-12T00:00:00Z","5464":"2016-11-12T00:00:00Z","5465":"2016-11-11T00:00:00Z","5466":"2016-11-11T00:00:00Z","5467":"2016-11-11T00:00:00Z","5468":"2016-11-11T00:00:00Z","5469":"2016-11-11T00:00:00Z","5470":"2016-11-11T00:00:00Z","5471":"2016-11-11T00:00:00Z","5473":"2016-11-10T00:00:00Z","5475":"2016-11-10T00:00:00Z","5476":"2016-11-09T00:00:00Z","5477":"2016-11-09T00:00:00Z","5478":"2016-11-08T00:00:00Z","5479":"2016-11-07T00:00:00Z","5480":"2016-11-07T00:00:00Z","5481":"2016-11-07T00:00:00Z","5483":"2016-11-06T00:00:00Z","5484":"2016-11-06T00:00:00Z","5485":"2016-11-06T00:00:00Z","5486":"2016-11-06T00:00:00Z","5487":"2016-11-05T00:00:00Z","5488":"2016-11-05T00:00:00Z","5489":"2016-11-05T00:00:00Z","5490":"2016-11-05T00:00:00Z","5491":"2016-11-04T00:00:00Z","5492":"2016-11-04T00:00:00Z","5493":"2016-11-03T00:00:00Z","5494":"2016-11-03T00:00:00Z","5495":"2016-11-03T00:00:00Z","5496":"2016-11-03T00:00:00Z","5498":"2016-11-02T00:00:00Z","5499":"2016-11-02T00:00:00Z","5500":"2016-11-02T00:00:00Z","5501":"2016-11-02T00:00:00Z","5502":"2016-11-02T00:00:00Z","5503":"2016-11-02T00:00:00Z","5506":"2016-11-01T00:00:00Z","5507":"2016-11-01T00:00:00Z","5508":"2016-10-31T00:00:00Z","5509":"2016-10-30T00:00:00Z","5510":"2016-10-30T00:00:00Z","5511":"2016-10-29T00:00:00Z","5512":"2016-10-29T00:00:00Z","5513":"2016-10-29T00:00:00Z","5514":"2016-10-29T00:00:00Z","5515":"2016-10-28T00:00:00Z","5516":"2016-10-28T00:00:00Z","5517":"2016-10-28T00:00:00Z","5518":"2016-10-27T00:00:00Z","5519":"2016-10-27T00:00:00Z","5520":"2016-10-26T00:00:00Z","5521":"2016-10-26T00:00:00Z","5522":"2016-10-25T00:00:00Z","5524":"2016-10-24T00:00:00Z","5525":"2016-10-24T00:00:00Z","5526":"2016-10-24T00:00:00Z","5527":"2016-10-24T00:00:00Z","5528":"2016-10-23T00:00:00Z","5529":"2016-10-23T00:00:00Z","5530":"2016-10-23T00:00:00Z","5531":"2016-10-23T00:00:00Z","5532":"2016-10-23T00:00:00Z","5533":"2016-10-23T00:00:00Z","5534":"2016-10-23T00:00:00Z","5535":"2016-10-23T00:00:00Z","5536":"2016-10-23T00:00:00Z","5537":"2016-10-23T00:00:00Z","5538":"2016-10-22T00:00:00Z","5539":"2016-10-22T00:00:00Z","5540":"2016-10-21T00:00:00Z","5541":"2016-10-21T00:00:00Z","5542":"2016-10-21T00:00:00Z","5545":"2016-10-20T00:00:00Z","5546":"2016-10-20T00:00:00Z","5547":"2016-10-20T00:00:00Z","5548":"2016-10-20T00:00:00Z","5550":"2016-10-19T00:00:00Z","5551":"2016-10-19T00:00:00Z","5552":"2016-10-19T00:00:00Z","5553":"2016-10-19T00:00:00Z","5554":"2016-10-19T00:00:00Z","5557":"2016-10-19T00:00:00Z","5559":"2016-10-17T00:00:00Z","5560":"2016-10-17T00:00:00Z","5561":"2016-10-16T00:00:00Z","5562":"2016-10-16T00:00:00Z","5563":"2016-10-16T00:00:00Z","5564":"2016-10-16T00:00:00Z","5565":"2016-10-16T00:00:00Z","5566":"2016-10-15T00:00:00Z","5567":"2016-10-15T00:00:00Z","5568":"2016-10-15T00:00:00Z","5569":"2016-10-15T00:00:00Z","5570":"2016-10-15T00:00:00Z","5571":"2016-10-15T00:00:00Z","5572":"2016-10-14T00:00:00Z","5573":"2016-10-14T00:00:00Z","5574":"2016-10-14T00:00:00Z","5575":"2016-10-12T00:00:00Z","5576":"2016-10-11T00:00:00Z","5577":"2016-10-11T00:00:00Z","5578":"2016-10-11T00:00:00Z","5579":"2016-10-10T00:00:00Z","5580":"2016-10-10T00:00:00Z","5581":"2016-10-10T00:00:00Z","5582":"2016-10-10T00:00:00Z","5583":"2016-10-09T00:00:00Z","5584":"2016-10-09T00:00:00Z","5585":"2016-10-09T00:00:00Z","5586":"2016-10-09T00:00:00Z","5587":"2016-10-09T00:00:00Z","5588":"2016-10-09T00:00:00Z","5589":"2016-10-08T00:00:00Z","5590":"2016-10-08T00:00:00Z","5591":"2016-10-08T00:00:00Z","5592":"2016-10-08T00:00:00Z","5593":"2016-10-08T00:00:00Z","5594":"2016-10-08T00:00:00Z","5595":"2016-10-07T00:00:00Z","5597":"2016-10-07T00:00:00Z","5598":"2016-10-07T00:00:00Z","5599":"2016-10-06T00:00:00Z","5600":"2016-10-06T00:00:00Z","5601":"2016-10-06T00:00:00Z","5602":"2016-10-05T00:00:00Z","5603":"2016-10-05T00:00:00Z","5604":"2016-10-05T00:00:00Z","5606":"2016-10-05T00:00:00Z","5607":"2016-10-05T00:00:00Z","5608":"2016-10-05T00:00:00Z","5609":"2016-10-05T00:00:00Z","5610":"2016-10-04T00:00:00Z","5612":"2016-10-04T00:00:00Z","5614":"2016-10-04T00:00:00Z","5617":"2016-10-03T00:00:00Z","5622":"2016-10-03T00:00:00Z","5623":"2016-10-03T00:00:00Z","5624":"2016-10-03T00:00:00Z","5625":"2016-10-03T00:00:00Z","5626":"2016-10-03T00:00:00Z","5627":"2016-10-02T00:00:00Z","5628":"2016-10-01T00:00:00Z","5629":"2016-10-01T00:00:00Z","5630":"2016-09-30T00:00:00Z","5631":"2016-09-30T00:00:00Z","5632":"2016-09-30T00:00:00Z","5633":"2016-09-29T00:00:00Z","5634":"2016-09-29T00:00:00Z","5635":"2016-09-28T00:00:00Z","5636":"2016-09-28T00:00:00Z","5637":"2016-09-27T00:00:00Z","5638":"2016-09-27T00:00:00Z","5639":"2016-09-27T00:00:00Z","5640":"2016-09-27T00:00:00Z","5641":"2016-09-27T00:00:00Z","5642":"2016-09-27T00:00:00Z","5644":"2016-09-26T00:00:00Z","5646":"2016-09-26T00:00:00Z","5647":"2016-09-26T00:00:00Z","5648":"2016-09-26T00:00:00Z","5649":"2016-09-26T00:00:00Z","5650":"2016-09-26T00:00:00Z","5651":"2016-09-25T00:00:00Z","5652":"2016-09-25T00:00:00Z","5653":"2016-09-25T00:00:00Z","5654":"2016-09-25T00:00:00Z","5655":"2016-09-25T00:00:00Z","5656":"2016-09-25T00:00:00Z","5657":"2016-09-24T00:00:00Z","5658":"2016-09-24T00:00:00Z","5659":"2016-09-24T00:00:00Z","5660":"2016-09-23T00:00:00Z","5661":"2016-09-23T00:00:00Z","5662":"2016-09-22T00:00:00Z","5663":"2016-09-22T00:00:00Z","5664":"2016-09-20T00:00:00Z","5665":"2016-09-20T00:00:00Z","5666":"2016-09-20T00:00:00Z","5668":"2016-09-19T00:00:00Z","5670":"2016-09-19T00:00:00Z","5671":"2016-09-18T00:00:00Z","5672":"2016-09-18T00:00:00Z","5673":"2016-09-17T00:00:00Z","5674":"2016-09-17T00:00:00Z","5675":"2016-09-17T00:00:00Z","5676":"2016-09-17T00:00:00Z","5677":"2016-09-17T00:00:00Z","5678":"2016-09-17T00:00:00Z","5679":"2016-09-17T00:00:00Z","5680":"2016-09-17T00:00:00Z","5681":"2016-09-15T00:00:00Z","5682":"2016-09-15T00:00:00Z","5683":"2016-09-14T00:00:00Z","5684":"2016-09-14T00:00:00Z","5685":"2016-09-14T00:00:00Z","5686":"2016-09-14T00:00:00Z","5687":"2016-09-14T00:00:00Z","5688":"2016-09-14T00:00:00Z","5689":"2016-09-14T00:00:00Z","5690":"2016-09-13T00:00:00Z","5691":"2016-09-13T00:00:00Z","5692":"2016-09-13T00:00:00Z","5693":"2016-09-13T00:00:00Z","5694":"2016-09-12T00:00:00Z","5695":"2016-09-12T00:00:00Z","5696":"2016-09-12T00:00:00Z","5698":"2016-09-12T00:00:00Z","5700":"2016-09-12T00:00:00Z","5701":"2016-09-12T00:00:00Z","5702":"2016-09-12T00:00:00Z","5704":"2016-09-12T00:00:00Z","5705":"2016-09-12T00:00:00Z","5706":"2016-09-12T00:00:00Z","5707":"2016-09-12T00:00:00Z","5712":"2016-09-08T00:00:00Z","5713":"2016-09-08T00:00:00Z","5714":"2016-09-08T00:00:00Z","5715":"2016-09-08T00:00:00Z","5718":"2016-09-06T00:00:00Z","5719":"2016-09-06T00:00:00Z","5721":"2016-09-03T00:00:00Z","5722":"2016-09-02T00:00:00Z","5723":"2016-09-02T00:00:00Z","5724":"2016-09-01T00:00:00Z","5725":"2016-09-01T00:00:00Z","5727":"2016-09-01T00:00:00Z","5729":"2016-09-01T00:00:00Z","5730":"2016-09-01T00:00:00Z","5731":"2016-08-31T00:00:00Z","5732":"2016-08-31T00:00:00Z","5733":"2016-08-31T00:00:00Z","5734":"2016-08-31T00:00:00Z","5735":"2016-08-31T00:00:00Z","5736":"2016-08-31T00:00:00Z","5737":"2016-08-31T00:00:00Z","5738":"2016-08-31T00:00:00Z","5739":"2016-08-31T00:00:00Z","5740":"2016-08-31T00:00:00Z","5741":"2016-08-30T00:00:00Z","5742":"2016-08-30T00:00:00Z","5744":"2016-08-30T00:00:00Z","5745":"2016-08-30T00:00:00Z","5746":"2016-08-30T00:00:00Z","5747":"2016-08-30T00:00:00Z","5748":"2016-08-30T00:00:00Z","5749":"2016-08-29T00:00:00Z","5750":"2016-08-29T00:00:00Z","5751":"2016-08-29T00:00:00Z","5752":"2016-08-29T00:00:00Z","5753":"2016-08-29T00:00:00Z","5754":"2016-08-29T00:00:00Z","5755":"2016-08-19T00:00:00Z","5756":"2016-08-19T00:00:00Z","5758":"2016-08-17T00:00:00Z","5759":"2016-08-17T00:00:00Z","5760":"2016-08-17T00:00:00Z","5761":"2016-08-17T00:00:00Z","5762":"2016-08-17T00:00:00Z","5764":"2016-08-16T00:00:00Z","5765":"2016-08-15T00:00:00Z","5766":"2016-08-15T00:00:00Z","5767":"2016-08-13T00:00:00Z","5768":"2016-08-13T00:00:00Z","5769":"2016-08-13T00:00:00Z","5770":"2016-08-13T00:00:00Z","5771":"2016-08-12T00:00:00Z","5772":"2016-08-12T00:00:00Z","5773":"2016-08-12T00:00:00Z","5774":"2016-08-12T00:00:00Z","5775":"2016-08-12T00:00:00Z","5778":"2016-08-11T00:00:00Z","5779":"2016-08-11T00:00:00Z","5780":"2016-08-11T00:00:00Z","5781":"2016-08-11T00:00:00Z","5782":"2016-08-11T00:00:00Z","5783":"2016-08-10T00:00:00Z","5785":"2016-08-10T00:00:00Z","5786":"2016-08-10T00:00:00Z","5787":"2016-08-10T00:00:00Z","5788":"2016-08-10T00:00:00Z","5789":"2016-08-09T00:00:00Z","5790":"2016-08-09T00:00:00Z","5791":"2016-08-09T00:00:00Z","5792":"2016-08-09T00:00:00Z","5793":"2016-08-08T00:00:00Z","5795":"2016-08-08T00:00:00Z","5796":"2016-08-08T00:00:00Z","5797":"2016-08-08T00:00:00Z","5798":"2016-08-08T00:00:00Z","5799":"2016-08-07T00:00:00Z","5800":"2016-08-06T00:00:00Z","5801":"2016-08-06T00:00:00Z","5802":"2016-08-06T00:00:00Z","5803":"2016-08-06T00:00:00Z","5804":"2016-08-06T00:00:00Z","5805":"2016-08-06T00:00:00Z","5806":"2016-08-06T00:00:00Z","5807":"2016-08-06T00:00:00Z","5808":"2016-08-05T00:00:00Z","5809":"2016-08-05T00:00:00Z","5810":"2016-08-04T00:00:00Z","5811":"2016-08-04T00:00:00Z","5812":"2016-08-04T00:00:00Z","5813":"2016-08-04T00:00:00Z","5814":"2016-08-04T00:00:00Z","5816":"2016-08-04T00:00:00Z","5817":"2016-08-03T00:00:00Z","5819":"2016-08-03T00:00:00Z","5820":"2016-08-03T00:00:00Z","5821":"2016-08-03T00:00:00Z","5822":"2016-08-03T00:00:00Z","5823":"2016-08-03T00:00:00Z","5824":"2016-08-03T00:00:00Z","5825":"2016-08-03T00:00:00Z","5826":"2016-08-03T00:00:00Z","5827":"2016-08-02T00:00:00Z","5828":"2016-08-02T00:00:00Z","5829":"2016-08-02T00:00:00Z","5831":"2016-08-01T00:00:00Z","5832":"2016-08-01T00:00:00Z","5833":"2016-08-01T00:00:00Z","5834":"2016-07-30T00:00:00Z","5835":"2016-07-30T00:00:00Z","5836":"2016-07-29T00:00:00Z","5837":"2016-07-29T00:00:00Z","5839":"2016-07-28T00:00:00Z","5840":"2016-07-28T00:00:00Z","5841":"2016-07-28T00:00:00Z","5842":"2016-07-28T00:00:00Z","5843":"2016-07-28T00:00:00Z","5844":"2016-07-28T00:00:00Z","5845":"2016-07-28T00:00:00Z","5847":"2016-07-27T00:00:00Z","5848":"2016-07-27T00:00:00Z","5849":"2016-07-27T00:00:00Z","5850":"2016-07-27T00:00:00Z","5851":"2016-07-27T00:00:00Z","5852":"2016-07-27T00:00:00Z","5853":"2016-07-27T00:00:00Z","5854":"2016-07-27T00:00:00Z","5855":"2016-07-27T00:00:00Z","5856":"2016-07-26T00:00:00Z","5857":"2016-07-26T00:00:00Z","5858":"2016-07-26T00:00:00Z","5859":"2016-07-26T00:00:00Z","5860":"2016-07-26T00:00:00Z","5861":"2016-07-26T00:00:00Z","5862":"2016-07-25T00:00:00Z","5863":"2016-07-25T00:00:00Z","5864":"2016-07-25T00:00:00Z","5866":"2016-07-25T00:00:00Z","5867":"2016-07-25T00:00:00Z","5869":"2016-07-25T00:00:00Z","5870":"2016-07-25T00:00:00Z","5871":"2016-07-25T00:00:00Z","5872":"2016-07-25T00:00:00Z","5874":"2016-07-25T00:00:00Z","5875":"2016-07-25T00:00:00Z","5876":"2016-07-25T00:00:00Z","5877":"2016-07-22T00:00:00Z","5878":"2016-07-22T00:00:00Z","5879":"2016-07-18T00:00:00Z","5880":"2016-07-18T00:00:00Z","5882":"2016-07-13T00:00:00Z","5883":"2016-07-13T00:00:00Z","5884":"2016-07-13T00:00:00Z","5887":"2016-07-11T00:00:00Z","5889":"2016-07-08T00:00:00Z","5890":"2016-07-08T00:00:00Z","5891":"2016-07-08T00:00:00Z","5892":"2016-07-08T00:00:00Z","5893":"2016-07-08T00:00:00Z","5894":"2016-07-08T00:00:00Z","5895":"2016-07-07T00:00:00Z","5896":"2016-07-07T00:00:00Z","5897":"2016-07-07T00:00:00Z","5898":"2016-07-07T00:00:00Z","5899":"2016-07-06T00:00:00Z","5900":"2016-07-05T00:00:00Z","5902":"2016-07-05T00:00:00Z","5903":"2016-07-05T00:00:00Z","5906":"2016-06-17T00:00:00Z","5907":"2016-06-17T00:00:00Z","5908":"2016-06-17T00:00:00Z","5909":"2016-06-16T00:00:00Z","5910":"2016-06-15T00:00:00Z","5911":"2016-06-13T00:00:00Z","5912":"2016-06-13T00:00:00Z","5913":"2016-06-13T00:00:00Z","5915":"2016-06-13T00:00:00Z","5917":"2016-06-13T00:00:00Z","5918":"2016-06-13T00:00:00Z","5919":"2016-06-13T00:00:00Z","5922":"2016-06-10T00:00:00Z","5923":"2016-06-10T00:00:00Z","5924":"2016-06-10T00:00:00Z","5925":"2016-06-10T00:00:00Z","5926":"2016-06-10T00:00:00Z","5927":"2016-06-10T00:00:00Z","5929":"2016-06-09T00:00:00Z","5931":"2016-06-09T00:00:00Z","5933":"2016-06-08T00:00:00Z","5934":"2016-06-07T00:00:00Z","5935":"2016-06-07T00:00:00Z","5936":"2016-06-07T00:00:00Z","5937":"2016-06-07T00:00:00Z","5938":"2016-06-07T00:00:00Z","5940":"2016-06-06T00:00:00Z","5941":"2016-06-06T00:00:00Z","5943":"2016-06-05T00:00:00Z","5944":"2016-06-05T00:00:00Z","5945":"2016-06-04T00:00:00Z","5946":"2016-06-03T00:00:00Z","5947":"2016-06-03T00:00:00Z","5949":"2016-06-02T00:00:00Z","5950":"2016-06-02T00:00:00Z","5951":"2016-06-02T00:00:00Z","5952":"2016-06-01T00:00:00Z","5953":"2016-06-01T00:00:00Z","5954":"2016-05-30T00:00:00Z","5955":"2016-05-30T00:00:00Z","5956":"2016-05-30T00:00:00Z","5957":"2016-05-30T00:00:00Z","5958":"2016-05-29T00:00:00Z","5959":"2016-05-29T00:00:00Z","5960":"2016-05-27T00:00:00Z","5961":"2016-05-26T00:00:00Z","5968":"2016-05-23T00:00:00Z","5969":"2016-05-23T00:00:00Z","5971":"2016-05-23T00:00:00Z","5972":"2016-05-22T00:00:00Z","5973":"2016-05-21T00:00:00Z","5974":"2016-05-21T00:00:00Z","5975":"2016-05-20T00:00:00Z","5976":"2016-05-20T00:00:00Z","5977":"2016-05-20T00:00:00Z","5978":"2016-05-19T00:00:00Z","5979":"2016-05-18T00:00:00Z","5982":"2016-05-16T00:00:00Z","5983":"2016-05-16T00:00:00Z","5984":"2016-05-16T00:00:00Z","5985":"2016-05-16T00:00:00Z","5986":"2016-05-16T00:00:00Z","5987":"2016-05-16T00:00:00Z","5988":"2016-05-16T00:00:00Z","5989":"2016-05-13T00:00:00Z","5990":"2016-05-13T00:00:00Z","5991":"2016-05-13T00:00:00Z","5992":"2016-05-13T00:00:00Z","5993":"2016-05-13T00:00:00Z","5994":"2016-05-12T00:00:00Z","5995":"2016-05-12T00:00:00Z","5996":"2016-05-12T00:00:00Z","5997":"2016-05-12T00:00:00Z","5998":"2016-05-12T00:00:00Z","5999":"2016-05-11T00:00:00Z","6000":"2016-05-11T00:00:00Z","6001":"2016-05-11T00:00:00Z","6002":"2016-05-10T00:00:00Z","6006":"2016-05-10T00:00:00Z","6007":"2016-05-10T00:00:00Z","6010":"2016-05-09T00:00:00Z","6012":"2016-05-09T00:00:00Z","6013":"2016-05-09T00:00:00Z","6014":"2016-05-09T00:00:00Z","6016":"2016-05-09T00:00:00Z","6017":"2016-05-06T00:00:00Z","6018":"2016-05-06T00:00:00Z","6019":"2016-05-06T00:00:00Z","6020":"2016-05-06T00:00:00Z","6021":"2016-05-06T00:00:00Z","6022":"2016-05-05T00:00:00Z","6023":"2016-05-05T00:00:00Z","6024":"2016-05-05T00:00:00Z","6025":"2016-05-05T00:00:00Z","6026":"2016-05-05T00:00:00Z","6027":"2016-05-05T00:00:00Z","6029":"2016-05-04T00:00:00Z","6030":"2016-05-03T00:00:00Z","6031":"2016-05-03T00:00:00Z","6032":"2016-05-03T00:00:00Z","6033":"2016-05-02T00:00:00Z","6034":"2016-05-02T00:00:00Z","6035":"2016-05-02T00:00:00Z","6036":"2016-05-02T00:00:00Z","6037":"2016-05-02T00:00:00Z","6038":"2016-05-02T00:00:00Z","6039":"2016-05-02T00:00:00Z","6040":"2016-05-02T00:00:00Z","6041":"2016-05-02T00:00:00Z","6042":"2016-05-02T00:00:00Z","6043":"2016-05-02T00:00:00Z","6044":"2016-05-02T00:00:00Z","6045":"2016-05-02T00:00:00Z","6046":"2016-04-29T00:00:00Z","6047":"2016-04-29T00:00:00Z","6048":"2016-04-28T00:00:00Z","6049":"2016-04-28T00:00:00Z","6050":"2016-04-28T00:00:00Z","6051":"2016-04-27T00:00:00Z","6052":"2016-04-27T00:00:00Z","6053":"2016-04-27T00:00:00Z","6054":"2016-04-27T00:00:00Z","6055":"2016-04-26T00:00:00Z","6056":"2016-04-25T00:00:00Z","6057":"2016-04-25T00:00:00Z","6059":"2016-04-25T00:00:00Z","6060":"2016-04-25T00:00:00Z","6061":"2016-04-22T00:00:00Z","6062":"2016-04-21T00:00:00Z","6063":"2016-04-21T00:00:00Z","6064":"2016-04-21T00:00:00Z","6065":"2016-04-21T00:00:00Z","6066":"2016-04-20T00:00:00Z","6067":"2016-04-20T00:00:00Z","6068":"2016-04-20T00:00:00Z","6069":"2016-04-19T00:00:00Z","6070":"2016-04-19T00:00:00Z","6071":"2016-04-19T00:00:00Z","6072":"2016-04-19T00:00:00Z","6073":"2016-04-18T00:00:00Z","6074":"2016-04-18T00:00:00Z","6075":"2016-04-18T00:00:00Z","6076":"2016-04-18T00:00:00Z","6077":"2016-04-18T00:00:00Z","6078":"2016-04-18T00:00:00Z","6079":"2016-04-18T00:00:00Z","6080":"2016-04-18T00:00:00Z","6081":"2016-04-18T00:00:00Z","6082":"2016-04-15T00:00:00Z","6083":"2016-04-15T00:00:00Z","6084":"2016-04-14T00:00:00Z","6085":"2016-04-12T00:00:00Z","6087":"2016-04-11T00:00:00Z","6088":"2016-04-11T00:00:00Z","6089":"2016-04-11T00:00:00Z","6090":"2016-04-11T00:00:00Z","6091":"2016-04-11T00:00:00Z","6092":"2016-04-11T00:00:00Z","6093":"2016-04-11T00:00:00Z","6094":"2016-04-11T00:00:00Z","6095":"2016-04-11T00:00:00Z","6096":"2016-04-11T00:00:00Z","6097":"2016-04-11T00:00:00Z","6098":"2016-04-11T00:00:00Z","6099":"2016-04-08T00:00:00Z","6100":"2016-04-08T00:00:00Z","6101":"2016-04-08T00:00:00Z","6102":"2016-04-08T00:00:00Z","6103":"2016-04-08T00:00:00Z","6104":"2016-04-07T00:00:00Z","6105":"2016-04-06T00:00:00Z","6106":"2016-04-06T00:00:00Z","6107":"2016-04-06T00:00:00Z","6108":"2016-04-05T00:00:00Z","6109":"2016-04-05T00:00:00Z","6110":"2016-04-05T00:00:00Z","6111":"2016-04-04T00:00:00Z","6112":"2016-04-04T00:00:00Z","6113":"2016-04-04T00:00:00Z","6114":"2016-04-04T00:00:00Z","6115":"2016-04-04T00:00:00Z","6116":"2016-04-04T00:00:00Z","6117":"2016-04-04T00:00:00Z","6118":"2016-04-04T00:00:00Z","6119":"2016-04-04T00:00:00Z","6120":"2016-04-04T00:00:00Z","6121":"2016-04-04T00:00:00Z","6123":"2016-04-04T00:00:00Z","6124":"2016-04-04T00:00:00Z","6126":"2016-04-01T00:00:00Z","6127":"2016-04-01T00:00:00Z","6128":"2016-03-31T00:00:00Z","6130":"2016-03-31T00:00:00Z","6131":"2016-03-30T00:00:00Z","6132":"2016-03-29T00:00:00Z","6133":"2016-03-29T00:00:00Z","6134":"2016-03-29T00:00:00Z","6135":"2016-03-29T00:00:00Z","6136":"2016-03-28T00:00:00Z","6137":"2016-03-28T00:00:00Z","6138":"2016-03-28T00:00:00Z","6139":"2016-03-28T00:00:00Z","6140":"2016-03-28T00:00:00Z","6141":"2016-03-25T00:00:00Z","6143":"2016-03-24T00:00:00Z","6144":"2016-03-24T00:00:00Z","6145":"2016-03-23T00:00:00Z","6146":"2016-03-22T00:00:00Z","6148":"2016-03-21T00:00:00Z","6149":"2016-03-21T00:00:00Z","6151":"2016-03-18T00:00:00Z","6152":"2016-03-18T00:00:00Z","6153":"2016-03-18T00:00:00Z","6154":"2016-03-17T00:00:00Z","6155":"2016-03-17T00:00:00Z","6156":"2016-03-17T00:00:00Z","6157":"2016-03-16T00:00:00Z","6158":"2016-03-16T00:00:00Z","6160":"2016-03-13T00:00:00Z","6161":"2016-03-12T00:00:00Z","6162":"2016-03-12T00:00:00Z","6163":"2016-03-12T00:00:00Z","6164":"2016-03-12T00:00:00Z","6165":"2016-03-12T00:00:00Z","6166":"2016-03-12T00:00:00Z","6167":"2016-03-12T00:00:00Z","6168":"2016-03-11T00:00:00Z","6169":"2016-03-11T00:00:00Z","6170":"2016-03-11T00:00:00Z","6171":"2016-03-11T00:00:00Z","6173":"2016-03-10T00:00:00Z","6175":"2016-03-09T00:00:00Z","6176":"2016-03-08T00:00:00Z","6177":"2016-03-07T00:00:00Z","6178":"2016-03-07T00:00:00Z","6179":"2016-03-07T00:00:00Z","6180":"2016-03-07T00:00:00Z","6181":"2016-03-07T00:00:00Z","6182":"2016-03-06T00:00:00Z","6183":"2016-03-06T00:00:00Z","6184":"2016-03-06T00:00:00Z","6185":"2016-03-05T00:00:00Z","6186":"2016-03-05T00:00:00Z","6187":"2016-03-05T00:00:00Z","6188":"2016-03-05T00:00:00Z","6189":"2016-03-05T00:00:00Z","6190":"2016-03-04T00:00:00Z","6191":"2016-03-04T00:00:00Z","6192":"2016-03-03T00:00:00Z","6194":"2016-03-02T00:00:00Z","6195":"2016-03-02T00:00:00Z","6197":"2016-03-02T00:00:00Z","6198":"2016-02-29T00:00:00Z","6199":"2016-02-29T00:00:00Z","6200":"2016-02-29T00:00:00Z","6201":"2016-02-29T00:00:00Z","6202":"2016-02-29T00:00:00Z","6203":"2016-02-28T00:00:00Z","6204":"2016-02-28T00:00:00Z","6205":"2016-02-28T00:00:00Z","6206":"2016-02-28T00:00:00Z","6207":"2016-02-28T00:00:00Z","6208":"2016-02-25T00:00:00Z","6210":"2016-02-24T00:00:00Z","6211":"2016-02-21T00:00:00Z","6212":"2016-02-21T00:00:00Z","6213":"2016-02-21T00:00:00Z","6214":"2016-02-21T00:00:00Z","6215":"2016-02-21T00:00:00Z","6216":"2016-02-20T00:00:00Z","6217":"2016-02-20T00:00:00Z","6218":"2016-02-20T00:00:00Z","6219":"2016-02-20T00:00:00Z","6220":"2016-02-20T00:00:00Z","6221":"2016-02-18T00:00:00Z","6222":"2016-02-18T00:00:00Z","6223":"2016-02-17T00:00:00Z","6225":"2016-02-11T00:00:00Z","6226":"2016-02-11T00:00:00Z","6227":"2016-02-11T00:00:00Z","6228":"2016-02-11T00:00:00Z","6229":"2016-02-10T00:00:00Z","6231":"2016-02-10T00:00:00Z","6232":"2016-02-08T00:00:00Z","6233":"2016-02-06T00:00:00Z","6234":"2016-02-06T00:00:00Z","6235":"2016-02-06T00:00:00Z","6236":"2016-02-06T00:00:00Z","6237":"2016-02-05T00:00:00Z","6238":"2016-02-05T00:00:00Z","6239":"2016-02-05T00:00:00Z","6240":"2016-02-05T00:00:00Z","6241":"2016-01-30T00:00:00Z","6242":"2016-01-29T00:00:00Z","6243":"2016-01-28T00:00:00Z","6244":"2016-01-28T00:00:00Z","6245":"2016-01-27T00:00:00Z","6246":"2016-01-26T00:00:00Z","6247":"2016-01-26T00:00:00Z","6248":"2016-01-17T00:00:00Z","6249":"2016-01-16T00:00:00Z","6250":"2016-01-16T00:00:00Z","6251":"2016-01-16T00:00:00Z","6252":"2016-01-16T00:00:00Z","6253":"2016-01-15T00:00:00Z","6254":"2016-01-15T00:00:00Z","6255":"2016-01-15T00:00:00Z","6256":"2016-01-14T00:00:00Z","6257":"2016-01-14T00:00:00Z","6258":"2016-01-14T00:00:00Z","6259":"2016-01-13T00:00:00Z","6260":"2016-01-12T00:00:00Z","6261":"2016-01-11T00:00:00Z","6262":"2016-01-11T00:00:00Z","6264":"2016-01-11T00:00:00Z","6267":"2016-01-10T00:00:00Z","6268":"2016-01-10T00:00:00Z","6269":"2016-01-10T00:00:00Z","6270":"2016-01-10T00:00:00Z","6271":"2016-01-10T00:00:00Z","6272":"2016-01-09T00:00:00Z","6274":"2016-01-07T00:00:00Z","6275":"2016-01-07T00:00:00Z","6276":"2016-01-07T00:00:00Z","6277":"2016-01-07T00:00:00Z","6278":"2016-01-06T00:00:00Z","6279":"2016-01-06T00:00:00Z","6280":"2016-01-06T00:00:00Z","6282":"2016-01-05T00:00:00Z","6285":"2016-01-04T00:00:00Z","6286":"2016-01-04T00:00:00Z","6287":"2016-01-04T00:00:00Z","6288":"2016-01-04T00:00:00Z","6289":"2016-01-04T00:00:00Z","6290":"2016-01-04T00:00:00Z","6292":"2016-01-04T00:00:00Z","6293":"2016-01-04T00:00:00Z","6294":"2016-01-04T00:00:00Z","6295":"2016-01-04T00:00:00Z","6296":"2016-01-04T00:00:00Z","6297":"2016-01-04T00:00:00Z","6298":"2015-12-31T00:00:00Z","6299":"2015-12-31T00:00:00Z","6300":"2015-12-31T00:00:00Z","6301":"2015-12-30T00:00:00Z","6302":"2015-12-30T00:00:00Z","6303":"2015-12-28T00:00:00Z","6305":"2015-12-23T00:00:00Z","6306":"2015-12-23T00:00:00Z","6307":"2015-12-23T00:00:00Z","6308":"2015-12-22T00:00:00Z","6309":"2015-12-22T00:00:00Z","6310":"2015-12-22T00:00:00Z","6311":"2015-12-22T00:00:00Z","6312":"2015-12-22T00:00:00Z","6313":"2015-12-22T00:00:00Z","6314":"2015-12-21T00:00:00Z","6315":"2015-12-21T00:00:00Z","6316":"2015-12-21T00:00:00Z","6317":"2015-12-21T00:00:00Z","6318":"2015-12-21T00:00:00Z","6319":"2015-12-21T00:00:00Z","6320":"2015-12-21T00:00:00Z","6322":"2015-12-21T00:00:00Z","6323":"2015-12-18T00:00:00Z","6324":"2015-12-17T00:00:00Z","6325":"2015-12-17T00:00:00Z","6326":"2015-12-17T00:00:00Z","6327":"2015-12-17T00:00:00Z","6328":"2015-12-17T00:00:00Z","6329":"2015-12-17T00:00:00Z","6330":"2015-12-17T00:00:00Z","6331":"2015-12-16T00:00:00Z","6332":"2015-12-16T00:00:00Z","6333":"2015-12-16T00:00:00Z","6334":"2015-12-15T00:00:00Z","6335":"2015-12-15T00:00:00Z","6336":"2015-12-14T00:00:00Z","6337":"2015-12-14T00:00:00Z","6338":"2015-12-14T00:00:00Z","6339":"2015-12-14T00:00:00Z","6340":"2015-12-14T00:00:00Z","6341":"2015-12-14T00:00:00Z","6342":"2015-12-14T00:00:00Z","6343":"2015-12-14T00:00:00Z","6344":"2015-12-14T00:00:00Z","6345":"2015-12-12T00:00:00Z","6346":"2015-12-11T00:00:00Z","6348":"2015-12-11T00:00:00Z","6349":"2015-12-10T00:00:00Z","6350":"2015-12-10T00:00:00Z","6351":"2015-12-10T00:00:00Z","6352":"2015-12-10T00:00:00Z","6353":"2015-12-10T00:00:00Z","6355":"2015-12-09T00:00:00Z","6356":"2015-12-08T00:00:00Z","6357":"2015-12-08T00:00:00Z","6358":"2015-12-07T00:00:00Z","6359":"2015-12-07T00:00:00Z","6360":"2015-12-07T00:00:00Z","6361":"2015-12-07T00:00:00Z","6362":"2015-12-07T00:00:00Z","6363":"2015-12-07T00:00:00Z","6364":"2015-12-07T00:00:00Z","6365":"2015-12-07T00:00:00Z","6366":"2015-12-04T00:00:00Z","6367":"2015-12-04T00:00:00Z","6373":"2015-12-02T00:00:00Z","6374":"2015-12-02T00:00:00Z","6375":"2015-12-01T00:00:00Z","6376":"2015-11-30T00:00:00Z","6377":"2015-11-30T00:00:00Z","6378":"2015-11-30T00:00:00Z","6379":"2015-11-30T00:00:00Z","6380":"2015-11-29T00:00:00Z","6381":"2015-11-28T00:00:00Z","6382":"2015-11-28T00:00:00Z","6383":"2015-11-28T00:00:00Z","6384":"2015-11-28T00:00:00Z","6385":"2015-11-28T00:00:00Z","6386":"2015-11-27T00:00:00Z","6387":"2015-11-27T00:00:00Z","6388":"2015-11-27T00:00:00Z","6389":"2015-11-27T00:00:00Z","6390":"2015-11-27T00:00:00Z","6391":"2015-11-27T00:00:00Z","6392":"2015-11-26T00:00:00Z","6393":"2015-11-26T00:00:00Z","6394":"2015-11-25T00:00:00Z","6395":"2015-11-25T00:00:00Z","6396":"2015-11-25T00:00:00Z","6398":"2015-11-24T00:00:00Z","6401":"2015-11-23T00:00:00Z","6402":"2015-11-23T00:00:00Z","6403":"2015-11-23T00:00:00Z","6404":"2015-11-23T00:00:00Z","6405":"2015-11-23T00:00:00Z","6406":"2015-11-23T00:00:00Z","6407":"2015-11-23T00:00:00Z","6408":"2015-11-21T00:00:00Z","6409":"2015-11-21T00:00:00Z","6410":"2015-11-21T00:00:00Z","6411":"2015-11-21T00:00:00Z","6412":"2015-11-21T00:00:00Z","6413":"2015-11-20T00:00:00Z","6415":"2015-11-19T00:00:00Z","6416":"2015-11-18T00:00:00Z","6417":"2015-11-18T00:00:00Z","6418":"2015-11-18T00:00:00Z","6419":"2015-11-18T00:00:00Z","6420":"2015-11-18T00:00:00Z","6421":"2015-11-18T00:00:00Z","6422":"2015-11-17T00:00:00Z","6423":"2015-11-16T00:00:00Z","6424":"2015-11-16T00:00:00Z","6425":"2015-11-16T00:00:00Z","6426":"2015-11-16T00:00:00Z","6427":"2015-11-16T00:00:00Z","6428":"2015-11-16T00:00:00Z","6429":"2015-11-13T00:00:00Z","6430":"2015-11-12T00:00:00Z","6431":"2015-11-11T00:00:00Z","6432":"2015-11-11T00:00:00Z","6433":"2015-11-10T00:00:00Z","6435":"2015-11-10T00:00:00Z","6436":"2015-11-10T00:00:00Z","6437":"2015-11-09T00:00:00Z","6438":"2015-11-09T00:00:00Z","6439":"2015-11-09T00:00:00Z","6440":"2015-11-09T00:00:00Z","6441":"2015-11-09T00:00:00Z","6442":"2015-11-07T00:00:00Z","6443":"2015-11-06T00:00:00Z","6444":"2015-11-06T00:00:00Z","6445":"2015-11-06T00:00:00Z","6446":"2015-11-06T00:00:00Z","6447":"2015-11-06T00:00:00Z","6448":"2015-11-05T00:00:00Z","6449":"2015-11-05T00:00:00Z","6450":"2015-11-05T00:00:00Z","6451":"2015-11-05T00:00:00Z","6453":"2015-11-05T00:00:00Z","6454":"2015-11-05T00:00:00Z","6455":"2015-11-04T00:00:00Z","6456":"2015-11-04T00:00:00Z","6457":"2015-11-04T00:00:00Z","6458":"2015-11-04T00:00:00Z","6459":"2015-11-04T00:00:00Z","6460":"2015-11-04T00:00:00Z","6461":"2015-11-03T00:00:00Z","6462":"2015-11-03T00:00:00Z","6463":"2015-11-03T00:00:00Z","6464":"2015-11-03T00:00:00Z","6465":"2015-11-02T00:00:00Z","6466":"2015-11-02T00:00:00Z","6467":"2015-11-02T00:00:00Z","6468":"2015-11-02T00:00:00Z","6469":"2015-11-02T00:00:00Z","6470":"2015-11-02T00:00:00Z","6471":"2015-11-02T00:00:00Z","6472":"2015-11-02T00:00:00Z","6473":"2015-11-02T00:00:00Z","6474":"2015-11-02T00:00:00Z","6475":"2015-11-02T00:00:00Z","6476":"2015-11-02T00:00:00Z","6477":"2015-11-02T00:00:00Z","6478":"2015-11-02T00:00:00Z","6479":"2015-11-02T00:00:00Z","6480":"2015-10-30T00:00:00Z","6481":"2015-10-30T00:00:00Z","6482":"2015-10-30T00:00:00Z","6483":"2015-10-29T00:00:00Z","6484":"2015-10-29T00:00:00Z","6485":"2015-10-29T00:00:00Z","6486":"2015-10-29T00:00:00Z","6487":"2015-10-29T00:00:00Z","6488":"2015-10-28T00:00:00Z","6491":"2015-10-27T00:00:00Z","6492":"2015-10-26T00:00:00Z","6493":"2015-10-26T00:00:00Z","6495":"2015-10-26T00:00:00Z","6496":"2015-10-26T00:00:00Z","6497":"2015-10-26T00:00:00Z","6498":"2015-10-26T00:00:00Z","6499":"2015-10-26T00:00:00Z","6500":"2015-10-26T00:00:00Z","6501":"2015-10-25T00:00:00Z","6502":"2015-10-23T00:00:00Z","6503":"2015-10-23T00:00:00Z","6504":"2015-10-23T00:00:00Z","6505":"2015-10-22T00:00:00Z","6506":"2015-10-22T00:00:00Z","6507":"2015-10-21T00:00:00Z","6508":"2015-10-21T00:00:00Z","6509":"2015-10-21T00:00:00Z","6510":"2015-10-21T00:00:00Z","6511":"2015-10-21T00:00:00Z","6512":"2015-10-21T00:00:00Z","6513":"2015-10-21T00:00:00Z","6514":"2015-10-21T00:00:00Z","6515":"2015-10-21T00:00:00Z","6516":"2015-10-20T00:00:00Z","6517":"2015-10-20T00:00:00Z","6519":"2015-10-19T00:00:00Z","6520":"2015-10-19T00:00:00Z","6521":"2015-10-19T00:00:00Z","6522":"2015-10-19T00:00:00Z","6523":"2015-10-19T00:00:00Z","6524":"2015-10-19T00:00:00Z","6525":"2015-10-19T00:00:00Z","6526":"2015-10-19T00:00:00Z","6527":"2015-10-19T00:00:00Z","6528":"2015-10-16T00:00:00Z","6529":"2015-10-16T00:00:00Z","6530":"2015-10-16T00:00:00Z","6531":"2015-10-16T00:00:00Z","6532":"2015-10-16T00:00:00Z","6533":"2015-10-15T00:00:00Z","6534":"2015-10-15T00:00:00Z","6535":"2015-10-15T00:00:00Z","6536":"2015-10-15T00:00:00Z","6537":"2015-10-15T00:00:00Z","6538":"2015-10-15T00:00:00Z","6539":"2015-10-15T00:00:00Z","6540":"2015-10-14T00:00:00Z","6541":"2015-10-14T00:00:00Z","6542":"2015-10-14T00:00:00Z","6543":"2015-10-14T00:00:00Z","6544":"2015-10-13T00:00:00Z","6545":"2015-10-13T00:00:00Z","6546":"2015-10-13T00:00:00Z","6547":"2015-10-13T00:00:00Z","6548":"2015-10-13T00:00:00Z","6549":"2015-10-13T00:00:00Z","6550":"2015-10-13T00:00:00Z","6551":"2015-10-13T00:00:00Z","6552":"2015-10-13T00:00:00Z","6553":"2015-10-13T00:00:00Z","6554":"2015-10-12T00:00:00Z","6555":"2015-10-12T00:00:00Z","6556":"2015-10-12T00:00:00Z","6557":"2015-10-11T00:00:00Z","6558":"2015-10-10T00:00:00Z","6559":"2015-10-09T00:00:00Z","6561":"2015-10-08T00:00:00Z","6562":"2015-10-07T00:00:00Z","6563":"2015-10-07T00:00:00Z","6564":"2015-10-07T00:00:00Z","6565":"2015-10-06T00:00:00Z","6567":"2015-10-06T00:00:00Z","6569":"2015-10-05T00:00:00Z","6570":"2015-10-05T00:00:00Z","6571":"2015-10-05T00:00:00Z","6572":"2015-10-05T00:00:00Z","6573":"2015-10-05T00:00:00Z","6574":"2015-10-05T00:00:00Z","6575":"2015-10-05T00:00:00Z","6576":"2015-10-05T00:00:00Z","6577":"2015-10-02T00:00:00Z","6578":"2015-10-02T00:00:00Z","6580":"2015-10-01T00:00:00Z","6581":"2015-09-30T00:00:00Z","6582":"2015-09-30T00:00:00Z","6583":"2015-09-30T00:00:00Z","6584":"2015-09-30T00:00:00Z","6585":"2015-09-30T00:00:00Z","6586":"2015-09-29T00:00:00Z","6587":"2015-09-29T00:00:00Z","6588":"2015-09-29T00:00:00Z","6589":"2015-09-29T00:00:00Z","6590":"2015-09-29T00:00:00Z","6591":"2015-09-28T00:00:00Z","6592":"2015-09-28T00:00:00Z","6593":"2015-09-28T00:00:00Z","6594":"2015-09-28T00:00:00Z","6595":"2015-09-28T00:00:00Z","6596":"2015-09-28T00:00:00Z","6597":"2015-09-28T00:00:00Z","6598":"2015-09-25T00:00:00Z","6599":"2015-09-25T00:00:00Z","6600":"2015-09-25T00:00:00Z","6602":"2015-09-24T00:00:00Z","6603":"2015-09-24T00:00:00Z","6604":"2015-09-23T00:00:00Z","6605":"2015-09-23T00:00:00Z","6606":"2015-09-23T00:00:00Z","6607":"2015-09-22T00:00:00Z","6609":"2015-09-21T00:00:00Z","6610":"2015-09-21T00:00:00Z","6611":"2015-09-21T00:00:00Z","6612":"2015-09-21T00:00:00Z","6613":"2015-09-21T00:00:00Z","6614":"2015-09-21T00:00:00Z","6615":"2015-09-21T00:00:00Z","6616":"2015-09-21T00:00:00Z","6617":"2015-09-21T00:00:00Z","6618":"2015-09-21T00:00:00Z","6619":"2015-09-21T00:00:00Z","6620":"2015-09-21T00:00:00Z","6621":"2015-09-21T00:00:00Z","6622":"2015-09-21T00:00:00Z","6623":"2015-09-21T00:00:00Z","6624":"2015-09-21T00:00:00Z","6625":"2015-09-21T00:00:00Z","6627":"2015-09-21T00:00:00Z","6628":"2015-09-19T00:00:00Z","6629":"2015-09-19T00:00:00Z","6630":"2015-09-19T00:00:00Z","6631":"2015-09-18T00:00:00Z","6632":"2015-09-18T00:00:00Z","6633":"2015-09-18T00:00:00Z","6634":"2015-09-18T00:00:00Z","6635":"2015-09-18T00:00:00Z","6636":"2015-09-18T00:00:00Z","6637":"2015-09-16T00:00:00Z","6638":"2015-09-14T00:00:00Z","6639":"2015-09-14T00:00:00Z","6640":"2015-09-14T00:00:00Z","6641":"2015-09-13T00:00:00Z","6642":"2015-09-13T00:00:00Z","6643":"2015-09-12T00:00:00Z","6644":"2015-09-12T00:00:00Z","6645":"2015-09-12T00:00:00Z","6646":"2015-09-11T00:00:00Z","6647":"2015-09-11T00:00:00Z","6648":"2015-09-11T00:00:00Z","6649":"2015-09-11T00:00:00Z","6650":"2015-09-11T00:00:00Z","6652":"2015-09-10T00:00:00Z","6653":"2015-09-10T00:00:00Z","6654":"2015-09-10T00:00:00Z","6655":"2015-09-10T00:00:00Z","6656":"2015-09-10T00:00:00Z","6657":"2015-09-10T00:00:00Z","6658":"2015-09-10T00:00:00Z","6659":"2015-09-10T00:00:00Z","6660":"2015-09-10T00:00:00Z","6662":"2015-09-10T00:00:00Z","6664":"2015-09-10T00:00:00Z","6665":"2015-09-09T00:00:00Z","6666":"2015-09-08T00:00:00Z","6667":"2015-09-08T00:00:00Z","6668":"2015-09-08T00:00:00Z","6669":"2015-09-08T00:00:00Z","6670":"2015-09-08T00:00:00Z","6671":"2015-09-08T00:00:00Z","6672":"2015-09-08T00:00:00Z","6673":"2015-09-08T00:00:00Z","6674":"2015-09-08T00:00:00Z","6675":"2015-09-08T00:00:00Z","6676":"2015-09-08T00:00:00Z","6677":"2015-09-08T00:00:00Z","6678":"2015-09-08T00:00:00Z","6679":"2015-09-08T00:00:00Z","6680":"2015-09-08T00:00:00Z","6681":"2015-09-08T00:00:00Z","6682":"2015-09-04T00:00:00Z","6683":"2015-09-04T00:00:00Z","6684":"2015-09-04T00:00:00Z","6685":"2015-09-03T00:00:00Z","6687":"2015-09-02T00:00:00Z","6688":"2015-09-02T00:00:00Z","6690":"2015-09-02T00:00:00Z","6691":"2015-08-31T00:00:00Z","6692":"2015-08-31T00:00:00Z","6693":"2015-08-31T00:00:00Z","6694":"2015-08-31T00:00:00Z","6695":"2015-08-31T00:00:00Z","6696":"2015-08-31T00:00:00Z","6697":"2015-08-31T00:00:00Z","6698":"2015-08-31T00:00:00Z","6699":"2015-08-31T00:00:00Z","6700":"2015-08-31T00:00:00Z","6701":"2015-08-31T00:00:00Z","6702":"2015-08-31T00:00:00Z","6703":"2015-08-31T00:00:00Z","6704":"2015-08-31T00:00:00Z","6705":"2015-08-31T00:00:00Z","6706":"2015-08-31T00:00:00Z","6707":"2015-08-28T00:00:00Z","6708":"2015-08-28T00:00:00Z","6709":"2015-08-28T00:00:00Z","6710":"2015-08-27T00:00:00Z","6711":"2015-08-27T00:00:00Z","6712":"2015-08-26T00:00:00Z","6713":"2015-08-26T00:00:00Z","6715":"2015-08-25T00:00:00Z","6716":"2015-08-24T00:00:00Z","6717":"2015-08-24T00:00:00Z","6719":"2015-08-24T00:00:00Z","6721":"2015-08-24T00:00:00Z","6722":"2015-08-23T00:00:00Z","6723":"2015-08-23T00:00:00Z","6724":"2015-08-22T00:00:00Z","6725":"2015-08-22T00:00:00Z","6726":"2015-08-22T00:00:00Z","6727":"2015-08-22T00:00:00Z","6728":"2015-08-22T00:00:00Z","6729":"2015-08-21T00:00:00Z","6730":"2015-08-21T00:00:00Z","6731":"2015-08-21T00:00:00Z","6733":"2015-08-21T00:00:00Z","6734":"2015-08-20T00:00:00Z","6735":"2015-08-20T00:00:00Z","6736":"2015-08-20T00:00:00Z","6737":"2015-08-19T00:00:00Z","6738":"2015-08-19T00:00:00Z","6739":"2015-08-19T00:00:00Z","6740":"2015-08-19T00:00:00Z","6741":"2015-08-19T00:00:00Z","6742":"2015-08-19T00:00:00Z","6743":"2015-08-17T00:00:00Z","6744":"2015-08-17T00:00:00Z","6745":"2015-08-17T00:00:00Z","6746":"2015-08-17T00:00:00Z","6747":"2015-08-17T00:00:00Z","6748":"2015-08-16T00:00:00Z","6749":"2015-08-16T00:00:00Z","6751":"2015-08-14T00:00:00Z","6752":"2015-08-14T00:00:00Z","6753":"2015-08-14T00:00:00Z","6756":"2015-08-13T00:00:00Z","6757":"2015-08-12T00:00:00Z","6764":"2015-08-10T00:00:00Z","6765":"2015-08-10T00:00:00Z","6766":"2015-08-10T00:00:00Z","6767":"2015-08-09T00:00:00Z","6768":"2015-08-09T00:00:00Z","6769":"2015-08-09T00:00:00Z","6770":"2015-08-09T00:00:00Z","6771":"2015-08-08T00:00:00Z","6772":"2015-08-08T00:00:00Z","6773":"2015-08-07T00:00:00Z","6774":"2015-08-07T00:00:00Z","6775":"2015-08-06T00:00:00Z","6776":"2015-08-06T00:00:00Z","6777":"2015-08-05T00:00:00Z","6779":"2015-08-04T00:00:00Z","6780":"2015-08-04T00:00:00Z","6783":"2015-08-03T00:00:00Z","6784":"2015-08-03T00:00:00Z","6785":"2015-08-03T00:00:00Z","6786":"2015-08-03T00:00:00Z","6787":"2015-08-03T00:00:00Z","6788":"2015-08-03T00:00:00Z","6789":"2015-08-02T00:00:00Z","6790":"2015-08-02T00:00:00Z","6791":"2015-08-01T00:00:00Z","6792":"2015-07-31T00:00:00Z","6793":"2015-07-31T00:00:00Z","6794":"2015-07-31T00:00:00Z","6795":"2015-07-31T00:00:00Z","6796":"2015-07-31T00:00:00Z","6797":"2015-07-31T00:00:00Z","6798":"2015-07-30T00:00:00Z","6799":"2015-07-30T00:00:00Z","6800":"2015-07-30T00:00:00Z","6801":"2015-07-30T00:00:00Z","6803":"2015-07-30T00:00:00Z","6805":"2015-07-30T00:00:00Z","6806":"2015-07-30T00:00:00Z","6807":"2015-07-30T00:00:00Z","6808":"2015-07-30T00:00:00Z","6809":"2015-07-29T00:00:00Z","6810":"2015-07-29T00:00:00Z","6811":"2015-07-28T00:00:00Z","6812":"2015-07-28T00:00:00Z","6813":"2015-07-27T00:00:00Z","6814":"2015-07-27T00:00:00Z","6815":"2015-07-27T00:00:00Z","6816":"2015-07-27T00:00:00Z","6817":"2015-07-24T00:00:00Z","6818":"2015-07-24T00:00:00Z","6819":"2015-07-24T00:00:00Z","6822":"2015-07-23T00:00:00Z","6823":"2015-07-23T00:00:00Z","6824":"2015-07-23T00:00:00Z","6826":"2015-07-23T00:00:00Z","6827":"2015-07-23T00:00:00Z","6828":"2015-07-22T00:00:00Z","6829":"2015-07-22T00:00:00Z","6830":"2015-07-21T00:00:00Z","6831":"2015-07-20T00:00:00Z","6832":"2015-07-20T00:00:00Z","6833":"2015-07-20T00:00:00Z","6835":"2015-07-17T00:00:00Z","6836":"2015-07-17T00:00:00Z","6837":"2015-07-16T00:00:00Z","6838":"2015-07-15T00:00:00Z","6839":"2015-07-15T00:00:00Z","6840":"2015-07-15T00:00:00Z","6841":"2015-07-14T00:00:00Z","6842":"2015-07-14T00:00:00Z","6844":"2015-07-13T00:00:00Z","6845":"2015-07-13T00:00:00Z","6846":"2015-07-13T00:00:00Z","6847":"2015-07-13T00:00:00Z","6848":"2015-07-13T00:00:00Z","6849":"2015-07-13T00:00:00Z","6850":"2015-07-13T00:00:00Z","6851":"2015-07-13T00:00:00Z","6852":"2015-07-13T00:00:00Z","6853":"2015-07-13T00:00:00Z","6855":"2015-07-10T00:00:00Z","6856":"2015-07-10T00:00:00Z","6858":"2015-07-10T00:00:00Z","6859":"2015-07-10T00:00:00Z","6861":"2015-07-09T00:00:00Z","6862":"2015-07-09T00:00:00Z","6863":"2015-07-09T00:00:00Z","6864":"2015-07-09T00:00:00Z","6865":"2015-07-08T00:00:00Z","6866":"2015-07-07T00:00:00Z","6867":"2015-07-06T00:00:00Z","6868":"2015-07-06T00:00:00Z","6869":"2015-07-03T00:00:00Z","6870":"2015-07-03T00:00:00Z","6871":"2015-07-03T00:00:00Z","6872":"2015-07-03T00:00:00Z","6874":"2015-07-02T00:00:00Z","6875":"2015-07-02T00:00:00Z","6876":"2015-07-01T00:00:00Z","6877":"2015-07-01T00:00:00Z","6878":"2015-07-01T00:00:00Z","6879":"2015-06-30T00:00:00Z","6880":"2015-06-29T00:00:00Z","6881":"2015-06-29T00:00:00Z","6882":"2015-06-29T00:00:00Z","6883":"2015-06-29T00:00:00Z","6884":"2015-06-29T00:00:00Z","6885":"2015-06-29T00:00:00Z","6886":"2015-06-26T00:00:00Z","6889":"2015-06-25T00:00:00Z","6891":"2015-06-24T00:00:00Z","6892":"2015-06-24T00:00:00Z","6893":"2015-06-23T00:00:00Z","6894":"2015-06-23T00:00:00Z","6895":"2015-06-22T00:00:00Z","6896":"2015-06-22T00:00:00Z","6897":"2015-06-22T00:00:00Z","6898":"2015-06-22T00:00:00Z","6899":"2015-06-22T00:00:00Z","6900":"2015-06-19T00:00:00Z","6901":"2015-06-19T00:00:00Z","6902":"2015-06-19T00:00:00Z","6903":"2015-06-15T00:00:00Z","6904":"2015-06-15T00:00:00Z","6905":"2015-06-15T00:00:00Z","6906":"2015-06-15T00:00:00Z","6907":"2015-06-15T00:00:00Z","6908":"2015-06-15T00:00:00Z","6909":"2015-06-15T00:00:00Z","6910":"2015-06-15T00:00:00Z","6911":"2015-06-12T00:00:00Z","6912":"2015-06-12T00:00:00Z","6915":"2015-06-11T00:00:00Z","6916":"2015-06-11T00:00:00Z","6917":"2015-06-11T00:00:00Z","6918":"2015-06-11T00:00:00Z","6919":"2015-06-10T00:00:00Z","6920":"2015-06-10T00:00:00Z","6922":"2015-06-10T00:00:00Z","6924":"2015-06-08T00:00:00Z","6925":"2015-06-08T00:00:00Z","6927":"2015-06-08T00:00:00Z","6928":"2015-06-08T00:00:00Z","6929":"2015-06-05T00:00:00Z","6930":"2015-06-05T00:00:00Z","6931":"2015-06-05T00:00:00Z","6932":"2015-06-04T00:00:00Z","6933":"2015-06-04T00:00:00Z","6934":"2015-06-04T00:00:00Z","6935":"2015-06-04T00:00:00Z","6937":"2015-06-02T00:00:00Z","6938":"2015-06-02T00:00:00Z","6939":"2015-06-02T00:00:00Z","6940":"2015-06-02T00:00:00Z","6941":"2015-06-02T00:00:00Z","6942":"2015-06-02T00:00:00Z","6944":"2015-06-01T00:00:00Z","6945":"2015-06-01T00:00:00Z","6946":"2015-06-01T00:00:00Z","6948":"2015-06-01T00:00:00Z","6949":"2015-06-01T00:00:00Z","6950":"2015-06-01T00:00:00Z","6951":"2015-06-01T00:00:00Z","6952":"2015-06-01T00:00:00Z","6953":"2015-06-01T00:00:00Z","6954":"2015-05-27T00:00:00Z","6955":"2015-05-26T00:00:00Z","6956":"2015-05-26T00:00:00Z","6957":"2015-05-26T00:00:00Z","6958":"2015-05-26T00:00:00Z","6959":"2015-05-26T00:00:00Z","6960":"2015-05-26T00:00:00Z","6962":"2015-05-22T00:00:00Z","6963":"2015-05-22T00:00:00Z","6964":"2015-05-21T00:00:00Z","6965":"2015-05-20T00:00:00Z","6966":"2015-05-20T00:00:00Z","6967":"2015-05-18T00:00:00Z","6968":"2015-05-18T00:00:00Z","6969":"2015-05-18T00:00:00Z","6970":"2015-05-18T00:00:00Z","6971":"2015-05-18T00:00:00Z","6972":"2015-05-15T00:00:00Z","6973":"2015-05-15T00:00:00Z","6974":"2015-05-15T00:00:00Z","6975":"2015-05-15T00:00:00Z","6976":"2015-05-15T00:00:00Z","6977":"2015-05-14T00:00:00Z","6978":"2015-05-14T00:00:00Z","6979":"2015-05-14T00:00:00Z","6980":"2015-05-14T00:00:00Z","6981":"2015-05-14T00:00:00Z","6982":"2015-05-14T00:00:00Z","6983":"2015-05-14T00:00:00Z","6984":"2015-05-13T00:00:00Z","6985":"2015-05-12T00:00:00Z","6986":"2015-05-12T00:00:00Z","6988":"2015-05-11T00:00:00Z","6989":"2015-05-11T00:00:00Z","6990":"2015-05-11T00:00:00Z","6991":"2015-05-11T00:00:00Z","6992":"2015-05-11T00:00:00Z","6993":"2015-05-11T00:00:00Z","6995":"2015-05-11T00:00:00Z","6996":"2015-05-11T00:00:00Z","6999":"2015-05-08T00:00:00Z","7000":"2015-05-07T00:00:00Z","7001":"2015-05-05T00:00:00Z","7003":"2015-05-04T00:00:00Z","7004":"2015-05-04T00:00:00Z","7005":"2015-05-04T00:00:00Z","7006":"2015-05-04T00:00:00Z","7007":"2015-05-04T00:00:00Z","7008":"2015-05-04T00:00:00Z","7009":"2015-05-04T00:00:00Z","7010":"2015-05-04T00:00:00Z","7011":"2015-05-04T00:00:00Z","7012":"2015-05-04T00:00:00Z","7013":"2015-05-04T00:00:00Z","7014":"2015-05-01T00:00:00Z","7015":"2015-04-30T00:00:00Z","7016":"2015-04-30T00:00:00Z","7017":"2015-04-29T00:00:00Z","7018":"2015-04-28T00:00:00Z","7019":"2015-04-28T00:00:00Z","7020":"2015-04-28T00:00:00Z","7021":"2015-04-28T00:00:00Z","7022":"2015-04-28T00:00:00Z","7023":"2015-04-27T00:00:00Z","7024":"2015-04-27T00:00:00Z","7025":"2015-04-27T00:00:00Z","7026":"2015-04-27T00:00:00Z","7028":"2015-04-23T00:00:00Z","7029":"2015-04-22T00:00:00Z","7030":"2015-04-22T00:00:00Z","7031":"2015-04-21T00:00:00Z","7032":"2015-04-21T00:00:00Z","7034":"2015-04-20T00:00:00Z","7035":"2015-04-20T00:00:00Z","7036":"2015-04-20T00:00:00Z","7038":"2015-04-16T00:00:00Z","7039":"2015-04-16T00:00:00Z","7040":"2015-04-16T00:00:00Z","7041":"2015-04-13T00:00:00Z","7043":"2015-04-10T00:00:00Z","7044":"2015-04-10T00:00:00Z","7046":"2015-04-09T00:00:00Z","7048":"2015-04-06T00:00:00Z","7049":"2015-04-06T00:00:00Z","7051":"2015-04-06T00:00:00Z","7052":"2015-04-06T00:00:00Z","7053":"2015-04-06T00:00:00Z","7054":"2015-04-06T00:00:00Z","7055":"2015-04-06T00:00:00Z","7056":"2015-04-06T00:00:00Z","7057":"2015-04-06T00:00:00Z","7058":"2015-04-06T00:00:00Z","7059":"2015-04-06T00:00:00Z","7060":"2015-04-06T00:00:00Z","7063":"2015-04-03T00:00:00Z","7066":"2015-04-03T00:00:00Z","7067":"2015-04-03T00:00:00Z","7068":"2015-04-03T00:00:00Z","7069":"2015-04-03T00:00:00Z","7070":"2015-04-03T00:00:00Z","7071":"2015-04-02T00:00:00Z","7072":"2015-04-02T00:00:00Z","7073":"2015-04-02T00:00:00Z","7075":"2015-03-30T00:00:00Z","7076":"2015-03-30T00:00:00Z","7077":"2015-03-30T00:00:00Z","7078":"2015-03-30T00:00:00Z","7079":"2015-03-30T00:00:00Z","7080":"2015-03-30T00:00:00Z","7081":"2015-03-30T00:00:00Z","7082":"2015-03-27T00:00:00Z","7083":"2015-03-27T00:00:00Z","7084":"2015-03-27T00:00:00Z","7085":"2015-03-26T00:00:00Z","7086":"2015-03-26T00:00:00Z","7088":"2015-03-25T00:00:00Z","7090":"2015-03-25T00:00:00Z","7091":"2015-03-25T00:00:00Z","7092":"2015-03-25T00:00:00Z","7093":"2015-03-25T00:00:00Z","7094":"2015-03-25T00:00:00Z","7097":"2015-03-24T00:00:00Z","7098":"2015-03-24T00:00:00Z","7099":"2015-03-24T00:00:00Z","7100":"2015-03-23T00:00:00Z","7101":"2015-03-23T00:00:00Z","7102":"2015-03-23T00:00:00Z","7103":"2015-03-23T00:00:00Z","7104":"2015-03-23T00:00:00Z","7106":"2015-03-23T00:00:00Z","7107":"2015-03-20T00:00:00Z","7108":"2015-03-20T00:00:00Z","7109":"2015-03-20T00:00:00Z","7110":"2015-03-20T00:00:00Z","7111":"2015-03-20T00:00:00Z","7112":"2015-03-19T00:00:00Z","7113":"2015-03-18T00:00:00Z","7114":"2015-03-17T00:00:00Z","7115":"2015-03-16T00:00:00Z","7116":"2015-03-16T00:00:00Z","7117":"2015-03-16T00:00:00Z","7118":"2015-03-16T00:00:00Z","7119":"2015-03-16T00:00:00Z","7120":"2015-03-13T00:00:00Z","7121":"2015-03-13T00:00:00Z","7122":"2015-03-12T00:00:00Z","7126":"2015-03-11T00:00:00Z","7127":"2015-03-11T00:00:00Z","7130":"2015-03-10T00:00:00Z","7131":"2015-03-10T00:00:00Z","7133":"2015-03-09T00:00:00Z","7134":"2015-03-09T00:00:00Z","7135":"2015-03-09T00:00:00Z","7136":"2015-03-09T00:00:00Z","7137":"2015-03-06T00:00:00Z","7138":"2015-03-06T00:00:00Z","7139":"2015-03-06T00:00:00Z","7140":"2015-03-05T00:00:00Z","7142":"2015-03-04T00:00:00Z","7143":"2015-03-02T00:00:00Z","7144":"2015-03-02T00:00:00Z","7145":"2015-03-02T00:00:00Z","7146":"2015-03-02T00:00:00Z","7147":"2015-03-02T00:00:00Z","7148":"2015-02-27T00:00:00Z","7149":"2015-02-27T00:00:00Z","7150":"2015-02-26T00:00:00Z","7151":"2015-02-26T00:00:00Z","7152":"2015-02-25T00:00:00Z","7154":"2015-02-24T00:00:00Z","7155":"2015-02-24T00:00:00Z","7158":"2015-02-23T00:00:00Z","7159":"2015-02-23T00:00:00Z","7160":"2015-02-23T00:00:00Z","7163":"2015-02-23T00:00:00Z","7164":"2015-02-23T00:00:00Z","7165":"2015-02-23T00:00:00Z","7166":"2015-02-23T00:00:00Z","7167":"2015-02-19T00:00:00Z","7168":"2015-02-19T00:00:00Z","7169":"2015-02-18T00:00:00Z","7170":"2015-02-18T00:00:00Z","7172":"2015-02-17T00:00:00Z","7173":"2015-02-17T00:00:00Z","7175":"2015-02-17T00:00:00Z","7176":"2015-02-17T00:00:00Z","7177":"2015-02-17T00:00:00Z","7178":"2015-02-17T00:00:00Z","7179":"2015-02-17T00:00:00Z","7180":"2015-02-13T00:00:00Z","7185":"2015-02-09T00:00:00Z","7186":"2015-02-09T00:00:00Z","7187":"2015-02-09T00:00:00Z","7188":"2015-01-30T00:00:00Z","7189":"2015-01-30T00:00:00Z","7190":"2015-01-30T00:00:00Z","7191":"2015-01-29T00:00:00Z","7192":"2015-01-27T00:00:00Z","7193":"2015-01-27T00:00:00Z","7194":"2015-01-26T00:00:00Z","7195":"2015-01-26T00:00:00Z","7197":"2015-01-26T00:00:00Z","7198":"2015-01-26T00:00:00Z","7200":"2015-01-23T00:00:00Z","7201":"2015-01-23T00:00:00Z","7203":"2015-01-22T00:00:00Z","7204":"2015-01-20T00:00:00Z","7205":"2015-01-20T00:00:00Z","7206":"2015-01-20T00:00:00Z","7207":"2015-01-20T00:00:00Z","7208":"2015-01-20T00:00:00Z","7209":"2015-01-16T00:00:00Z","7210":"2015-01-15T00:00:00Z","7211":"2015-01-15T00:00:00Z","7212":"2015-01-15T00:00:00Z","7213":"2015-01-15T00:00:00Z","7216":"2015-01-13T00:00:00Z","7217":"2015-01-13T00:00:00Z","7218":"2015-01-13T00:00:00Z","7219":"2015-01-13T00:00:00Z","7220":"2015-01-13T00:00:00Z","7221":"2015-01-13T00:00:00Z","7224":"2015-01-12T00:00:00Z","7225":"2015-01-12T00:00:00Z","7227":"2015-01-12T00:00:00Z","7228":"2015-01-12T00:00:00Z","7229":"2015-01-12T00:00:00Z","7230":"2015-01-12T00:00:00Z","7231":"2015-01-12T00:00:00Z","7232":"2015-01-12T00:00:00Z","7233":"2015-01-12T00:00:00Z","7234":"2015-01-12T00:00:00Z","7235":"2015-01-12T00:00:00Z","7236":"2015-01-12T00:00:00Z","7237":"2015-01-12T00:00:00Z","7238":"2015-01-12T00:00:00Z","7239":"2015-01-12T00:00:00Z","7240":"2015-01-12T00:00:00Z","7241":"2015-01-12T00:00:00Z","7243":"2015-01-12T00:00:00Z","7244":"2015-01-09T00:00:00Z","7246":"2015-01-09T00:00:00Z","7247":"2015-01-08T00:00:00Z","7248":"2015-01-07T00:00:00Z","7249":"2015-01-07T00:00:00Z","7250":"2015-01-06T00:00:00Z","7251":"2015-01-06T00:00:00Z","7252":"2015-01-06T00:00:00Z","7253":"2015-01-05T00:00:00Z","7254":"2015-01-05T00:00:00Z","7255":"2015-01-05T00:00:00Z","7256":"2015-01-05T00:00:00Z","7257":"2015-01-05T00:00:00Z","7258":"2015-01-05T00:00:00Z","7260":"2015-01-02T00:00:00Z","7261":"2015-01-02T00:00:00Z","7262":"2015-01-02T00:00:00Z","7263":"2015-01-02T00:00:00Z","7264":"2014-12-30T00:00:00Z","7265":"2014-12-30T00:00:00Z","7266":"2014-12-30T00:00:00Z","7267":"2014-12-29T00:00:00Z","7268":"2014-12-29T00:00:00Z","7269":"2014-12-26T00:00:00Z","7270":"2014-12-26T00:00:00Z","7271":"2014-12-26T00:00:00Z","7272":"2014-12-24T00:00:00Z","7274":"2014-12-24T00:00:00Z","7275":"2014-12-23T00:00:00Z","7276":"2014-12-23T00:00:00Z","7277":"2014-12-23T00:00:00Z","7278":"2014-12-22T00:00:00Z","7279":"2014-12-22T00:00:00Z","7280":"2014-12-22T00:00:00Z","7281":"2014-12-22T00:00:00Z","7282":"2014-12-22T00:00:00Z","7283":"2014-12-22T00:00:00Z","7284":"2014-12-22T00:00:00Z","7285":"2014-12-22T00:00:00Z","7286":"2014-12-22T00:00:00Z","7287":"2014-12-22T00:00:00Z","7289":"2014-12-22T00:00:00Z","7290":"2014-12-22T00:00:00Z","7291":"2014-12-22T00:00:00Z","7292":"2014-12-22T00:00:00Z","7293":"2014-12-22T00:00:00Z","7294":"2014-12-22T00:00:00Z","7295":"2014-12-22T00:00:00Z","7296":"2014-12-22T00:00:00Z","7297":"2014-12-22T00:00:00Z","7298":"2014-12-22T00:00:00Z","7299":"2014-12-22T00:00:00Z","7301":"2014-12-19T00:00:00Z","7302":"2014-12-19T00:00:00Z","7303":"2014-12-19T00:00:00Z","7304":"2014-12-19T00:00:00Z","7306":"2014-12-19T00:00:00Z","7307":"2014-12-19T00:00:00Z","7308":"2014-12-18T00:00:00Z","7309":"2014-12-18T00:00:00Z","7310":"2014-12-18T00:00:00Z","7311":"2014-12-18T00:00:00Z","7314":"2014-12-17T00:00:00Z","7315":"2014-12-17T00:00:00Z","7316":"2014-12-17T00:00:00Z","7317":"2014-12-17T00:00:00Z","7318":"2014-12-17T00:00:00Z","7321":"2014-12-16T00:00:00Z","7322":"2014-12-16T00:00:00Z","7323":"2014-12-16T00:00:00Z","7324":"2014-12-16T00:00:00Z","7325":"2014-12-16T00:00:00Z","7326":"2014-12-16T00:00:00Z","7327":"2014-12-16T00:00:00Z","7328":"2014-12-15T00:00:00Z","7329":"2014-12-15T00:00:00Z","7330":"2014-12-15T00:00:00Z","7331":"2014-12-15T00:00:00Z","7332":"2014-12-15T00:00:00Z","7333":"2014-12-15T00:00:00Z","7334":"2014-12-15T00:00:00Z","7335":"2014-12-15T00:00:00Z","7336":"2014-12-12T00:00:00Z","7338":"2014-12-10T00:00:00Z","7339":"2014-12-10T00:00:00Z","7342":"2014-12-08T00:00:00Z","7343":"2014-12-08T00:00:00Z","7344":"2014-12-08T00:00:00Z","7345":"2014-12-08T00:00:00Z","7346":"2014-12-08T00:00:00Z","7348":"2014-12-08T00:00:00Z","7349":"2014-12-08T00:00:00Z","7351":"2014-12-08T00:00:00Z","7352":"2014-12-04T00:00:00Z","7353":"2014-12-04T00:00:00Z","7354":"2014-12-03T00:00:00Z","7355":"2014-12-03T00:00:00Z","7357":"2014-12-03T00:00:00Z","7358":"2014-12-01T00:00:00Z","7359":"2014-12-01T00:00:00Z","7360":"2014-12-01T00:00:00Z","7361":"2014-11-26T00:00:00Z","7362":"2014-11-26T00:00:00Z","7364":"2014-11-24T00:00:00Z","7365":"2014-11-24T00:00:00Z","7366":"2014-11-24T00:00:00Z","7367":"2014-11-24T00:00:00Z","7368":"2014-11-24T00:00:00Z","7369":"2014-11-24T00:00:00Z","7370":"2014-11-24T00:00:00Z","7371":"2014-11-24T00:00:00Z","7373":"2014-11-19T00:00:00Z","7374":"2014-11-19T00:00:00Z","7375":"2014-11-17T00:00:00Z","7376":"2014-11-17T00:00:00Z","7383":"2014-11-10T00:00:00Z","7385":"2014-11-10T00:00:00Z","7386":"2014-11-10T00:00:00Z","7387":"2014-11-10T00:00:00Z","7388":"2014-11-10T00:00:00Z","7390":"2014-11-10T00:00:00Z","7391":"2014-11-05T00:00:00Z","7392":"2014-11-05T00:00:00Z","7394":"2014-11-05T00:00:00Z","7395":"2014-11-05T00:00:00Z","7396":"2014-11-05T00:00:00Z","7398":"2014-11-03T00:00:00Z","7399":"2014-11-03T00:00:00Z","7400":"2014-11-03T00:00:00Z","7401":"2014-11-03T00:00:00Z","7402":"2014-11-03T00:00:00Z","7404":"2014-11-03T00:00:00Z","7405":"2014-10-31T00:00:00Z","7406":"2014-10-30T00:00:00Z","7408":"2014-10-29T00:00:00Z","7409":"2014-10-28T00:00:00Z","7410":"2014-10-28T00:00:00Z","7411":"2014-10-28T00:00:00Z","7412":"2014-10-28T00:00:00Z","7413":"2014-10-27T00:00:00Z","7414":"2014-10-27T00:00:00Z","7415":"2014-10-27T00:00:00Z","7416":"2014-10-27T00:00:00Z","7417":"2014-10-27T00:00:00Z","7418":"2014-10-27T00:00:00Z","7419":"2014-10-27T00:00:00Z","7420":"2014-10-27T00:00:00Z","7421":"2014-10-27T00:00:00Z","7422":"2014-10-27T00:00:00Z","7423":"2014-10-27T00:00:00Z","7424":"2014-10-24T00:00:00Z","7426":"2014-10-24T00:00:00Z","7427":"2014-10-23T00:00:00Z","7428":"2014-10-22T00:00:00Z","7429":"2014-10-22T00:00:00Z","7430":"2014-10-22T00:00:00Z","7431":"2014-10-20T00:00:00Z","7432":"2014-10-20T00:00:00Z","7433":"2014-10-20T00:00:00Z","7434":"2014-10-20T00:00:00Z","7435":"2014-10-20T00:00:00Z","7436":"2014-10-20T00:00:00Z","7437":"2014-10-20T00:00:00Z","7438":"2014-10-20T00:00:00Z","7439":"2014-10-20T00:00:00Z","7440":"2014-10-20T00:00:00Z","7441":"2014-10-20T00:00:00Z","7442":"2014-10-17T00:00:00Z","7443":"2014-10-17T00:00:00Z","7444":"2014-10-17T00:00:00Z","7446":"2014-10-16T00:00:00Z","7447":"2014-10-16T00:00:00Z","7448":"2014-10-15T00:00:00Z","7449":"2014-10-15T00:00:00Z","7450":"2014-10-15T00:00:00Z","7451":"2014-10-14T00:00:00Z","7452":"2014-10-14T00:00:00Z","7453":"2014-10-14T00:00:00Z","7454":"2014-10-14T00:00:00Z","7455":"2014-10-14T00:00:00Z","7456":"2014-10-14T00:00:00Z","7457":"2014-10-14T00:00:00Z","7458":"2014-10-14T00:00:00Z","7459":"2014-10-14T00:00:00Z","7460":"2014-10-14T00:00:00Z","7461":"2014-10-14T00:00:00Z","7462":"2014-10-14T00:00:00Z","7463":"2014-10-14T00:00:00Z","7465":"2014-10-09T00:00:00Z","7467":"2014-10-08T00:00:00Z","7468":"2014-10-07T00:00:00Z","7469":"2014-10-06T00:00:00Z","7470":"2014-10-06T00:00:00Z","7471":"2014-10-06T00:00:00Z","7472":"2014-10-06T00:00:00Z","7473":"2014-10-06T00:00:00Z","7475":"2014-10-03T00:00:00Z","7476":"2014-10-02T00:00:00Z","7477":"2014-10-01T00:00:00Z","7478":"2014-09-29T00:00:00Z","7480":"2014-09-29T00:00:00Z","7481":"2014-09-29T00:00:00Z","7482":"2014-09-25T00:00:00Z","7483":"2014-09-25T00:00:00Z","7484":"2014-09-25T00:00:00Z","7485":"2014-09-24T00:00:00Z","7486":"2014-09-24T00:00:00Z","7487":"2014-09-22T00:00:00Z","7488":"2014-09-22T00:00:00Z","7489":"2014-09-22T00:00:00Z","7490":"2014-09-22T00:00:00Z","7491":"2014-09-22T00:00:00Z","7492":"2014-09-22T00:00:00Z","7493":"2014-09-19T00:00:00Z","7494":"2014-09-18T00:00:00Z","7495":"2014-09-18T00:00:00Z","7496":"2014-09-17T00:00:00Z","7497":"2014-09-16T00:00:00Z","7498":"2014-09-15T00:00:00Z","7499":"2014-09-15T00:00:00Z","7500":"2014-09-15T00:00:00Z","7501":"2014-09-15T00:00:00Z","7502":"2014-09-12T00:00:00Z","7503":"2014-09-12T00:00:00Z","7504":"2014-09-11T00:00:00Z","7505":"2014-09-11T00:00:00Z","7507":"2014-09-10T00:00:00Z","7508":"2014-09-09T00:00:00Z","7509":"2014-09-09T00:00:00Z","7510":"2014-09-09T00:00:00Z","7511":"2014-09-09T00:00:00Z","7513":"2014-09-08T00:00:00Z","7514":"2014-09-08T00:00:00Z","7515":"2014-09-08T00:00:00Z","7516":"2014-09-08T00:00:00Z","7517":"2014-09-05T00:00:00Z","7519":"2014-09-04T00:00:00Z","7520":"2014-09-04T00:00:00Z","7521":"2014-09-04T00:00:00Z","7523":"2014-09-03T00:00:00Z","7526":"2014-08-29T00:00:00Z","7527":"2014-08-28T00:00:00Z","7528":"2014-08-28T00:00:00Z","7529":"2014-08-27T00:00:00Z","7530":"2014-08-27T00:00:00Z","7531":"2014-08-26T00:00:00Z","7532":"2014-08-25T00:00:00Z","7533":"2014-08-25T00:00:00Z","7536":"2014-08-25T00:00:00Z","7537":"2014-08-25T00:00:00Z","7538":"2014-08-25T00:00:00Z","7539":"2014-08-25T00:00:00Z","7540":"2014-08-25T00:00:00Z","7541":"2014-08-25T00:00:00Z","7542":"2014-08-25T00:00:00Z","7543":"2014-08-25T00:00:00Z","7544":"2014-08-25T00:00:00Z","7545":"2014-08-22T00:00:00Z","7546":"2014-08-22T00:00:00Z","7547":"2014-08-22T00:00:00Z","7548":"2014-08-22T00:00:00Z","7549":"2014-08-21T00:00:00Z","7550":"2014-08-20T00:00:00Z","7551":"2014-08-20T00:00:00Z","7552":"2014-08-20T00:00:00Z","7553":"2014-08-20T00:00:00Z","7554":"2014-08-18T00:00:00Z","7555":"2014-08-18T00:00:00Z","7556":"2014-08-18T00:00:00Z","7558":"2014-08-15T00:00:00Z","7560":"2014-08-14T00:00:00Z","7561":"2014-08-14T00:00:00Z","7562":"2014-08-14T00:00:00Z","7563":"2014-08-14T00:00:00Z","7565":"2014-08-11T00:00:00Z","7567":"2014-08-05T00:00:00Z","7568":"2014-08-04T00:00:00Z","7570":"2014-08-01T00:00:00Z","7571":"2014-07-31T00:00:00Z","7572":"2014-07-25T00:00:00Z","7573":"2014-07-21T00:00:00Z","7574":"2014-07-21T00:00:00Z","7575":"2014-07-18T00:00:00Z","7576":"2014-07-17T00:00:00Z","7577":"2014-07-16T00:00:00Z","7578":"2014-07-15T00:00:00Z","7579":"2014-07-14T00:00:00Z","7580":"2014-07-11T00:00:00Z","7581":"2014-07-11T00:00:00Z","7582":"2014-07-11T00:00:00Z","7583":"2014-07-10T00:00:00Z","7585":"2014-07-10T00:00:00Z","7586":"2014-07-09T00:00:00Z","7587":"2014-07-09T00:00:00Z","7588":"2014-07-08T00:00:00Z","7589":"2014-07-08T00:00:00Z","7590":"2014-07-07T00:00:00Z","7591":"2014-07-07T00:00:00Z","7592":"2014-07-07T00:00:00Z","7593":"2014-07-07T00:00:00Z","7594":"2014-07-07T00:00:00Z","7595":"2014-07-07T00:00:00Z","7596":"2014-07-07T00:00:00Z","7597":"2014-07-03T00:00:00Z","7598":"2014-07-03T00:00:00Z","7599":"2014-07-03T00:00:00Z","7601":"2014-07-03T00:00:00Z","7602":"2014-07-03T00:00:00Z","7603":"2014-07-03T00:00:00Z","7604":"2014-07-02T00:00:00Z","7605":"2014-07-02T00:00:00Z","7606":"2014-07-02T00:00:00Z","7607":"2014-07-02T00:00:00Z","7608":"2014-07-01T00:00:00Z","7610":"2014-07-01T00:00:00Z","7613":"2014-06-30T00:00:00Z","7614":"2014-06-30T00:00:00Z","7615":"2014-06-26T00:00:00Z","7617":"2014-06-24T00:00:00Z","7618":"2014-06-16T00:00:00Z","7619":"2014-06-16T00:00:00Z","7620":"2014-06-16T00:00:00Z","7622":"2014-06-09T00:00:00Z","7623":"2014-06-09T00:00:00Z","7625":"2014-06-02T00:00:00Z","7626":"2014-06-02T00:00:00Z","7627":"2014-06-02T00:00:00Z","7630":"2014-05-27T00:00:00Z","7631":"2014-05-27T00:00:00Z","7632":"2014-05-27T00:00:00Z","7633":"2014-05-27T00:00:00Z","7634":"2014-05-27T00:00:00Z","7635":"2014-05-27T00:00:00Z","7636":"2014-05-27T00:00:00Z","7637":"2014-05-23T00:00:00Z","7638":"2014-05-23T00:00:00Z","7639":"2014-05-22T00:00:00Z","7641":"2014-05-21T00:00:00Z","7642":"2014-05-20T00:00:00Z","7643":"2014-05-20T00:00:00Z","7644":"2014-05-20T00:00:00Z","7645":"2014-05-20T00:00:00Z","7646":"2014-05-19T00:00:00Z","7647":"2014-05-19T00:00:00Z","7648":"2014-05-19T00:00:00Z","7649":"2014-05-19T00:00:00Z","7650":"2014-05-19T00:00:00Z","7651":"2014-05-19T00:00:00Z","7652":"2014-05-19T00:00:00Z","7653":"2014-05-16T00:00:00Z","7654":"2014-05-15T00:00:00Z","7656":"2014-05-12T00:00:00Z","7657":"2014-05-12T00:00:00Z","7659":"2014-05-08T00:00:00Z","7660":"2014-05-08T00:00:00Z","7662":"2014-05-07T00:00:00Z","7663":"2014-05-05T00:00:00Z","7664":"2014-05-05T00:00:00Z","7665":"2014-05-05T00:00:00Z","7666":"2014-05-05T00:00:00Z","7668":"2014-05-05T00:00:00Z","7670":"2014-05-02T00:00:00Z","7671":"2014-05-01T00:00:00Z","7672":"2014-04-30T00:00:00Z","7673":"2014-04-30T00:00:00Z","7674":"2014-04-30T00:00:00Z","7676":"2014-04-29T00:00:00Z","7677":"2014-04-28T00:00:00Z","7678":"2014-04-28T00:00:00Z","7679":"2014-04-28T00:00:00Z","7680":"2014-04-24T00:00:00Z","7681":"2014-04-24T00:00:00Z","7683":"2014-04-21T00:00:00Z","7684":"2014-04-18T00:00:00Z","7685":"2014-04-16T00:00:00Z","7686":"2014-04-16T00:00:00Z","7687":"2014-04-15T00:00:00Z","7688":"2014-04-14T00:00:00Z","7689":"2014-04-14T00:00:00Z","7690":"2014-04-14T00:00:00Z","7691":"2014-04-14T00:00:00Z","7692":"2014-04-14T00:00:00Z","7693":"2014-04-11T00:00:00Z","7694":"2014-04-11T00:00:00Z","7695":"2014-04-11T00:00:00Z","7696":"2014-04-11T00:00:00Z","7697":"2014-04-11T00:00:00Z","7698":"2014-04-10T00:00:00Z","7699":"2014-04-09T00:00:00Z","7700":"2014-04-08T00:00:00Z","7701":"2014-04-08T00:00:00Z","7702":"2014-04-07T00:00:00Z","7703":"2014-04-07T00:00:00Z","7704":"2014-04-07T00:00:00Z","7706":"2014-04-07T00:00:00Z","7707":"2014-04-07T00:00:00Z","7708":"2014-04-07T00:00:00Z","7709":"2014-04-07T00:00:00Z","7710":"2014-04-07T00:00:00Z","7712":"2014-04-03T00:00:00Z","7713":"2014-04-02T00:00:00Z","7714":"2014-04-02T00:00:00Z","7715":"2014-04-01T00:00:00Z","7716":"2014-04-01T00:00:00Z","7718":"2014-03-31T00:00:00Z","7719":"2014-03-28T00:00:00Z","7720":"2014-03-27T00:00:00Z","7721":"2014-03-26T00:00:00Z","7722":"2014-03-26T00:00:00Z","7723":"2014-03-24T00:00:00Z","7724":"2014-03-24T00:00:00Z","7725":"2014-03-24T00:00:00Z","7726":"2014-03-24T00:00:00Z","7727":"2014-03-24T00:00:00Z","7728":"2014-03-24T00:00:00Z","7730":"2014-03-20T00:00:00Z","7732":"2014-03-17T00:00:00Z","7733":"2014-03-17T00:00:00Z","7735":"2014-03-17T00:00:00Z","7736":"2014-03-17T00:00:00Z","7737":"2014-03-17T00:00:00Z","7738":"2014-03-17T00:00:00Z","7739":"2014-03-17T00:00:00Z","7740":"2014-03-17T00:00:00Z","7741":"2014-03-14T00:00:00Z","7742":"2014-03-14T00:00:00Z","7743":"2014-03-11T00:00:00Z","7744":"2014-03-10T00:00:00Z","7745":"2014-03-10T00:00:00Z","7746":"2014-03-10T00:00:00Z","7747":"2014-03-10T00:00:00Z","7748":"2014-03-10T00:00:00Z","7749":"2014-03-10T00:00:00Z","7750":"2014-03-10T00:00:00Z","7751":"2014-03-10T00:00:00Z","7752":"2014-03-07T00:00:00Z","7753":"2014-03-07T00:00:00Z","7755":"2014-03-05T00:00:00Z","7756":"2014-03-05T00:00:00Z","7757":"2014-03-05T00:00:00Z","7758":"2014-03-05T00:00:00Z","7759":"2014-03-05T00:00:00Z","7760":"2014-03-04T00:00:00Z","7761":"2014-03-03T00:00:00Z","7762":"2014-03-03T00:00:00Z","7763":"2014-03-03T00:00:00Z","7764":"2014-03-03T00:00:00Z","7765":"2014-03-03T00:00:00Z","7766":"2014-02-28T00:00:00Z","7767":"2014-02-28T00:00:00Z","7768":"2014-02-28T00:00:00Z","7769":"2014-02-28T00:00:00Z","7770":"2014-02-27T00:00:00Z","7771":"2014-02-27T00:00:00Z","7774":"2014-02-20T00:00:00Z","7775":"2014-02-20T00:00:00Z","7776":"2014-02-18T00:00:00Z","7777":"2014-02-18T00:00:00Z","7778":"2014-02-18T00:00:00Z","7779":"2014-02-18T00:00:00Z","7780":"2014-02-18T00:00:00Z","7781":"2014-02-18T00:00:00Z","7782":"2014-02-18T00:00:00Z","7783":"2014-02-18T00:00:00Z","7784":"2014-02-18T00:00:00Z","7785":"2014-02-14T00:00:00Z","7786":"2014-02-10T00:00:00Z","7787":"2014-02-10T00:00:00Z","7788":"2014-02-10T00:00:00Z","7789":"2014-02-10T00:00:00Z","7790":"2014-02-10T00:00:00Z","7791":"2014-02-10T00:00:00Z","7792":"2014-02-10T00:00:00Z","7793":"2014-02-07T00:00:00Z","7795":"2014-02-07T00:00:00Z","7797":"2014-02-05T00:00:00Z","7798":"2014-02-04T00:00:00Z","7799":"2014-02-03T00:00:00Z","7800":"2014-02-03T00:00:00Z","7801":"2014-02-03T00:00:00Z","7803":"2014-02-03T00:00:00Z","7804":"2014-02-03T00:00:00Z","7805":"2014-02-03T00:00:00Z","7806":"2014-02-03T00:00:00Z","7807":"2014-02-03T00:00:00Z","7808":"2014-01-31T00:00:00Z","7809":"2014-01-31T00:00:00Z","7810":"2014-01-31T00:00:00Z","7811":"2014-01-30T00:00:00Z","7812":"2014-01-29T00:00:00Z","7813":"2014-01-29T00:00:00Z","7814":"2014-01-28T00:00:00Z","7815":"2014-01-28T00:00:00Z","7816":"2014-01-28T00:00:00Z","7817":"2014-01-27T00:00:00Z","7818":"2014-01-27T00:00:00Z","7819":"2014-01-27T00:00:00Z","7821":"2014-01-27T00:00:00Z","7822":"2014-01-27T00:00:00Z","7823":"2014-01-24T00:00:00Z","7824":"2014-01-24T00:00:00Z","7825":"2014-01-24T00:00:00Z","7828":"2014-01-24T00:00:00Z","7830":"2014-01-23T00:00:00Z","7831":"2014-01-23T00:00:00Z","7833":"2014-01-22T00:00:00Z","7835":"2014-01-21T00:00:00Z","7836":"2014-01-21T00:00:00Z","7838":"2014-01-21T00:00:00Z","7839":"2014-01-21T00:00:00Z","7841":"2014-01-16T00:00:00Z","7842":"2014-01-14T00:00:00Z","7843":"2014-01-14T00:00:00Z","7844":"2014-01-13T00:00:00Z","7846":"2014-01-13T00:00:00Z","7847":"2014-01-13T00:00:00Z","7848":"2014-01-13T00:00:00Z","7849":"2014-01-13T00:00:00Z","7850":"2014-01-10T00:00:00Z","7851":"2014-01-10T00:00:00Z","7852":"2014-01-10T00:00:00Z","7853":"2014-01-10T00:00:00Z","7854":"2014-01-09T00:00:00Z","7855":"2014-01-08T00:00:00Z","7856":"2014-01-07T00:00:00Z","7857":"2014-01-07T00:00:00Z","7858":"2014-01-06T00:00:00Z","7859":"2014-01-06T00:00:00Z","7860":"2014-01-06T00:00:00Z","7861":"2014-01-06T00:00:00Z","7862":"2014-01-06T00:00:00Z","7864":"2014-01-06T00:00:00Z","7865":"2014-01-06T00:00:00Z","7866":"2014-01-03T00:00:00Z","7867":"2014-01-03T00:00:00Z","7868":"2014-01-03T00:00:00Z","7869":"2014-01-03T00:00:00Z","7870":"2014-01-02T00:00:00Z","7872":"2014-01-02T00:00:00Z","7873":"2014-01-02T00:00:00Z","7874":"2014-01-02T00:00:00Z","7876":"2013-12-31T00:00:00Z","7877":"2013-12-31T00:00:00Z","7880":"2013-12-27T00:00:00Z","7881":"2013-12-23T00:00:00Z","7882":"2013-12-23T00:00:00Z","7883":"2013-12-23T00:00:00Z","7884":"2013-12-23T00:00:00Z","7885":"2013-12-20T00:00:00Z","7886":"2013-12-20T00:00:00Z","7887":"2013-12-20T00:00:00Z","7889":"2013-12-19T00:00:00Z","7890":"2013-12-18T00:00:00Z","7891":"2013-12-18T00:00:00Z","7892":"2013-12-18T00:00:00Z","7894":"2013-12-17T00:00:00Z","7895":"2013-12-16T00:00:00Z","7897":"2013-12-16T00:00:00Z","7898":"2013-12-16T00:00:00Z","7899":"2013-12-16T00:00:00Z","7900":"2013-12-16T00:00:00Z","7901":"2013-12-16T00:00:00Z","7902":"2013-12-13T00:00:00Z","7903":"2013-12-13T00:00:00Z","7904":"2013-12-11T00:00:00Z","7905":"2013-12-09T00:00:00Z","7906":"2013-12-09T00:00:00Z","7907":"2013-12-09T00:00:00Z","7908":"2013-12-09T00:00:00Z","7909":"2013-12-09T00:00:00Z","7910":"2013-12-09T00:00:00Z","7913":"2013-12-06T00:00:00Z","7914":"2013-12-06T00:00:00Z","7916":"2013-12-05T00:00:00Z","7917":"2013-12-05T00:00:00Z","7920":"2013-12-04T00:00:00Z","7921":"2013-12-03T00:00:00Z","7922":"2013-12-03T00:00:00Z","7924":"2013-11-27T00:00:00Z","7925":"2013-11-26T00:00:00Z","7926":"2013-11-26T00:00:00Z","7927":"2013-11-26T00:00:00Z","7928":"2013-11-26T00:00:00Z","7929":"2013-11-26T00:00:00Z","7930":"2013-11-26T00:00:00Z","7931":"2013-11-25T00:00:00Z","7932":"2013-11-25T00:00:00Z","7933":"2013-11-25T00:00:00Z","7934":"2013-11-25T00:00:00Z","7936":"2013-11-25T00:00:00Z","7937":"2013-11-25T00:00:00Z","7938":"2013-11-25T00:00:00Z","7939":"2013-11-22T00:00:00Z","7941":"2013-11-22T00:00:00Z","7942":"2013-11-21T00:00:00Z","7943":"2013-11-21T00:00:00Z","7944":"2013-11-21T00:00:00Z","7945":"2013-11-21T00:00:00Z","7946":"2013-11-21T00:00:00Z","7947":"2013-11-20T00:00:00Z","7948":"2013-11-20T00:00:00Z","7949":"2013-11-19T00:00:00Z","7950":"2013-11-19T00:00:00Z","7951":"2013-11-19T00:00:00Z","7952":"2013-11-18T00:00:00Z","7953":"2013-11-18T00:00:00Z","7954":"2013-11-18T00:00:00Z","7955":"2013-11-18T00:00:00Z","7956":"2013-11-18T00:00:00Z","7957":"2013-11-18T00:00:00Z","7958":"2013-11-18T00:00:00Z","7959":"2013-11-18T00:00:00Z","7960":"2013-11-18T00:00:00Z","7961":"2013-11-15T00:00:00Z","7962":"2013-11-15T00:00:00Z","7963":"2013-11-15T00:00:00Z","7964":"2013-11-15T00:00:00Z","7966":"2013-11-15T00:00:00Z","7967":"2013-11-15T00:00:00Z","7968":"2013-11-14T00:00:00Z","7969":"2013-11-14T00:00:00Z","7970":"2013-11-13T00:00:00Z","7971":"2013-11-13T00:00:00Z","7972":"2013-11-12T00:00:00Z","7973":"2013-11-12T00:00:00Z","7974":"2013-11-12T00:00:00Z","7975":"2013-11-12T00:00:00Z","7976":"2013-11-12T00:00:00Z","7977":"2013-11-12T00:00:00Z","7978":"2013-11-12T00:00:00Z","7979":"2013-11-12T00:00:00Z","7980":"2013-11-08T00:00:00Z","7981":"2013-11-08T00:00:00Z","7983":"2013-11-07T00:00:00Z","7984":"2013-11-06T00:00:00Z","7985":"2013-11-06T00:00:00Z","7987":"2013-11-05T00:00:00Z","7988":"2013-11-04T00:00:00Z","7989":"2013-11-04T00:00:00Z","7990":"2013-11-04T00:00:00Z","7991":"2013-11-04T00:00:00Z","7992":"2013-11-04T00:00:00Z","7993":"2013-11-01T00:00:00Z","7994":"2013-10-31T00:00:00Z","7995":"2013-10-31T00:00:00Z","7997":"2013-10-31T00:00:00Z","7998":"2013-10-30T00:00:00Z","8000":"2013-10-29T00:00:00Z","8001":"2013-10-29T00:00:00Z","8002":"2013-10-29T00:00:00Z","8005":"2013-10-28T00:00:00Z","8007":"2013-10-28T00:00:00Z","8008":"2013-10-28T00:00:00Z","8009":"2013-10-28T00:00:00Z","8010":"2013-10-28T00:00:00Z","8011":"2013-10-28T00:00:00Z","8013":"2013-10-25T00:00:00Z","8015":"2013-10-16T00:00:00Z","8019":"2013-10-15T00:00:00Z","8020":"2013-10-15T00:00:00Z","8021":"2013-10-15T00:00:00Z","8022":"2013-10-15T00:00:00Z","8023":"2013-10-15T00:00:00Z","8024":"2013-10-15T00:00:00Z","8025":"2013-10-15T00:00:00Z","8027":"2013-10-15T00:00:00Z","8028":"2013-10-11T00:00:00Z","8031":"2013-10-09T00:00:00Z","8033":"2013-10-08T00:00:00Z","8034":"2013-10-08T00:00:00Z","8035":"2013-10-07T00:00:00Z","8036":"2013-10-07T00:00:00Z","8037":"2013-10-07T00:00:00Z","8038":"2013-10-07T00:00:00Z","8039":"2013-10-07T00:00:00Z","8040":"2013-10-07T00:00:00Z","8042":"2013-10-07T00:00:00Z","8043":"2013-10-03T00:00:00Z","8045":"2013-10-03T00:00:00Z","8047":"2013-10-02T00:00:00Z","8048":"2013-10-02T00:00:00Z","8049":"2013-10-02T00:00:00Z","8050":"2013-10-02T00:00:00Z","8051":"2013-10-01T00:00:00Z","8052":"2013-10-01T00:00:00Z","8053":"2013-09-30T00:00:00Z","8054":"2013-09-30T00:00:00Z","8055":"2013-09-30T00:00:00Z","8056":"2013-09-30T00:00:00Z","8058":"2013-09-25T00:00:00Z","8059":"2013-09-25T00:00:00Z","8060":"2013-09-23T00:00:00Z","8063":"2013-09-09T00:00:00Z","8065":"2013-09-06T00:00:00Z","8067":"2013-09-05T00:00:00Z","8068":"2013-09-04T00:00:00Z","8069":"2013-09-04T00:00:00Z","8070":"2013-09-04T00:00:00Z","8072":"2013-09-03T00:00:00Z","8073":"2013-09-03T00:00:00Z","8074":"2013-09-03T00:00:00Z","8075":"2013-09-03T00:00:00Z","8076":"2013-09-03T00:00:00Z","8077":"2013-09-03T00:00:00Z","8078":"2013-09-03T00:00:00Z","8079":"2013-09-03T00:00:00Z","8080":"2013-09-03T00:00:00Z","8081":"2013-09-03T00:00:00Z","8082":"2013-09-03T00:00:00Z","8083":"2013-08-30T00:00:00Z","8084":"2013-08-30T00:00:00Z","8085":"2013-08-30T00:00:00Z","8087":"2013-08-29T00:00:00Z","8088":"2013-08-29T00:00:00Z","8089":"2013-08-28T00:00:00Z","8090":"2013-08-27T00:00:00Z","8091":"2013-08-26T00:00:00Z","8092":"2013-08-26T00:00:00Z","8093":"2013-08-26T00:00:00Z","8094":"2013-08-26T00:00:00Z","8096":"2013-08-26T00:00:00Z","8097":"2013-08-26T00:00:00Z","8098":"2013-08-23T00:00:00Z","8099":"2013-08-23T00:00:00Z","8100":"2013-08-22T00:00:00Z","8101":"2013-08-22T00:00:00Z","8102":"2013-08-22T00:00:00Z","8103":"2013-08-22T00:00:00Z","8104":"2013-08-22T00:00:00Z","8106":"2013-08-19T00:00:00Z","8107":"2013-08-19T00:00:00Z","8109":"2013-08-15T00:00:00Z","8110":"2013-08-13T00:00:00Z","8111":"2013-08-12T00:00:00Z","8112":"2013-08-12T00:00:00Z","8113":"2013-08-08T00:00:00Z","8115":"2013-08-06T00:00:00Z","8116":"2013-08-06T00:00:00Z","8117":"2013-08-06T00:00:00Z","8118":"2013-08-06T00:00:00Z","8119":"2013-08-05T00:00:00Z","8120":"2013-08-05T00:00:00Z","8121":"2013-08-05T00:00:00Z","8123":"2013-08-05T00:00:00Z","8124":"2013-08-05T00:00:00Z","8125":"2013-08-05T00:00:00Z","8126":"2013-08-05T00:00:00Z","8127":"2013-08-02T00:00:00Z","8129":"2013-08-02T00:00:00Z","8130":"2013-08-02T00:00:00Z","8131":"2013-08-02T00:00:00Z","8132":"2013-08-01T00:00:00Z","8134":"2013-08-01T00:00:00Z","8135":"2013-08-01T00:00:00Z","8136":"2013-07-31T00:00:00Z","8137":"2013-07-31T00:00:00Z","8138":"2013-07-31T00:00:00Z","8139":"2013-07-30T00:00:00Z","8140":"2013-07-29T00:00:00Z","8141":"2013-07-29T00:00:00Z","8142":"2013-07-26T00:00:00Z","8143":"2013-07-24T00:00:00Z","8144":"2013-07-22T00:00:00Z","8145":"2013-07-22T00:00:00Z","8146":"2013-07-22T00:00:00Z","8148":"2013-07-22T00:00:00Z","8149":"2013-07-19T00:00:00Z","8150":"2013-07-19T00:00:00Z","8151":"2013-07-19T00:00:00Z","8152":"2013-07-18T00:00:00Z","8153":"2013-07-18T00:00:00Z","8154":"2013-07-17T00:00:00Z","8155":"2013-07-17T00:00:00Z","8156":"2013-07-17T00:00:00Z","8157":"2013-07-16T00:00:00Z","8159":"2013-07-15T00:00:00Z","8161":"2013-07-12T00:00:00Z","8162":"2013-07-08T00:00:00Z","8163":"2013-07-08T00:00:00Z","8164":"2013-07-08T00:00:00Z","8165":"2013-07-08T00:00:00Z","8166":"2013-07-08T00:00:00Z","8167":"2013-07-05T00:00:00Z","8168":"2013-07-05T00:00:00Z","8169":"2013-07-05T00:00:00Z","8170":"2013-07-05T00:00:00Z","8171":"2013-07-05T00:00:00Z","8173":"2013-07-05T00:00:00Z","8174":"2013-07-05T00:00:00Z","8175":"2013-07-05T00:00:00Z","8177":"2013-07-05T00:00:00Z","8178":"2013-07-03T00:00:00Z","8180":"2013-07-03T00:00:00Z","8181":"2013-07-03T00:00:00Z","8184":"2013-07-02T00:00:00Z","8187":"2013-07-01T00:00:00Z","8188":"2013-07-01T00:00:00Z","8189":"2013-07-01T00:00:00Z","8190":"2013-07-01T00:00:00Z","8191":"2013-07-01T00:00:00Z","8192":"2013-07-01T00:00:00Z","8195":"2013-06-25T00:00:00Z","8196":"2013-06-24T00:00:00Z","8197":"2013-06-24T00:00:00Z","8199":"2013-06-17T00:00:00Z","8200":"2013-06-17T00:00:00Z","8201":"2013-06-14T00:00:00Z","8202":"2013-06-13T00:00:00Z","8203":"2013-06-13T00:00:00Z","8204":"2013-06-12T00:00:00Z","8205":"2013-06-11T00:00:00Z","8206":"2013-06-11T00:00:00Z","8207":"2013-06-11T00:00:00Z","8208":"2013-06-10T00:00:00Z"},"Amount":{"5":88.19,"8":17.44,"9":19.36,"10":16.52,"11":3.99,"12":15.42,"13":0.25,"14":6.59,"15":2.28,"16":14.98,"17":44.36,"18":7.76,"19":1.75,"20":4.31,"21":2.0,"22":1.65,"23":7.76,"24":26.46,"25":1.0,"26":13.0,"27":6.58,"28":11.08,"29":0.07,"30":68.36,"31":23.12,"32":9.03,"33":5.03,"34":5.85,"35":4.0,"38":34.4,"45":8.63,"46":37.88,"47":26.74,"48":0.99,"49":10.81,"50":30.87,"52":16.18,"53":16.23,"54":176.19,"55":22.49,"56":21.05,"61":5.0,"63":27.43,"64":54.47,"65":37.23,"66":9.37,"67":20.0,"68":110.24,"69":43.29,"70":54.12,"71":8.65,"72":1000.0,"73":44.65,"74":3.0,"77":199.0,"78":5.26,"79":0.18,"80":3.5,"83":17.7,"84":8.0,"85":10.28,"88":0.38,"89":0.5,"90":1.0,"91":5.0,"92":21.8,"95":114.3,"96":2.0,"97":2.0,"98":1.5,"99":65.13,"100":20.77,"101":1.0,"102":0.45,"103":1.5,"104":2.25,"105":2.0,"107":70.12,"108":13.0,"110":12.9,"113":18.21,"114":23.17,"116":12.0,"117":1300.0,"118":23.33,"119":95.0,"120":2.15,"121":10.98,"122":125.0,"123":26.82,"124":2.0,"125":1.0,"126":13.04,"127":15.95,"128":15.42,"129":16.46,"130":7.94,"131":2.0,"132":2.0,"134":10.79,"135":0.98,"136":7.64,"137":12.0,"138":68.36,"141":2.15,"142":95.0,"143":99.2,"144":414.33,"145":0.07,"146":12.29,"147":28.08,"148":1.0,"149":66.42,"150":6.38,"151":88.64,"152":14.32,"153":18.3,"154":17.88,"155":7.76,"156":1.25,"157":20.08,"158":8.63,"161":49.53,"162":17.4,"163":10.97,"164":65.89,"165":31.0,"166":4.32,"167":0.99,"168":40.5,"169":79.38,"171":33.25,"172":10.81,"173":2.24,"174":7.01,"175":2.67,"176":90.84,"177":4.32,"184":16.18,"185":3.0,"186":25.65,"187":16.23,"188":1.62,"189":61.48,"190":23.29,"191":27.21,"192":19.05,"193":3.0,"194":5.7,"195":56.8,"196":28.0,"197":23.21,"198":5.0,"199":3.0,"200":9.0,"201":14.29,"202":34.72,"203":49.53,"204":16.58,"205":8.17,"206":18.78,"207":16.48,"208":1000.0,"209":1.93,"210":1.5,"211":1.5,"212":0.25,"213":0.25,"214":1.43,"216":16.88,"217":19.37,"218":121.87,"219":330.0,"221":16.87,"222":20.89,"223":16.9,"224":54.76,"225":37.34,"226":154.66,"227":14.29,"228":6.38,"229":2.5,"230":63.3,"232":114.3,"233":20.08,"234":4.55,"236":15.0,"237":19.6,"248":15.65,"249":2.0,"252":216.8,"253":4.32,"254":20.63,"255":35.22,"256":37.08,"257":1300.0,"260":99.0,"261":16.36,"262":10.79,"263":15.42,"264":33.04,"265":4.32,"266":4.72,"267":7.0,"268":61.69,"269":14.95,"270":4.0,"271":68.36,"272":9.47,"273":5.5,"274":0.07,"276":70.28,"277":12.09,"278":5.98,"279":2.0,"280":2.25,"282":14.09,"283":6.58,"284":7.46,"287":6.57,"288":8.63,"289":33.01,"290":9.66,"291":18.0,"292":2.55,"293":2.55,"294":11.0,"295":81.9,"296":81.47,"297":78.15,"298":104.54,"299":0.99,"300":4.32,"301":221.54,"302":27.74,"303":10.81,"304":53.48,"305":35.99,"306":28.74,"307":16.18,"309":9.66,"310":958.55,"311":16.23,"313":14.0,"314":84.6,"315":46.14,"316":1.49,"317":66.15,"318":104.91,"323":5.0,"324":31.22,"333":10.95,"334":5.0,"335":6.55,"336":3.49,"337":7.45,"338":3.0,"339":120.0,"340":1000.0,"342":2.5,"343":7.77,"344":3.0,"345":24.35,"346":12.06,"347":4.95,"348":27.08,"349":9.99,"350":8.37,"351":13.85,"352":505.36,"353":13.0,"354":2.25,"355":15.72,"356":736.0,"357":531.31,"358":114.3,"359":13.0,"360":2.55,"361":25.5,"362":23.5,"363":4.19,"364":2.89,"365":23.85,"366":2.0,"367":14.72,"369":8.45,"370":11.84,"371":37.32,"373":264.62,"374":3.72,"375":11.99,"376":21.07,"377":6.48,"378":28.14,"379":47.01,"380":23.61,"381":437.09,"382":44.93,"384":35.0,"385":2.71,"386":165.0,"387":18.41,"389":145.3,"390":34.67,"391":4.32,"392":70.0,"393":98.0,"394":2.71,"395":2.99,"396":14.98,"397":1.08,"398":4.0,"399":100.0,"400":23.35,"401":2.5,"402":276.9,"403":10.79,"404":16.0,"405":2.5,"406":31.32,"407":130.09,"408":307.6,"409":23.98,"410":25.29,"411":3.48,"412":15.42,"413":0.25,"414":1.66,"415":0.5,"416":0.5,"417":14.38,"418":0.05,"419":1.0,"420":400.0,"421":318.09,"422":16.26,"423":4.58,"425":68.21,"426":1.25,"427":2.85,"428":14.18,"429":0.5,"430":4.58,"431":32.69,"432":0.12,"433":2.0,"434":0.75,"435":31.7,"436":17.18,"437":68.36,"438":27.08,"439":0.17,"440":1.5,"441":1.5,"442":0.5,"443":5.07,"444":0.07,"445":19.49,"446":28.14,"451":0.42,"452":14.18,"453":0.25,"455":4.58,"460":31.61,"461":10.86,"462":81.92,"463":23.54,"464":16.0,"465":7.0,"466":4.58,"467":16.46,"468":2.0,"469":2.0,"470":72.63,"471":0.99,"472":21.58,"473":12.98,"474":18.19,"475":8.63,"476":178.5,"477":12.58,"478":17.05,"479":432.45,"481":1.91,"482":130.84,"483":35.62,"484":16.23,"485":3.86,"486":93.66,"487":193.38,"490":58.73,"491":13.22,"492":14.0,"493":3.9,"494":178.6,"495":2.0,"496":71.54,"497":16.8,"500":3.0,"501":200.0,"504":2.5,"507":31.3,"508":70.34,"509":1000.0,"510":18.68,"511":427.78,"512":3.49,"514":4.38,"517":4.13,"518":4.58,"519":10.42,"520":43.03,"521":1.95,"522":1.08,"523":71.4,"524":78.4,"525":212.22,"526":22.04,"527":114.65,"528":0.81,"529":7.65,"530":6.5,"531":58.4,"532":68.5,"533":283.5,"540":19.03,"541":5.4,"542":50.93,"545":13.23,"546":5.4,"547":4.05,"548":4.05,"549":17.72,"551":23.46,"552":17.58,"553":128.82,"554":5.4,"556":22.13,"557":2033.31,"558":4.41,"559":3.2,"560":13.23,"561":66.06,"562":2033.31,"563":9.38,"564":4.58,"565":17.33,"566":53.1,"567":55.44,"568":6.04,"569":101.43,"570":19.2,"571":3.0,"572":15.42,"575":50.93,"576":2.74,"577":14.4,"578":4.87,"579":52.97,"580":110.6,"582":59.78,"583":4.92,"584":19.17,"585":0.07,"586":3.43,"587":17.5,"588":9.87,"589":59.65,"590":11.71,"591":3.45,"592":4.0,"593":765.99,"594":4.95,"595":27.99,"596":28.97,"600":16.16,"601":32.0,"602":8.65,"603":18.98,"604":53.3,"605":6.99,"606":0.99,"607":16.15,"610":20.0,"611":20.0,"612":16.2,"613":36.91,"614":3.77,"615":14.0,"616":4.51,"617":14.35,"618":8.61,"619":48.39,"620":3.15,"621":237.04,"622":16.23,"623":38.75,"624":100.0,"625":26.94,"626":8.58,"627":5.5,"629":24.3,"632":18.67,"633":4.75,"637":225.3,"638":1.99,"639":5.1,"640":16.16,"641":26.93,"643":1000.0,"645":47.03,"647":54.21,"648":25.95,"649":30.3,"650":50.29,"651":4.5,"652":28.0,"654":4.25,"655":4.75,"656":17.56,"657":4.11,"658":142.2,"659":4.75,"660":59.74,"661":250.0,"662":4.79,"663":30.08,"664":10.89,"665":37.74,"666":4.79,"667":6.64,"668":13.73,"669":4.47,"670":3.69,"674":8.99,"675":4.0,"676":10.68,"677":16.16,"678":53.06,"679":4.5,"681":5.27,"687":4.75,"688":21.6,"689":20.0,"691":4.75,"692":13.77,"693":102.0,"694":29.09,"695":31.27,"696":3.5,"697":100.0,"698":2.5,"700":117.8,"701":15.42,"702":28.61,"703":18.85,"704":117.0,"705":37.0,"706":19.47,"708":71.75,"709":2.0,"714":50.76,"715":475.0,"716":0.07,"721":17.51,"722":48.87,"724":114.48,"725":38.5,"726":53.03,"727":4.16,"728":3.71,"729":25.5,"730":6.8,"731":27.63,"732":45.32,"733":48.13,"734":11.36,"735":8.65,"737":135.98,"738":41.93,"739":0.99,"740":8.72,"741":4.22,"742":25.58,"743":8.0,"744":2.99,"745":2.91,"746":9.02,"747":3.24,"748":4.06,"750":5.33,"751":9.02,"752":16.23,"755":59.0,"756":21.06,"758":46.79,"761":1000.0,"764":4.79,"765":101.0,"766":4.97,"767":7.91,"768":3.85,"769":9.02,"770":50.0,"772":41.77,"773":12.52,"776":9.02,"777":11.47,"779":3.0,"781":29.49,"782":4.22,"783":55.65,"784":7.33,"785":19.31,"786":24.36,"787":10.16,"788":3.85,"789":11.88,"790":4.22,"791":11.67,"792":9.02,"793":21.98,"794":27.05,"795":81.11,"798":4.52,"800":12.3,"804":4.52,"805":9.02,"806":37.72,"807":1.81,"808":25.64,"809":11.88,"810":4.22,"811":10.82,"812":15.42,"813":17.3,"816":13.47,"818":475.0,"819":4.25,"820":0.07,"826":3.85,"827":4.22,"828":14.48,"829":3.35,"830":5.67,"831":40.0,"832":12.43,"833":83.0,"834":42.27,"835":245.18,"836":8.65,"837":5.11,"838":13.47,"839":516.15,"840":0.99,"841":4.52,"842":17.76,"844":9.47,"847":10.44,"848":4.52,"849":15.16,"851":5.33,"853":13.35,"854":6.41,"858":16.23,"859":4.79,"860":4.52,"861":17.06,"862":13.05,"863":26.92,"864":14.37,"865":10.25,"866":15.45,"867":5.06,"868":19.5,"872":1000.0,"875":24.78,"876":47.86,"877":13.63,"878":3.92,"879":5.04,"880":19.56,"881":11.02,"882":52.0,"883":4.0,"885":5.0,"886":6.48,"887":110.01,"888":0.0,"891":42.99,"892":8.78,"893":24.98,"896":2.23,"897":34.72,"898":64.94,"899":3.5,"900":5.64,"901":11.02,"902":22.82,"903":25.0,"904":24.46,"905":5.06,"906":126.1,"907":15.42,"908":2.33,"911":6.14,"912":1936.53,"913":223.59,"914":20.75,"916":5.6,"917":4.52,"918":11.9,"919":11.5,"920":0.07,"921":35.16,"922":4.31,"923":475.0,"924":35.73,"925":6.91,"926":4.33,"927":8.39,"928":5.12,"929":75.62,"931":37.21,"937":8.65,"938":24.64,"939":0.99,"940":2.0,"941":4.74,"942":11.02,"944":48.62,"945":7.88,"946":20.0,"947":9.02,"948":8.48,"949":14.12,"950":16.23,"951":2.0,"952":11.36,"953":4.52,"954":9.02,"955":37.88,"956":48.7,"958":4.52,"959":3462.37,"960":1000.0,"961":13.76,"962":9.07,"963":25.98,"964":35.49,"965":171.69,"966":26.63,"967":26.41,"968":20.0,"970":24.32,"972":117.99,"974":29.31,"975":6.7,"976":205.62,"978":12.97,"980":11.36,"986":34.63,"987":15.42,"989":66.04,"990":4.32,"992":6.0,"993":475.0,"994":0.07,"995":32.46,"996":222.15,"997":48.71,"1000":7.06,"1001":37.57,"1003":48.6,"1004":200.0,"1005":10.92,"1006":8.65,"1007":7.07,"1008":191.57,"1009":0.99,"1010":13.7,"1012":30.07,"1013":27.18,"1015":22.01,"1016":40.28,"1017":50.0,"1018":14.27,"1019":3.29,"1022":13.0,"1028":24.16,"1029":11.99,"1030":44.46,"1031":7.06,"1032":35.39,"1033":4.52,"1034":1000.0,"1035":3.5,"1036":147.91,"1037":40.21,"1038":16.23,"1039":624.26,"1040":4.52,"1041":32.13,"1042":2.0,"1043":4.52,"1044":4.52,"1045":31.33,"1046":16.22,"1047":5.4,"1048":33.11,"1049":4.39,"1050":22.01,"1051":11.36,"1052":12.94,"1055":280.0,"1056":11.36,"1057":233.97,"1058":15.4,"1059":100.0,"1060":117.22,"1061":140.0,"1062":108.11,"1063":4.9,"1064":7.73,"1065":20.83,"1066":0.0,"1067":4.52,"1068":25.38,"1069":0.07,"1070":37.56,"1071":35.11,"1072":11.92,"1073":2.91,"1074":12.99,"1077":33.38,"1078":54.12,"1086":1000.0,"1088":27.23,"1089":7.57,"1090":0.99,"1091":20.99,"1092":15.73,"1096":44.81,"1097":29.04,"1099":36.54,"1100":3.34,"1101":8.0,"1102":31.46,"1103":13.02,"1104":11.01,"1105":10.0,"1106":54.97,"1107":1000.0,"1108":13.01,"1109":52.42,"1110":206.03,"1111":72.92,"1112":166.61,"1113":22.76,"1114":17.88,"1115":28.86,"1116":142.86,"1117":19.12,"1118":82.66,"1120":330.73,"1121":37.95,"1126":52.0,"1127":2.91,"1130":56.38,"1131":27.01,"1132":40.84,"1133":29.19,"1134":15.4,"1136":51.1,"1139":10.81,"1140":4.39,"1141":1.05,"1142":120.0,"1145":2.91,"1146":54.12,"1155":30.51,"1156":0.07,"1157":20.09,"1158":163.56,"1159":23.57,"1160":29.78,"1161":11.9,"1162":190.0,"1163":21.36,"1164":7.79,"1166":71.75,"1167":7.89,"1168":11.45,"1169":7.57,"1170":5.06,"1171":165.95,"1172":2.0,"1173":105.14,"1174":8.06,"1175":0.99,"1176":15.65,"1177":8.06,"1179":8.06,"1180":4.52,"1181":23.4,"1182":18.93,"1183":8.06,"1184":44.35,"1185":432.24,"1186":4.32,"1187":1000.0,"1188":10.0,"1191":45.39,"1192":4.32,"1193":10.54,"1194":21.08,"1195":8.81,"1196":12.0,"1197":300.11,"1198":41.4,"1199":52.71,"1200":80.11,"1203":14.34,"1205":8.06,"1210":8.06,"1215":47.0,"1216":15.4,"1217":2144.87,"1218":9.19,"1222":17.25,"1223":28.52,"1224":1.5,"1225":75.16,"1226":2.0,"1227":2.06,"1228":32.0,"1229":27.73,"1230":0.07,"1231":105.0,"1233":14.58,"1234":23.56,"1235":3.78,"1236":7.57,"1237":16.46,"1238":0.99,"1239":21.3,"1240":16.95,"1242":8.57,"1243":12.09,"1244":185.0,"1245":8.06,"1246":72.1,"1248":1000.0,"1250":54.34,"1251":7.0,"1252":47.11,"1253":83.91,"1254":5.35,"1255":26.02,"1256":18.48,"1258":6.86,"1259":348.6,"1260":81.5,"1262":12.0,"1263":80.11,"1264":32.51,"1266":7.92,"1267":12.13,"1272":7.92,"1273":190.0,"1278":7.98,"1279":7.92,"1281":10.0,"1282":27.49,"1283":9.01,"1284":5.31,"1285":54.13,"1286":185.99,"1287":14.3,"1288":25.32,"1289":14.06,"1291":2140.7,"1293":31.16,"1294":11.09,"1295":133.1,"1296":0.02,"1297":32.0,"1298":550.55,"1301":33.02,"1302":1.09,"1303":190.0,"1305":22.31,"1306":16.2,"1307":7.92,"1308":7.57,"1309":49.03,"1310":5.19,"1311":0.99,"1312":23.97,"1314":33.82,"1315":47.0,"1316":4.25,"1317":20.82,"1318":7.92,"1319":17.05,"1320":3.0,"1321":7.92,"1322":12.21,"1323":1000.0,"1324":190.0,"1325":12.0,"1326":7.92,"1328":80.11,"1329":30.59,"1330":32.46,"1331":21.09,"1332":295.18,"1333":11.0,"1334":3.29,"1338":18.38,"1339":10.0,"1340":7.92,"1345":41.59,"1347":28.81,"1348":68.81,"1349":28.79,"1350":7.65,"1351":14.3,"1352":10.0,"1353":62.77,"1356":2130.24,"1357":190.0,"1358":5.4,"1359":11.34,"1361":9.58,"1362":24.44,"1363":25.49,"1364":32.0,"1366":95.0,"1367":4.0,"1368":4.96,"1369":15.25,"1370":27.34,"1371":33.02,"1372":47.0,"1374":7.92,"1375":117.5,"1376":0.99,"1378":6.9,"1380":8.67,"1381":84.89,"1382":12.0,"1383":72.9,"1385":54.0,"1387":8.67,"1388":175.0,"1389":1000.0,"1390":86.77,"1391":32.74,"1392":59.2,"1393":39.06,"1394":33.34,"1395":12.0,"1397":11.0,"1398":80.11,"1402":24.03,"1407":32.99,"1408":12.06,"1409":41.59,"1411":184.02,"1412":3.78,"1413":179.26,"1414":13.22,"1419":32.25,"1420":50.0,"1421":8.16,"1422":95.0,"1423":108.1,"1424":13.25,"1426":14.3,"1427":2.64,"1428":2.11,"1429":2.11,"1430":15.88,"1431":17.06,"1432":16.18,"1433":8.65,"1436":1119.2,"1437":32.0,"1438":38.67,"1439":77.24,"1441":33.02,"1443":10.0,"1444":175.0,"1447":21.65,"1448":43.57,"1449":0.99,"1452":329.3,"1453":8.51,"1454":97.42,"1456":1000.0,"1458":24.33,"1459":60.44,"1460":17.95,"1461":2.6,"1463":12.0,"1464":27.7,"1466":12.98,"1467":78.0,"1471":11.0,"1472":33.04,"1473":12.98,"1475":53.22,"1476":21.65,"1477":43.51,"1479":2.0,"1480":46.16,"1484":8.51,"1486":16.97,"1487":5.61,"1488":150.0,"1489":9.92,"1490":178.61,"1491":14.36,"1492":9.39,"1493":38.74,"1494":3.7,"1495":38.36,"1496":14.3,"1497":98.04,"1501":2050.46,"1502":32.0,"1504":175.0,"1507":33.02,"1508":220.75,"1509":3.24,"1510":28.25,"1511":0.99,"1512":22.76,"1514":36.07,"1517":8.59,"1518":17.88,"1519":41.29,"1520":184.96,"1524":1000.0,"1525":86.55,"1528":11.0,"1531":27.13,"1536":6.0,"1537":235.04,"1538":2034.77,"1540":14.3,"1542":32.0,"1544":21.04,"1546":33.02,"1547":0.99,"1548":56.53,"1550":1000.0,"1555":15.12,"1557":11.0,"1559":37.57,"1563":14.3,"1564":2054.09,"1565":32.0,"1566":33.02,"1569":0.99,"1574":22.8,"1577":1000.0,"1581":11.0,"1584":212.09,"1589":14.3,"1590":2043.62,"1593":3.29,"1597":0.99,"1602":1000.0,"1606":66.05,"1607":7.57,"1615":23.8,"1617":22.88,"1618":38.33,"1619":14.3,"1620":20.93,"1622":6.12,"1623":45.8,"1625":2040.35,"1626":27.46,"1627":348.03,"1630":63.31,"1631":81.72,"1633":0.99,"1634":10.18,"1635":6.06,"1636":10.99,"1637":66.21,"1638":11.9,"1639":16.18,"1640":25.93,"1642":65.32,"1643":5.0,"1647":22.06,"1649":20.73,"1650":8.63,"1651":19.51,"1652":24.31,"1655":57.36,"1658":70.3,"1659":112.17,"1661":45.77,"1664":11.9,"1665":11.9,"1666":17.39,"1667":17.3,"1669":87.09,"1672":21.64,"1673":29.22,"1674":76.83,"1675":12.1,"1676":16.37,"1680":21.64,"1681":14.3,"1682":45.8,"1683":99.48,"1685":2037.27,"1686":18.39,"1687":89.82,"1688":22.95,"1689":7.31,"1691":121.26,"1692":4.32,"1695":3.24,"1696":17.84,"1697":24.62,"1698":23.38,"1699":19.24,"1700":0.99,"1701":10.99,"1702":16.18,"1703":11.9,"1704":5.4,"1705":12.45,"1707":22.01,"1708":75.75,"1709":22.71,"1710":20.57,"1711":22.67,"1712":70.38,"1713":5.0,"1714":330.78,"1716":18.92,"1717":500.0,"1718":22.06,"1721":284.03,"1722":11.49,"1723":112.17,"1724":6.48,"1726":8.64,"1727":35.0,"1728":35.0,"1731":0.97,"1732":215.7,"1733":113.09,"1734":40.68,"1735":4.82,"1736":6.55,"1737":0.09,"1738":3.13,"1739":2.41,"1740":8.31,"1741":3.28,"1742":12.19,"1743":1.95,"1744":50.0,"1745":3.31,"1747":14.56,"1748":27.67,"1749":16.41,"1750":54.28,"1751":24.64,"1752":3.84,"1753":8.08,"1754":1.12,"1755":8.08,"1756":9.14,"1757":7.77,"1758":8.7,"1759":24.64,"1760":90.09,"1761":29.7,"1762":24.0,"1763":35.42,"1764":38.0,"1766":18.22,"1767":0.0,"1770":14.3,"1771":23.43,"1772":45.8,"1773":645.8,"1774":2041.18,"1776":15.55,"1777":21.81,"1778":8.0,"1779":42.0,"1780":131.07,"1781":21.48,"1782":48.71,"1783":631.07,"1784":7.7,"1785":21.0,"1786":46.74,"1787":16.0,"1790":0.99,"1791":17.14,"1792":5.4,"1793":27.27,"1794":146.48,"1795":10.99,"1796":57.09,"1797":8.34,"1798":16.18,"1799":14.71,"1800":19.89,"1801":21.65,"1802":7.5,"1803":20.72,"1804":14.32,"1806":5.0,"1810":500.0,"1811":22.06,"1812":152.07,"1813":1.62,"1814":10.87,"1815":10.0,"1820":15.57,"1821":4.11,"1822":9.4,"1823":30.74,"1824":16.1,"1825":8.92,"1826":40.0,"1827":17.18,"1828":5.87,"1829":64.93,"1830":18.64,"1831":11.0,"1833":114.17,"1834":20.7,"1835":19.43,"1836":32.98,"1837":10.81,"1840":21.64,"1841":21.64,"1842":18.21,"1843":54.61,"1844":15.14,"1845":2.0,"1846":258.1,"1847":140.71,"1848":56.78,"1849":38.95,"1850":17.37,"1851":8.34,"1852":19.89,"1853":120.0,"1854":20.68,"1856":10.59,"1857":84.44,"1859":292.0,"1860":9.0,"1862":31.12,"1864":14.3,"1865":45.32,"1866":27.67,"1867":45.8,"1868":2040.35,"1869":27.48,"1870":7.41,"1871":20.8,"1872":7.95,"1873":49.0,"1874":9.0,"1876":41.0,"1877":130.77,"1878":40.0,"1879":3.91,"1880":20.65,"1881":36.06,"1882":2.5,"1884":220.0,"1885":7.99,"1886":13.63,"1888":9.42,"1889":39.32,"1890":0.99,"1891":7.91,"1892":4.65,"1893":7.38,"1894":3.62,"1895":167.33,"1896":6.03,"1897":4.98,"1898":3.95,"1899":7.19,"1900":3.89,"1901":7.91,"1902":8.56,"1903":51.13,"1904":29.82,"1905":10.99,"1906":12.03,"1907":42.0,"1908":16.29,"1909":16.46,"1910":61.98,"1911":36.19,"1912":14.21,"1913":39.74,"1914":6.92,"1915":15.94,"1916":42.08,"1920":22.06,"1921":2.67,"1922":4.1,"1923":26.72,"1924":42.34,"1925":5.5,"1926":5.0,"1930":500.0,"1931":30.85,"1932":6.6,"1933":52.35,"1934":18.0,"1935":7.0,"1936":15.66,"1937":5.5,"1938":18.67,"1939":5.5,"1940":31.23,"1941":1.1,"1942":1.97,"1943":6.61,"1944":17.74,"1945":27.0,"1946":17.72,"1947":3.2,"1948":18.0,"1949":22.57,"1950":8.75,"1951":7.67,"1952":11.96,"1953":37.76,"1954":3.24,"1955":14.69,"1956":19.0,"1957":19.51,"1958":11.86,"1959":15.64,"1960":5.0,"1961":16.89,"1962":47.98,"1964":12.47,"1965":33.9,"1966":2.17,"1967":70.3,"1968":40.0,"1969":149.0,"1970":848.85,"1972":64.51,"1973":1.62,"1974":112.43,"1975":157.69,"1976":19.14,"1977":79.68,"1978":54.56,"1979":125.16,"1980":8.41,"1982":28.13,"1983":15.14,"1985":15.59,"1987":27.05,"1988":7.08,"1989":28.13,"1990":8.0,"1991":21.64,"1992":19.64,"1993":10.95,"1994":10.0,"1995":20.07,"1996":36.31,"1998":9.73,"1999":2.17,"2003":2036.79,"2004":32.66,"2005":8.66,"2006":45.44,"2007":206.8,"2008":14.3,"2009":2407.88,"2010":44.72,"2011":22.75,"2012":12.68,"2013":36.52,"2014":9.41,"2015":48.69,"2016":24.56,"2017":12.89,"2018":40.0,"2019":130.77,"2020":21.64,"2021":8.96,"2022":32.04,"2023":32.01,"2025":171.95,"2026":377.79,"2027":174.19,"2028":13.94,"2029":27.2,"2030":5.09,"2031":49.24,"2032":29.07,"2033":55.06,"2034":0.99,"2035":55.75,"2036":26.61,"2037":33.0,"2038":31.53,"2039":10.99,"2040":11.8,"2042":16.46,"2043":34.49,"2044":43.13,"2045":53.35,"2046":2.72,"2047":19.01,"2048":510.0,"2050":31.08,"2051":16.33,"2052":36.78,"2053":21.34,"2054":12.99,"2055":62.76,"2056":17.02,"2057":1.49,"2058":5.0,"2060":12.65,"2061":23.91,"2063":12.22,"2064":16.26,"2065":157.69,"2067":125.1,"2068":108.03,"2071":8.14,"2072":108.24,"2073":26.09,"2074":40.0,"2075":15.0,"2077":60.81,"2079":8.19,"2080":4.95,"2081":5.04,"2082":65.3,"2083":12.4,"2084":12.03,"2085":6.61,"2086":11.68,"2087":8.25,"2088":82.02,"2089":66.85,"2090":4.5,"2091":132.11,"2093":11.0,"2094":121.1,"2095":40.0,"2096":16.82,"2097":1100.95,"2098":1075.23,"2099":198.17,"2100":6.59,"2101":13.2,"2102":4.39,"2103":10.12,"2104":4.95,"2105":12.51,"2106":3.69,"2107":8.0,"2108":5.78,"2109":9.67,"2110":100.0,"2111":3.62,"2112":16.5,"2113":4.39,"2114":2.7,"2115":4.28,"2116":12.51,"2117":9.67,"2118":2.7,"2119":14.3,"2120":9.67,"2121":2.7,"2122":102.39,"2123":1599.0,"2124":9.0,"2125":11.68,"2126":10.43,"2127":12.98,"2128":23.18,"2129":16.8,"2130":8.24,"2131":8.8,"2132":12.85,"2133":21.79,"2134":25.6,"2135":7.68,"2136":22.5,"2137":40.0,"2138":19.81,"2139":20.37,"2140":36.31,"2141":7.72,"2142":14.5,"2143":7.0,"2144":12.53,"2147":3.69,"2148":2.4,"2149":92.17,"2150":4.28,"2151":8.25,"2152":21.99,"2153":4.26,"2154":7.97,"2155":7.42,"2156":17.3,"2157":3.85,"2158":14.47,"2159":2.7,"2160":0.49,"2161":9.83,"2162":8.25,"2163":44.57,"2164":29.72,"2165":4.28,"2166":10.29,"2167":23.86,"2168":15.0,"2169":4.39,"2170":4.39,"2171":12.79,"2172":14.49,"2173":4.39,"2174":3.51,"2175":4.83,"2176":5.49,"2177":62.59,"2178":48.05,"2179":8.03,"2180":6.57,"2181":4.38,"2182":1338.71,"2183":16.46,"2184":16.5,"2185":3.69,"2186":8.96,"2188":4.11,"2189":104.13,"2190":22.13,"2192":10.5,"2193":1.79,"2194":11.43,"2195":8.58,"2196":5.0,"2197":47.79,"2199":29.99,"2200":6.59,"2201":146.08,"2202":6.98,"2203":10.0,"2204":18.71,"2205":18.01,"2206":18.16,"2207":15.57,"2208":44.47,"2209":16.19,"2210":11.36,"2211":1050.33,"2212":10.34,"2213":25.5,"2214":1.89,"2215":8.59,"2216":15.57,"2217":1409.85,"2218":5.41,"2219":9.07,"2220":8.82,"2221":8.94,"2222":1.89,"2223":6.47,"2224":14.13,"2225":10.97,"2226":13.0,"2227":58.99,"2228":4.28,"2229":3.62,"2230":2.53,"2232":5.25,"2233":112.59,"2234":32.48,"2235":34.53,"2236":3.0,"2237":46.05,"2239":60.81,"2240":27.76,"2241":21.62,"2242":16.95,"2243":160.31,"2244":17.99,"2245":3.24,"2246":8.0,"2247":5.0,"2248":11.05,"2249":4.68,"2250":34.47,"2252":11.31,"2253":4.02,"2254":3.29,"2255":8.8,"2256":11.43,"2257":5.0,"2259":40.0,"2260":5.0,"2261":18.17,"2262":5.0,"2263":2.52,"2264":10.76,"2265":8.47,"2266":9.9,"2267":16.5,"2268":4.39,"2269":11.83,"2270":9.24,"2271":19.67,"2272":21.38,"2273":9.8,"2274":9.5,"2275":3.84,"2276":72.56,"2278":3.62,"2279":3.84,"2280":14.3,"2281":53.36,"2282":53.5,"2283":102.39,"2284":82.07,"2285":8.07,"2286":24.77,"2287":31.24,"2288":15.46,"2289":40.01,"2290":1599.0,"2291":85.02,"2292":11.42,"2295":5.54,"2296":6.99,"2297":14.77,"2298":17.6,"2299":49.99,"2300":10.17,"2301":34.12,"2302":8.63,"2303":117.0,"2304":202.06,"2305":16.14,"2306":4.95,"2307":5.45,"2309":18.69,"2311":46.64,"2312":5.49,"2313":14.5,"2314":16.46,"2316":33.02,"2317":1.79,"2318":41.04,"2320":100.0,"2321":20.35,"2322":11.17,"2323":2.7,"2324":12.94,"2326":30.59,"2327":8.75,"2328":14.51,"2329":4.39,"2330":8.5,"2332":23.27,"2333":32.49,"2334":6.03,"2335":6.49,"2337":6.18,"2338":5.83,"2341":4.39,"2342":107.9,"2343":15.2,"2344":33.59,"2345":19.8,"2346":33.01,"2347":5.49,"2348":13.5,"2349":198.09,"2350":13.5,"2351":6.33,"2352":60.81,"2353":2.25,"2354":23.82,"2355":10.02,"2356":3.0,"2359":16.32,"2360":1.98,"2361":167.28,"2364":16.82,"2365":2.04,"2366":7.04,"2367":40.0,"2368":6.04,"2369":10.01,"2373":23.7,"2374":4.84,"2375":16.5,"2376":5.99,"2377":44.9,"2378":27.33,"2379":21.3,"2380":22.01,"2381":14.3,"2382":102.39,"2383":14.19,"2384":15.8,"2385":9.72,"2386":6.08,"2387":9.84,"2388":37.73,"2389":10.17,"2390":3.98,"2391":1599.0,"2392":12.22,"2393":1.64,"2394":1.98,"2397":332.0,"2398":8.03,"2399":4.95,"2400":8.84,"2401":38.56,"2402":12.38,"2403":11.36,"2404":10.8,"2405":2.74,"2406":32.96,"2407":9.81,"2408":10.17,"2409":4.84,"2410":6.59,"2412":3.62,"2413":63.42,"2414":3.85,"2415":6.89,"2417":5.27,"2418":1.34,"2419":6.59,"2420":8.71,"2421":3.25,"2422":8.4,"2423":19.82,"2424":23.55,"2426":100.0,"2427":31.24,"2428":10.28,"2429":8.68,"2430":5.49,"2431":14.5,"2432":5.54,"2433":61.75,"2434":52.09,"2435":8.7,"2436":33.02,"2437":6.61,"2438":30.0,"2439":12.87,"2440":6.04,"2441":4.55,"2442":27.49,"2443":106.54,"2444":7.78,"2445":9.35,"2446":48.09,"2447":8.0,"2448":60.75,"2449":18.85,"2450":14.22,"2452":3.02,"2454":10.75,"2455":5.0,"2456":10.81,"2457":31.53,"2458":17.07,"2459":4.32,"2460":13.96,"2461":37.87,"2462":4.39,"2463":63.03,"2464":35.0,"2465":5.86,"2466":22.22,"2467":54.25,"2468":21.59,"2469":3.78,"2470":4.98,"2471":19.98,"2472":5.67,"2473":23.27,"2474":74.55,"2475":5.11,"2476":8.0,"2477":11.41,"2478":4.33,"2479":40.93,"2480":180.73,"2481":14.56,"2482":5.11,"2483":24.6,"2484":12.81,"2485":60.81,"2487":13.88,"2488":8.75,"2490":2.8,"2491":14.79,"2492":10.8,"2493":10.86,"2494":173.3,"2496":29.69,"2497":25.81,"2498":12.0,"2499":47.15,"2500":27.24,"2501":40.0,"2502":9.0,"2503":2.71,"2504":17.0,"2505":4.21,"2506":43.42,"2507":12.0,"2508":7.19,"2509":3.73,"2510":6.44,"2511":4.33,"2512":4.55,"2513":17.0,"2514":14.56,"2515":14.78,"2516":85.73,"2517":16.12,"2518":12.1,"2519":17.8,"2520":1599.0,"2521":102.39,"2522":550.04,"2523":6.14,"2524":10.22,"2525":37.88,"2526":9.08,"2527":10.81,"2528":16.5,"2529":7.57,"2530":134.21,"2531":11.04,"2532":48.96,"2533":10.39,"2534":134.21,"2536":56.17,"2537":6.04,"2539":25.97,"2540":5.4,"2541":12.75,"2543":14.5,"2544":5.49,"2545":82.23,"2546":19.9,"2547":2.3,"2548":33.02,"2549":5.33,"2550":12.45,"2551":17.07,"2552":95.0,"2553":14.05,"2554":24.59,"2555":1.4,"2556":15.69,"2557":2.8,"2558":14.38,"2559":20.58,"2562":27.98,"2563":7.57,"2564":14.05,"2565":6.48,"2568":12.38,"2569":9.26,"2570":9.9,"2571":14.0,"2572":18.3,"2573":15.28,"2574":9.39,"2575":16.3,"2576":19.78,"2577":8.95,"2578":2.76,"2579":2.14,"2580":14.73,"2581":17.19,"2582":8.77,"2583":6.48,"2584":6.04,"2585":15.86,"2588":74.38,"2589":60.81,"2592":6.04,"2593":13.31,"2594":113.55,"2595":1.08,"2596":16.0,"2597":31.37,"2598":10.16,"2599":40.0,"2600":10.75,"2601":9.18,"2602":2.8,"2603":40.05,"2604":17.0,"2606":7.88,"2607":7.57,"2608":40.9,"2609":6.04,"2610":12.1,"2611":9.32,"2612":1599.0,"2613":64.96,"2614":8.65,"2615":61.33,"2616":7.88,"2617":17.56,"2618":2.8,"2619":5.4,"2620":37.88,"2621":12.97,"2622":88.0,"2623":16.5,"2624":8.43,"2625":25.51,"2626":2.17,"2627":5.0,"2628":55.17,"2629":25.01,"2630":36.0,"2631":7.31,"2632":14.95,"2633":16.73,"2634":11.64,"2635":29.83,"2636":383.97,"2637":15.91,"2638":13.2,"2639":29.21,"2640":42.72,"2641":5.71,"2642":3.98,"2643":15.96,"2644":9.4,"2645":13.32,"2646":38.1,"2647":3.94,"2648":27.98,"2649":6.04,"2651":14.28,"2652":2.22,"2654":40.0,"2655":17.23,"2656":9.72,"2657":7.57,"2658":4.68,"2659":62.4,"2660":2.22,"2661":27.47,"2662":21.11,"2663":10.16,"2664":19.51,"2665":9.66,"2666":15.32,"2667":13.24,"2668":43.23,"2669":6.04,"2670":5.49,"2671":10.38,"2672":14.5,"2673":17.3,"2674":500.0,"2676":9.45,"2677":33.02,"2679":26.12,"2680":5.07,"2682":2.8,"2683":1.09,"2684":1.09,"2685":59.95,"2686":11.85,"2688":46.23,"2689":40.0,"2690":51.0,"2691":53.0,"2692":5.14,"2693":7.57,"2694":18.07,"2695":5.33,"2696":9.0,"2697":4.28,"2698":11.0,"2699":10.99,"2700":27.88,"2701":57.04,"2702":9.9,"2703":4.0,"2704":16.45,"2705":9.3,"2706":14.35,"2707":17.71,"2708":16.04,"2712":21.05,"2713":32.44,"2714":10.99,"2715":21.16,"2717":60.83,"2718":271.4,"2719":5.57,"2720":10.0,"2721":9.0,"2722":10.81,"2723":2.8,"2724":14.3,"2725":50.0,"2726":10.6,"2727":14.9,"2728":181.75,"2729":9.99,"2731":19.69,"2732":16.22,"2733":17.63,"2734":40.0,"2735":1599.0,"2736":7.87,"2737":86.59,"2738":2.49,"2739":4.31,"2740":12.1,"2741":64.96,"2742":8.5,"2743":5.99,"2744":75.75,"2745":8.11,"2746":16.23,"2747":198.39,"2748":6.04,"2750":75.48,"2751":17.06,"2752":16.5,"2753":3.98,"2754":3.09,"2755":16.08,"2757":114.42,"2758":3.0,"2759":3.75,"2760":102.62,"2761":8.68,"2762":2.65,"2763":14.5,"2764":5.49,"2775":33.02,"2776":23.61,"2777":68.38,"2778":15.0,"2779":18.37,"2781":544.62,"2783":4.21,"2784":3.66,"2785":4.54,"2786":41.75,"2787":9.9,"2788":32.45,"2789":114.48,"2790":6.15,"2791":23.75,"2792":1.62,"2793":33.83,"2794":60.8,"2797":4.27,"2798":38.51,"2799":14.72,"2801":9.88,"2802":35.7,"2803":40.0,"2804":6.76,"2805":111.51,"2806":14.4,"2807":19.52,"2808":14.08,"2809":164.48,"2810":6.04,"2811":6.04,"2812":3.85,"2813":15.01,"2814":27.81,"2815":12.1,"2816":12.1,"2817":7.23,"2818":64.96,"2819":11.82,"2820":1599.0,"2821":1.89,"2822":8.03,"2823":6.89,"2825":16.5,"2826":4.39,"2827":60.07,"2828":12.77,"2829":23.8,"2830":4.7,"2831":12.02,"2832":16.68,"2833":6.04,"2835":4.68,"2836":10.81,"2837":6.04,"2838":3.79,"2839":14.5,"2840":26.95,"2841":8.38,"2842":10.0,"2843":9.46,"2844":11.88,"2846":5.0,"2847":5.0,"2848":11.9,"2849":49.5,"2850":20.88,"2851":5.49,"2852":11.0,"2853":33.02,"2854":220.2,"2855":29.99,"2856":40.14,"2857":19.78,"2858":25.29,"2861":9.89,"2862":2.19,"2863":2.39,"2864":44.03,"2865":18.41,"2866":6.79,"2867":5.79,"2868":13.09,"2869":23.12,"2870":7.72,"2871":1.89,"2872":4.37,"2873":36.01,"2874":9.9,"2875":11.0,"2876":3.29,"2877":4.88,"2878":5.05,"2879":2.59,"2880":7.72,"2881":30.22,"2882":21.99,"2883":6.04,"2884":10.64,"2885":23.58,"2886":8.24,"2887":8.42,"2888":11.0,"2889":6.88,"2890":3.62,"2891":106.43,"2893":12.19,"2894":3.29,"2895":6.59,"2896":30.0,"2898":60.8,"2899":61.6,"2900":90.23,"2901":6.91,"2902":5.4,"2903":13.56,"2904":39.0,"2905":21.47,"2906":26.42,"2907":23.81,"2908":7.04,"2909":17.59,"2910":9.44,"2912":40.0,"2913":11.45,"2914":3.07,"2915":57.85,"2916":3.07,"2917":64.99,"2918":8.03,"2919":5.79,"2920":10.17,"2921":8.81,"2922":3.19,"2923":22.45,"2924":12.1,"2925":8.25,"2926":442.0,"2927":2.7,"2928":3.29,"2929":102.39,"2930":14.95,"2931":218.36,"2932":11.03,"2933":7.7,"2934":13.38,"2935":7.15,"2936":13.5,"2937":25.0,"2938":18.85,"2939":155.9,"2940":6.5,"2941":29.15,"2942":16.5,"2943":53.95,"2944":1599.0,"2945":1.79,"2946":29.27,"2947":3.62,"2948":7.38,"2949":3.48,"2950":4.39,"2951":7.43,"2955":3.92,"2956":140.93,"2957":1.89,"2958":16.28,"2959":19.04,"2960":6.5,"2961":30.51,"2962":6.83,"2963":53.04,"2964":186.34,"2965":7.43,"2966":6.12,"2967":2.59,"2968":18.48,"2969":51.0,"2970":11.75,"2972":10.99,"2973":14.5,"2974":7.72,"2975":26.3,"2976":2.7,"2977":9.99,"2978":2.08,"2980":8.03,"2981":4.93,"2983":19.52,"2984":39.85,"2985":2.74,"2986":9.9,"2988":3.29,"2989":7.41,"2990":11.0,"2991":15.0,"2992":274.15,"2993":113.54,"2994":294.15,"2995":20.7,"2996":11.0,"2997":7.7,"2998":15.21,"2999":29.25,"3000":48.0,"3001":18.0,"3002":4.93,"3003":7.02,"3004":15.17,"3006":60.8,"3008":152.28,"3009":12.0,"3010":68.3,"3011":274.15,"3012":48.33,"3015":6.14,"3016":15.74,"3017":10.7,"3018":126.2,"3020":23.8,"3021":40.0,"3022":1.89,"3023":8.0,"3024":41.31,"3025":8.0,"3026":8.37,"3027":11.9,"3028":10.99,"3029":12.1,"3030":4.39,"3031":7.43,"3032":102.39,"3033":7.43,"3034":3.07,"3035":7.43,"3036":1599.0,"3037":4.4,"3038":66.42,"3039":5.28,"3040":7.68,"3043":6.04,"3044":66.05,"3045":33.02,"3046":19.76,"3047":5.7,"3048":52.03,"3049":170.04,"3050":4.39,"3051":9.18,"3052":17.88,"3054":29.47,"3055":2.86,"3056":566.26,"3060":11.92,"3062":16.23,"3063":47.4,"3064":8.78,"3065":8.99,"3066":18.4,"3067":6.28,"3068":9.35,"3069":15.4,"3070":16.51,"3072":14.5,"3073":5.49,"3074":23.43,"3075":42.69,"3076":9.99,"3077":72.93,"3078":4.9,"3081":6.96,"3083":3.28,"3084":11.44,"3085":9.9,"3086":13.5,"3087":1.69,"3088":7.43,"3089":5.49,"3090":8.31,"3091":0.99,"3092":3.29,"3093":11.86,"3094":15.35,"3095":26.94,"3096":1.72,"3097":5.93,"3098":10.36,"3099":46.33,"3100":53.84,"3101":209.48,"3103":11.0,"3107":11.0,"3108":60.8,"3111":11.04,"3114":2.5,"3115":60.0,"3116":2.5,"3119":100.6,"3120":4.32,"3121":2.7,"3122":8.45,"3123":74.71,"3124":16.89,"3125":10.22,"3126":40.0,"3127":93.77,"3128":12.08,"3129":11.88,"3130":12.1,"3132":1599.0,"3133":7.73,"3134":114.35,"3135":49.61,"3136":102.39,"3138":33.74,"3139":10.31,"3140":34.77,"3141":16.63,"3142":37.95,"3143":20.27,"3144":47.64,"3145":15.03,"3146":4.45,"3147":13.5,"3148":19.75,"3149":22.0,"3150":11.98,"3151":39.98,"3152":48.0,"3153":12.65,"3154":89.95,"3155":13.8,"3157":12.5,"3158":113.36,"3159":15.11,"3160":25.83,"3161":16.88,"3162":9.04,"3163":13.5,"3164":63.25,"3165":7.43,"3167":2.59,"3168":5.0,"3169":52.0,"3170":3.07,"3171":3.59,"3172":199.2,"3174":8.8,"3175":6.03,"3177":38.64,"3179":7.68,"3180":7.9,"3181":4.23,"3182":52.15,"3183":11.42,"3184":5.33,"3185":3.07,"3186":105.53,"3187":11.0,"3188":14.5,"3189":17.98,"3190":11.2,"3191":5.49,"3192":19.19,"3193":5.4,"3194":10.24,"3195":9.99,"3197":49.33,"3198":17.07,"3199":57.53,"3200":6.04,"3201":5.26,"3202":456.1,"3203":8.48,"3204":49.26,"3205":3.07,"3208":100.03,"3209":132.12,"3210":69.0,"3211":7.28,"3212":9.9,"3213":165.15,"3214":20.37,"3215":4.37,"3216":9.48,"3217":6.69,"3218":28.02,"3219":16.0,"3220":2.65,"3221":3.8,"3222":11.82,"3223":10.98,"3224":3.8,"3225":11.0,"3226":93.96,"3227":7.43,"3228":3.73,"3229":5.74,"3231":11.0,"3232":60.01,"3233":12.1,"3236":12.39,"3237":5.48,"3238":40.0,"3239":5.89,"3240":5.95,"3241":3.07,"3242":131.02,"3243":3.07,"3244":9.72,"3245":6.88,"3246":2.63,"3247":4.95,"3248":6.88,"3249":1.89,"3250":7.43,"3251":16.19,"3252":12.1,"3253":53.95,"3254":10.42,"3255":91.01,"3256":6.03,"3257":11.8,"3258":23.25,"3259":8.24,"3260":6.9,"3261":5.73,"3262":19.55,"3263":51.45,"3264":3.85,"3265":11.0,"3266":95.0,"3267":9.95,"3268":22.83,"3271":6.88,"3272":6.28,"3273":6.93,"3274":5.89,"3275":9.9,"3276":3.25,"3278":50.0,"3279":46.21,"3280":54.73,"3281":3.53,"3282":25.31,"3283":21.01,"3284":2.65,"3285":3.59,"3286":9.99,"3287":5.49,"3289":165.15,"3290":7.14,"3291":3.15,"3292":6.98,"3293":2.39,"3294":68.59,"3295":1.26,"3296":32.84,"3297":6.69,"3298":10.33,"3299":10.35,"3300":7.43,"3301":7.71,"3302":47.84,"3303":5.54,"3304":7.04,"3305":1.75,"3306":7.43,"3307":9.9,"3308":15.14,"3310":44.99,"3311":54.46,"3312":14.29,"3313":13.64,"3314":2.25,"3315":80.0,"3316":2.5,"3317":38.51,"3318":16.21,"3319":16.15,"3320":20.24,"3321":1.29,"3322":11.0,"3323":8.4,"3324":45.87,"3325":16.21,"3326":40.5,"3327":43.16,"3328":8.4,"3329":19.0,"3330":113.71,"3331":56.71,"3332":11.0,"3334":1599.0,"3335":1599.0,"3336":111.3,"3338":62.56,"3339":7.43,"3340":7.35,"3341":3.06,"3342":4.08,"3343":8.88,"3344":3.06,"3345":5.98,"3347":20.89,"3348":40.0,"3349":10.48,"3350":54.08,"3351":23.11,"3352":18.27,"3353":15.87,"3354":16.5,"3355":18.14,"3356":800.0,"3357":12.1,"3358":53.95,"3361":7.43,"3362":5.73,"3364":70.0,"3365":2.39,"3367":20.21,"3368":157.34,"3369":39.64,"3370":13.07,"3371":7.7,"3372":10.89,"3373":19.36,"3374":27.51,"3375":4.21,"3376":7.87,"3377":16.49,"3378":13.81,"3379":5.68,"3380":3.25,"3381":59.86,"3382":9.89,"3383":7.86,"3384":8.03,"3387":13.29,"3388":194.62,"3389":5.49,"3390":4.39,"3391":9.99,"3393":62.57,"3394":104.84,"3395":8.8,"3396":32.51,"3397":6.48,"3398":10.5,"3399":3.19,"3400":14.3,"3401":3.25,"3402":7.43,"3403":3.47,"3404":3.25,"3405":14.3,"3406":7.61,"3407":2.96,"3409":15.46,"3410":9.9,"3411":11.55,"3412":15.46,"3413":7.57,"3414":36.61,"3415":3.25,"3416":18.77,"3417":31.95,"3419":18.71,"3420":55.36,"3421":51.47,"3422":3.49,"3423":13.21,"3424":11.46,"3425":3.25,"3426":93.96,"3427":8.88,"3428":62.61,"3431":6.72,"3432":4.35,"3434":40.0,"3439":18.4,"3440":71.02,"3441":6.62,"3442":19.81,"3443":19.91,"3444":3.62,"3445":53.95,"3446":1.0,"3447":20.37,"3448":5.51,"3449":4.0,"3450":3.62,"3452":53.95,"3453":11.97,"3454":12.1,"3455":7.71,"3459":7.43,"3462":59.4,"3463":218.04,"3464":5.91,"3465":8.1,"3466":23.97,"3467":10.85,"3468":20.0,"3469":6.77,"3470":13.6,"3471":6.13,"3472":7.7,"3473":8.88,"3474":9.9,"3475":12.09,"3476":94.67,"3477":24.99,"3478":2.5,"3479":39.08,"3480":5.0,"3481":9.99,"3482":15.72,"3483":6.47,"3484":3.98,"3486":5.49,"3487":41.66,"3488":49.67,"3489":72.44,"3490":3.62,"3491":7.43,"3493":27.0,"3494":31.9,"3495":34.82,"3496":9.69,"3497":17.03,"3498":19.6,"3499":69.4,"3500":57.4,"3501":18.4,"3502":44.88,"3503":9.9,"3504":205.25,"3505":4.14,"3506":3.73,"3507":3.95,"3508":4.48,"3509":8.25,"3510":8.03,"3511":16.05,"3512":29.33,"3513":13.7,"3515":43.93,"3516":22.28,"3517":67.32,"3518":31.01,"3519":18.55,"3520":62.61,"3521":22.09,"3522":11.37,"3523":17.33,"3525":8.88,"3526":242.58,"3528":3.18,"3529":12.1,"3530":40.0,"3531":47.21,"3532":14.78,"3533":2.7,"3534":9.92,"3535":1.89,"3536":7.92,"3537":24.8,"3538":94.32,"3539":78.51,"3540":12.1,"3541":53.95,"3542":16.11,"3543":265.0,"3544":7.43,"3546":7.7,"3547":5.1,"3548":10.09,"3550":1.6,"3552":36.64,"3553":6.11,"3554":84.09,"3555":43.53,"3556":27.6,"3557":3.51,"3558":7.71,"3559":5.14,"3560":8.35,"3561":5.49,"3562":31.98,"3563":8.88,"3564":3.85,"3565":30.48,"3566":9.99,"3567":9.9,"3568":16.12,"3570":41.82,"3571":2.59,"3572":3.35,"3573":22.43,"3574":16.0,"3575":23.4,"3576":6.78,"3577":23.97,"3578":23.7,"3579":16.49,"3580":10.32,"3581":29.4,"3582":3.78,"3583":18.89,"3584":50.55,"3585":2.41,"3586":17.77,"3587":9.9,"3588":14.0,"3589":3.35,"3590":20.0,"3591":4.79,"3592":8.8,"3593":6.29,"3597":6.38,"3601":61.65,"3602":32.56,"3603":12.98,"3604":15.98,"3605":32.6,"3606":53.95,"3607":3.27,"3608":26.42,"3609":4.39,"3610":2.59,"3611":16.0,"3612":34.94,"3613":7.71,"3614":16.49,"3615":3.62,"3616":8.88,"3617":11.69,"3618":62.61,"3621":7.16,"3622":3.19,"3623":9.73,"3624":7.7,"3626":40.0,"3627":4.23,"3628":1.0,"3629":2.91,"3630":17.6,"3631":4.39,"3632":9.58,"3633":19.81,"3634":65.99,"3635":53.95,"3636":3.25,"3637":2.48,"3638":8.8,"3639":4.07,"3640":9.99,"3641":5.87,"3642":5.49,"3643":81.44,"3644":7.43,"3646":6.28,"3647":14.82,"3648":18.77,"3649":1.87,"3650":5.49,"3651":66.57,"3652":1.87,"3653":49.82,"3654":6.13,"3655":55.4,"3656":3.96,"3657":12.5,"3658":9.87,"3659":21.67,"3660":23.3,"3661":4.25,"3662":7.7,"3663":8.88,"3664":5.29,"3665":7.11,"3666":8.26,"3668":33.53,"3669":30.39,"3670":26.23,"3671":33.15,"3672":9.9,"3673":12.64,"3674":34.73,"3675":8.8,"3676":3.25,"3677":100.0,"3678":1.89,"3679":4.39,"3680":62.61,"3681":139.98,"3682":9.98,"3683":51.76,"3689":12.35,"3690":1.89,"3691":17.03,"3692":9.0,"3693":22.01,"3694":7.03,"3695":5.49,"3696":7.56,"3698":40.0,"3699":9.83,"3700":6.18,"3701":52.45,"3702":10.79,"3703":3.85,"3704":10.79,"3705":4.61,"3706":10.13,"3707":5.27,"3708":15.39,"3709":12.1,"3710":9.9,"3712":53.95,"3713":32.18,"3714":10.86,"3715":7.98,"3716":36.64,"3717":40.2,"3718":7.98,"3719":35.28,"3720":5.48,"3721":8.88,"3722":11.05,"3723":8.88,"3724":3.25,"3725":4.39,"3726":4.11,"3727":5.84,"3728":11.0,"3729":20.38,"3730":21.5,"3731":278.7,"3734":10.47,"3735":29.43,"3736":16.73,"3737":5.49,"3738":9.99,"3739":20.0,"3740":11.0,"3741":11.68,"3742":9.8,"3743":8.03,"3744":22.02,"3745":11.65,"3746":8.88,"3747":5.1,"3748":41.19,"3749":9.9,"3750":7.74,"3751":6.91,"3752":2.74,"3753":7.98,"3754":8.42,"3755":9.89,"3756":7.98,"3757":63.05,"3758":15.96,"3759":16.76,"3760":1.89,"3761":6.18,"3762":0.98,"3763":7.61,"3764":5.83,"3766":13.18,"3767":96.85,"3768":12.93,"3769":4.05,"3770":20.84,"3772":62.61,"3774":10.88,"3775":37.52,"3776":6.0,"3777":13.47,"3778":31.1,"3779":2.07,"3780":8.88,"3781":6.4,"3783":40.0,"3784":67.56,"3785":9.03,"3786":15.25,"3787":17.37,"3788":1.68,"3789":12.1,"3793":66.59,"3795":53.95,"3796":11.87,"3797":17.89,"3798":29.18,"3800":58.94,"3801":9.28,"3802":7.35,"3803":22.62,"3804":3.25,"3805":15.4,"3806":11.78,"3807":3.25,"3808":6.98,"3809":3.25,"3810":4.4,"3811":51.42,"3812":34.29,"3813":13.2,"3814":2.49,"3815":18.69,"3816":23.88,"3817":10.89,"3818":13.2,"3819":37.16,"3820":9.99,"3821":3.25,"3822":9.51,"3824":5.49,"3825":8.8,"3826":23.63,"3827":9.44,"3828":8.75,"3829":3.25,"3830":12.98,"3833":71.89,"3834":4.4,"3835":6.94,"3836":7.7,"3837":27.39,"3838":8.8,"3839":5.9,"3840":29.17,"3841":5.99,"3842":44.99,"3843":6.99,"3844":4.11,"3845":9.17,"3846":239.6,"3847":9.9,"3848":13.31,"3849":58.04,"3850":87.97,"3851":47.93,"3852":5.15,"3853":84.99,"3854":22.6,"3855":27.98,"3856":55.41,"3859":100.0,"3860":18.74,"3861":78.06,"3865":128.02,"3866":50.0,"3867":40.0,"3868":8.81,"3869":12.07,"3870":45.39,"3871":14.85,"3872":27.28,"3873":12.1,"3874":53.95,"3875":93.06,"3878":14.59,"3879":48.52,"3880":3.3,"3883":192.3,"3884":64.34,"3885":8.12,"3886":6.78,"3887":29.98,"3888":15.16,"3889":19.62,"3890":21.21,"3891":7.31,"3892":16.0,"3893":10.74,"3894":16.78,"3895":47.66,"3896":8.39,"3897":28.12,"3898":9.73,"3899":16.23,"3900":9.99,"3901":5.49,"3902":4.39,"3903":51.09,"3904":17.0,"3905":20.0,"3906":4.8,"3907":5.99,"3908":9.99,"3909":45.0,"3910":9.4,"3912":16.5,"3913":46.53,"3914":31.38,"3915":40.58,"3917":2.05,"3919":21.65,"3920":3.7,"3921":125.24,"3922":10.58,"3923":7.43,"3924":3.25,"3925":9.9,"3926":76.62,"3927":9.9,"3929":150.0,"3930":109.8,"3931":4.99,"3932":4.39,"3934":4.23,"3935":75.23,"3936":12.51,"3937":6.43,"3938":9.24,"3939":11.01,"3940":37.83,"3941":11.0,"3942":3.4,"3943":3.25,"3944":24.91,"3945":8.8,"3946":11.98,"3947":3.19,"3949":8.03,"3950":22.21,"3951":3.5,"3952":55.45,"3954":12.99,"3955":14.3,"3956":6.06,"3957":11.0,"3958":6.75,"3959":5.05,"3961":40.0,"3962":10.8,"3963":19.67,"3964":9.27,"3965":6.0,"3966":4.0,"3967":12.33,"3968":16.03,"3969":9.0,"3970":6.88,"3971":3.25,"3972":15.41,"3973":4.52,"3974":18.12,"3977":3.62,"3978":8.88,"3979":3.25,"3980":19.48,"3982":2.41,"3983":9.66,"3984":11.0,"3985":40.73,"3986":15.78,"3987":12.99,"3988":53.95,"3989":4.13,"3992":3.0,"3993":49.34,"3994":12.0,"3995":16.17,"3996":5.43,"3998":18.99,"3999":1.79,"4000":25.47,"4001":52.21,"4002":4.39,"4003":3.62,"4004":24.58,"4005":9.66,"4007":7.98,"4008":5.51,"4009":0.99,"4010":8.03,"4011":14.75,"4012":9.9,"4013":17.58,"4014":15.23,"4015":200.0,"4016":9.99,"4017":36.36,"4018":36.39,"4019":15.86,"4020":11.88,"4021":15.39,"4022":8.8,"4023":5.49,"4024":2.96,"4025":13.16,"4026":11.21,"4027":18.77,"4028":14.78,"4029":22.01,"4030":13.74,"4033":8.8,"4034":21.98,"4035":41.52,"4036":49.0,"4037":8.84,"4038":2.41,"4039":13.99,"4040":6.33,"4041":2.48,"4042":9.9,"4043":13.42,"4044":25.94,"4045":26.91,"4046":11.0,"4047":48.93,"4048":21.33,"4049":8.8,"4050":8.25,"4051":1.74,"4052":40.0,"4053":4.39,"4054":109.8,"4055":14.0,"4056":3.62,"4057":9.52,"4058":62.76,"4059":93.2,"4060":3.85,"4063":3.6,"4066":39.15,"4068":32.71,"4069":55.45,"4070":41.74,"4071":124.7,"4072":90.45,"4073":7.15,"4074":8.8,"4075":20.68,"4076":27.0,"4078":55.99,"4079":7.98,"4080":3.85,"4081":4.39,"4082":6.55,"4084":53.95,"4085":5.08,"4086":7.98,"4087":11.0,"4089":11.88,"4090":72.36,"4091":13.0,"4092":16.51,"4093":12.01,"4094":39.42,"4095":12.43,"4096":1.49,"4097":34.62,"4098":13.02,"4099":5.74,"4100":2.49,"4101":5.05,"4102":9.9,"4103":4.39,"4105":6.93,"4107":4.39,"4108":27.05,"4109":36.59,"4110":8.03,"4111":8.76,"4112":30.7,"4113":3.42,"4114":73.75,"4117":2.49,"4118":17.42,"4119":24.15,"4120":7.72,"4121":26.12,"4122":3.0,"4123":4.32,"4124":22.01,"4125":9.99,"4127":28.74,"4128":11.0,"4129":54.99,"4131":46.77,"4134":24.15,"4135":12.19,"4136":23.37,"4138":13.08,"4139":5.49,"4140":109.8,"4141":10.93,"4142":5.44,"4143":11.6,"4144":4.39,"4145":8.03,"4149":90.98,"4150":5.74,"4151":30.99,"4152":4.39,"4153":21.78,"4154":8.0,"4155":8.28,"4156":23.82,"4157":4.29,"4158":20.16,"4160":33.82,"4161":19.45,"4162":19.0,"4163":3.8,"4165":60.19,"4166":1.0,"4167":1.0,"4168":28.32,"4170":10.29,"4171":11.0,"4174":8.59,"4175":15.96,"4176":55.04,"4177":8.88,"4178":27.6,"4179":10.52,"4182":5.71,"4183":6.75,"4184":4.25,"4185":9.0,"4186":82.0,"4187":37.7,"4188":25.3,"4190":21.94,"4191":36.33,"4192":93.32,"4193":28.22,"4194":15.46,"4195":6.43,"4196":59.65,"4197":42.83,"4198":15.4,"4199":6.5,"4200":55.34,"4201":5.58,"4202":11.05,"4203":4.7,"4204":7.98,"4205":46.67,"4207":9.24,"4208":65.41,"4209":4.39,"4211":39.95,"4213":19.39,"4215":8.8,"4216":40.0,"4217":14.5,"4218":23.43,"4219":57.17,"4220":3.19,"4221":48.96,"4222":7.01,"4223":109.79,"4224":51.0,"4225":11.88,"4226":4.39,"4227":8.87,"4228":2.2,"4229":95.75,"4230":10.52,"4231":6.59,"4232":25.08,"4233":59.46,"4234":11.19,"4235":16.0,"4236":7.0,"4237":53.95,"4239":20.0,"4240":13.76,"4241":2.07,"4243":23.29,"4244":11.0,"4245":23.96,"4246":42.61,"4247":1.69,"4248":20.5,"4250":18.0,"4251":5.9,"4252":74.73,"4253":40.0,"4255":8.88,"4256":20.22,"4257":14.0,"4258":11.58,"4259":2.48,"4260":7.63,"4261":20.85,"4262":14.83,"4263":38.52,"4264":7.52,"4265":5.5,"4266":32.48,"4267":4.39,"4268":5.0,"4269":5.45,"4270":17.85,"4271":14.3,"4272":1.29,"4273":13.94,"4274":26.4,"4275":14.28,"4276":8.88,"4277":70.61,"4279":39.53,"4280":4.18,"4281":32.64,"4282":338.95,"4283":250.0,"4284":375.38,"4288":3.62,"4289":87.1,"4290":12.63,"4291":11.83,"4292":40.65,"4293":2.74,"4294":26.7,"4295":6.36,"4296":2.49,"4297":23.12,"4298":2.48,"4299":7.62,"4300":34.1,"4301":9.99,"4302":10.29,"4303":2.49,"4304":3.83,"4305":7.98,"4306":6.59,"4307":10.98,"4308":14.65,"4309":24.08,"4310":5.45,"4311":109.8,"4312":29.14,"4313":7.0,"4314":24.0,"4315":38.09,"4316":8.5,"4318":6.59,"4319":3.62,"4320":106.99,"4321":10.98,"4322":90.19,"4323":6.75,"4324":2.49,"4325":2.52,"4326":4.39,"4327":25.8,"4328":2.49,"4329":40.23,"4331":9.5,"4332":30.03,"4333":226.95,"4334":8.53,"4335":100.81,"4336":2.48,"4338":42.23,"4343":75.47,"4344":30.99,"4345":2.49,"4347":13.92,"4348":28.13,"4349":9.01,"4351":9.13,"4352":11.98,"4353":16.26,"4354":33.01,"4355":32.42,"4356":40.0,"4357":51.21,"4358":16.54,"4359":69.89,"4360":8.0,"4361":2.8,"4362":2.75,"4363":12.93,"4364":10.0,"4366":9.88,"4367":13.2,"4368":63.68,"4369":48.57,"4370":10.84,"4371":3.95,"4372":6.88,"4373":74.23,"4374":11.0,"4375":10.73,"4376":4.13,"4377":6.88,"4379":15.41,"4380":15.0,"4382":3.25,"4383":8.8,"4384":7.38,"4385":15.4,"4386":2.67,"4387":8.84,"4388":23.46,"4389":63.95,"4390":7.36,"4391":8.79,"4392":76.92,"4393":4.39,"4396":13.76,"4397":42.76,"4398":10.0,"4399":3.5,"4400":4.39,"4401":49.47,"4402":22.92,"4403":10.45,"4406":19.8,"4407":9.52,"4408":6.88,"4409":33.02,"4410":105.53,"4411":9.85,"4412":20.97,"4413":39.3,"4414":6.08,"4415":8.8,"4416":5.82,"4417":32.68,"4418":7.85,"4419":48.42,"4422":6.15,"4423":13.13,"4424":10.0,"4425":29.32,"4426":12.64,"4427":266.94,"4428":1.42,"4429":9.99,"4430":6.59,"4431":5.87,"4432":41.52,"4433":72.01,"4434":9.9,"4435":12.0,"4436":98.58,"4437":217.93,"4438":8.84,"4439":8.98,"4440":72.0,"4441":2.74,"4442":91.8,"4443":12.42,"4444":30.69,"4445":7.5,"4446":10.1,"4447":41.51,"4448":112.48,"4449":170.0,"4450":256.41,"4451":23.4,"4452":17.92,"4453":0.01,"4454":13.0,"4455":10.38,"4456":19.34,"4457":8.8,"4458":256.41,"4459":256.41,"4460":50.0,"4461":33.01,"4462":109.8,"4463":35.87,"4464":5.0,"4465":11.78,"4466":31.95,"4467":34.23,"4468":19.14,"4469":5.95,"4470":14.86,"4471":13.13,"4473":12.64,"4474":36.43,"4475":39.28,"4476":19.0,"4477":25.97,"4478":17.7,"4479":15.75,"4480":8.17,"4481":296.75,"4482":31.97,"4483":50.47,"4487":6.88,"4488":10.73,"4489":17.07,"4490":13.52,"4491":13.01,"4492":109.0,"4494":217.93,"4495":12.46,"4496":18.65,"4497":40.0,"4498":11.17,"4499":15.19,"4500":23.7,"4501":21.17,"4502":22.76,"4503":20.9,"4504":39.87,"4505":31.4,"4506":32.73,"4507":11.0,"4508":10.0,"4509":14.49,"4510":2.48,"4511":140.25,"4512":32.3,"4513":11.0,"4514":16.94,"4515":2250.0,"4517":8.8,"4518":17.77,"4519":22.15,"4520":60.0,"4523":9.13,"4526":12.99,"4527":14.29,"4528":4.94,"4529":64.96,"4530":37.42,"4531":6.0,"4532":3.03,"4533":98.58,"4534":4.46,"4535":4.99,"4536":14.4,"4537":135.0,"4538":8.8,"4539":4.73,"4540":25.5,"4541":27.65,"4542":3.43,"4543":15.64,"4544":22.66,"4545":2.48,"4546":8.48,"4547":30.0,"4548":6.88,"4549":4.39,"4550":10.84,"4551":3.04,"4552":5.64,"4553":9.43,"4556":9.99,"4557":23.46,"4558":9.1,"4559":22.36,"4560":100.0,"4561":9.36,"4562":13.05,"4563":7.38,"4564":5.68,"4565":15.39,"4566":2.49,"4567":41.17,"4568":19.41,"4569":109.45,"4570":4.85,"4571":7.12,"4572":7.5,"4574":10.66,"4575":8.63,"4576":9.99,"4577":4.85,"4578":52.98,"4579":3.88,"4580":17.99,"4581":7.29,"4582":10.84,"4583":20.64,"4585":32.64,"4587":21.13,"4588":4.16,"4589":15.98,"4590":14.64,"4591":109.8,"4592":31.33,"4593":4.85,"4594":3.58,"4597":23.59,"4598":11.04,"4600":22.42,"4601":8.8,"4602":4.98,"4603":3.58,"4604":5.68,"4605":6.99,"4606":8.48,"4607":4.85,"4608":5.54,"4609":4.25,"4611":7.7,"4612":57.37,"4613":18.99,"4614":40.0,"4615":1.62,"4616":8.8,"4617":6.43,"4618":10.28,"4619":11.09,"4620":2.49,"4621":11.0,"4622":16.97,"4623":4.58,"4624":7.7,"4625":7.49,"4626":4.5,"4627":9.74,"4628":14.87,"4630":16.19,"4631":2.95,"4632":16.92,"4633":17.87,"4634":10.49,"4635":11.0,"4636":14.61,"4637":79.16,"4639":37.23,"4640":45.72,"4641":13.19,"4642":6.99,"4643":3.08,"4645":10.84,"4647":8.57,"4648":11.69,"4649":5.49,"4651":49.87,"4652":10.21,"4653":23.88,"4654":12.89,"4655":8.38,"4656":2.71,"4657":1.3,"4658":9.48,"4659":27.43,"4661":8.79,"4662":15.0,"4663":6.0,"4664":13.5,"4665":45.0,"4666":2.5,"4667":4.0,"4668":29.48,"4669":16.39,"4670":9.99,"4672":9.74,"4673":7.38,"4674":14.0,"4677":2.98,"4678":3.36,"4679":150.0,"4680":7.5,"4684":24.24,"4685":30.03,"4686":4.45,"4687":14.29,"4688":13.23,"4689":4.14,"4690":28.0,"4691":4.85,"4692":162.0,"4693":9.74,"4694":25.9,"4695":7.29,"4696":26.94,"4697":109.8,"4698":102.08,"4699":9.74,"4700":125.0,"4703":8.48,"4704":6.97,"4707":10.8,"4708":7.7,"4709":7.38,"4711":2.09,"4712":15.99,"4713":41.12,"4714":21.98,"4715":59.5,"4716":30.49,"4717":12.97,"4718":12.83,"4719":10.49,"4720":5.04,"4723":2.48,"4724":40.0,"4725":8.23,"4726":1.1,"4727":15.79,"4728":3.67,"4729":34.28,"4730":24.79,"4731":10.81,"4732":7.38,"4733":4.3,"4734":17.21,"4735":7.7,"4736":10.43,"4737":18.0,"4738":7.79,"4739":1.79,"4740":7.01,"4741":7.7,"4742":8.47,"4744":11.0,"4745":28.2,"4746":4.94,"4747":227.5,"4748":29.36,"4749":51.05,"4750":4.15,"4751":10.24,"4752":9.04,"4753":7.79,"4755":5.49,"4756":5.49,"4757":33.25,"4759":7.38,"4760":2.2,"4761":79.99,"4762":3.79,"4763":54.95,"4764":15.78,"4767":8.65,"4768":5.49,"4769":12.25,"4770":8.0,"4771":14.51,"4772":28.09,"4773":13.62,"4774":16.23,"4776":9.15,"4777":8.8,"4778":5.69,"4779":16.06,"4780":17.0,"4781":28.9,"4782":14.37,"4783":9.99,"4784":17.6,"4785":16.08,"4786":17.14,"4788":25.72,"4789":17.02,"4790":47.92,"4791":12.99,"4792":27.29,"4793":1.99,"4794":11.0,"4795":7.38,"4797":6.8,"4798":7.38,"4799":3.69,"4800":18.42,"4801":6.88,"4802":3.25,"4803":7.78,"4804":13.4,"4805":37.97,"4806":7.38,"4807":51.78,"4808":1.49,"4809":3.51,"4810":37.97,"4811":13.2,"4812":9.74,"4813":31.16,"4814":10.8,"4816":18.69,"4817":15.44,"4818":13.54,"4819":10.55,"4820":6.29,"4822":6.55,"4823":7.38,"4824":7.7,"4825":16.24,"4826":67.04,"4831":15.24,"4832":3.79,"4833":3.51,"4834":4.24,"4835":100.0,"4836":121.8,"4837":3.25,"4839":40.0,"4840":13.2,"4842":4.06,"4844":11.0,"4845":9.74,"4846":14.2,"4847":70.29,"4848":9.74,"4849":7.38,"4850":7.11,"4851":21.25,"4852":3.35,"4853":7.38,"4855":7.38,"4856":3.87,"4857":34.41,"4858":4.86,"4861":9.01,"4862":6.88,"4863":7.38,"4864":15.82,"4865":24.92,"4866":14.28,"4867":13.91,"4868":3.35,"4869":6.45,"4870":10.49,"4871":1.65,"4873":79.36,"4874":4.06,"4875":6.76,"4876":9.99,"4877":7.38,"4880":12.5,"4881":15.14,"4882":9.74,"4883":6.64,"4884":6.25,"4886":2.04,"4887":7.38,"4888":11.39,"4889":6.88,"4890":11.25,"4891":2.48,"4892":8.74,"4893":18.45,"4894":9.37,"4896":10.92,"4897":124.16,"4898":14.27,"4899":23.3,"4900":4.0,"4901":10.75,"4902":6.88,"4903":12.11,"4904":9.74,"4905":9.0,"4906":6.88,"4910":85.87,"4911":2.37,"4912":6.88,"4913":9.74,"4914":6.88,"4915":4.16,"4916":12.11,"4918":7.38,"4919":13.06,"4920":6.88,"4921":8.8,"4922":40.0,"4924":6.22,"4925":5.0,"4926":7.7,"4927":3.25,"4928":53.39,"4929":32.46,"4930":40.68,"4931":20.0,"4932":7.95,"4933":3.25,"4935":22.79,"4936":38.0,"4937":36.88,"4938":10.36,"4939":11.0,"4940":11.11,"4941":11.34,"4942":42.83,"4943":4.64,"4944":3.29,"4945":6.88,"4946":315.0,"4947":10.94,"4948":4.56,"4950":6.88,"4951":6.88,"4952":5.0,"4953":11.36,"4954":21.98,"4955":17.38,"4956":3.3,"4957":5.63,"4958":112.08,"4959":6.85,"4960":47.0,"4961":5.22,"4962":3.5,"4964":6.85,"4965":6.5,"4966":26.0,"4969":6.22,"4970":10.95,"4971":62.42,"4973":2.49,"4974":6.85,"4975":16.76,"4976":6.85,"4978":11.29,"4979":23.93,"4980":9.99,"4981":7.66,"4982":20.82,"4983":7.98,"4984":16.8,"4985":25.2,"4986":16.8,"4987":6.85,"4988":55.4,"4989":31.4,"4990":5.9,"4991":16.85,"4993":14.99,"4994":39.82,"4995":21.92,"4996":18.33,"4997":5.51,"4998":13.0,"4999":13.35,"5000":117.19,"5001":7.03,"5002":9.85,"5003":2.49,"5004":6.13,"5006":6.0,"5007":6.71,"5008":16.44,"5009":76.27,"5010":7.96,"5011":5.46,"5012":8.49,"5013":7.29,"5014":12.28,"5015":7.39,"5016":11.12,"5017":3.86,"5018":6.85,"5019":6.85,"5020":21.69,"5021":500.0,"5022":2.49,"5023":301.4,"5026":2.49,"5027":3.77,"5029":90.89,"5030":9.34,"5031":10.0,"5032":73.47,"5033":12.5,"5034":9.43,"5035":50.0,"5036":12.19,"5037":25.0,"5038":25.6,"5039":75.5,"5040":3.69,"5041":12.87,"5043":40.0,"5044":8.61,"5045":3.29,"5046":2.99,"5047":28.61,"5048":8.43,"5049":9.34,"5050":9.39,"5051":2.47,"5052":9.34,"5053":7.5,"5054":14.86,"5055":47.23,"5056":10.95,"5057":10.25,"5058":1.99,"5059":50.36,"5060":21.7,"5061":15.04,"5062":10.67,"5063":11.66,"5064":9.12,"5065":36.49,"5066":22.75,"5067":3.75,"5068":6.56,"5069":3.39,"5070":33.0,"5071":7.19,"5072":3.5,"5073":6.85,"5074":8.95,"5075":5.38,"5076":4.29,"5081":31.0,"5083":15.64,"5084":3.49,"5085":2.49,"5086":1.89,"5087":2.49,"5088":2.47,"5089":9.58,"5090":16.58,"5091":6.85,"5092":3.28,"5093":6.85,"5094":9.99,"5095":16.87,"5096":8.56,"5097":92.27,"5098":40.0,"5099":11.82,"5100":8.96,"5101":8.61,"5103":13.62,"5104":30.0,"5105":6.85,"5106":10.93,"5107":10.31,"5108":20.89,"5109":4.89,"5111":11.85,"5112":72.28,"5113":15.52,"5114":2.2,"5115":3.5,"5116":8.86,"5117":56.25,"5118":14.99,"5119":6.85,"5120":10.0,"5121":43.5,"5122":5.25,"5123":6.0,"5124":9.05,"5125":5.0,"5126":30.27,"5127":18.0,"5128":3.99,"5129":10.5,"5130":5.15,"5131":10.5,"5132":14.0,"5133":44.0,"5134":3.69,"5135":14.51,"5136":11.69,"5137":2.47,"5139":27.38,"5140":2.47,"5141":11.56,"5142":3.5,"5143":68.76,"5144":8.22,"5145":1.0,"5146":500.0,"5147":7.66,"5148":3.5,"5149":86.85,"5151":58.4,"5153":33.43,"5154":28.99,"5155":3.5,"5157":40.0,"5158":13.34,"5159":36.91,"5160":12.59,"5161":7.98,"5162":10.0,"5163":13.0,"5164":7.66,"5165":17.92,"5166":2.47,"5167":14.18,"5168":23.93,"5170":11.79,"5171":7.74,"5172":10.95,"5173":41.47,"5174":6.73,"5175":9.91,"5176":22.83,"5177":74.0,"5178":4.32,"5179":4.05,"5180":4.67,"5181":20.03,"5183":40.7,"5184":6.85,"5185":41.6,"5186":10.7,"5188":2.2,"5189":7.66,"5194":40.0,"5195":13.15,"5196":63.06,"5197":12.34,"5198":3.99,"5199":1.39,"5200":6.78,"5201":104.43,"5203":7.66,"5204":41.79,"5205":14.99,"5206":9.99,"5207":69.9,"5208":8.99,"5209":9.5,"5210":27.4,"5211":6.36,"5212":38.81,"5213":89.19,"5214":7.87,"5215":7.66,"5216":13.79,"5217":13.0,"5218":56.06,"5219":11.47,"5220":2.4,"5221":8.5,"5222":12.5,"5223":12.58,"5224":5.99,"5225":10.23,"5226":6.85,"5227":9.7,"5228":2.59,"5229":7.66,"5231":21.85,"5232":3.69,"5233":7.66,"5234":9.34,"5235":6.03,"5236":8.79,"5237":2.59,"5238":6.78,"5239":30.28,"5240":6.88,"5241":7.66,"5242":6.0,"5243":11.77,"5244":139.0,"5245":2.49,"5246":55.9,"5247":16.43,"5248":9.34,"5249":22.45,"5250":21.32,"5251":7.66,"5252":15.73,"5254":3.69,"5255":6.85,"5256":68.76,"5257":2.59,"5258":7.66,"5259":12.58,"5260":4.04,"5261":59.54,"5262":19.73,"5263":500.0,"5264":33.5,"5265":3.99,"5266":2.59,"5267":7.66,"5268":9.06,"5269":16.27,"5270":72.34,"5271":7.66,"5272":4.92,"5273":3.69,"5274":150.0,"5275":1.69,"5276":6.03,"5277":6.2,"5279":4.92,"5280":89.85,"5281":315.0,"5282":40.0,"5285":6.0,"5286":11.89,"5287":4.92,"5288":5.49,"5289":7.99,"5290":13.17,"5291":45.0,"5292":12.63,"5293":44.78,"5294":4.04,"5295":19.72,"5296":22.63,"5297":4.84,"5298":19.46,"5299":191.82,"5300":10.95,"5301":6.09,"5304":14.05,"5306":36.15,"5307":7.32,"5309":2.77,"5310":7.7,"5311":4.93,"5312":27.91,"5313":5.0,"5314":26.29,"5315":27.51,"5316":30.71,"5318":9.99,"5319":263.1,"5320":33.32,"5321":8.37,"5322":19.22,"5323":11.35,"5324":40.92,"5325":6.0,"5326":17.31,"5327":27.49,"5329":17.0,"5330":26.0,"5331":13.37,"5332":75.47,"5333":20.46,"5334":27.78,"5335":3.25,"5336":10.25,"5337":25.92,"5339":16.47,"5340":10.95,"5341":35.0,"5342":65.07,"5343":55.8,"5344":8.47,"5345":45.62,"5346":500.0,"5347":52.45,"5348":13.4,"5349":6.85,"5350":5.02,"5351":9.17,"5352":7.66,"5353":29.89,"5354":6.85,"5355":6.01,"5357":451.18,"5358":40.0,"5359":40.65,"5360":30.42,"5361":5.22,"5362":2.49,"5363":16.44,"5364":58.22,"5365":8.0,"5367":6.85,"5368":7.66,"5372":100.89,"5373":60.89,"5374":13.7,"5375":8.2,"5376":7.57,"5377":38.7,"5378":7.66,"5379":11.16,"5380":5.97,"5381":52.0,"5382":2.79,"5383":8.47,"5384":2.74,"5385":10.95,"5391":41.35,"5392":43.0,"5393":41.67,"5394":33.35,"5395":2.75,"5396":26.11,"5397":12.85,"5398":44.78,"5399":13.44,"5400":9.99,"5401":7.66,"5402":8.42,"5403":48.88,"5404":33.63,"5405":27.03,"5406":24.0,"5407":6.16,"5408":5.98,"5409":10.81,"5410":79.5,"5412":14.31,"5413":15.75,"5414":9.44,"5415":18.14,"5416":1.88,"5417":26.76,"5420":2.66,"5422":59.05,"5423":39.32,"5424":74.69,"5425":37.39,"5426":36.31,"5427":21.65,"5428":39.81,"5429":26.68,"5430":873.36,"5431":4.35,"5432":4.35,"5433":10.88,"5435":16.98,"5436":15.77,"5437":17.94,"5438":14.68,"5439":20.0,"5440":11.31,"5441":4.0,"5442":13.59,"5443":18.05,"5444":20.66,"5445":10.55,"5446":14.68,"5447":11.96,"5448":10.83,"5449":11.53,"5450":13.87,"5451":14.68,"5452":13.87,"5453":102.63,"5456":4.62,"5457":21.48,"5458":7.61,"5459":375.0,"5460":25.4,"5461":1500.0,"5463":12.23,"5464":17.06,"5465":14.92,"5466":4.38,"5467":10.7,"5468":19.58,"5469":18.58,"5470":19.98,"5471":27.7,"5473":34.34,"5475":40.0,"5476":3.56,"5477":2.44,"5478":10.95,"5479":32.87,"5480":12.58,"5481":10.95,"5483":4.78,"5484":52.34,"5485":5.55,"5486":7.28,"5487":17.8,"5488":44.3,"5489":7.66,"5490":6.85,"5491":7.61,"5492":25.93,"5493":11.16,"5494":6.85,"5495":84.0,"5496":9.9,"5498":3.56,"5499":7.66,"5500":3.56,"5501":10.95,"5502":11.86,"5503":7.66,"5506":23.69,"5507":11.23,"5508":63.2,"5509":49.09,"5510":9.99,"5511":344.14,"5512":92.86,"5513":7.68,"5514":11.99,"5515":4.7,"5516":5.0,"5517":47.94,"5518":3.23,"5519":8.97,"5520":13.98,"5521":9.34,"5522":11.35,"5524":7.66,"5525":262.2,"5526":33.32,"5527":30.15,"5528":6.85,"5529":6.85,"5530":38.15,"5531":6.6,"5532":22.21,"5533":22.18,"5534":195.23,"5535":44.77,"5536":10.2,"5537":26.78,"5538":11.4,"5539":5.86,"5540":7.66,"5541":3.71,"5542":101.18,"5545":7.66,"5546":10.95,"5547":1.55,"5548":6.85,"5550":9.31,"5551":6.78,"5552":12.37,"5553":39.48,"5554":6.08,"5557":12.85,"5559":4.12,"5560":3.0,"5561":5.0,"5562":87.85,"5563":30.0,"5564":50.64,"5565":8.85,"5566":49.12,"5567":9.19,"5568":15.0,"5569":212.01,"5570":5.0,"5571":27.57,"5572":24.95,"5573":11.64,"5574":12.78,"5575":19.34,"5576":40.0,"5577":9.57,"5578":8.76,"5579":6.99,"5580":282.1,"5581":54.0,"5582":6.7,"5583":13.1,"5584":23.99,"5585":67.14,"5586":2.02,"5587":12.37,"5588":94.08,"5589":7.0,"5590":19.95,"5591":27.04,"5592":9.0,"5593":2.29,"5594":8.76,"5595":315.0,"5597":22.23,"5598":62.95,"5599":6.85,"5600":9.85,"5601":11.0,"5602":9.21,"5603":5.75,"5604":2.73,"5606":65.21,"5607":55.07,"5608":13.09,"5609":168.18,"5610":50.0,"5612":369.0,"5614":12.37,"5617":8.37,"5622":147.1,"5623":211.2,"5624":6.85,"5625":97.1,"5626":48.47,"5627":39.39,"5628":5.12,"5629":17.81,"5630":60.0,"5631":6.83,"5632":9.99,"5633":10.2,"5634":17.2,"5635":6.85,"5636":12.59,"5637":52.55,"5638":2.11,"5639":8.77,"5640":291.2,"5641":5.75,"5642":10.65,"5644":52.4,"5646":40.82,"5647":51.0,"5648":11.14,"5649":3.5,"5650":51.0,"5651":6.85,"5652":28.0,"5653":191.75,"5654":27.27,"5655":291.2,"5656":16.44,"5657":12.0,"5658":1.64,"5659":8.88,"5660":5.78,"5661":6.85,"5662":3.5,"5663":2.49,"5664":9.62,"5665":10.95,"5666":3.69,"5668":100.0,"5670":16.23,"5671":8.34,"5672":12.33,"5673":6.85,"5674":11.36,"5675":12.59,"5676":97.42,"5677":3.69,"5678":2.94,"5679":50.63,"5680":6.85,"5681":6.85,"5682":22.0,"5683":7.98,"5684":2.59,"5685":6.85,"5686":11.64,"5687":14.4,"5688":10.2,"5689":3.5,"5690":28.2,"5691":6.85,"5692":6.28,"5693":10.95,"5694":12.14,"5695":2.66,"5696":4.25,"5698":9.5,"5700":10.95,"5701":1.48,"5702":3.69,"5704":6.85,"5705":10.95,"5706":40.0,"5707":6.85,"5712":8.63,"5713":6.85,"5714":2.47,"5715":2.99,"5718":5.86,"5719":9.85,"5721":15.67,"5722":13.82,"5723":15.0,"5724":9.34,"5725":3.56,"5727":6.85,"5729":10.83,"5730":2.4,"5731":6.85,"5732":7.33,"5733":3.56,"5734":6.85,"5735":86.58,"5736":4.44,"5737":9.99,"5738":4.51,"5739":46.69,"5740":9.96,"5741":7.49,"5742":12.25,"5744":4.34,"5745":10.41,"5746":54.0,"5747":7.26,"5748":27.79,"5749":283.2,"5750":6.85,"5751":2.69,"5752":105.53,"5753":51.47,"5754":49.02,"5755":3.56,"5756":38.58,"5758":35.36,"5759":3.56,"5760":10.62,"5761":35.58,"5762":35.97,"5764":20.52,"5765":60.07,"5766":28.0,"5767":18.06,"5768":27.45,"5769":35.4,"5770":5.0,"5771":30.3,"5772":65.35,"5773":17.64,"5774":3.39,"5775":13.0,"5778":2.92,"5779":14.99,"5780":3.39,"5781":3.56,"5782":36.58,"5783":40.0,"5785":108.5,"5786":3.56,"5787":7.49,"5788":13.74,"5789":4.12,"5790":9.2,"5791":2.47,"5792":6.2,"5793":9.85,"5795":20.16,"5796":3.39,"5797":2.47,"5798":9.39,"5799":28.29,"5800":33.24,"5801":40.0,"5802":7.0,"5803":7.0,"5804":9.18,"5805":24.02,"5806":4.12,"5807":50.84,"5808":47.4,"5809":2.69,"5810":10.97,"5811":2.47,"5812":2.47,"5813":43.84,"5814":4.66,"5816":2.73,"5817":24.0,"5819":52.56,"5820":6.37,"5821":9.59,"5822":2.74,"5823":3.56,"5824":11.67,"5825":27.38,"5826":120.1,"5827":3.01,"5828":10.97,"5829":2.47,"5831":46.46,"5832":10.02,"5833":3.01,"5834":44.89,"5835":17.08,"5836":5.9,"5837":6.01,"5839":12.0,"5840":2.47,"5841":3.69,"5842":20.55,"5843":8.75,"5844":13.55,"5845":2.0,"5847":27.57,"5848":3.99,"5849":24.0,"5850":9.1,"5851":3.39,"5852":12.35,"5853":6.86,"5854":49.19,"5855":24.99,"5856":3.39,"5857":2.47,"5858":20.0,"5859":78.24,"5860":26.07,"5861":10.36,"5862":0.58,"5863":19.23,"5864":5.0,"5866":74.94,"5867":2.11,"5869":9.87,"5870":0.31,"5871":10.27,"5872":0.3,"5874":0.06,"5875":0.21,"5876":0.09,"5877":89.9,"5878":5.0,"5879":149.65,"5880":5.0,"5882":222.4,"5883":5.45,"5884":181.75,"5887":40.0,"5889":2.0,"5890":5.0,"5891":44.89,"5892":2.0,"5893":2.0,"5894":2.0,"5895":149.62,"5896":149.62,"5897":5.0,"5898":5.0,"5899":9.85,"5900":5.0,"5902":150.3,"5903":2.0,"5906":91.18,"5907":5.0,"5908":2.0,"5909":25.61,"5910":24.99,"5911":2.0,"5912":91.4,"5913":3.25,"5915":108.48,"5917":5.0,"5918":144.59,"5919":4.34,"5922":3.59,"5923":0.32,"5924":2.09,"5925":0.06,"5926":10.76,"5927":40.0,"5929":16.0,"5931":74.84,"5933":2.47,"5934":32.95,"5935":24.99,"5936":24.99,"5937":4.64,"5938":3.39,"5940":9.85,"5941":3.39,"5943":42.16,"5944":6.56,"5945":43.47,"5946":15.65,"5947":1.9,"5949":5.59,"5950":6.0,"5951":60.0,"5952":12.05,"5953":11.94,"5954":7.0,"5955":11.77,"5956":11.43,"5957":23.63,"5958":34.94,"5959":10.95,"5960":141.78,"5961":10.25,"5968":24.04,"5969":11.26,"5971":5.84,"5972":134.08,"5973":15.97,"5974":7.68,"5975":4.42,"5976":14.25,"5977":23.64,"5978":11.65,"5979":6.63,"5982":18.0,"5983":17.86,"5984":30.33,"5985":56.11,"5986":11.87,"5987":19.16,"5988":42.93,"5989":13.0,"5990":18.0,"5991":39.83,"5992":8.95,"5993":25.5,"5994":15.85,"5995":20.66,"5996":5.94,"5997":24.37,"5998":12.97,"5999":42.41,"6000":2.69,"6001":68.27,"6002":10.4,"6006":40.0,"6007":9.85,"6010":12.72,"6012":7.03,"6013":19.34,"6014":29.91,"6016":22.95,"6017":2.47,"6018":9.85,"6019":3.69,"6020":10.65,"6021":18.25,"6022":3.41,"6023":4.12,"6024":21.79,"6025":10.0,"6026":12.05,"6027":29.33,"6029":3.99,"6030":103.91,"6031":45.77,"6032":14.59,"6033":1.48,"6034":8.37,"6035":13.15,"6036":34.63,"6037":5.0,"6038":8.12,"6039":9.87,"6040":148.6,"6041":7.58,"6042":4.01,"6043":3.39,"6044":5.84,"6045":16.74,"6046":2.47,"6047":4.48,"6048":113.94,"6049":11.31,"6050":3.69,"6051":3.69,"6052":106.18,"6053":4.26,"6054":3.46,"6055":31.39,"6056":10.25,"6057":4.99,"6059":18.34,"6060":5.79,"6061":2.47,"6062":18.3,"6063":2.47,"6064":7.33,"6065":10.11,"6066":40.78,"6067":40.0,"6068":4.3,"6069":316.0,"6070":34.0,"6071":18.34,"6072":31.66,"6073":38.31,"6074":16.42,"6075":6.48,"6076":15.64,"6077":23.62,"6078":5.49,"6079":12.13,"6080":7.56,"6081":53.47,"6082":27.39,"6083":10.56,"6084":61.77,"6085":22.95,"6087":24.49,"6088":6.99,"6089":10.95,"6090":12.99,"6091":21.49,"6092":4.72,"6093":40.0,"6094":87.59,"6095":11.51,"6096":12.43,"6097":24.57,"6098":149.08,"6099":39.24,"6100":8.57,"6101":4.12,"6102":4.84,"6103":6.48,"6104":16.38,"6105":3.17,"6106":86.0,"6107":9.85,"6108":5.91,"6109":50.15,"6110":38.35,"6111":25.76,"6112":5.18,"6113":16.61,"6114":7.88,"6115":12.07,"6116":8.76,"6117":4.55,"6118":3.24,"6119":2.47,"6120":12.95,"6121":73.05,"6123":5.18,"6124":11.23,"6126":5.18,"6127":8.62,"6128":6.1,"6130":10.48,"6131":10.65,"6132":32.6,"6133":14.8,"6134":9.61,"6135":2.49,"6136":14.65,"6137":30.0,"6138":14.0,"6139":5.08,"6140":50.05,"6141":108.1,"6143":83.09,"6144":8.29,"6145":19.96,"6146":27.66,"6148":27.44,"6149":11.25,"6151":27.47,"6152":15.11,"6153":53.73,"6154":110.37,"6155":5.49,"6156":117.81,"6157":35.22,"6158":2.69,"6160":41.88,"6161":2.0,"6162":41.71,"6163":6.57,"6164":15.17,"6165":9.81,"6166":1.0,"6167":27.11,"6168":35.46,"6169":10.85,"6170":3.58,"6171":7.33,"6173":3.69,"6175":2.39,"6176":1.79,"6177":4.64,"6178":73.08,"6179":21.8,"6180":39.98,"6181":9.85,"6182":16.44,"6183":115.96,"6184":2.69,"6185":21.73,"6186":32.87,"6187":12.08,"6188":18.0,"6189":9.0,"6190":320.2,"6191":3.58,"6192":32.4,"6194":73.18,"6195":29.48,"6197":29.29,"6198":92.85,"6199":6.08,"6200":132.47,"6201":27.05,"6202":2.56,"6203":5.9,"6204":2.51,"6205":8.76,"6206":36.34,"6207":78.62,"6208":6.78,"6210":44.19,"6211":81.1,"6212":10.38,"6213":19.71,"6214":20.0,"6215":153.42,"6216":11.0,"6217":12.0,"6218":12.0,"6219":76.71,"6220":77.12,"6221":1.47,"6222":3.99,"6223":51.17,"6225":28.05,"6226":8.75,"6227":7.0,"6228":1.47,"6229":40.0,"6231":14.0,"6232":9.85,"6233":40.52,"6234":19.89,"6235":10.0,"6236":12.77,"6237":3.0,"6238":19.96,"6239":15.5,"6240":27.0,"6241":19.07,"6242":14.42,"6243":9.95,"6244":2.14,"6245":54.99,"6246":3.75,"6247":10.81,"6248":10.17,"6249":25.31,"6250":19.79,"6251":5.69,"6252":74.0,"6253":9.31,"6254":9.77,"6255":3.56,"6256":119.63,"6257":12.81,"6258":46.99,"6259":164.39,"6260":3.39,"6261":10.89,"6262":82.6,"6264":40.0,"6267":10.81,"6268":12.49,"6269":14.65,"6270":14.0,"6271":10.0,"6272":11.86,"6274":3.39,"6275":10.81,"6276":11.86,"6277":316.0,"6278":5.18,"6279":26.84,"6280":9.85,"6282":101.52,"6285":15.41,"6286":5.98,"6287":1043.63,"6288":4.01,"6289":14.76,"6290":267.0,"6292":17.98,"6293":66.73,"6294":29.95,"6295":12.75,"6296":20.01,"6297":10.48,"6298":267.0,"6299":21.49,"6300":60.37,"6301":7.56,"6302":13.16,"6303":24.33,"6305":12.97,"6306":32.52,"6307":21.81,"6308":6.48,"6309":94.31,"6310":14.06,"6311":131.24,"6312":8.01,"6313":75.3,"6314":28.32,"6315":6.64,"6316":74.5,"6317":26.95,"6318":3.1,"6319":33.9,"6320":14.0,"6322":7.5,"6323":93.33,"6324":54.8,"6325":19.12,"6326":49.78,"6327":3.39,"6328":2.69,"6329":39.51,"6330":29.77,"6331":11.27,"6332":10.31,"6333":2.69,"6334":3.4,"6335":16.43,"6336":17.18,"6337":2.47,"6338":45.13,"6339":54.62,"6340":16.5,"6341":76.08,"6342":62.5,"6343":297.99,"6344":21.94,"6345":14.89,"6346":42.93,"6348":59.15,"6349":3.69,"6350":4.36,"6351":21.63,"6352":12.31,"6353":40.0,"6355":8.97,"6356":73.35,"6357":43.99,"6358":37.64,"6359":9.85,"6360":6.57,"6361":17.95,"6362":82.19,"6363":13.57,"6364":3.39,"6365":10.4,"6366":3.06,"6367":5.87,"6373":1.12,"6374":4.14,"6375":103.6,"6376":252.2,"6377":9.06,"6378":13.28,"6379":2.8,"6380":45.14,"6381":50.21,"6382":24.0,"6383":21.92,"6384":23.3,"6385":11.8,"6386":8.49,"6387":142.72,"6388":23.35,"6389":9.75,"6390":84.82,"6391":2.47,"6392":109.6,"6393":48.74,"6394":27.62,"6395":39.46,"6396":4.0,"6398":12.1,"6401":91.98,"6402":47.44,"6403":19.79,"6404":49.77,"6405":13.81,"6406":13.95,"6407":34.99,"6408":50.0,"6409":51.59,"6410":87.58,"6411":3.83,"6412":38.0,"6413":22.73,"6415":3.69,"6416":6.63,"6417":4.0,"6418":10.0,"6419":2.47,"6420":478.93,"6421":3.69,"6422":54.8,"6423":2.14,"6424":59.92,"6425":8.28,"6426":5.99,"6427":35.5,"6428":24.99,"6429":13.07,"6430":5.46,"6431":122.13,"6432":39.9,"6433":8.76,"6435":25.06,"6436":40.0,"6437":60.04,"6438":12.47,"6439":3.39,"6440":3.39,"6441":43.5,"6442":21.73,"6443":9.85,"6444":86.19,"6445":3.69,"6446":35.01,"6447":51.0,"6448":27.98,"6449":3.69,"6450":3.69,"6451":85.3,"6453":4.87,"6454":8.65,"6455":3.69,"6456":80.32,"6457":21.82,"6458":3.69,"6459":9.25,"6460":12.58,"6461":11.63,"6462":92.49,"6463":37.88,"6464":4.1,"6465":54.8,"6466":12.03,"6467":5.0,"6468":18.09,"6469":18.34,"6470":8.08,"6471":33.09,"6472":4.22,"6473":20.52,"6474":16.59,"6475":22.73,"6476":7.8,"6477":11.8,"6478":14.91,"6479":20.61,"6480":42.33,"6481":3.0,"6482":29.11,"6483":15.99,"6484":25.27,"6485":8.0,"6486":26.26,"6487":53.97,"6488":9.17,"6491":6.74,"6492":15.99,"6493":19.07,"6495":115.0,"6496":4.64,"6497":2.0,"6498":2.46,"6499":1.75,"6500":2.0,"6501":5.0,"6502":80.19,"6503":8.08,"6504":49.38,"6505":6.3,"6506":11.96,"6507":5.4,"6508":16.71,"6509":14.0,"6510":2.89,"6511":15.0,"6512":33.11,"6513":59.0,"6514":9.15,"6515":26.32,"6516":6.98,"6517":17.24,"6519":16.12,"6520":16.37,"6521":11.0,"6522":35.2,"6523":3.29,"6524":45.9,"6525":27.99,"6526":41.0,"6527":5.38,"6528":18.2,"6529":20.0,"6530":1.5,"6531":100.0,"6532":2.5,"6533":19.4,"6534":6.0,"6535":7.5,"6536":6.0,"6537":6.0,"6538":17.25,"6539":14.0,"6540":4.96,"6541":14.94,"6542":3.12,"6543":49.99,"6544":8.71,"6545":65.88,"6546":40.0,"6547":5.99,"6548":7.99,"6549":9.59,"6550":15.0,"6551":2.99,"6552":66.0,"6553":85.3,"6554":6.65,"6555":21.23,"6556":3.12,"6557":35.44,"6558":30.67,"6559":80.35,"6561":316.0,"6562":76.0,"6563":3.39,"6564":84.08,"6565":73.54,"6567":9.85,"6569":10.16,"6570":7.0,"6571":10.0,"6572":3.25,"6573":158.15,"6574":7.89,"6575":196.2,"6576":6.4,"6577":8.75,"6578":27.21,"6580":29.09,"6581":14.87,"6582":6.63,"6583":4.04,"6584":63.27,"6585":15.09,"6586":40.82,"6587":7.0,"6588":76.6,"6589":5.99,"6590":102.0,"6591":47.14,"6592":15.14,"6593":60.0,"6594":24.58,"6595":8.45,"6596":30.16,"6597":29.07,"6598":18.5,"6599":26.02,"6600":15.0,"6602":178.25,"6603":11.0,"6604":5.24,"6605":5.48,"6606":146.72,"6607":42.45,"6609":6.9,"6610":3.93,"6611":2.36,"6612":3.93,"6613":6.95,"6614":17.52,"6615":8.15,"6616":3.93,"6617":101.19,"6618":25.92,"6619":6.95,"6620":37.55,"6621":6.95,"6622":24.91,"6623":37.7,"6624":2.1,"6625":6.95,"6627":6.95,"6628":16.0,"6629":15.0,"6630":18.05,"6631":44.39,"6632":54.8,"6633":111.38,"6634":25.0,"6635":30.38,"6636":4.62,"6637":93.14,"6638":13.63,"6639":83.23,"6640":98.63,"6641":9.09,"6642":35.25,"6643":14.19,"6644":17.16,"6645":6.86,"6646":16.0,"6647":15.05,"6648":3.0,"6649":10.0,"6650":9.6,"6652":3.0,"6653":7.82,"6654":6.36,"6655":4.89,"6656":8.92,"6657":8.4,"6658":35.0,"6659":58.45,"6660":40.28,"6662":34.62,"6664":40.0,"6665":10.89,"6666":62.54,"6667":18.9,"6668":2.85,"6669":19.0,"6670":4.05,"6671":50.0,"6672":151.93,"6673":35.0,"6674":41.88,"6675":6.42,"6676":9.73,"6677":21.0,"6678":14.0,"6679":29.26,"6680":3.27,"6681":11.0,"6682":39.07,"6683":5.86,"6684":18.52,"6685":9.29,"6687":70.25,"6688":3.85,"6690":6.57,"6691":14.75,"6692":10.0,"6693":3.5,"6694":25.82,"6695":61.65,"6696":27.9,"6697":168.2,"6698":7.75,"6699":8.08,"6700":2.64,"6701":32.89,"6702":8.84,"6703":10.96,"6704":12.0,"6705":35.0,"6706":26.0,"6707":74.37,"6708":136.47,"6709":66.38,"6710":42.29,"6711":105.54,"6712":13.88,"6713":3.99,"6715":22.82,"6716":20.75,"6717":103.64,"6719":40.48,"6721":93.57,"6722":6.48,"6723":5.57,"6724":5.5,"6725":10.5,"6726":26.5,"6727":6.0,"6728":8.67,"6729":18.0,"6730":0.25,"6731":16.69,"6733":12.5,"6734":10.0,"6735":10.65,"6736":21.0,"6737":19.99,"6738":9.99,"6739":3.78,"6740":10.4,"6741":20.04,"6742":75.0,"6743":46.0,"6744":10.96,"6745":1.0,"6746":260.0,"6747":89.29,"6748":265.19,"6749":74.48,"6751":3.28,"6752":220.2,"6753":300.0,"6756":40.0,"6757":16.97,"6764":40.0,"6765":107.17,"6766":26.92,"6767":50.0,"6768":34.58,"6769":20.0,"6770":5.0,"6771":55.35,"6772":18.0,"6773":24.63,"6774":5.47,"6775":9.73,"6776":7.99,"6777":18.2,"6779":7.98,"6780":5.99,"6783":5.43,"6784":37.49,"6785":96.94,"6786":3.0,"6787":1850.0,"6788":16.0,"6789":2.99,"6790":40.0,"6791":20.0,"6792":37.0,"6793":12.96,"6794":9.5,"6795":30.46,"6796":62.61,"6797":95.67,"6798":7.78,"6799":14.58,"6800":5.99,"6801":8.75,"6803":300.0,"6805":7.1,"6806":19.73,"6807":1.0,"6808":8.41,"6809":4.04,"6810":31.74,"6811":17.37,"6812":14.13,"6813":30.29,"6814":73.8,"6815":18.48,"6816":77.91,"6817":3.24,"6818":11.75,"6819":20.54,"6822":174.25,"6823":100.0,"6824":18.12,"6826":19.5,"6827":39.08,"6828":7.12,"6829":89.0,"6830":19.99,"6831":3.56,"6832":19.18,"6833":13.28,"6835":8.68,"6836":5.31,"6837":12.59,"6838":7.67,"6839":47.0,"6840":13.38,"6841":9.77,"6842":35.96,"6844":24.8,"6845":4.0,"6846":15.82,"6847":28.7,"6848":6.48,"6849":14.28,"6850":6.0,"6851":20.3,"6852":15.37,"6853":60.26,"6855":200.0,"6856":146.19,"6858":40.0,"6859":22.71,"6861":11.83,"6862":13.99,"6863":11.2,"6864":62.35,"6865":34.72,"6866":23.76,"6867":20.05,"6868":9.73,"6869":7.58,"6870":36.82,"6871":1.75,"6872":8.67,"6874":26.13,"6875":27.56,"6876":12.95,"6877":73.74,"6878":3.24,"6879":8.23,"6880":27.39,"6881":2.15,"6882":60.98,"6883":50.0,"6884":250.0,"6885":29.35,"6886":111.09,"6889":34.74,"6891":12.76,"6892":23.84,"6893":99.68,"6894":63.0,"6895":172.21,"6896":9.28,"6897":214.68,"6898":8.4,"6899":2.26,"6900":10.62,"6901":39.58,"6902":9.22,"6903":24.77,"6904":2.25,"6905":6.9,"6906":37.78,"6907":29.48,"6908":22.7,"6909":25.2,"6910":31.65,"6911":20.13,"6912":11.65,"6915":3.0,"6916":25.63,"6917":300.0,"6918":6.57,"6919":40.0,"6920":45.82,"6922":53.64,"6924":9.73,"6925":66.71,"6927":89.96,"6928":75.7,"6929":17.49,"6930":216.05,"6931":352.2,"6932":10.0,"6933":21.25,"6934":23.97,"6935":16.32,"6937":7.66,"6938":305.78,"6939":75.69,"6940":919.54,"6941":80.0,"6942":4.98,"6944":19.99,"6945":46.57,"6946":31.5,"6948":9.42,"6949":5.95,"6950":87.68,"6951":15.41,"6952":45.86,"6953":20.13,"6954":24.05,"6955":10.31,"6956":15.05,"6957":9.75,"6958":81.88,"6959":3.99,"6960":3.25,"6962":42.98,"6963":18.95,"6964":8.67,"6965":13.37,"6966":14.25,"6967":9.49,"6968":1.57,"6969":80.0,"6970":23.48,"6971":52.63,"6972":12.83,"6973":3.41,"6974":27.37,"6975":7.6,"6976":1.42,"6977":22.83,"6978":10.56,"6979":4.14,"6980":5.03,"6981":13.01,"6982":21.0,"6983":84.93,"6984":98.05,"6985":36.75,"6986":50.86,"6988":4.04,"6989":198.6,"6990":19.55,"6991":9.73,"6992":2.24,"6993":10.77,"6995":23.07,"6996":40.0,"6999":80.0,"7000":16.57,"7001":13.7,"7003":6.58,"7004":28.18,"7005":65.99,"7006":2.58,"7007":20.82,"7008":20.01,"7009":6.47,"7010":18.0,"7011":2.74,"7012":27.05,"7013":6.47,"7014":40.45,"7015":8.09,"7016":35.06,"7017":15.22,"7018":3.91,"7019":180.0,"7020":112.4,"7021":28.17,"7022":35.0,"7023":61.85,"7024":43.19,"7025":43.47,"7026":132.2,"7028":16.16,"7029":80.0,"7030":10.75,"7031":397.0,"7032":13.96,"7034":275.64,"7035":7.84,"7036":14.18,"7038":8.75,"7039":13.41,"7040":500.0,"7041":156.98,"7043":14.25,"7044":40.0,"7046":12.98,"7048":3.06,"7049":9.73,"7051":10.84,"7052":43.73,"7053":26.49,"7054":25.01,"7055":11.64,"7056":21.92,"7057":10.6,"7058":85.63,"7059":60.19,"7060":1850.0,"7063":14.26,"7066":69.38,"7067":20.0,"7068":23.97,"7069":16.96,"7070":24.99,"7071":18.56,"7072":12.5,"7073":12.4,"7075":29.26,"7076":13.69,"7077":3.24,"7078":38.0,"7079":3.69,"7080":118.25,"7081":16.14,"7082":10.0,"7083":20.0,"7084":27.39,"7085":147.72,"7086":43.0,"7088":8.0,"7090":390.2,"7091":18.09,"7092":18.82,"7093":8.62,"7094":9.99,"7097":38.69,"7098":25.0,"7099":12.5,"7100":19.7,"7101":176.79,"7102":11.0,"7103":12.5,"7104":38.92,"7106":45.34,"7107":56.5,"7108":160.74,"7109":5.03,"7110":6.73,"7111":29.1,"7112":1.69,"7113":7.04,"7114":15.47,"7115":54.38,"7116":34.0,"7117":20.0,"7118":7.17,"7119":7.17,"7120":128.79,"7121":27.11,"7122":6.62,"7126":4.83,"7127":83.12,"7130":50.7,"7131":40.0,"7133":7.77,"7134":10.81,"7135":59.28,"7136":13.5,"7137":119.62,"7138":9.73,"7139":102.31,"7140":25.23,"7142":13.54,"7143":24.83,"7144":20.39,"7145":150.0,"7146":29.23,"7147":16.24,"7148":39.68,"7149":23.08,"7150":20.0,"7151":31.5,"7152":26.23,"7154":8.5,"7155":25.08,"7158":22.09,"7159":59.11,"7160":18.0,"7163":65.7,"7164":154.93,"7165":10.81,"7166":6.88,"7167":5.48,"7168":25.6,"7169":454.2,"7170":1.49,"7172":7.45,"7173":24.31,"7175":32.92,"7176":16.87,"7177":39.99,"7178":8.98,"7179":130.44,"7180":1.19,"7185":9.91,"7186":114.36,"7187":9.73,"7188":6.56,"7189":21.99,"7190":38.75,"7191":6.56,"7192":11.94,"7193":397.0,"7194":2.24,"7195":4.71,"7197":2.95,"7198":80.67,"7200":18.0,"7201":60.0,"7203":2.95,"7204":10.77,"7205":8.67,"7206":95.2,"7207":29.13,"7208":3.59,"7209":20.63,"7210":85.19,"7211":153.0,"7212":90.0,"7213":155.49,"7216":2.3,"7217":10.0,"7218":19.47,"7219":11.5,"7220":43.6,"7221":14.0,"7224":14.51,"7225":8.73,"7227":8.53,"7228":34.5,"7229":9.5,"7230":2.24,"7231":10.8,"7232":15.52,"7233":34.61,"7234":2.96,"7235":40.46,"7236":2.46,"7237":2.57,"7238":14.78,"7239":30.0,"7240":18.0,"7241":60.0,"7243":6.01,"7244":5.98,"7246":28.67,"7247":4.26,"7248":21.97,"7249":11.0,"7250":9.73,"7251":8.0,"7252":50.71,"7253":35.87,"7254":30.79,"7255":32.93,"7256":29.33,"7257":34.33,"7258":114.95,"7260":89.5,"7261":32.37,"7262":87.4,"7263":1.69,"7264":20.89,"7265":178.88,"7266":95.0,"7267":31.85,"7268":55.49,"7269":115.75,"7270":7.29,"7271":48.65,"7272":29.32,"7274":26.55,"7275":15.33,"7276":104.8,"7277":15.5,"7278":184.85,"7279":32.05,"7280":9.22,"7281":52.0,"7282":9.19,"7283":61.91,"7284":27.96,"7285":35.74,"7286":13.75,"7287":19.13,"7289":33.77,"7290":100.53,"7291":8.76,"7292":8.11,"7293":12.58,"7294":50.0,"7295":5.35,"7296":11.89,"7297":37.68,"7298":18.0,"7299":25.0,"7301":24.78,"7302":10.16,"7303":465.76,"7304":2.5,"7306":18.13,"7307":65.99,"7308":214.58,"7309":74.59,"7310":17.88,"7311":42.02,"7314":8.07,"7315":200.0,"7316":31.75,"7317":40.77,"7318":56.27,"7321":100.8,"7322":48.57,"7323":3.27,"7324":9.69,"7325":3.0,"7326":43.75,"7327":8.19,"7328":7.0,"7329":6.0,"7330":13.85,"7331":5.48,"7332":15.0,"7333":56.99,"7334":11.0,"7335":349.79,"7336":18.6,"7338":9.99,"7339":6.3,"7342":23.9,"7343":49.03,"7344":159.13,"7345":14.13,"7346":1.07,"7348":1850.0,"7349":32.34,"7351":3.66,"7352":93.6,"7353":63.1,"7354":202.55,"7355":28.15,"7357":27.0,"7358":156.55,"7359":4.32,"7360":35.97,"7361":21.85,"7362":19.56,"7364":38.21,"7365":50.71,"7366":41.0,"7367":13.05,"7368":4.37,"7369":5.0,"7370":5.5,"7371":8.75,"7373":7.02,"7374":12.66,"7375":169.43,"7376":1.97,"7383":5.75,"7385":13.81,"7386":25.38,"7387":2.24,"7388":17.0,"7390":87.14,"7391":9.58,"7392":2.24,"7394":691.2,"7395":2.1,"7396":1.69,"7398":6.3,"7399":102.43,"7400":9.8,"7401":12.0,"7402":7.75,"7404":20.0,"7405":9.93,"7406":2.24,"7408":2.24,"7409":50.0,"7410":5.97,"7411":29.95,"7412":27.07,"7413":1.6,"7414":5.83,"7415":6.7,"7416":9.29,"7417":67.8,"7418":45.49,"7419":1.6,"7420":8.25,"7421":6.73,"7422":65.36,"7423":21.0,"7424":51.29,"7426":2.24,"7427":46.35,"7428":2.5,"7429":4.34,"7430":2.24,"7431":2.5,"7432":40.0,"7433":15.24,"7434":6.45,"7435":25.0,"7436":6.73,"7437":3.79,"7438":2.5,"7439":112.13,"7440":2.52,"7441":2.24,"7442":30.07,"7443":7.57,"7444":4.19,"7446":2.7,"7447":3.79,"7448":21.9,"7449":8.76,"7450":13.9,"7451":28.35,"7452":13.0,"7453":31.74,"7454":3.3,"7455":13.95,"7456":4.49,"7457":7.25,"7458":2.57,"7459":19.0,"7460":3.79,"7461":100.0,"7462":20.89,"7463":5.75,"7465":30.73,"7467":2.57,"7468":37.19,"7469":4.32,"7470":4.12,"7471":1.55,"7472":41.68,"7473":2.57,"7475":5.93,"7476":75.94,"7477":19.91,"7478":16.08,"7480":22.89,"7481":7.83,"7482":13.87,"7483":189.92,"7484":4.37,"7485":4.87,"7486":2.24,"7487":20.69,"7488":55.48,"7489":54.96,"7490":11.63,"7491":13.95,"7492":7.95,"7493":39.39,"7494":5.4,"7495":22.09,"7496":18.62,"7497":5.38,"7498":12.0,"7499":5.95,"7500":7.44,"7501":42.37,"7502":3.27,"7503":3.9,"7504":108.32,"7505":34.38,"7507":294.38,"7508":35.0,"7509":49.21,"7510":164.24,"7511":50.07,"7513":48.67,"7514":10.81,"7515":10.98,"7516":52.2,"7517":38.85,"7519":29.35,"7520":45.32,"7521":49.32,"7523":90.52,"7526":32.86,"7527":12.89,"7528":39.43,"7529":3.79,"7530":171.05,"7531":33.96,"7532":10.5,"7533":64.28,"7536":43.46,"7537":17.41,"7538":20.3,"7539":23.7,"7540":14.36,"7541":25.45,"7542":9.64,"7543":47.9,"7544":32.48,"7545":39.43,"7546":91.03,"7547":151.54,"7548":14.86,"7549":11.9,"7550":8.65,"7551":50.23,"7552":10.81,"7553":3.27,"7554":2.52,"7555":11.4,"7556":7.95,"7558":92.38,"7560":300.0,"7561":4.99,"7562":30.88,"7563":54.11,"7565":107.17,"7567":310.98,"7568":124.92,"7570":27.34,"7571":2.5,"7572":43.83,"7573":14.8,"7574":16.46,"7575":45.35,"7576":13.62,"7577":41.71,"7578":39.25,"7579":15.82,"7580":34.15,"7581":9.69,"7582":14.06,"7583":53.77,"7585":31.35,"7586":9.19,"7587":51.83,"7588":10.5,"7589":16.21,"7590":11.48,"7591":16.18,"7592":1.34,"7593":1.99,"7594":6.28,"7595":8.29,"7596":9.67,"7597":43.26,"7598":47.61,"7599":3.68,"7601":141.93,"7602":10.88,"7603":16.2,"7604":197.67,"7605":875.5,"7606":58.79,"7607":13.65,"7608":3.49,"7610":48.11,"7613":37.04,"7614":45.46,"7615":15.52,"7617":36.83,"7618":54.32,"7619":2.0,"7620":5.0,"7622":200.0,"7623":169.42,"7625":895.06,"7626":5.0,"7627":204.76,"7630":0.73,"7631":0.11,"7632":5.0,"7633":24.59,"7634":3.86,"7635":136.62,"7636":19.27,"7637":28.52,"7638":126.3,"7639":8.99,"7641":8.64,"7642":7.03,"7643":27.05,"7644":2.13,"7645":47.7,"7646":10.18,"7647":14.39,"7648":3.11,"7649":69.46,"7650":10.81,"7651":9.9,"7652":9.74,"7653":14.26,"7654":21.04,"7656":33.25,"7657":61.69,"7659":6.03,"7660":6.03,"7662":33.31,"7663":2.25,"7664":0.96,"7665":1.0,"7666":166.66,"7668":6.69,"7670":902.76,"7671":2.7,"7672":9.74,"7673":2.25,"7674":11.73,"7676":2.25,"7677":2.25,"7678":2.25,"7679":2.15,"7680":2.17,"7681":1.54,"7683":2.25,"7684":12.41,"7685":5.94,"7686":1.81,"7687":32.67,"7688":2.25,"7689":2.58,"7690":76.42,"7691":4.25,"7692":5.79,"7693":2.95,"7694":40.0,"7695":2.5,"7696":2.25,"7697":2.25,"7698":10.31,"7699":58.79,"7700":8.94,"7701":9.72,"7702":9.3,"7703":4.6,"7704":4.65,"7706":6.04,"7707":23.79,"7708":2.91,"7709":101.39,"7710":37.93,"7712":2.95,"7713":132.68,"7714":1.28,"7715":927.35,"7716":47.18,"7718":7.96,"7719":3.29,"7720":3.5,"7721":2.52,"7722":1.72,"7723":2.17,"7724":8.33,"7725":4.77,"7726":3.64,"7727":12.83,"7728":92.1,"7730":15.64,"7732":6.44,"7733":4.33,"7735":64.0,"7736":8.77,"7737":29.7,"7738":2.0,"7739":2.48,"7740":15.27,"7741":36.44,"7742":11.9,"7743":7.5,"7744":52.03,"7745":29.44,"7746":8.0,"7747":19.06,"7748":43.53,"7749":11.66,"7750":18.46,"7751":2.0,"7752":19.5,"7753":3.02,"7755":30.09,"7756":18.7,"7757":11.92,"7758":20.0,"7759":5.3,"7760":3.88,"7761":19.71,"7762":2.14,"7763":2.25,"7764":2.52,"7765":139.56,"7766":18.7,"7767":2.29,"7768":53.34,"7769":875.5,"7770":17.8,"7771":2.35,"7774":20.0,"7775":1.61,"7776":19.99,"7777":37.82,"7778":71.15,"7779":9.95,"7780":1.08,"7781":27.98,"7782":1.17,"7783":57.35,"7784":3.01,"7785":2.9,"7786":54.61,"7787":5.94,"7788":1.0,"7789":12.99,"7790":18.41,"7791":140.35,"7792":13.72,"7793":26.26,"7795":38.96,"7797":31.46,"7798":32.78,"7799":10.15,"7800":3.75,"7801":8.66,"7803":8.15,"7804":2.15,"7805":66.12,"7806":1.0,"7807":87.87,"7808":2.7,"7809":8.76,"7810":1.47,"7811":2.15,"7812":9.8,"7813":903.63,"7814":53.34,"7815":7.57,"7816":155.16,"7817":2.9,"7818":292.0,"7819":5.0,"7821":44.06,"7822":2.15,"7823":3.0,"7824":20.0,"7825":19.0,"7828":2.5,"7830":5.12,"7831":35.0,"7833":2.99,"7835":14.57,"7836":4.45,"7838":2.25,"7839":4.33,"7841":80.0,"7842":47.88,"7843":18.4,"7844":34.9,"7846":41.89,"7847":27.8,"7848":8.99,"7849":1.5,"7850":110.0,"7851":85.0,"7852":14.88,"7853":28.13,"7854":1.38,"7855":12.44,"7856":16.14,"7857":5.41,"7858":3.56,"7859":3.44,"7860":62.06,"7861":118.9,"7862":6.77,"7864":24.09,"7865":6.0,"7866":18.38,"7867":12.88,"7868":12.09,"7869":85.0,"7870":56.0,"7872":100.0,"7873":146.05,"7874":902.44,"7876":62.06,"7877":75.0,"7880":40.29,"7881":34.85,"7882":5.4,"7883":1.25,"7884":4.2,"7885":23.58,"7886":9.3,"7887":2.15,"7889":1.0,"7890":4.08,"7891":200.0,"7892":189.43,"7894":19.92,"7895":2.06,"7897":2.8,"7898":26.89,"7899":10.28,"7900":83.02,"7901":15.53,"7902":100.0,"7903":12.95,"7904":1.84,"7905":1.0,"7906":2.25,"7907":5.73,"7908":6.66,"7909":128.3,"7910":5.94,"7913":2.06,"7914":40.0,"7916":1.54,"7917":3.01,"7920":904.13,"7921":62.06,"7922":172.54,"7924":20.99,"7925":48.96,"7926":29.69,"7927":75.76,"7928":16.32,"7929":12.96,"7930":80.0,"7931":15.37,"7932":26.0,"7933":2.06,"7934":12.97,"7936":1.0,"7937":71.96,"7938":7.13,"7939":6.1,"7941":1.0,"7942":19.95,"7943":118.31,"7944":17.57,"7945":10.52,"7946":6.74,"7947":45.95,"7948":19.95,"7949":10.83,"7950":5.04,"7951":42.38,"7952":6.91,"7953":44.65,"7954":3.01,"7955":1.79,"7956":124.89,"7957":12.95,"7958":80.0,"7959":2.9,"7960":1.5,"7961":1.54,"7962":17.38,"7963":2.25,"7964":3.79,"7966":1.54,"7967":6.91,"7968":6.12,"7969":5.0,"7970":183.91,"7971":1.0,"7972":1.0,"7973":5.72,"7974":49.77,"7975":9.61,"7976":27.88,"7977":19.41,"7978":3.56,"7979":7.57,"7980":20.0,"7981":5.28,"7983":3.79,"7984":27.6,"7985":27.6,"7987":5.94,"7988":8.59,"7989":7.35,"7990":35.11,"7991":2.0,"7992":9.18,"7993":10.0,"7994":903.42,"7995":1.0,"7997":1.0,"7998":43.3,"8000":14.26,"8001":4.28,"8002":77.33,"8005":1.0,"8007":43.5,"8008":20.0,"8009":2.0,"8010":4.06,"8011":2.15,"8013":8.65,"8015":5.73,"8019":1.0,"8020":4.33,"8021":4.57,"8022":58.43,"8023":4.22,"8024":13.17,"8025":15.62,"8027":42.2,"8028":11.3,"8031":35.0,"8033":11.43,"8034":907.42,"8035":6.69,"8036":4.85,"8037":5.12,"8038":315.76,"8039":11.91,"8040":78.38,"8042":14.67,"8043":2.08,"8045":1.08,"8047":24.79,"8048":5.73,"8049":3.02,"8050":7.57,"8051":6.27,"8052":4.36,"8053":4.13,"8054":11.45,"8055":3.98,"8056":40.84,"8058":29.89,"8059":99.8,"8060":37.7,"8063":296.71,"8065":3.99,"8067":5.23,"8068":31.97,"8069":909.89,"8070":4.69,"8072":14.07,"8073":2.15,"8074":10.11,"8075":24.89,"8076":93.05,"8077":2.15,"8078":8.93,"8079":2.48,"8080":2.15,"8081":5.59,"8082":14.28,"8083":1.08,"8084":2.15,"8085":21.0,"8087":7.57,"8088":40.96,"8089":34.52,"8090":41.1,"8091":11.72,"8092":6.06,"8093":2.15,"8094":7.21,"8096":13.39,"8097":8.66,"8098":128.72,"8099":1.0,"8100":40.47,"8101":7.57,"8102":11.12,"8103":23.76,"8104":9.39,"8106":19.47,"8107":144.43,"8109":2.25,"8110":12.79,"8111":14.6,"8112":42.22,"8113":252.6,"8115":87.69,"8116":11.99,"8117":9.74,"8118":905.55,"8119":43.93,"8120":12.98,"8121":6.22,"8123":44.51,"8124":24.0,"8125":16.0,"8126":25.96,"8127":13.53,"8129":1.08,"8130":42.9,"8131":22.72,"8132":47.41,"8134":16.92,"8135":4.98,"8136":3.0,"8137":10.41,"8138":7.03,"8139":2.25,"8140":3.52,"8141":71.45,"8142":27.62,"8143":2.25,"8144":9.56,"8145":12.99,"8146":5.28,"8148":42.08,"8149":284.05,"8150":30.39,"8151":1.08,"8152":3.69,"8153":35.48,"8154":29.15,"8155":59.2,"8156":30.39,"8157":43.53,"8159":20.31,"8161":9.0,"8162":5.41,"8163":6.38,"8164":5.15,"8165":2.0,"8166":24.36,"8167":6.18,"8168":1.83,"8169":37.86,"8170":8.65,"8171":20.88,"8173":0.99,"8174":243.13,"8175":23.75,"8177":13.1,"8178":4.96,"8180":869.68,"8181":1.39,"8184":111.13,"8187":7.2,"8188":5.88,"8189":6.45,"8190":38.63,"8191":7.38,"8192":13.57,"8195":1.39,"8196":6.8,"8197":76.64,"8199":31.92,"8200":32.42,"8201":24.93,"8202":29.22,"8203":5.65,"8204":8.5,"8205":14.44,"8206":0.36,"8207":3.29,"8208":11.32},"Category":{"5":"Electronics & Software","8":"Restaurants","9":"Restaurants","10":"Shopping","11":"Groceries","12":"Television","13":"Parking","14":"Groceries","15":"Parking","16":"Shopping","17":"Restaurants","18":"Coffee Shops","19":"Parking","20":"Restaurants","21":"Parking","22":"Parking","23":"Coffee Shops","24":"Gym","25":"Parking","26":"Restaurants","27":"Restaurants","28":"Fast Food","29":"Advertising","30":"Entertainment","31":"Entertainment","32":"Fast Food","33":"Coffee Shops","34":"Coffee Shops","35":"Clothing","38":"Gas & Fuel","45":"Travel","46":"Furnishings","47":"Gas & Fuel","48":"Electronics & Software","49":"Music","50":"Groceries","52":"Books","53":"Movies & DVDs","54":"Service & Parts","55":"Fast Food","56":"Groceries","61":"Newspapers & Magazines","63":"Gas & Fuel","64":"Restaurants","65":"Groceries","66":"Groceries","67":"Arts","68":"Electronics & Software","69":"Gas & Fuel","70":"Gym","71":"Groceries","72":"Student Loan","73":"Restaurants","74":"Public Transportation","77":"Electronics & Software","78":"Restaurants","79":"Finance Charge","80":"Finance Charge","83":"Coffee Shops","84":"Parking","85":"Shopping","88":"Parking","89":"Parking","90":"Parking","91":"Groceries","92":"Restaurants","95":"Auto Insurance","96":"Parking","97":"Parking","98":"Parking","99":"Groceries","100":"Ride Share","101":"Parking","102":"Parking","103":"Parking","104":"Public Transportation","105":"Parking","107":"Gas & Fuel","108":"Restaurants","110":"Restaurants","113":"Groceries","114":"Shopping","116":"Business Services","117":"Mortgage & Rent","118":"Groceries","119":"Fees & Charges","120":"Spa & Massage","121":"Coffee Shops","122":"Shopping","123":"Shopping","124":"Parking","125":"Parking","126":"Coffee Shops","127":"Pharmacy","128":"Television","129":"Coffee Shops","130":"Fast Food","131":"Parking","132":"Parking","134":"Music","135":"Parking","136":"Coffee Shops","137":"Advertising","138":"Entertainment","141":"Movies & DVDs","142":"Service Fee","143":"Gift","144":"Groceries","145":"Advertising","146":"Pharmacy","147":"Shopping","148":"Parking","149":"Gas & Fuel","150":"Pharmacy","151":"Restaurants","152":"Alcohol & Bars","153":"Fast Food","154":"Restaurants","155":"Coffee Shops","156":"Parking","157":"Restaurants","158":"Television","161":"Groceries","162":"Coffee Shops","163":"Groceries","164":"Groceries","165":"Auto & Transport","166":"Movies & DVDs","167":"Electronics & Software","168":"Shopping","169":"Restaurants","171":"Shopping","172":"Music","173":"Food & Dining","174":"Gas & Fuel","175":"Restaurants","176":"Rental Car & Taxi","177":"Movies & DVDs","184":"Books","185":"Public Transportation","186":"Restaurants","187":"Movies & DVDs","188":"Pharmacy","189":"Pharmacy","190":"Restaurants","191":"Restaurants","192":"Restaurants","193":"Public Transportation","194":"Pharmacy","195":"Rental Car & Taxi","196":"Public Transportation","197":"Shopping","198":"Entertainment","199":"Public Transportation","200":"Restaurants","201":"Fast Food","202":"Rental Car & Taxi","203":"Shopping","204":"Restaurants","205":"Restaurants","206":"Restaurants","207":"Rental Car & Taxi","208":"Student Loan","209":"Parking","210":"Parking","211":"Parking","212":"Parking","213":"Parking","214":"Parking","216":"Groceries","217":"Gas & Fuel","218":"Travel","219":"Hotel","221":"Groceries","222":"Groceries","223":"Rental Car & Taxi","224":"Restaurants","225":"Restaurants","226":"Rental Car & Taxi","227":"Food & Dining","228":"Gas & Fuel","229":"ATM Fee","230":"Cash & ATM","232":"Auto Insurance","233":"Restaurants","234":"Fast Food","236":"Restaurants","237":"Restaurants","248":"Restaurants","249":"Fast Food","252":"Air Travel","253":"Movies & DVDs","254":"Restaurants","255":"Shopping","256":"Groceries","257":"Mortgage & Rent","260":"Service Fee","261":"Entertainment","262":"Music","263":"Television","264":"Pharmacy","265":"Movies & DVDs","266":"Restaurants","267":"Gas & Fuel","268":"Groceries","269":"Restaurants","270":"Restaurants","271":"Entertainment","272":"Coffee Shops","273":"Public Transportation","274":"Advertising","276":"Groceries","277":"Restaurants","278":"Restaurants","279":"Fast Food","280":"Business Services","282":"Restaurants","283":"Restaurants","284":"Restaurants","287":"Shopping","288":"Television","289":"Shopping","290":"Restaurants","291":"Restaurants","292":"Fast Food","293":"Fast Food","294":"Shopping","295":"Shopping","296":"Sporting Goods","297":"Restaurants","298":"Shopping","299":"Electronics & Software","300":"Movies & DVDs","301":"Groceries","302":"Gas & Fuel","303":"Music","304":"Restaurants","305":"Rental Car & Taxi","306":"Restaurants","307":"Books","309":"Restaurants","310":"Air Travel","311":"Charity","313":"Travel","314":"Shopping","315":"Fast Food","316":"Groceries","317":"Internet","318":"Groceries","323":"Entertainment","324":"Restaurants","333":"Clothing","334":"Restaurants","335":"Coffee Shops","336":"Fast Food","337":"Books","338":"ATM Fee","339":"Cash & ATM","340":"Student Loan","342":"ATM Fee","343":"Utilities","344":"Public Transportation","345":"Utilities","346":"Fast Food","347":"Restaurants","348":"Shopping","349":"Food & Dining","350":"Gas & Fuel","351":"Fast Food","352":"Shopping","353":"Restaurants","354":"Business Services","355":"Groceries","356":"Mortgage & Rent","357":"Sporting Goods","358":"Auto Insurance","359":"Restaurants","360":"Fast Food","361":"Groceries","362":"Gas & Fuel","363":"Vacation","364":"Gas & Fuel","365":"Gas & Fuel","366":"Travel","367":"Gas & Fuel","369":"Entertainment","370":"Groceries","371":"Restaurants","373":"Hotel","374":"Coffee Shops","375":"Rental Car & Taxi","376":"Gas & Fuel","377":"Gas & Fuel","378":"Shopping","379":"Groceries","380":"Rental Car & Taxi","381":"Hotel","382":"Restaurants","384":"Travel","385":"Coffee Shops","386":"Vacation","387":"Alcohol & Bars","389":"Vacation","390":"Vacation","391":"Movies & DVDs","392":"Service & Parts","393":"Vacation","394":"Coffee Shops","395":"Alcohol & Bars","396":"Coffee Shops","397":"Restaurants","398":"ATM Fee","399":"Cash & ATM","400":"Groceries","401":"ATM Fee","402":"Hotel","403":"Coffee Shops","404":"Alcohol & Bars","405":"Fast Food","406":"Shopping","407":"Electronics & Software","408":"Sporting Goods","409":"Rental Car & Taxi","410":"Rental Car & Taxi","411":"Groceries","412":"Television","413":"Parking","414":"Gym","415":"Parking","416":"Parking","417":"Groceries","418":"Parking","419":"Parking","420":"Mortgage & Rent","421":"Travel","422":"Fast Food","423":"Coffee Shops","425":"Gas & Fuel","426":"Parking","427":"Gas & Fuel","428":"Restaurants","429":"Parking","430":"Coffee Shops","431":"Groceries","432":"Parking","433":"Parking","434":"Parking","435":"Movies & DVDs","436":"Pharmacy","437":"Entertainment","438":"Shopping","439":"Parking","440":"Parking","441":"Parking","442":"Parking","443":"Restaurants","444":"Advertising","445":"Restaurants","446":"Shopping","451":"Parking","452":"Restaurants","453":"Parking","455":"Coffee Shops","460":"Groceries","461":"Restaurants","462":"Vacation","463":"Groceries","464":"Parking","465":"Parking","466":"Coffee Shops","467":"Movies & DVDs","468":"Parking","469":"Parking","470":"Clothing","471":"Electronics & Software","472":"Shopping","473":"Movies & DVDs","474":"Restaurants","475":"Television","476":"Air Travel","477":"Groceries","478":"Shopping","479":"Entertainment","481":"Parking","482":"Business Services","483":"Restaurants","484":"Charity","485":"Gym","486":"Shopping","487":"Shopping","490":"Gas & Fuel","491":"Groceries","492":"Travel","493":"Vacation","494":"Groceries","495":"Sporting Goods","496":"Restaurants","497":"Restaurants","500":"ATM Fee","501":"Cash & ATM","504":"ATM Fee","507":"Restaurants","508":"Gas & Fuel","509":"Student Loan","510":"Fast Food","511":"Hotel","512":"Groceries","514":"Restaurants","517":"Books","518":"Coffee Shops","519":"Restaurants","520":"Gas & Fuel","521":"Groceries","522":"Groceries","523":"Air Travel","524":"Air Travel","525":"Groceries","526":"Electronics & Software","527":"Service & Parts","528":"Parking","529":"Coffee Shops","530":"Coffee Shops","531":"Air Travel","532":"Air Travel","533":"Hotel","540":"Restaurants","541":"Coffee Shops","542":"Restaurants","545":"Books","546":"Coffee Shops","547":"Restaurants","548":"Restaurants","549":"Restaurants","551":"Groceries","552":"Food & Dining","553":"Shopping","554":"Coffee Shops","556":"Restaurants","557":"Hotel","558":"Books","559":"Restaurants","560":"Books","561":"Gas & Fuel","562":"Hotel","563":"Groceries","564":"Coffee Shops","565":"Restaurants","566":"Groceries","567":"Gas & Fuel","568":"Groceries","569":"Gym","570":"Fast Food","571":"Fast Food","572":"Television","575":"Gas & Fuel","576":"Gas & Fuel","577":"Food & Dining","578":"Groceries","579":"Gas & Fuel","580":"Groceries","582":"Gas & Fuel","583":"Gas & Fuel","584":"Restaurants","585":"Advertising","586":"Fast Food","587":"Groceries","588":"Groceries","589":"Gas & Fuel","590":"Coffee Shops","591":"Coffee Shops","592":"Parking","593":"Clothing","594":"Shopping","595":"Clothing","596":"Groceries","600":"Coffee Shops","601":"Food & Dining","602":"Television","603":"Food & Dining","604":"Groceries","605":"Groceries","606":"Electronics & Software","607":"Fast Food","610":"Food & Dining","611":"Food & Dining","612":"Restaurants","613":"Groceries","614":"Restaurants","615":"Fast Food","616":"Fast Food","617":"Coffee Shops","618":"Fast Food","619":"Gas & Fuel","620":"Gas & Fuel","621":"Vacation","622":"Movies & DVDs","623":"Restaurants","624":"Service & Parts","625":"Books","626":"Groceries","627":"Restaurants","629":"Gym","632":"Groceries","633":"Coffee Shops","637":"Shopping","638":"Groceries","639":"Shopping","640":"Restaurants","641":"Shopping","643":"Student Loan","645":"Groceries","647":"Gas & Fuel","648":"Groceries","649":"Restaurants","650":"Shopping","651":"Coffee Shops","652":"Restaurants","654":"Fast Food","655":"Coffee Shops","656":"Groceries","657":"Restaurants","658":"Hotel","659":"Coffee Shops","660":"Restaurants","661":"Hotel","662":"Fast Food","663":"Groceries","664":"Fast Food","665":"Restaurants","666":"Fast Food","667":"Restaurants","668":"Fast Food","669":"Fast Food","670":"Restaurants","674":"Groceries","675":"Coffee Shops","676":"Shopping","677":"Coffee Shops","678":"Gas & Fuel","679":"Coffee Shops","681":"Gas & Fuel","687":"Coffee Shops","688":"Groceries","689":"Restaurants","691":"Coffee Shops","692":"Groceries","693":"Gym","694":"Restaurants","695":"Shopping","696":"ATM Fee","697":"Cash & ATM","698":"ATM Fee","700":"Amusement","701":"Movies & DVDs","702":"Gas & Fuel","703":"Groceries","704":"Hotel","705":"Business Services","706":"Business Services","708":"Parking","709":"Bank Fee","714":"Restaurants","715":"Gym","716":"Advertising","721":"Gas & Fuel","722":"Groceries","724":"Sporting Goods","725":"Gas & Fuel","726":"Gas & Fuel","727":"Gas & Fuel","728":"Gas & Fuel","729":"Service & Parts","730":"Gas & Fuel","731":"Groceries","732":"Gas & Fuel","733":"Gas & Fuel","734":"Fast Food","735":"Movies & DVDs","737":"Groceries","738":"Gas & Fuel","739":"Electronics & Software","740":"Groceries","741":"Coffee Shops","742":"Gas & Fuel","743":"Groceries","744":"Groceries","745":"Fast Food","746":"Fast Food","747":"Fast Food","748":"Coffee Shops","750":"Coffee Shops","751":"Fast Food","752":"Charity","755":"Restaurants","756":"Groceries","758":"Gas & Fuel","761":"Student Loan","764":"Restaurants","765":"Restaurants","766":"Coffee Shops","767":"Coffee Shops","768":"Coffee Shops","769":"Fast Food","770":"Hair","772":"Gas & Fuel","773":"Groceries","776":"Fast Food","777":"Groceries","779":"Gym","781":"Gas & Fuel","782":"Coffee Shops","783":"Restaurants","784":"Coffee Shops","785":"Groceries","786":"Gym","787":"Fast Food","788":"Coffee Shops","789":"Coffee Shops","790":"Coffee Shops","791":"Groceries","792":"Fast Food","793":"Groceries","794":"Shopping","795":"Entertainment","798":"Coffee Shops","800":"Groceries","804":"Coffee Shops","805":"Fast Food","806":"Gas & Fuel","807":"Coffee Shops","808":"Restaurants","809":"Coffee Shops","810":"Coffee Shops","811":"Coffee Shops","812":"Movies & DVDs","813":"Groceries","816":"Restaurants","818":"Gym","819":"Coffee Shops","820":"Advertising","826":"Coffee Shops","827":"Coffee Shops","828":"Coffee Shops","829":"Coffee Shops","830":"Groceries","831":"Printing","832":"Groceries","833":"Personal Care","834":"Gas & Fuel","835":"Groceries","836":"Movies & DVDs","837":"Coffee Shops","838":"Restaurants","839":"Service & Parts","840":"Electronics & Software","841":"Coffee Shops","842":"Groceries","844":"Groceries","847":"Restaurants","848":"Coffee Shops","849":"Groceries","851":"Coffee Shops","853":"Public Transportation","854":"Coffee Shops","858":"Charity","859":"Coffee Shops","860":"Coffee Shops","861":"Alcohol & Bars","862":"Restaurants","863":"Restaurants","864":"Restaurants","865":"Shopping","866":"Restaurants","867":"Coffee Shops","868":"Shopping","872":"Student Loan","875":"Groceries","876":"Gas & Fuel","877":"Coffee Shops","878":"Coffee Shops","879":"Coffee Shops","880":"Restaurants","881":"Coffee Shops","882":"Hair","883":"Alcohol & Bars","885":"Alcohol & Bars","886":"Groceries","887":"Restaurants","888":"Service Fee","891":"Gas & Fuel","892":"Groceries","893":"Groceries","896":"Groceries","897":"Alcohol & Bars","898":"Groceries","899":"Coffee Shops","900":"Coffee Shops","901":"Coffee Shops","902":"Restaurants","903":"Books","904":"Groceries","905":"Coffee Shops","906":"Entertainment","907":"Movies & DVDs","908":"Groceries","911":"Coffee Shops","912":"Hotel","913":"Groceries","914":"Gas & Fuel","916":"Coffee Shops","917":"Coffee Shops","918":"Groceries","919":"Coffee Shops","920":"Electronics & Software","921":"Shopping","922":"Fast Food","923":"Gym","924":"Gas & Fuel","925":"Fast Food","926":"Restaurants","927":"Groceries","928":"Groceries","929":"Restaurants","931":"Restaurants","937":"Movies & DVDs","938":"Gas & Fuel","939":"Electronics & Software","940":"Parking","941":"Coffee Shops","942":"Coffee Shops","944":"Groceries","945":"Groceries","946":"Gym","947":"Fast Food","948":"Coffee Shops","949":"Fast Food","950":"Charity","951":"Parking","952":"Fast Food","953":"Coffee Shops","954":"Fast Food","955":"Shopping","956":"Shopping","958":"Coffee Shops","959":"Hotel","960":"Student Loan","961":"Coffee Shops","962":"Restaurants","963":"Restaurants","964":"Gas & Fuel","965":"Groceries","966":"Groceries","967":"Restaurants","968":"Gym","970":"Groceries","972":"Shopping","974":"Home Improvement","975":"Fast Food","976":"Music","978":"Books","980":"Fast Food","986":"Groceries","987":"Movies & DVDs","989":"Restaurants","990":"Movies & DVDs","992":"Parking","993":"Gym","994":"Advertising","995":"Gas & Fuel","996":"Groceries","997":"Gym","1000":"Coffee Shops","1001":"Groceries","1003":"Internet","1004":"Gym","1005":"Restaurants","1006":"Movies & DVDs","1007":"Fast Food","1008":"Furnishings","1009":"Electronics & Software","1010":"Restaurants","1012":"Groceries","1013":"Groceries","1015":"Entertainment","1016":"Gas & Fuel","1017":"Hair","1018":"Fast Food","1019":"Fast Food","1022":"Groceries","1028":"Gas & Fuel","1029":"Gas & Fuel","1030":"Gas & Fuel","1031":"Gas & Fuel","1032":"Gas & Fuel","1033":"Coffee Shops","1034":"Student Loan","1035":"Gym","1036":"Groceries","1037":"Gas & Fuel","1038":"Shopping","1039":"Furnishings","1040":"Coffee Shops","1041":"Groceries","1042":"Business Services","1043":"Coffee Shops","1044":"Coffee Shops","1045":"Home Improvement","1046":"Groceries","1047":"Shopping","1048":"Shopping","1049":"Electronics & Software","1050":"Electronics & Software","1051":"Fast Food","1052":"Groceries","1055":"Hotel","1056":"Fast Food","1057":"Groceries","1058":"Movies & DVDs","1059":"Hair","1060":"Home Improvement","1061":"Hotel","1062":"Hotel","1063":"Coffee Shops","1064":"Groceries","1065":"Groceries","1066":"Service Fee","1067":"Coffee Shops","1068":"Restaurants","1069":"Electronics & Software","1070":"Gas & Fuel","1071":"Home Improvement","1072":"Groceries","1073":"Fast Food","1074":"Gym","1077":"Groceries","1078":"Gym","1086":"Student Loan","1088":"Restaurants","1089":"Movies & DVDs","1090":"Electronics & Software","1091":"Groceries","1092":"Groceries","1096":"Mortgage & Rent","1097":"Home Improvement","1099":"Groceries","1100":"Travel","1101":"Groceries","1102":"Travel","1103":"Restaurants","1104":"Restaurants","1105":"Business Services","1106":"Restaurants","1107":"Student Loan","1108":"Travel","1109":"Restaurants","1110":"Vacation","1111":"Vacation","1112":"Groceries","1113":"Restaurants","1114":"Utilities","1115":"Coffee Shops","1116":"Groceries","1117":"Gas & Fuel","1118":"Groceries","1120":"Vacation","1121":"Groceries","1126":"Hair","1127":"Fast Food","1130":"Groceries","1131":"Home Improvement","1132":"Books","1133":"Gas & Fuel","1134":"Movies & DVDs","1136":"Restaurants","1139":"Shopping","1140":"Electronics & Software","1141":"Shipping","1142":"Groceries","1145":"Fast Food","1146":"Gym","1155":"Groceries","1156":"Advertising","1157":"Gas & Fuel","1158":"Home Services","1159":"Restaurants","1160":"Gas & Fuel","1161":"Auto & Transport","1162":"Doctor","1163":"Restaurants","1164":"Groceries","1166":"Service & Parts","1167":"Shopping","1168":"Shopping","1169":"Movies & DVDs","1170":"Coffee Shops","1171":"Shopping","1172":"Bank Fee","1173":"Service & Parts","1174":"Groceries","1175":"Electronics & Software","1176":"Restaurants","1177":"Groceries","1179":"Groceries","1180":"Coffee Shops","1181":"Groceries","1182":"Restaurants","1183":"Groceries","1184":"Restaurants","1185":"Home Services","1186":"Movies & DVDs","1187":"Student Loan","1188":"Personal Care","1191":"Restaurants","1192":"Movies & DVDs","1193":"Fast Food","1194":"Shopping","1195":"Groceries","1196":"Business Services","1197":"Groceries","1198":"Shopping","1199":"Groceries","1200":"Gym","1203":"Sports","1205":"Groceries","1210":"Groceries","1215":"Hair","1216":"Movies & DVDs","1217":"Mortgage & Rent","1218":"Shopping","1222":"Gas & Fuel","1223":"Groceries","1224":"Service & Parts","1225":"Groceries","1226":"Fast Food","1227":"Utilities","1228":"Gym","1229":"Groceries","1230":"Advertising","1231":"Public Transportation","1233":"Restaurants","1234":"Groceries","1235":"Fast Food","1236":"Movies & DVDs","1237":"Shopping","1238":"Electronics & Software","1239":"Gas & Fuel","1240":"Groceries","1242":"Gift","1243":"Food & Dining","1244":"Air Travel","1245":"Restaurants","1246":"Restaurants","1248":"Student Loan","1250":"Groceries","1251":"Hotel","1252":"Rental Car & Taxi","1253":"Rental Car & Taxi","1254":"Gift","1255":"Gift","1256":"Shopping","1258":"Groceries","1259":"Air Travel","1260":"Air Travel","1262":"Business Services","1263":"Gym","1264":"Rental Car & Taxi","1266":"Groceries","1267":"Fast Food","1272":"Groceries","1273":"Doctor","1278":"Entertainment","1279":"Groceries","1281":"Personal Care","1282":"Entertainment","1283":"Groceries","1284":"Home Improvement","1285":"Shopping","1286":"Groceries","1287":"Movies & DVDs","1288":"Restaurants","1289":"Groceries","1291":"Mortgage & Rent","1293":"Air Travel","1294":"Electronics & Software","1295":"Air Travel","1296":"Advertising","1297":"Gym","1298":"Hotel","1301":"Business Services","1302":"Electronics & Software","1303":"Doctor","1305":"Groceries","1306":"Gas & Fuel","1307":"Groceries","1308":"Movies & DVDs","1309":"Groceries","1310":"Pharmacy","1311":"Electronics & Software","1312":"Groceries","1314":"Groceries","1315":"Personal Care","1316":"Restaurants","1317":"Food & Dining","1318":"Groceries","1319":"Restaurants","1320":"Restaurants","1321":"Groceries","1322":"Restaurants","1323":"Student Loan","1324":"Doctor","1325":"Business Services","1326":"Groceries","1328":"Gym","1329":"Groceries","1330":"Alcohol & Bars","1331":"Groceries","1332":"Groceries","1333":"Entertainment","1334":"Electronics & Software","1338":"Entertainment","1339":"Hair","1340":"Groceries","1345":"Entertainment","1347":"Gas & Fuel","1348":"Groceries","1349":"Restaurants","1350":"Groceries","1351":"Movies & DVDs","1352":"Restaurants","1353":"Restaurants","1356":"Mortgage & Rent","1357":"Doctor","1358":"Entertainment","1359":"Groceries","1361":"Restaurants","1362":"Restaurants","1363":"Groceries","1364":"Gym","1366":"Service Fee","1367":"Parking","1368":"Alcohol & Bars","1369":"Groceries","1370":"Groceries","1371":"Business Services","1372":"Hair","1374":"Groceries","1375":"Gift","1376":"Electronics & Software","1378":"Coffee Shops","1380":"Groceries","1381":"Groceries","1382":"Advertising","1383":"Doctor","1385":"Auto & Transport","1387":"Groceries","1388":"Doctor","1389":"Student Loan","1390":"Shopping","1391":"Alcohol & Bars","1392":"Shopping","1393":"Groceries","1394":"Rental Car & Taxi","1395":"Business Services","1397":"Entertainment","1398":"Gym","1402":"Restaurants","1407":"Gas & Fuel","1408":"Groceries","1409":"Entertainment","1411":"Gym","1412":"Fast Food","1413":"Clothing","1414":"Groceries","1419":"Air Travel","1420":"Air Travel","1421":"Hotel","1422":"Service Fee","1423":"Air Travel","1424":"Air Travel","1426":"Movies & DVDs","1427":"Gift","1428":"Restaurants","1429":"Restaurants","1430":"Travel","1431":"Food & Dining","1432":"Gas & Fuel","1433":"Alcohol & Bars","1436":"Mortgage & Rent","1437":"Gym","1438":"Groceries","1439":"Restaurants","1441":"Business Services","1443":"Hair","1444":"Doctor","1447":"Gym","1448":"Groceries","1449":"Electronics & Software","1452":"Groceries","1453":"Groceries","1454":"Doctor","1456":"Student Loan","1458":"Furnishings","1459":"Groceries","1460":"Groceries","1461":"Fast Food","1463":"Business Services","1464":"Groceries","1466":"Entertainment","1467":"Hair","1471":"Entertainment","1472":"Gas & Fuel","1473":"Coffee Shops","1475":"Groceries","1476":"Gym","1477":"Entertainment","1479":"Shopping","1480":"Entertainment","1484":"Groceries","1486":"Restaurants","1487":"Restaurants","1488":"Spa & Massage","1489":"Rental Car & Taxi","1490":"Books","1491":"Fast Food","1492":"Rental Car & Taxi","1493":"Groceries","1494":"Fast Food","1495":"Restaurants","1496":"Movies & DVDs","1497":"Groceries","1501":"Mortgage & Rent","1502":"Gym","1504":"Doctor","1507":"Business Services","1508":"Groceries","1509":"Groceries","1510":"Groceries","1511":"Electronics & Software","1512":"Restaurants","1514":"Groceries","1517":"Rental Car & Taxi","1518":"Groceries","1519":"Restaurants","1520":"Groceries","1524":"Student Loan","1525":"Sporting Goods","1528":"Entertainment","1531":"Entertainment","1536":"Groceries","1537":"Groceries","1538":"Mortgage & Rent","1540":"Movies & DVDs","1542":"Gym","1544":"Groceries","1546":"Business Services","1547":"Electronics & Software","1548":"Groceries","1550":"Student Loan","1555":"Groceries","1557":"Entertainment","1559":"Business Services","1563":"Movies & DVDs","1564":"Mortgage & Rent","1565":"Gym","1566":"Business Services","1569":"Electronics & Software","1574":"Restaurants","1577":"Student Loan","1581":"Entertainment","1584":"Groceries","1589":"Movies & DVDs","1590":"Mortgage & Rent","1593":"Electronics & Software","1597":"Electronics & Software","1602":"Student Loan","1606":"Business Services","1607":"Pharmacy","1615":"Groceries","1617":"Gas & Fuel","1618":"Restaurants","1619":"Movies & DVDs","1620":"Restaurants","1622":"Shopping","1623":"Internet","1625":"Mortgage & Rent","1626":"Pharmacy","1627":"Groceries","1630":"Shopping","1631":"Groceries","1633":"Electronics & Software","1634":"Gas & Fuel","1635":"Gas & Fuel","1636":"Music","1637":"Pharmacy","1638":"Restaurants","1639":"Books","1640":"Shopping","1642":"Groceries","1643":"Business Services","1647":"Home Insurance","1649":"Gas & Fuel","1650":"Fast Food","1651":"Gas & Fuel","1652":"Gas & Fuel","1655":"Shopping","1658":"Groceries","1659":"Auto Insurance","1661":"Bank Fee","1664":"Gas & Fuel","1665":"Gas & Fuel","1666":"Pharmacy","1667":"Pharmacy","1669":"Groceries","1672":"Entertainment","1673":"Entertainment","1674":"Gas & Fuel","1675":"Pharmacy","1676":"Groceries","1680":"Pharmacy","1681":"Movies & DVDs","1682":"Internet","1683":"Utilities","1685":"Mortgage & Rent","1686":"Entertainment","1687":"Gift","1688":"Pharmacy","1689":"Pharmacy","1691":"Utilities","1692":"Pharmacy","1695":"Entertainment","1696":"Pharmacy","1697":"Pharmacy","1698":"Restaurants","1699":"Pharmacy","1700":"Electronics & Software","1701":"Music","1702":"Books","1703":"Restaurants","1704":"Pharmacy","1705":"Fast Food","1707":"Entertainment","1708":"Food & Dining","1709":"Pharmacy","1710":"Shopping","1711":"Shopping","1712":"Groceries","1713":"Business Services","1714":"Shopping","1716":"Pharmacy","1717":"Student Loan","1718":"Home Insurance","1721":"Groceries","1722":"Gas & Fuel","1723":"Auto Insurance","1724":"Shopping","1726":"Pharmacy","1727":"Shopping","1728":"Parking","1731":"Food & Dining","1732":"Restaurants","1733":"Food & Dining","1734":"Gas & Fuel","1735":"Hotel","1736":"Food & Dining","1737":"Food & Dining","1738":"Alcohol & Bars","1739":"Restaurants","1740":"Food & Dining","1741":"Food & Dining","1742":"Alcohol & Bars","1743":"Pharmacy","1744":"Hotel","1745":"Food & Dining","1747":"Restaurants","1748":"Alcohol & Bars","1749":"Pharmacy","1750":"Alcohol & Bars","1751":"Alcohol & Bars","1752":"Food & Dining","1753":"Alcohol & Bars","1754":"Food & Dining","1755":"Alcohol & Bars","1756":"Electronics & Software","1757":"Alcohol & Bars","1758":"Restaurants","1759":"Alcohol & Bars","1760":"Groceries","1761":"Gas & Fuel","1762":"Restaurants","1763":"Home Improvement","1764":"Shopping","1766":"Restaurants","1767":"Service Fee","1770":"Movies & DVDs","1771":"Food & Dining","1772":"Internet","1773":"Air Travel","1774":"Mortgage & Rent","1776":"Rental Car & Taxi","1777":"Groceries","1778":"Music","1779":"Gym","1780":"Gym","1781":"Rental Car & Taxi","1782":"Personal Care","1783":"Music","1784":"Groceries","1785":"Furnishings","1786":"Restaurants","1787":"Shopping","1790":"Electronics & Software","1791":"Restaurants","1792":"Shopping","1793":"Gas & Fuel","1794":"Shopping","1795":"Music","1796":"Entertainment","1797":"Restaurants","1798":"Books","1799":"Shopping","1800":"Pharmacy","1801":"Gym","1802":"Restaurants","1803":"Groceries","1804":"Restaurants","1806":"Business Services","1810":"Student Loan","1811":"Home Insurance","1812":"Shopping","1813":"Shopping","1814":"Gas & Fuel","1815":"Entertainment","1820":"Rental Car & Taxi","1821":"Movies & DVDs","1822":"Life Insurance","1823":"Gas & Fuel","1824":"Music","1825":"Home Improvement","1826":"Personal Care","1827":"Gifts & Donations","1828":"Gas & Fuel","1829":"Food & Dining","1830":"Groceries","1831":"Music","1833":"Auto Insurance","1834":"Shopping","1835":"Shopping","1836":"Entertainment","1837":"Shopping","1840":"Restaurants","1841":"Shopping","1842":"Pharmacy","1843":"Shopping","1844":"Shopping","1845":"Life Insurance","1846":"Utilities","1847":"Lawn & Garden","1848":"Alcohol & Bars","1849":"Shopping","1850":"Gas & Fuel","1851":"Life Insurance","1852":"Groceries","1853":"Shopping","1854":"Restaurants","1856":"Shopping","1857":"Service & Parts","1859":"Shopping","1860":"Groceries","1862":"Gas & Fuel","1864":"Movies & DVDs","1865":"Shopping","1866":"Groceries","1867":"Television","1868":"Mortgage & Rent","1869":"Restaurants","1870":"Alcohol & Bars","1871":"Restaurants","1872":"Food & Dining","1873":"Music","1874":"Air Travel","1876":"Parking","1877":"Gym","1878":"Gym","1879":"Travel","1880":"Travel","1881":"Fast Food","1882":"ATM Fee","1884":"Cash & ATM","1885":"Rental Car & Taxi","1886":"Groceries","1888":"Life Insurance","1889":"Hotel","1890":"Electronics & Software","1891":"Fast Food","1892":"Fast Food","1893":"Groceries","1894":"Restaurants","1895":"Hotel","1896":"Fast Food","1897":"Travel","1898":"Restaurants","1899":"Fast Food","1900":"Kids","1901":"Entertainment","1902":"Pharmacy","1903":"Restaurants","1904":"Restaurants","1905":"Music","1906":"Restaurants","1907":"Parking","1908":"Restaurants","1909":"Books","1910":"Business Services","1911":"Groceries","1912":"Groceries","1913":"Rental Car & Taxi","1914":"Gas & Fuel","1915":"Gas & Fuel","1916":"Food & Dining","1920":"Home Insurance","1921":"Gas & Fuel","1922":"Gas & Fuel","1923":"Gas & Fuel","1924":"Gas & Fuel","1925":"Restaurants","1926":"Business Services","1930":"Student Loan","1931":"Rental Car & Taxi","1932":"Restaurants","1933":"Restaurants","1934":"Movies & DVDs","1935":"Coffee Shops","1936":"Gas & Fuel","1937":"Restaurants","1938":"Restaurants","1939":"Restaurants","1940":"Rental Car & Taxi","1941":"Rental Car & Taxi","1942":"Rental Car & Taxi","1943":"Restaurants","1944":"Restaurants","1945":"Public Transportation","1946":"Groceries","1947":"Restaurants","1948":"Air Travel","1949":"Coffee Shops","1950":"Movies & DVDs","1951":"Movies & DVDs","1952":"Restaurants","1953":"Pharmacy","1954":"Life Insurance","1955":"Restaurants","1956":"Movies & DVDs","1957":"Rental Car & Taxi","1958":"Rental Car & Taxi","1959":"Rental Car & Taxi","1960":"Rental Car & Taxi","1961":"Rental Car & Taxi","1962":"Rental Car & Taxi","1964":"Gifts & Donations","1965":"Gas & Fuel","1966":"Life Insurance","1967":"Clothing","1968":"Personal Care","1969":"Air Travel","1970":"Air Travel","1972":"Shopping","1973":"Shopping","1974":"Clothing","1975":"Auto Insurance","1976":"Groceries","1977":"Restaurants","1978":"Clothing","1979":"Groceries","1980":"Restaurants","1982":"Gas & Fuel","1983":"Food & Dining","1985":"Restaurants","1987":"Restaurants","1988":"Gas & Fuel","1989":"Gas & Fuel","1990":"Fast Food","1991":"Restaurants","1992":"Restaurants","1993":"Movies & DVDs","1994":"Alcohol & Bars","1995":"Movies & DVDs","1996":"Movies & DVDs","1998":"Restaurants","1999":"Groceries","2003":"Mortgage & Rent","2004":"Gas & Fuel","2005":"Lawn & Garden","2006":"Restaurants","2007":"Air Travel","2008":"Movies & DVDs","2009":"Air Travel","2010":"Television","2011":"Life Insurance","2012":"Gas & Fuel","2013":"Restaurants","2014":"Fast Food","2015":"Shopping","2016":"Gas & Fuel","2017":"Restaurants","2018":"Gym","2019":"Gym","2020":"Gas & Fuel","2021":"Gas & Fuel","2022":"Gas & Fuel","2023":"Groceries","2025":"Hotel","2026":"Sporting Goods","2027":"Shopping","2028":"Gas & Fuel","2029":"Movies & DVDs","2030":"Groceries","2031":"Restaurants","2032":"Gas & Fuel","2033":"Restaurants","2034":"Electronics & Software","2035":"Food & Dining","2036":"Travel","2037":"Alcohol & Bars","2038":"Restaurants","2039":"Music","2040":"Restaurants","2042":"Books","2043":"Food & Dining","2044":"Food & Dining","2045":"Groceries","2046":"Travel","2047":"Travel","2048":"Travel","2050":"Gas & Fuel","2051":"Pharmacy","2052":"Restaurants","2053":"Gas & Fuel","2054":"Life Insurance","2055":"Gas & Fuel","2056":"Groceries","2057":"Food & Dining","2058":"Business Services","2060":"Life Insurance","2061":"Restaurants","2063":"Rental Car & Taxi","2064":"Rental Car & Taxi","2065":"Auto Insurance","2067":"Shopping","2068":"Shopping","2071":"Shopping","2072":"Shopping","2073":"Entertainment","2074":"Alcohol & Bars","2075":"Alcohol & Bars","2077":"Auto Insurance","2079":"Food & Dining","2080":"Food & Dining","2081":"Groceries","2082":"Books","2083":"Groceries","2084":"Groceries","2085":"Food & Dining","2086":"Restaurants","2087":"Food & Dining","2088":"Groceries","2089":"Tuition","2090":"Food & Dining","2091":"Shopping","2093":"Groceries","2094":"Shopping","2095":"Alcohol & Bars","2096":"Groceries","2097":"Sporting Goods","2098":"Sporting Goods","2099":"Sporting Goods","2100":"Gas & Fuel","2101":"Shopping","2102":"Fast Food","2103":"Groceries","2104":"Food & Dining","2105":"Restaurants","2106":"Food & Dining","2107":"Restaurants","2108":"Parking","2109":"Groceries","2110":"Cash & ATM","2111":"Food & Dining","2112":"Shopping","2113":"Fast Food","2114":"Food & Dining","2115":"Food & Dining","2116":"Restaurants","2117":"Groceries","2118":"Food & Dining","2119":"Movies & DVDs","2120":"Groceries","2121":"Food & Dining","2122":"Gym","2123":"Check","2124":"Restaurants","2125":"Restaurants","2126":"Fast Food","2127":"Groceries","2128":"Groceries","2129":"Rental Car & Taxi","2130":"Fast Food","2131":"Groceries","2132":"Groceries","2133":"Rental Car & Taxi","2134":"Fast Food","2135":"Fast Food","2136":"Food & Dining","2137":"Hair","2138":"Fast Food","2139":"Fast Food","2140":"Groceries","2141":"Fast Food","2142":"Tuition","2143":"Tuition","2144":"Food & Dining","2147":"Food & Dining","2148":"Restaurants","2149":"Restaurants","2150":"Food & Dining","2151":"Food & Dining","2152":"Shopping","2153":"Groceries","2154":"Food & Dining","2155":"Food & Dining","2156":"Restaurants","2157":"Food & Dining","2158":"Home Improvement","2159":"Food & Dining","2160":"Food & Dining","2161":"Groceries","2162":"Food & Dining","2163":"Sporting Goods","2164":"Entertainment","2165":"Food & Dining","2166":"Groceries","2167":"Groceries","2168":"Coffee Shops","2169":"Movies & DVDs","2170":"Shopping","2171":"Groceries","2172":"Home Insurance","2173":"Movies & DVDs","2174":"Fast Food","2175":"Groceries","2176":"Music","2177":"Sporting Goods","2178":"Restaurants","2179":"Food & Dining","2180":"Home Improvement","2181":"Groceries","2182":"Sporting Goods","2183":"Books","2184":"Shopping","2185":"Food & Dining","2186":"Food & Dining","2188":"Food & Dining","2189":"Education","2190":"Restaurants","2192":"Groceries","2193":"Food & Dining","2194":"Restaurants","2195":"Food & Dining","2196":"Coffee Shops","2197":"Restaurants","2199":"Shopping","2200":"Groceries","2201":"Groceries","2202":"Food & Dining","2203":"Coffee Shops","2204":"Shopping","2205":"Groceries","2206":"Groceries","2207":"Groceries","2208":"Sporting Goods","2209":"Restaurants","2210":"Rental Car & Taxi","2211":"Sporting Goods","2212":"Groceries","2213":"Restaurants","2214":"Groceries","2215":"Food & Dining","2216":"Groceries","2217":"Business Services","2218":"Food & Dining","2219":"Food & Dining","2220":"Groceries","2221":"Pharmacy","2222":"Groceries","2223":"Food & Dining","2224":"Shopping","2225":"Food & Dining","2226":"Tuition","2227":"Groceries","2228":"Food & Dining","2229":"Food & Dining","2230":"Fast Food","2232":"Coffee Shops","2233":"Restaurants","2234":"Restaurants","2235":"Gas & Fuel","2236":"Auto & Transport","2237":"Restaurants","2239":"Auto Insurance","2240":"Groceries","2241":"Groceries","2242":"Fast Food","2243":"Groceries","2244":"Personal Care","2245":"Groceries","2246":"Alcohol & Bars","2247":"Restaurants","2248":"Fast Food","2249":"Coffee Shops","2250":"Alcohol & Bars","2252":"Groceries","2253":"Food & Dining","2254":"Food & Dining","2255":"Restaurants","2256":"Restaurants","2257":"Coffee Shops","2259":"Alcohol & Bars","2260":"Restaurants","2261":"Groceries","2262":"Restaurants","2263":"Food & Dining","2264":"Groceries","2265":"Fast Food","2266":"Food & Dining","2267":"Movies & DVDs","2268":"Food & Dining","2269":"Restaurants","2270":"Food & Dining","2271":"Music","2272":"Shopping","2273":"Rental Car & Taxi","2274":"Rental Car & Taxi","2275":"Fast Food","2276":"Restaurants","2278":"Food & Dining","2279":"Groceries","2280":"Movies & DVDs","2281":"Restaurants","2282":"Hair","2283":"Gym","2284":"Groceries","2285":"Groceries","2286":"Alcohol & Bars","2287":"Movies & DVDs","2288":"Alcohol & Bars","2289":"Restaurants","2290":"Check","2291":"Groceries","2292":"Groceries","2295":"Groceries","2296":"Groceries","2297":"Groceries","2298":"Shopping","2299":"Groceries","2300":"Restaurants","2301":"Groceries","2302":"Groceries","2303":"Shopping","2304":"Groceries","2305":"Home Improvement","2306":"Food & Dining","2307":"Food & Dining","2309":"Shopping","2311":"Shopping","2312":"Music","2313":"Home Insurance","2314":"Books","2316":"Business Services","2317":"Groceries","2318":"Gas & Fuel","2320":"Cash & ATM","2321":"Groceries","2322":"Groceries","2323":"Coffee Shops","2324":"Restaurants","2326":"Groceries","2327":"Groceries","2328":"Groceries","2329":"Fast Food","2330":"Restaurants","2332":"Groceries","2333":"Restaurants","2334":"Fast Food","2335":"Restaurants","2337":"Food & Dining","2338":"Fast Food","2341":"Food & Dining","2342":"Gym","2343":"Food & Dining","2344":"Groceries","2345":"Rental Car & Taxi","2346":"Gift","2347":"Groceries","2348":"Gym","2349":"Clothing","2350":"Gym","2351":"Coffee Shops","2352":"Auto Insurance","2353":"Public Transportation","2354":"Alcohol & Bars","2355":"Food & Dining","2356":"Coffee Shops","2359":"Groceries","2360":"Food & Dining","2361":"Air Travel","2364":"Groceries","2365":"Coffee Shops","2366":"Food & Dining","2367":"Alcohol & Bars","2368":"Food & Dining","2369":"Food & Dining","2373":"Fast Food","2374":"Fast Food","2375":"Movies & DVDs","2376":"Groceries","2377":"Groceries","2378":"Shopping","2379":"Groceries","2380":"Entertainment","2381":"Movies & DVDs","2382":"Gym","2383":"Food & Dining","2384":"Restaurants","2385":"Restaurants","2386":"Food & Dining","2387":"Fast Food","2388":"Restaurants","2389":"Restaurants","2390":"Food & Dining","2391":"Check","2392":"Groceries","2393":"Fast Food","2394":"Food & Dining","2397":"Tuition","2398":"Food & Dining","2399":"Food & Dining","2400":"Fast Food","2401":"Restaurants","2402":"Groceries","2403":"Groceries","2404":"Groceries","2405":"Gas & Fuel","2406":"Shopping","2407":"Restaurants","2408":"Restaurants","2409":"Fast Food","2410":"Gas & Fuel","2412":"Food & Dining","2413":"Uncategorized","2414":"Food & Dining","2415":"Groceries","2417":"Food & Dining","2418":"Groceries","2419":"Gas & Fuel","2420":"Groceries","2421":"Fast Food","2422":"Alcohol & Bars","2423":"Shopping","2424":"Fast Food","2426":"Cash & ATM","2427":"Movies & DVDs","2428":"Food & Dining","2429":"Groceries","2430":"Music","2431":"Home Insurance","2432":"Fast Food","2433":"Business Services","2434":"Groceries","2435":"Gift","2436":"Business Services","2437":"Home Supplies","2438":"Air Travel","2439":"Coffee Shops","2440":"Pharmacy","2441":"Coffee Shops","2442":"Groceries","2443":"Groceries","2444":"Groceries","2445":"Fast Food","2446":"Gas & Fuel","2447":"Restaurants","2448":"Alcohol & Bars","2449":"Alcohol & Bars","2450":"Gas & Fuel","2452":"Gas & Fuel","2454":"Movies & DVDs","2455":"Alcohol & Bars","2456":"Gas & Fuel","2457":"Movies & DVDs","2458":"Restaurants","2459":"Gas & Fuel","2460":"Food & Dining","2461":"Gift","2462":"Fast Food","2463":"Restaurants","2464":"Alcohol & Bars","2465":"Coffee Shops","2466":"Fast Food","2467":"Gas & Fuel","2468":"Gas & Fuel","2469":"Fast Food","2470":"Fast Food","2471":"Restaurants","2472":"Shopping","2473":"Restaurants","2474":"Shopping","2475":"Coffee Shops","2476":"Alcohol & Bars","2477":"Shopping","2478":"Coffee Shops","2479":"Groceries","2480":"Hotel","2481":"Coffee Shops","2482":"Coffee Shops","2483":"Gas & Fuel","2484":"Restaurants","2485":"Auto Insurance","2487":"Groceries","2488":"Fast Food","2490":"Food & Dining","2491":"Groceries","2492":"Groceries","2493":"Coffee Shops","2494":"Air Travel","2496":"Groceries","2497":"Gas & Fuel","2498":"Gym","2499":"Alcohol & Bars","2500":"Restaurants","2501":"Alcohol & Bars","2502":"Alcohol & Bars","2503":"Restaurants","2504":"Parking","2505":"Food & Dining","2506":"Food & Dining","2507":"Gym","2508":"Coffee Shops","2509":"Restaurants","2510":"Restaurants","2511":"Coffee Shops","2512":"Coffee Shops","2513":"Parking","2514":"Restaurants","2515":"Gas & Fuel","2516":"Shopping","2517":"Restaurants","2518":"Movies & DVDs","2519":"Food & Dining","2520":"Check","2521":"Gym","2522":"Utilities","2523":"Coffee Shops","2524":"Coffee Shops","2525":"Gym","2526":"Gas & Fuel","2527":"Groceries","2528":"Movies & DVDs","2529":"Fast Food","2530":"Shopping","2531":"Coffee Shops","2532":"Groceries","2533":"Coffee Shops","2534":"Shopping","2536":"Groceries","2537":"Groceries","2539":"Clothing","2540":"Fast Food","2541":"Fast Food","2543":"Home Insurance","2544":"Music","2545":"Clothing","2546":"Gas & Fuel","2547":"Fast Food","2548":"Business Services","2549":"Fast Food","2550":"Fast Food","2551":"Gas & Fuel","2552":"Hair","2553":"Fast Food","2554":"Gas & Fuel","2555":"Gas & Fuel","2556":"Food & Dining","2557":"Food & Dining","2558":"Restaurants","2559":"Groceries","2562":"Shopping","2563":"Fast Food","2564":"Groceries","2565":"Fast Food","2568":"Groceries","2569":"Groceries","2570":"Movies & DVDs","2571":"Groceries","2572":"Groceries","2573":"Fast Food","2574":"Groceries","2575":"Gas & Fuel","2576":"Groceries","2577":"Gas & Fuel","2578":"Groceries","2579":"Alcohol & Bars","2580":"Food & Dining","2581":"Restaurants","2582":"Groceries","2583":"Fast Food","2584":"Food & Dining","2585":"Restaurants","2588":"Shopping","2589":"Auto Insurance","2592":"Groceries","2593":"Restaurants","2594":"Groceries","2595":"Groceries","2596":"Air Travel","2597":"Food & Dining","2598":"Fast Food","2599":"Alcohol & Bars","2600":"Movies & DVDs","2601":"Fast Food","2602":"Food & Dining","2603":"Groceries","2604":"Restaurants","2606":"Fast Food","2607":"Fast Food","2608":"Shopping","2609":"Groceries","2610":"Movies & DVDs","2611":"Shopping","2612":"Check","2613":"Gym","2614":"Fast Food","2615":"Rental Car & Taxi","2616":"Fast Food","2617":"Pharmacy","2618":"Food & Dining","2619":"Groceries","2620":"Gym","2621":"Fast Food","2622":"Parking","2623":"Movies & DVDs","2624":"Fast Food","2625":"Groceries","2626":"Gas & Fuel","2627":"Rental Car & Taxi","2628":"Alcohol & Bars","2629":"Alcohol & Bars","2630":"Spa & Massage","2631":"Alcohol & Bars","2632":"Restaurants","2633":"Fast Food","2634":"Restaurants","2635":"Pharmacy","2636":"Hotel","2637":"Rental Car & Taxi","2638":"Rental Car & Taxi","2639":"Pharmacy","2640":"Restaurants","2641":"Coffee Shops","2642":"Pharmacy","2643":"Pharmacy","2644":"Coffee Shops","2645":"Groceries","2646":"Restaurants","2647":"Shopping","2648":"Personal Care","2649":"Groceries","2651":"Restaurants","2652":"Gas & Fuel","2654":"Cash & ATM","2655":"Gas & Fuel","2656":"Fast Food","2657":"Sporting Goods","2658":"Fast Food","2659":"Shopping","2660":"Gas & Fuel","2661":"Groceries","2662":"Shopping","2663":"Movies & DVDs","2664":"Movies & DVDs","2665":"Fast Food","2666":"Restaurants","2667":"Shopping","2668":"Groceries","2669":"Food & Dining","2670":"Music","2671":"Restaurants","2672":"Home Insurance","2673":"Sporting Goods","2674":"Tuition","2676":"Shopping","2677":"Business Services","2679":"Shopping","2680":"Gas & Fuel","2682":"Groceries","2683":"Music","2684":"Music","2685":"Shopping","2686":"Shopping","2688":"Groceries","2689":"Air Travel","2690":"Groceries","2691":"Hair","2692":"Food & Dining","2693":"Fast Food","2694":"Gas & Fuel","2695":"Food & Dining","2696":"Movies & DVDs","2697":"Coffee Shops","2698":"Movies & DVDs","2699":"Shopping","2700":"Shopping","2701":"Rental Car & Taxi","2702":"Movies & DVDs","2703":"Alcohol & Bars","2704":"Clothing","2705":"Fast Food","2706":"Gas & Fuel","2707":"Shopping","2708":"Restaurants","2712":"Personal Care","2713":"Shopping","2714":"Shopping","2715":"Shopping","2717":"Auto Insurance","2718":"Entertainment","2719":"Coffee Shops","2720":"Movies & DVDs","2721":"Movies & DVDs","2722":"Shopping","2723":"Groceries","2724":"Groceries","2725":"Shopping","2726":"Shopping","2727":"Restaurants","2728":"Hotel","2729":"Air Travel","2731":"Clothing","2732":"Restaurants","2733":"Groceries","2734":"Alcohol & Bars","2735":"Check","2736":"Pharmacy","2737":"Sporting Goods","2738":"Sporting Goods","2739":"Shopping","2740":"Movies & DVDs","2741":"Gym","2742":"Shopping","2743":"Groceries","2744":"Service & Parts","2745":"Fast Food","2746":"Clothing","2747":"Shopping","2748":"Groceries","2750":"Shopping","2751":"Groceries","2752":"Movies & DVDs","2753":"Shopping","2754":"Gas & Fuel","2755":"Shopping","2757":"Groceries","2758":"Parking","2759":"Clothing","2760":"Books","2761":"Fast Food","2762":"Fast Food","2763":"Home Insurance","2764":"Music","2775":"Business Services","2776":"Restaurants","2777":"Groceries","2778":"Gym","2779":"Groceries","2781":"Hotel","2783":"Gas & Fuel","2784":"Groceries","2785":"Pharmacy","2786":"Gas & Fuel","2787":"Movies & DVDs","2788":"Electronics & Software","2789":"Pharmacy","2790":"Gas & Fuel","2791":"Restaurants","2792":"Shopping","2793":"Restaurants","2794":"Auto Insurance","2797":"Gas & Fuel","2798":"Entertainment","2799":"Rental Car & Taxi","2801":"Groceries","2802":"Fast Food","2803":"Alcohol & Bars","2804":"Rental Car & Taxi","2805":"Shopping","2806":"Groceries","2807":"Alcohol & Bars","2808":"Groceries","2809":"Groceries","2810":"Fast Food","2811":"Fast Food","2812":"Food & Dining","2813":"Food & Dining","2814":"Groceries","2815":"Shopping","2816":"Movies & DVDs","2817":"Fast Food","2818":"Gym","2819":"Food & Dining","2820":"Check","2821":"Groceries","2822":"Food & Dining","2823":"Restaurants","2825":"Movies & DVDs","2826":"Food & Dining","2827":"Groceries","2828":"Fast Food","2829":"Groceries","2830":"Restaurants","2831":"Food & Dining","2832":"Groceries","2833":"Fast Food","2835":"Food & Dining","2836":"Food & Dining","2837":"Fast Food","2838":"Groceries","2839":"Home Insurance","2840":"Fast Food","2841":"Food & Dining","2842":"Alcohol & Bars","2843":"Pharmacy","2844":"Rental Car & Taxi","2846":"Rental Car & Taxi","2847":"Rental Car & Taxi","2848":"Rental Car & Taxi","2849":"Restaurants","2850":"Groceries","2851":"Music","2852":"Business Services","2853":"Business Services","2854":"Air Travel","2855":"Groceries","2856":"Groceries","2857":"Groceries","2858":"Fast Food","2861":"Pharmacy","2862":"Fast Food","2863":"Groceries","2864":"Shopping","2865":"Tuition","2866":"Restaurants","2867":"Restaurants","2868":"Fast Food","2869":"Shopping","2870":"Restaurants","2871":"Groceries","2872":"Groceries","2873":"Fast Food","2874":"Movies & DVDs","2875":"Fast Food","2876":"Fast Food","2877":"Fast Food","2878":"Groceries","2879":"Groceries","2880":"Restaurants","2881":"Groceries","2882":"Home Improvement","2883":"Fast Food","2884":"Groceries","2885":"Groceries","2886":"Fast Food","2887":"Food & Dining","2888":"Fast Food","2889":"Food & Dining","2890":"Food & Dining","2891":"Utilities","2893":"Gas & Fuel","2894":"Fast Food","2895":"Pharmacy","2896":"Restaurants","2898":"Auto Insurance","2899":"Restaurants","2900":"Groceries","2901":"Office Supplies","2902":"Restaurants","2903":"Alcohol & Bars","2904":"Alcohol & Bars","2905":"Restaurants","2906":"Arts","2907":"Groceries","2908":"Food & Dining","2909":"Fast Food","2910":"Restaurants","2912":"Alcohol & Bars","2913":"Groceries","2914":"Fast Food","2915":"Groceries","2916":"Pharmacy","2917":"Groceries","2918":"Food & Dining","2919":"Restaurants","2920":"Restaurants","2921":"Fast Food","2922":"Food & Dining","2923":"Groceries","2924":"Movies & DVDs","2925":"Food & Dining","2926":"Tuition","2927":"Food & Dining","2928":"Fast Food","2929":"Gym","2930":"Rental Car & Taxi","2931":"Restaurants","2932":"Rental Car & Taxi","2933":"Fast Food","2934":"Fast Food","2935":"Food & Dining","2936":"Shopping","2937":"Shopping","2938":"Groceries","2939":"Clothing","2940":"Coffee Shops","2941":"Shopping","2942":"Movies & DVDs","2943":"Tuition","2944":"Check","2945":"Food & Dining","2946":"Restaurants","2947":"Food & Dining","2948":"Food & Dining","2949":"Groceries","2950":"Food & Dining","2951":"Food & Dining","2955":"Hotel","2956":"Clothing","2957":"Groceries","2958":"Tuition","2959":"Restaurants","2960":"Restaurants","2961":"Restaurants","2962":"Groceries","2963":"Restaurants","2964":"Shopping","2965":"Food & Dining","2966":"Gas & Fuel","2967":"Groceries","2968":"Groceries","2969":"Hair","2970":"Groceries","2972":"Music","2973":"Home Insurance","2974":"Restaurants","2975":"Groceries","2976":"Food & Dining","2977":"Music","2978":"Groceries","2980":"Food & Dining","2981":"Fast Food","2983":"Groceries","2984":"Gas & Fuel","2985":"Gas & Fuel","2986":"Shopping","2988":"Fast Food","2989":"Rental Car & Taxi","2990":"Books","2991":"Hair","2992":"Sporting Goods","2993":"Shopping","2994":"Sporting Goods","2995":"Groceries","2996":"Shopping","2997":"Restaurants","2998":"Food & Dining","2999":"Shopping","3000":"Restaurants","3001":"Food & Dining","3002":"Fast Food","3003":"Gas & Fuel","3004":"Gas & Fuel","3006":"Auto Insurance","3008":"Furnishings","3009":"Restaurants","3010":"Alcohol & Bars","3011":"Sporting Goods","3012":"Groceries","3015":"Food & Dining","3016":"Utilities","3017":"Groceries","3018":"Entertainment","3020":"Restaurants","3021":"Alcohol & Bars","3022":"Groceries","3023":"Air Travel","3024":"Restaurants","3025":"Air Travel","3026":"Gas & Fuel","3027":"Restaurants","3028":"Air Travel","3029":"Movies & DVDs","3030":"Fast Food","3031":"Food & Dining","3032":"Gym","3033":"Food & Dining","3034":"Fast Food","3035":"Food & Dining","3036":"Check","3037":"Restaurants","3038":"Restaurants","3039":"Food & Dining","3040":"Fast Food","3043":"Fast Food","3044":"Entertainment","3045":"Groceries","3046":"Shopping","3047":"Fast Food","3048":"Groceries","3049":"Home Improvement","3050":"Fast Food","3051":"Furnishings","3052":"Restaurants","3054":"Newspapers & Magazines","3055":"Coffee Shops","3056":"Hotel","3060":"Rental Car & Taxi","3062":"Hotel","3063":"Hotel","3064":"Food & Dining","3065":"Food & Dining","3066":"Hotel","3067":"Groceries","3068":"Food & Dining","3069":"Hotel","3070":"Groceries","3072":"Home Insurance","3073":"Music","3074":"Groceries","3075":"Hobbies","3076":"Music","3077":"Groceries","3078":"Groceries","3081":"Fast Food","3083":"Fast Food","3084":"Rental Car & Taxi","3085":"Movies & DVDs","3086":"Groceries","3087":"Groceries","3088":"Food & Dining","3089":"Music","3090":"Gas & Fuel","3091":"Food & Dining","3092":"Pharmacy","3093":"Fast Food","3094":"Fast Food","3095":"Fast Food","3096":"Fast Food","3097":"Alcohol & Bars","3098":"Restaurants","3099":"Home Improvement","3100":"Shopping","3101":"Furnishings","3103":"Entertainment","3107":"Books","3108":"Auto Insurance","3111":"Rental Car & Taxi","3114":"ATM Fee","3115":"Cash & ATM","3116":"ATM Fee","3119":"Groceries","3120":"Groceries","3121":"Parking","3122":"Alcohol & Bars","3123":"Home Improvement","3124":"Alcohol & Bars","3125":"Fast Food","3126":"Alcohol & Bars","3127":"Home Improvement","3128":"Home Improvement","3129":"Groceries","3130":"Movies & DVDs","3132":"Check","3133":"Fast Food","3134":"Shopping","3135":"Groceries","3136":"Gym","3138":"Gas & Fuel","3139":"Groceries","3140":"Fast Food","3141":"Groceries","3142":"Shopping","3143":"Groceries","3144":"Groceries","3145":"Restaurants","3146":"Groceries","3147":"Alcohol & Bars","3148":"Fast Food","3149":"Tuition","3150":"Electronics & Software","3151":"Books","3152":"Alcohol & Bars","3153":"Business Services","3154":"Alcohol & Bars","3155":"Restaurants","3157":"Alcohol & Bars","3158":"Home Improvement","3159":"Fast Food","3160":"Gas & Fuel","3161":"Printing","3162":"Groceries","3163":"Alcohol & Bars","3164":"Groceries","3165":"Food & Dining","3167":"Groceries","3168":"Advertising","3169":"Hair","3170":"Fast Food","3171":"Groceries","3172":"Printing","3174":"Shopping","3175":"Fast Food","3177":"Gas & Fuel","3179":"Fast Food","3180":"Gas & Fuel","3181":"Groceries","3182":"Groceries","3183":"Electronics & Software","3184":"Groceries","3185":"Fast Food","3186":"Electronics & Software","3187":"Entertainment","3188":"Home Insurance","3189":"Groceries","3190":"Groceries","3191":"Music","3192":"Home Improvement","3193":"Groceries","3194":"Parking","3195":"Music","3197":"Utilities","3198":"Groceries","3199":"Health & Fitness","3200":"Fast Food","3201":"Fast Food","3202":"Air Travel","3203":"Restaurants","3204":"Restaurants","3205":"Fast Food","3208":"Groceries","3209":"Clothing","3210":"Restaurants","3211":"Groceries","3212":"Shopping","3213":"Clothing","3214":"Clothing","3215":"Home Improvement","3216":"Groceries","3217":"Groceries","3218":"Fast Food","3219":"Movies & DVDs","3220":"Groceries","3221":"Fast Food","3222":"Shopping","3223":"Groceries","3224":"Fast Food","3225":"Entertainment","3226":"Gym","3227":"Food & Dining","3228":"Fast Food","3229":"Groceries","3231":"Books","3232":"Auto Insurance","3233":"Shopping","3236":"Food & Dining","3237":"Groceries","3238":"Alcohol & Bars","3239":"Food & Dining","3240":"Restaurants","3241":"Fast Food","3242":"Shopping","3243":"Fast Food","3244":"Restaurants","3245":"Food & Dining","3246":"Fast Food","3247":"Food & Dining","3248":"Food & Dining","3249":"Groceries","3250":"Food & Dining","3251":"Groceries","3252":"Movies & DVDs","3253":"Gym","3254":"Restaurants","3255":"Groceries","3256":"Groceries","3257":"Restaurants","3258":"Restaurants","3259":"Shopping","3260":"Alcohol & Bars","3261":"Coffee Shops","3262":"Alcohol & Bars","3263":"Groceries","3264":"Food & Dining","3265":"Groceries","3266":"Alcohol & Bars","3267":"Groceries","3268":"Groceries","3271":"Food & Dining","3272":"Food & Dining","3273":"Groceries","3274":"Parking","3275":"Food & Dining","3276":"Fast Food","3278":"Groceries","3279":"Home Services","3280":"Groceries","3281":"Fast Food","3282":"Shopping","3283":"Home Insurance","3284":"Groceries","3285":"Groceries","3286":"Music","3287":"Music","3289":"Clothing","3290":"Groceries","3291":"Groceries","3292":"Groceries","3293":"Groceries","3294":"Groceries","3295":"Groceries","3296":"Groceries","3297":"Groceries","3298":"Groceries","3299":"Restaurants","3300":"Food & Dining","3301":"Food & Dining","3302":"Restaurants","3303":"Fast Food","3304":"Food & Dining","3305":"Food & Dining","3306":"Food & Dining","3307":"Shopping","3308":"Tuition","3310":"Mobile Phone","3311":"Fast Food","3312":"Alcohol & Bars","3313":"Alcohol & Bars","3314":"ATM Fee","3315":"Cash & ATM","3316":"ATM Fee","3317":"Clothing","3318":"Alcohol & Bars","3319":"Restaurants","3320":"Alcohol & Bars","3321":"Food & Dining","3322":"Entertainment","3323":"Alcohol & Bars","3324":"Restaurants","3325":"Alcohol & Bars","3326":"Public Transportation","3327":"Groceries","3328":"Alcohol & Bars","3329":"Restaurants","3330":"Vacation","3331":"Groceries","3332":"Shopping","3334":"Check","3335":"Check","3336":"Groceries","3338":"Auto Insurance","3339":"Food & Dining","3340":"Food & Dining","3341":"Groceries","3342":"Groceries","3343":"Restaurants","3344":"Groceries","3345":"Food & Dining","3347":"Groceries","3348":"Alcohol & Bars","3349":"Groceries","3350":"Groceries","3351":"Groceries","3352":"Pharmacy","3353":"Groceries","3354":"Groceries","3355":"Groceries","3356":"Check","3357":"Movies & DVDs","3358":"Gym","3361":"Food & Dining","3362":"Coffee Shops","3364":"Hair","3365":"Groceries","3367":"Gas & Fuel","3368":"Shopping","3369":"Groceries","3370":"Groceries","3371":"Shopping","3372":"Pharmacy","3373":"Food & Dining","3374":"Entertainment","3375":"Parking","3376":"Food & Dining","3377":"Entertainment","3378":"Food & Dining","3379":"Coffee Shops","3380":"Fast Food","3381":"Groceries","3382":"Groceries","3383":"Groceries","3384":"Groceries","3387":"Fast Food","3388":"Utilities","3389":"Music","3390":"Pharmacy","3391":"Music","3393":"Restaurants","3394":"Shopping","3395":"Shopping","3396":"Gas & Fuel","3397":"Shopping","3398":"Groceries","3399":"Groceries","3400":"Groceries","3401":"Fast Food","3402":"Food & Dining","3403":"Fast Food","3404":"Fast Food","3405":"Shopping","3406":"Restaurants","3407":"Groceries","3409":"Shopping","3410":"Shopping","3411":"Groceries","3412":"Shopping","3413":"Pharmacy","3414":"Books","3415":"Books","3416":"Groceries","3417":"Gift","3419":"Entertainment","3420":"Shopping","3421":"Groceries","3422":"Groceries","3423":"Restaurants","3424":"Alcohol & Bars","3425":"Fast Food","3426":"Groceries","3427":"Restaurants","3428":"Auto Insurance","3431":"Rental Car & Taxi","3432":"Food & Dining","3434":"Alcohol & Bars","3439":"Alcohol & Bars","3440":"Groceries","3441":"Rental Car & Taxi","3442":"Groceries","3443":"Shopping","3444":"Food & Dining","3445":"Tuition","3446":"Fast Food","3447":"Fast Food","3448":"Food & Dining","3449":"Food & Dining","3450":"Food & Dining","3452":"Gym","3453":"Shopping","3454":"Movies & DVDs","3455":"Fast Food","3459":"Food & Dining","3462":"Groceries","3463":"Shopping","3464":"Groceries","3465":"Groceries","3466":"Groceries","3467":"Groceries","3468":"Clothing","3469":"Food & Dining","3470":"Groceries","3471":"Food & Dining","3472":"Food & Dining","3473":"Restaurants","3474":"Pharmacy","3475":"Groceries","3476":"Groceries","3477":"Fast Food","3478":"Gas & Fuel","3479":"Groceries","3480":"Gifts & Donations","3481":"Music","3482":"Pharmacy","3483":"Groceries","3484":"Groceries","3486":"Music","3487":"Shopping","3488":"Groceries","3489":"Groceries","3490":"Food & Dining","3491":"Food & Dining","3493":"Late Fee","3494":"Restaurants","3495":"Groceries","3496":"Groceries","3497":"Rental Car & Taxi","3498":"Groceries","3499":"Restaurants","3500":"Restaurants","3501":"Food & Dining","3502":"Restaurants","3503":"Shopping","3504":"Hotel","3505":"Restaurants","3506":"Pharmacy","3507":"Food & Dining","3508":"Groceries","3509":"Restaurants","3510":"Groceries","3511":"Groceries","3512":"Groceries","3513":"Tuition","3515":"Shopping","3516":"Groceries","3517":"Groceries","3518":"Groceries","3519":"Gas & Fuel","3520":"Auto Insurance","3521":"Groceries","3522":"Gas & Fuel","3523":"Groceries","3525":"Restaurants","3526":"Shopping","3528":"Food & Dining","3529":"Groceries","3530":"Alcohol & Bars","3531":"Shopping","3532":"Pharmacy","3533":"Food & Dining","3534":"Groceries","3535":"Food & Dining","3536":"Groceries","3537":"Food & Dining","3538":"Groceries","3539":"Restaurants","3540":"Movies & DVDs","3541":"Gym","3542":"Groceries","3543":"Restaurants","3544":"Food & Dining","3546":"Shopping","3547":"Food & Dining","3548":"Groceries","3550":"Groceries","3552":"Restaurants","3553":"Groceries","3554":"Shopping","3555":"Groceries","3556":"Restaurants","3557":"Pharmacy","3558":"Fast Food","3559":"Groceries","3560":"Food & Dining","3561":"Music","3562":"Groceries","3563":"Restaurants","3564":"Food & Dining","3565":"Groceries","3566":"Music","3567":"Groceries","3568":"Groceries","3570":"Shopping","3571":"Groceries","3572":"Groceries","3573":"Groceries","3574":"Air Travel","3575":"Alcohol & Bars","3576":"Food & Dining","3577":"Restaurants","3578":"Groceries","3579":"Pharmacy","3580":"Alcohol & Bars","3581":"Fast Food","3582":"Gas & Fuel","3583":"Groceries","3584":"Gas & Fuel","3585":"Food & Dining","3586":"Food & Dining","3587":"Shopping","3588":"Restaurants","3589":"Groceries","3590":"Restaurants","3591":"Groceries","3592":"Groceries","3593":"Restaurants","3597":"Pharmacy","3601":"Furnishings","3602":"Clothing","3603":"Groceries","3604":"Groceries","3605":"Shopping","3606":"Gym","3607":"Groceries","3608":"Gym","3609":"Pharmacy","3610":"Groceries","3611":"Restaurants","3612":"Groceries","3613":"Fast Food","3614":"Groceries","3615":"Food & Dining","3616":"Restaurants","3617":"Groceries","3618":"Auto Insurance","3621":"Groceries","3622":"Groceries","3623":"Groceries","3624":"Groceries","3626":"Alcohol & Bars","3627":"Groceries","3628":"Rental Car & Taxi","3629":"Rental Car & Taxi","3630":"Groceries","3631":"Pharmacy","3632":"Groceries","3633":"Shopping","3634":"Groceries","3635":"Gym","3636":"Fast Food","3637":"Groceries","3638":"Groceries","3639":"Groceries","3640":"Music","3641":"Pharmacy","3642":"Shopping","3643":"Shopping","3644":"Food & Dining","3646":"Groceries","3647":"Groceries","3648":"Food & Dining","3649":"Groceries","3650":"Music","3651":"Groceries","3652":"Groceries","3653":"Gas & Fuel","3654":"Groceries","3655":"Hair","3656":"Groceries","3657":"Groceries","3658":"Groceries","3659":"Groceries","3660":"Groceries","3661":"Groceries","3662":"Shopping","3663":"Restaurants","3664":"Groceries","3665":"Groceries","3666":"Groceries","3668":"Groceries","3669":"Groceries","3670":"Groceries","3671":"Fast Food","3672":"Shopping","3673":"Shopping","3674":"Restaurants","3675":"Groceries","3676":"Fast Food","3677":"Cash & ATM","3678":"Groceries","3679":"Pharmacy","3680":"Auto Insurance","3681":"Air Travel","3682":"Groceries","3683":"Groceries","3689":"Groceries","3690":"Groceries","3691":"Groceries","3692":"Groceries","3693":"Entertainment","3694":"Alcohol & Bars","3695":"Groceries","3696":"Groceries","3698":"Alcohol & Bars","3699":"Groceries","3700":"Food & Dining","3701":"Groceries","3702":"Groceries","3703":"Food & Dining","3704":"Groceries","3705":"Food & Dining","3706":"Tuition","3707":"Groceries","3708":"Groceries","3709":"Movies & DVDs","3710":"Pharmacy","3712":"Gym","3713":"Fast Food","3714":"Groceries","3715":"Groceries","3716":"Groceries","3717":"Groceries","3718":"Groceries","3719":"Restaurants","3720":"Groceries","3721":"Restaurants","3722":"Food & Dining","3723":"Restaurants","3724":"Fast Food","3725":"Pharmacy","3726":"Food & Dining","3727":"Food & Dining","3728":"Groceries","3729":"Groceries","3730":"Alcohol & Bars","3731":"Utilities","3734":"Groceries","3735":"Groceries","3736":"Groceries","3737":"Music","3738":"Music","3739":"Restaurants","3740":"Pharmacy","3741":"Groceries","3742":"Food & Dining","3743":"Food & Dining","3744":"Groceries","3745":"Food & Dining","3746":"Restaurants","3747":"Food & Dining","3748":"Groceries","3749":"Shopping","3750":"Groceries","3751":"Groceries","3752":"Pharmacy","3753":"Groceries","3754":"Groceries","3755":"Pharmacy","3756":"Groceries","3757":"Restaurants","3758":"Groceries","3759":"Restaurants","3760":"Food & Dining","3761":"Pharmacy","3762":"Food & Dining","3763":"Restaurants","3764":"Groceries","3766":"Charity","3767":"Restaurants","3768":"Home","3769":"Home","3770":"Shopping","3772":"Auto Insurance","3774":"Groceries","3775":"Groceries","3776":"Groceries","3777":"Groceries","3778":"Groceries","3779":"Groceries","3780":"Restaurants","3781":"Food & Dining","3783":"Alcohol & Bars","3784":"Groceries","3785":"Tuition","3786":"Coffee Shops","3787":"Shopping","3788":"Food & Dining","3789":"Movies & DVDs","3793":"Groceries","3795":"Gym","3796":"Rental Car & Taxi","3797":"Groceries","3798":"Groceries","3800":"Groceries","3801":"Restaurants","3802":"Parking","3803":"Alcohol & Bars","3804":"Fast Food","3805":"Food & Dining","3806":"Groceries","3807":"Fast Food","3808":"Groceries","3809":"Fast Food","3810":"Restaurants","3811":"Groceries","3812":"Gas & Fuel","3813":"Groceries","3814":"Groceries","3815":"Pharmacy","3816":"Groceries","3817":"Groceries","3818":"Pharmacy","3819":"Groceries","3820":"Music","3821":"Fast Food","3822":"Groceries","3824":"Music","3825":"Pharmacy","3826":"Groceries","3827":"Shopping","3828":"Shopping","3829":"Fast Food","3830":"Personal Care","3833":"Groceries","3834":"Home Supplies","3835":"Fast Food","3836":"Shopping","3837":"Groceries","3838":"Groceries","3839":"Groceries","3840":"Groceries","3841":"Groceries","3842":"Mobile Phone","3843":"Food & Dining","3844":"Food & Dining","3845":"Groceries","3846":"Check","3847":"Shopping","3848":"Shopping","3849":"Alcohol & Bars","3850":"Gift","3851":"Groceries","3852":"Groceries","3853":"Groceries","3854":"Groceries","3855":"Groceries","3856":"Auto Insurance","3859":"Cash & ATM","3860":"Groceries","3861":"Restaurants","3865":"Groceries","3866":"Check","3867":"Alcohol & Bars","3868":"Pets","3869":"Furnishings","3870":"Groceries","3871":"Groceries","3872":"Groceries","3873":"Movies & DVDs","3874":"Gym","3875":"Hotel","3878":"Restaurants","3879":"Groceries","3880":"Home Supplies","3883":"Air Travel","3884":"Gift","3885":"Alcohol & Bars","3886":"Newspapers & Magazines","3887":"Food & Dining","3888":"Groceries","3889":"Alcohol & Bars","3890":"Restaurants","3891":"Hotel","3892":"Air Travel","3893":"Alcohol & Bars","3894":"Fast Food","3895":"Spa & Massage","3896":"Fast Food","3897":"Groceries","3898":"Groceries","3899":"Groceries","3900":"Music","3901":"Music","3902":"Movies & DVDs","3903":"Electronics & Software","3904":"Arts","3905":"Alcohol & Bars","3906":"Food & Dining","3907":"Groceries","3908":"Air Travel","3909":"Restaurants","3910":"Groceries","3912":"Books","3913":"Shopping","3914":"Music","3915":"Food & Dining","3917":"Shopping","3919":"Shopping","3920":"Groceries","3921":"Shopping","3922":"Groceries","3923":"Food & Dining","3924":"Fast Food","3925":"Groceries","3926":"Shopping","3927":"Shopping","3929":"Cash & ATM","3930":"Check","3931":"Groceries","3932":"Pharmacy","3934":"Groceries","3935":"Groceries","3936":"Groceries","3937":"Rental Car & Taxi","3938":"Restaurants","3939":"Groceries","3940":"Shopping","3941":"Pharmacy","3942":"Groceries","3943":"Fast Food","3944":"Restaurants","3945":"Groceries","3946":"Groceries","3947":"Groceries","3949":"Movies & DVDs","3950":"Groceries","3951":"Groceries","3952":"Auto Insurance","3954":"Shopping","3955":"Groceries","3956":"Food & Dining","3957":"Groceries","3958":"Movies & DVDs","3959":"Food & Dining","3961":"Alcohol & Bars","3962":"Shopping","3963":"Groceries","3964":"Rental Car & Taxi","3965":"Business Services","3966":"Business Services","3967":"Fast Food","3968":"Groceries","3969":"Business Services","3970":"Groceries","3971":"Fast Food","3972":"Groceries","3973":"Rental Car & Taxi","3974":"Restaurants","3977":"Food & Dining","3978":"Restaurants","3979":"Fast Food","3980":"Groceries","3982":"Groceries","3983":"Restaurants","3984":"Movies & DVDs","3985":"Shopping","3986":"Shopping","3987":"Shopping","3988":"Gym","3989":"Groceries","3992":"Shopping","3993":"Groceries","3994":"Gifts & Donations","3995":"Groceries","3996":"Gas & Fuel","3998":"Groceries","3999":"Groceries","4000":"Groceries","4001":"Groceries","4002":"Groceries","4003":"Food & Dining","4004":"Groceries","4005":"Restaurants","4007":"Groceries","4008":"Food & Dining","4009":"Food & Dining","4010":"Food & Dining","4011":"Groceries","4012":"Groceries","4013":"Tuition","4014":"Pharmacy","4015":"Check","4016":"Music","4017":"Groceries","4018":"Groceries","4019":"Groceries","4020":"Groceries","4021":"Pharmacy","4022":"Groceries","4023":"Music","4024":"Groceries","4025":"Groceries","4026":"Groceries","4027":"Groceries","4028":"Groceries","4029":"Entertainment","4030":"Pharmacy","4033":"Groceries","4034":"Groceries","4035":"Groceries","4036":"Hair","4037":"Groceries","4038":"Groceries","4039":"Pharmacy","4040":"Groceries","4041":"Groceries","4042":"Shopping","4043":"Coffee Shops","4044":"Fast Food","4045":"Groceries","4046":"Restaurants","4047":"Gas & Fuel","4048":"Groceries","4049":"Pharmacy","4050":"Food & Dining","4051":"Groceries","4052":"Alcohol & Bars","4053":"Pharmacy","4054":"Check","4055":"Groceries","4056":"Food & Dining","4057":"Restaurants","4058":"Groceries","4059":"Hotel","4060":"Food & Dining","4063":"Groceries","4066":"Restaurants","4068":"Groceries","4069":"Auto Insurance","4070":"Clothing","4071":"Clothing","4072":"Restaurants","4073":"Shopping","4074":"Pharmacy","4075":"Groceries","4076":"Food & Dining","4078":"Groceries","4079":"Groceries","4080":"Food & Dining","4081":"Pharmacy","4082":"Food & Dining","4084":"Gym","4085":"Tuition","4086":"Groceries","4087":"Movies & DVDs","4089":"Restaurants","4090":"Groceries","4091":"Food & Dining","4092":"Alcohol & Bars","4093":"Groceries","4094":"Food & Dining","4095":"Fast Food","4096":"Food & Dining","4097":"Groceries","4098":"Groceries","4099":"Groceries","4100":"Groceries","4101":"Food & Dining","4102":"Shopping","4103":"Pharmacy","4105":"Food & Dining","4107":"Pharmacy","4108":"Groceries","4109":"Shopping","4110":"Food & Dining","4111":"Rental Car & Taxi","4112":"Alcohol & Bars","4113":"Fast Food","4114":"Groceries","4117":"Groceries","4118":"Groceries","4119":"Home Improvement","4120":"Groceries","4121":"Fast Food","4122":"Groceries","4123":"Home Improvement","4124":"Gift","4125":"Music","4127":"Utilities","4128":"Shopping","4129":"Shopping","4131":"Shopping","4134":"Home Improvement","4135":"Groceries","4136":"Groceries","4138":"Home Improvement","4139":"Music","4140":"Check","4141":"Restaurants","4142":"Groceries","4143":"Groceries","4144":"Pharmacy","4145":"Food & Dining","4149":"Groceries","4150":"Groceries","4151":"Shopping","4152":"Pharmacy","4153":"Shopping","4154":"Business Services","4155":"Groceries","4156":"Restaurants","4157":"Groceries","4158":"Home Improvement","4160":"Shopping","4161":"Restaurants","4162":"Business Services","4163":"Groceries","4165":"Groceries","4166":"Parking","4167":"Parking","4168":"Groceries","4170":"Groceries","4171":"Pharmacy","4174":"Groceries","4175":"Groceries","4176":"Shopping","4177":"Restaurants","4178":"Clothing","4179":"Restaurants","4182":"Food & Dining","4183":"Movies & DVDs","4184":"Coffee Shops","4185":"Alcohol & Bars","4186":"Alcohol & Bars","4187":"Fast Food","4188":"Groceries","4190":"Gift","4191":"Alcohol & Bars","4192":"Groceries","4193":"Restaurants","4194":"Rental Car & Taxi","4195":"Groceries","4196":"Restaurants","4197":"Groceries","4198":"Rental Car & Taxi","4199":"Auto & Transport","4200":"Alcohol & Bars","4201":"Groceries","4202":"Groceries","4203":"Groceries","4204":"Groceries","4205":"Groceries","4207":"Restaurants","4208":"Auto Insurance","4209":"Pharmacy","4211":"Gift","4213":"Restaurants","4215":"Groceries","4216":"Alcohol & Bars","4217":"Groceries","4218":"Shopping","4219":"Groceries","4220":"Groceries","4221":"Shopping","4222":"Groceries","4223":"Groceries","4224":"Hair","4225":"Groceries","4226":"Pharmacy","4227":"Groceries","4228":"Groceries","4229":"Restaurants","4230":"Restaurants","4231":"Groceries","4232":"Utilities","4233":"Rental Car & Taxi","4234":"Groceries","4235":"Air Travel","4236":"Air Travel","4237":"Gym","4239":"Hotel","4240":"Food & Dining","4241":"Coffee Shops","4243":"Pharmacy","4244":"Movies & DVDs","4245":"Groceries","4246":"Rental Car & Taxi","4247":"Pharmacy","4248":"Rental Car & Taxi","4250":"Shopping","4251":"Groceries","4252":"Rental Car & Taxi","4253":"Gym","4255":"Restaurants","4256":"Gym","4257":"Air Travel","4258":"Groceries","4259":"Groceries","4260":"Auto Payment","4261":"Auto Payment","4262":"Toys","4263":"Sporting Goods","4264":"Groceries","4265":"Internet","4266":"Fast Food","4267":"Pharmacy","4268":"Public Transportation","4269":"Rental Car & Taxi","4270":"Fast Food","4271":"Public Transportation","4272":"Home Improvement","4273":"Pharmacy","4274":"Clothing","4275":"Groceries","4276":"Restaurants","4277":"Groceries","4279":"Groceries","4280":"Groceries","4281":"Groceries","4282":"Hotel","4283":"Restaurants","4284":"Air Travel","4288":"Food & Dining","4289":"Groceries","4290":"Fast Food","4291":"Groceries","4292":"Gas & Fuel","4293":"Tuition","4294":"Restaurants","4295":"Groceries","4296":"Groceries","4297":"Groceries","4298":"Groceries","4299":"Pharmacy","4300":"Electronics & Software","4301":"Music","4302":"Pharmacy","4303":"Groceries","4304":"Music","4305":"Groceries","4306":"Groceries","4307":"Groceries","4308":"Groceries","4309":"Groceries","4310":"Groceries","4311":"Check","4312":"Groceries","4313":"Auto & Transport","4314":"Groceries","4315":"Shopping","4316":"Alcohol & Bars","4318":"Groceries","4319":"Food & Dining","4320":"Groceries","4321":"Groceries","4322":"Groceries","4323":"Movies & DVDs","4324":"Groceries","4325":"Groceries","4326":"Shopping","4327":"Groceries","4328":"Groceries","4329":"Groceries","4331":"Groceries","4332":"Groceries","4333":"Health & Fitness","4334":"Groceries","4335":"Gym","4336":"Groceries","4338":"Shopping","4343":"Auto Insurance","4344":"Groceries","4345":"Groceries","4347":"Restaurants","4348":"Groceries","4349":"Groceries","4351":"Groceries","4352":"Food & Dining","4353":"Groceries","4354":"Groceries","4355":"Rental Car & Taxi","4356":"Alcohol & Bars","4357":"Groceries","4358":"Groceries","4359":"Groceries","4360":"Air Travel","4361":"Gas & Fuel","4362":"Coffee Shops","4363":"Restaurants","4364":"Hair","4366":"Shopping","4367":"Tuition","4368":"Shopping","4369":"Restaurants","4370":"Fast Food","4371":"Food & Dining","4372":"Groceries","4373":"Groceries","4374":"Movies & DVDs","4375":"Groceries","4376":"Groceries","4377":"Groceries","4379":"Movies & DVDs","4380":"Shopping","4382":"Coffee Shops","4383":"Pharmacy","4384":"Groceries","4385":"Groceries","4386":"Gas & Fuel","4387":"Groceries","4388":"Groceries","4389":"Restaurants","4390":"Groceries","4391":"Food & Dining","4392":"Groceries","4393":"Pharmacy","4396":"Groceries","4397":"Restaurants","4398":"Coffee Shops","4399":"Coffee Shops","4400":"Pharmacy","4401":"Groceries","4402":"Utilities","4403":"Groceries","4406":"Groceries","4407":"Fast Food","4408":"Groceries","4409":"Entertainment","4410":"Electronics & Software","4411":"Coffee Shops","4412":"Groceries","4413":"Restaurants","4414":"Pharmacy","4415":"Pharmacy","4416":"Coffee Shops","4417":"Groceries","4418":"Groceries","4419":"Clothing","4422":"Shopping","4423":"Pharmacy","4424":"Alcohol & Bars","4425":"Groceries","4426":"Food & Dining","4427":"Music","4428":"Groceries","4429":"Music","4430":"Pharmacy","4431":"Pharmacy","4432":"Groceries","4433":"Restaurants","4434":"Groceries","4435":"Gym","4436":"Shopping","4437":"Shopping","4438":"Groceries","4439":"Groceries","4440":"Check","4441":"Tuition","4442":"Entertainment","4443":"Groceries","4444":"Groceries","4445":"Restaurants","4446":"Rental Car & Taxi","4447":"Alcohol & Bars","4448":"Groceries","4449":"Auto & Transport","4450":"Air Travel","4451":"Groceries","4452":"Alcohol & Bars","4453":"Air Travel","4454":"Auto & Transport","4455":"Groceries","4456":"Shopping","4457":"Pharmacy","4458":"Air Travel","4459":"Air Travel","4460":"Hair","4461":"Groceries","4462":"Check","4463":"Shopping","4464":"Alcohol & Bars","4465":"Groceries","4466":"Groceries","4467":"Restaurants","4468":"Restaurants","4469":"Gifts & Donations","4470":"Groceries","4471":"Personal Care","4473":"Food & Dining","4474":"Groceries","4475":"Groceries","4476":"Business Services","4477":"Groceries","4478":"Rental Car & Taxi","4479":"Groceries","4480":"Groceries","4481":"Auto & Transport","4482":"Alcohol & Bars","4483":"Auto Insurance","4487":"Groceries","4488":"Groceries","4489":"Groceries","4490":"Food & Dining","4491":"Restaurants","4492":"Shopping","4494":"Shopping","4495":"Pharmacy","4496":"Restaurants","4497":"Alcohol & Bars","4498":"Shipping","4499":"Uncategorized","4500":"Groceries","4501":"Groceries","4502":"Restaurants","4503":"Groceries","4504":"Restaurants","4505":"Gas & Fuel","4506":"Groceries","4507":"Movies & DVDs","4508":"Health & Fitness","4509":"Restaurants","4510":"Food & Dining","4511":"Groceries","4512":"Groceries","4513":"Groceries","4514":"Shopping","4515":"Mortgage & Rent","4517":"Pharmacy","4518":"Groceries","4519":"Utilities","4520":"Babysitter & Daycare","4523":"Tuition","4526":"Shopping","4527":"Pharmacy","4528":"Food & Dining","4529":"Shopping","4530":"Groceries","4531":"Alcohol & Bars","4532":"Food & Dining","4533":"Shopping","4534":"Groceries","4535":"Groceries","4536":"Groceries","4537":"Uncategorized","4538":"Groceries","4539":"Groceries","4540":"Alcohol & Bars","4541":"Food & Dining","4542":"Food & Dining","4543":"Groceries","4544":"Alcohol & Bars","4545":"Coffee Shops","4546":"Groceries","4547":"Groceries","4548":"Groceries","4549":"Pharmacy","4550":"Fast Food","4551":"Fast Food","4552":"Restaurants","4553":"Restaurants","4556":"Music","4557":"Groceries","4558":"Coffee Shops","4559":"Restaurants","4560":"Cash & ATM","4561":"Groceries","4562":"Groceries","4563":"Fast Food","4564":"Gifts & Donations","4565":"Pharmacy","4566":"Groceries","4567":"Groceries","4568":"Alcohol & Bars","4569":"Restaurants","4570":"Gifts & Donations","4571":"Groceries","4572":"Uncategorized","4574":"Pharmacy","4575":"Groceries","4576":"Air Travel","4577":"Gifts & Donations","4578":"Groceries","4579":"Food & Dining","4580":"Groceries","4581":"Sporting Goods","4582":"Fast Food","4583":"Groceries","4585":"Gas & Fuel","4587":"Rental Car & Taxi","4588":"Fast Food","4589":"Restaurants","4590":"Groceries","4591":"Check","4592":"Restaurants","4593":"Gifts & Donations","4594":"Coffee Shops","4597":"Pharmacy","4598":"Groceries","4600":"Groceries","4601":"Groceries","4602":"Groceries","4603":"Coffee Shops","4604":"Restaurants","4605":"Groceries","4606":"Fast Food","4607":"Gifts & Donations","4608":"Groceries","4609":"Coffee Shops","4611":"Food & Dining","4612":"Groceries","4613":"Groceries","4614":"Alcohol & Bars","4615":"Groceries","4616":"Pharmacy","4617":"Groceries","4618":"Groceries","4619":"Groceries","4620":"Groceries","4621":"Shopping","4622":"Groceries","4623":"Food & Dining","4624":"Food & Dining","4625":"Food & Dining","4626":"Groceries","4627":"Fast Food","4628":"Groceries","4630":"Business Services","4631":"Coffee Shops","4632":"Groceries","4633":"Pharmacy","4634":"Pharmacy","4635":"Movies & DVDs","4636":"Groceries","4637":"Groceries","4639":"Groceries","4640":"Groceries","4641":"Groceries","4642":"Groceries","4643":"Groceries","4645":"Fast Food","4647":"Rental Car & Taxi","4648":"Groceries","4649":"Groceries","4651":"Restaurants","4652":"Groceries","4653":"Rental Car & Taxi","4654":"Rental Car & Taxi","4655":"Groceries","4656":"Gas & Fuel","4657":"Groceries","4658":"Rental Car & Taxi","4659":"Rental Car & Taxi","4661":"Pharmacy","4662":"Uncategorized","4663":"Uncategorized","4664":"Uncategorized","4665":"Cash & ATM","4666":"ATM Fee","4667":"Uncategorized","4668":"Gas & Fuel","4669":"Groceries","4670":"Music","4672":"Fast Food","4673":"Fast Food","4674":"Groceries","4677":"Restaurants","4678":"Student Loan","4679":"Student Loan","4680":"Uncategorized","4684":"Shopping","4685":"Rental Car & Taxi","4686":"Groceries","4687":"Groceries","4688":"Groceries","4689":"Rental Car & Taxi","4690":"Groceries","4691":"Gifts & Donations","4692":"Hotel","4693":"Fast Food","4694":"Arts","4695":"Rental Car & Taxi","4696":"Groceries","4697":"Check","4698":"Restaurants","4699":"Fast Food","4700":"Entertainment","4703":"Fast Food","4704":"Food & Dining","4707":"Tuition","4708":"Groceries","4709":"Fast Food","4711":"Groceries","4712":"Pharmacy","4713":"Restaurants","4714":"Groceries","4715":"Restaurants","4716":"Groceries","4717":"Groceries","4718":"Groceries","4719":"Pharmacy","4720":"Groceries","4723":"Education","4724":"Alcohol & Bars","4725":"Groceries","4726":"Education","4727":"Pharmacy","4728":"Rental Car & Taxi","4729":"Groceries","4730":"Fast Food","4731":"Alcohol & Bars","4732":"Fast Food","4733":"Alcohol & Bars","4734":"Groceries","4735":"Pharmacy","4736":"Pharmacy","4737":"Restaurants","4738":"Groceries","4739":"Groceries","4740":"Groceries","4741":"Groceries","4742":"Rental Car & Taxi","4744":"Movies & DVDs","4745":"Groceries","4746":"Groceries","4747":"Entertainment","4748":"Groceries","4749":"Restaurants","4750":"Groceries","4751":"Fast Food","4752":"Groceries","4753":"Groceries","4755":"Pharmacy","4756":"Pharmacy","4757":"Groceries","4759":"Fast Food","4760":"Groceries","4761":"Groceries","4762":"Groceries","4763":"Doctor","4764":"Shopping","4767":"Groceries","4768":"Pharmacy","4769":"Restaurants","4770":"Air Travel","4771":"Furnishings","4772":"Groceries","4773":"Groceries","4774":"Coffee Shops","4776":"Pharmacy","4777":"Pharmacy","4778":"Restaurants","4779":"Restaurants","4780":"Alcohol & Bars","4781":"Restaurants","4782":"Coffee Shops","4783":"Music","4784":"Restaurants","4785":"Fast Food","4786":"Fast Food","4788":"Pharmacy","4789":"Groceries","4790":"Groceries","4791":"Groceries","4792":"Fast Food","4793":"Fast Food","4794":"Groceries","4795":"Fast Food","4797":"Groceries","4798":"Fast Food","4799":"Groceries","4800":"Restaurants","4801":"Groceries","4802":"Business Services","4803":"Rental Car & Taxi","4804":"Groceries","4805":"Fast Food","4806":"Fast Food","4807":"Alcohol & Bars","4808":"Food & Dining","4809":"Groceries","4810":"Groceries","4811":"Hotel","4812":"Fast Food","4813":"Pharmacy","4814":"Tuition","4816":"Pharmacy","4817":"Pharmacy","4818":"Groceries","4819":"Hotel","4820":"Groceries","4822":"Groceries","4823":"Fast Food","4824":"Groceries","4825":"Groceries","4826":"Groceries","4831":"Restaurants","4832":"Rental Car & Taxi","4833":"Groceries","4834":"Groceries","4835":"Cash & ATM","4836":"Check","4837":"Business Services","4839":"Alcohol & Bars","4840":"Hotel","4842":"Groceries","4844":"Movies & DVDs","4845":"Fast Food","4846":"Groceries","4847":"Groceries","4848":"Groceries","4849":"Fast Food","4850":"Groceries","4851":"Groceries","4852":"Groceries","4853":"Fast Food","4855":"Fast Food","4856":"Groceries","4857":"Groceries","4858":"Groceries","4861":"Groceries","4862":"Groceries","4863":"Fast Food","4864":"Gas & Fuel","4865":"Alcohol & Bars","4866":"Rental Car & Taxi","4867":"Rental Car & Taxi","4868":"Groceries","4869":"Groceries","4870":"Pharmacy","4871":"Groceries","4873":"Utilities","4874":"Groceries","4875":"Groceries","4876":"Music","4877":"Fast Food","4880":"Groceries","4881":"Groceries","4882":"Fast Food","4883":"Groceries","4884":"Coffee Shops","4886":"Hotel","4887":"Fast Food","4888":"Groceries","4889":"Groceries","4890":"Groceries","4891":"Groceries","4892":"Groceries","4893":"Groceries","4894":"Groceries","4896":"Groceries","4897":"Utilities","4898":"Restaurants","4899":"Groceries","4900":"Coffee Shops","4901":"Groceries","4902":"Groceries","4903":"Groceries","4904":"Fast Food","4905":"Groceries","4906":"Groceries","4910":"Electronics & Software","4911":"Groceries","4912":"Groceries","4913":"Fast Food","4914":"Groceries","4915":"Groceries","4916":"Groceries","4918":"Fast Food","4919":"Groceries","4920":"Groceries","4921":"Groceries","4922":"Alcohol & Bars","4924":"Groceries","4925":"Groceries","4926":"Groceries","4927":"Business Services","4928":"Food & Dining","4929":"Groceries","4930":"Groceries","4931":"Restaurants","4932":"Rental Car & Taxi","4933":"Business Services","4935":"Groceries","4936":"Fast Food","4937":"Groceries","4938":"Pharmacy","4939":"Movies & DVDs","4940":"Groceries","4941":"Groceries","4942":"Groceries","4943":"Groceries","4944":"Groceries","4945":"Groceries","4946":"Tuition","4947":"Groceries","4948":"Groceries","4950":"Groceries","4951":"Groceries","4952":"Groceries","4953":"Groceries","4954":"Alcohol & Bars","4955":"Restaurants","4956":"Groceries","4957":"Groceries","4958":"Groceries","4959":"Groceries","4960":"Gym","4961":"Groceries","4962":"Groceries","4964":"Groceries","4965":"Alcohol & Bars","4966":"Alcohol & Bars","4969":"Groceries","4970":"Groceries","4971":"Groceries","4973":"Groceries","4974":"Groceries","4975":"Groceries","4976":"Groceries","4978":"Groceries","4979":"Groceries","4980":"Uncategorized","4981":"Groceries","4982":"Groceries","4983":"Groceries","4984":"Amusement","4985":"Amusement","4986":"Amusement","4987":"Groceries","4988":"Restaurants","4989":"Fast Food","4990":"Groceries","4991":"Groceries","4993":"Groceries","4994":"Food & Dining","4995":"Rental Car & Taxi","4996":"Food & Dining","4997":"Groceries","4998":"Alcohol & Bars","4999":"Groceries","5000":"Home Phone","5001":"Alcohol & Bars","5002":"Groceries","5003":"Groceries","5004":"Groceries","5006":"Alcohol & Bars","5007":"Groceries","5008":"Groceries","5009":"Groceries","5010":"Groceries","5011":"Home Improvement","5012":"Groceries","5013":"Groceries","5014":"Pharmacy","5015":"Groceries","5016":"Groceries","5017":"Groceries","5018":"Groceries","5019":"Groceries","5020":"Groceries","5021":"Uncategorized","5022":"Groceries","5023":"Air Travel","5026":"Groceries","5027":"Groceries","5029":"Groceries","5030":"Groceries","5031":"Alcohol & Bars","5032":"Fast Food","5033":"Alcohol & Bars","5034":"Groceries","5035":"Alcohol & Bars","5036":"Groceries","5037":"Alcohol & Bars","5038":"Groceries","5039":"Alcohol & Bars","5040":"Groceries","5041":"Groceries","5043":"Alcohol & Bars","5044":"Groceries","5045":"Groceries","5046":"Groceries","5047":"Uncategorized","5048":"Groceries","5049":"Groceries","5050":"Groceries","5051":"Groceries","5052":"Groceries","5053":"Uncategorized","5054":"Groceries","5055":"Groceries","5056":"Movies & DVDs","5057":"Groceries","5058":"Restaurants","5059":"Fast Food","5060":"Alcohol & Bars","5061":"Restaurants","5062":"Restaurants","5063":"Restaurants","5064":"Restaurants","5065":"Groceries","5066":"Rental Car & Taxi","5067":"Coffee Shops","5068":"Rental Car & Taxi","5069":"Gas & Fuel","5070":"Restaurants","5071":"Groceries","5072":"Groceries","5073":"Groceries","5074":"Groceries","5075":"Coffee Shops","5076":"Coffee Shops","5081":"Alcohol & Bars","5083":"Groceries","5084":"Groceries","5085":"Groceries","5086":"Groceries","5087":"Groceries","5088":"Groceries","5089":"Groceries","5090":"Groceries","5091":"Groceries","5092":"Groceries","5093":"Groceries","5094":"Uncategorized","5095":"Groceries","5096":"Groceries","5097":"Groceries","5098":"Gym","5099":"Groceries","5100":"Coffee Shops","5101":"Groceries","5103":"Restaurants","5104":"Restaurants","5105":"Groceries","5106":"Groceries","5107":"Pharmacy","5108":"Groceries","5109":"Groceries","5111":"Groceries","5112":"Home Improvement","5113":"Groceries","5114":"Groceries","5115":"Groceries","5116":"Groceries","5117":"Groceries","5118":"Business Services","5119":"Groceries","5120":"Fast Food","5121":"Alcohol & Bars","5122":"Food & Dining","5123":"Uncategorized","5124":"Groceries","5125":"Sports","5126":"Business Services","5127":"Uncategorized","5128":"Groceries","5129":"Alcohol & Bars","5130":"Groceries","5131":"Sports","5132":"Alcohol & Bars","5133":"Fast Food","5134":"Groceries","5135":"Groceries","5136":"Groceries","5137":"Groceries","5139":"Groceries","5140":"Groceries","5141":"Groceries","5142":"Groceries","5143":"Groceries","5144":"Fast Food","5145":"Groceries","5146":"Uncategorized","5147":"Groceries","5148":"Groceries","5149":"Groceries","5151":"Groceries","5153":"Fast Food","5154":"Uncategorized","5155":"Groceries","5157":"Alcohol & Bars","5158":"Groceries","5159":"Groceries","5160":"Groceries","5161":"Groceries","5162":"Restaurants","5163":"Business Services","5164":"Groceries","5165":"Groceries","5166":"Groceries","5167":"Groceries","5168":"Groceries","5170":"Gas & Fuel","5171":"Groceries","5172":"Movies & DVDs","5173":"Uncategorized","5174":"Groceries","5175":"Groceries","5176":"Groceries","5177":"Hotel","5178":"Hotel","5179":"Hotel","5180":"Groceries","5181":"Groceries","5183":"Gas & Fuel","5184":"Groceries","5185":"Shopping","5186":"Groceries","5188":"Groceries","5189":"Groceries","5194":"Cash & ATM","5195":"Groceries","5196":"Groceries","5197":"Groceries","5198":"Groceries","5199":"Groceries","5200":"Groceries","5201":"Utilities","5203":"Groceries","5204":"Fast Food","5205":"Shopping","5206":"Uncategorized","5207":"Shopping","5208":"Shopping","5209":"Uncategorized","5210":"Groceries","5211":"Shopping","5212":"Restaurants","5213":"Groceries","5214":"Groceries","5215":"Groceries","5216":"Shopping","5217":"Alcohol & Bars","5218":"Groceries","5219":"Entertainment","5220":"Groceries","5221":"Alcohol & Bars","5222":"Gift","5223":"Groceries","5224":"Shopping","5225":"Groceries","5226":"Groceries","5227":"Fast Food","5228":"Groceries","5229":"Groceries","5231":"Groceries","5232":"Groceries","5233":"Groceries","5234":"Groceries","5235":"Alcohol & Bars","5236":"Groceries","5237":"Groceries","5238":"Groceries","5239":"Groceries","5240":"Groceries","5241":"Groceries","5242":"Alcohol & Bars","5243":"Groceries","5244":"Groceries","5245":"Groceries","5246":"Groceries","5247":"Entertainment","5248":"Groceries","5249":"Groceries","5250":"Groceries","5251":"Groceries","5252":"Groceries","5254":"Groceries","5255":"Groceries","5256":"Groceries","5257":"Groceries","5258":"Groceries","5259":"Groceries","5260":"Groceries","5261":"Groceries","5262":"Music","5263":"Uncategorized","5264":"Groceries","5265":"Coffee Shops","5266":"Groceries","5267":"Groceries","5268":"Groceries","5269":"Groceries","5270":"Alcohol & Bars","5271":"Groceries","5272":"Groceries","5273":"Groceries","5274":"Health & Fitness","5275":"Gas & Fuel","5276":"Alcohol & Bars","5277":"Groceries","5279":"Groceries","5280":"Utilities","5281":"Tuition","5282":"Alcohol & Bars","5285":"Alcohol & Bars","5286":"Groceries","5287":"Groceries","5288":"Shopping","5289":"Shopping","5290":"Shopping","5291":"Fast Food","5292":"Groceries","5293":"Groceries","5294":"Groceries","5295":"Groceries","5296":"Alcohol & Bars","5297":"Fast Food","5298":"Clothing","5299":"Clothing","5300":"Movies & DVDs","5301":"Groceries","5304":"Restaurants","5306":"Groceries","5307":"Groceries","5309":"Groceries","5310":"Groceries","5311":"Groceries","5312":"Groceries","5313":"Air Travel","5314":"Groceries","5315":"Fast Food","5316":"Fast Food","5318":"Uncategorized","5319":"Air Travel","5320":"Groceries","5321":"Fast Food","5322":"Furnishings","5323":"Fast Food","5324":"Groceries","5325":"Uncategorized","5326":"Pharmacy","5327":"Gas & Fuel","5329":"Air Travel","5330":"Alcohol & Bars","5331":"Fast Food","5332":"Uncategorized","5333":"Sporting Goods","5334":"Newspapers & Magazines","5335":"Fees & Charges","5336":"Groceries","5337":"Alcohol & Bars","5339":"Alcohol & Bars","5340":"Groceries","5341":"Alcohol & Bars","5342":"Groceries","5343":"Groceries","5344":"Groceries","5345":"Groceries","5346":"Uncategorized","5347":"Restaurants","5348":"Groceries","5349":"Groceries","5350":"Groceries","5351":"Tuition","5352":"Groceries","5353":"Groceries","5354":"Groceries","5355":"Groceries","5357":"Air Travel","5358":"Alcohol & Bars","5359":"Groceries","5360":"Home Improvement","5361":"Groceries","5362":"Groceries","5363":"Groceries","5364":"Home Improvement","5365":"Printing","5367":"Groceries","5368":"Groceries","5372":"Groceries","5373":"Groceries","5374":"Groceries","5375":"Groceries","5376":"Groceries","5377":"Groceries","5378":"Groceries","5379":"Gas & Fuel","5380":"Groceries","5381":"Personal Care","5382":"Gas & Fuel","5383":"Groceries","5384":"Groceries","5385":"Movies & DVDs","5391":"Utilities","5392":"Groceries","5393":"Groceries","5394":"Restaurants","5395":"Business Services","5396":"Home Improvement","5397":"Groceries","5398":"Groceries","5399":"Groceries","5400":"Uncategorized","5401":"Groceries","5402":"Groceries","5403":"Groceries","5404":"Groceries","5405":"Pharmacy","5406":"Food & Dining","5407":"Fast Food","5408":"Fast Food","5409":"Fast Food","5410":"Restaurants","5412":"Gas & Fuel","5413":"Air Travel","5414":"Restaurants","5415":"Pharmacy","5416":"Hair","5417":"Hair","5420":"Groceries","5422":"Uncategorized","5423":"Uncategorized","5424":"Uncategorized","5425":"Business Services","5426":"Restaurants","5427":"Books","5428":"Groceries","5429":"Uncategorized","5430":"Hotel","5431":"Business Services","5432":"Business Services","5433":"Business Services","5435":"Groceries","5436":"Business Services","5437":"Business Services","5438":"Business Services","5439":"Tuition","5440":"Fast Food","5441":"Public Transportation","5442":"Business Services","5443":"Business Services","5444":"Business Services","5445":"Hotel","5446":"Business Services","5447":"Business Services","5448":"Groceries","5449":"Business Services","5450":"Business Services","5451":"Business Services","5452":"Business Services","5453":"Shipping","5456":"Business Services","5457":"Business Services","5458":"Business Services","5459":"Uncategorized","5460":"Restaurants","5461":"Uncategorized","5463":"Business Services","5464":"Groceries","5465":"Sporting Goods","5466":"Fast Food","5467":"Food & Dining","5468":"Uncategorized","5469":"Uncategorized","5470":"Groceries","5471":"Pharmacy","5473":"Pharmacy","5475":"Alcohol & Bars","5476":"Coffee Shops","5477":"Rental Car & Taxi","5478":"Groceries","5479":"Groceries","5480":"Groceries","5481":"Movies & DVDs","5483":"Groceries","5484":"Groceries","5485":"Groceries","5486":"Groceries","5487":"Groceries","5488":"Gas & Fuel","5489":"Groceries","5490":"Groceries","5491":"Hotel","5492":"Utilities","5493":"Groceries","5494":"Groceries","5495":"Business Services","5496":"Business Services","5498":"Groceries","5499":"Groceries","5500":"Groceries","5501":"Groceries","5502":"Furnishings","5503":"Entertainment","5506":"Groceries","5507":"Groceries","5508":"Groceries","5509":"Groceries","5510":"Uncategorized","5511":"Parking","5512":"Shopping","5513":"Groceries","5514":"Shopping","5515":"Hotel","5516":"Coffee Shops","5517":"Groceries","5518":"Groceries","5519":"Groceries","5520":"Groceries","5521":"Groceries","5522":"Groceries","5524":"Groceries","5525":"Air Travel","5526":"Food & Dining","5527":"Shopping","5528":"Groceries","5529":"Groceries","5530":"Home Improvement","5531":"Groceries","5532":"Groceries","5533":"Tuition","5534":"Shopping","5535":"Groceries","5536":"Pharmacy","5537":"Shopping","5538":"Groceries","5539":"Groceries","5540":"Groceries","5541":"Groceries","5542":"Air Travel","5545":"Groceries","5546":"Groceries","5547":"Groceries","5548":"Groceries","5550":"Groceries","5551":"Groceries","5552":"Groceries","5553":"Groceries","5554":"Groceries","5557":"Restaurants","5559":"Coffee Shops","5560":"Restaurants","5561":"Shopping","5562":"Entertainment","5563":"Uncategorized","5564":"Uncategorized","5565":"Entertainment","5566":"Groceries","5567":"Groceries","5568":"Groceries","5569":"Clothing","5570":"Clothing","5571":"Restaurants","5572":"Alcohol & Bars","5573":"Groceries","5574":"Books","5575":"Restaurants","5576":"Alcohol & Bars","5577":"Groceries","5578":"Groceries","5579":"Shopping","5580":"Air Travel","5581":"Personal Care","5582":"Groceries","5583":"Groceries","5584":"Shopping","5585":"Groceries","5586":"Home Improvement","5587":"Groceries","5588":"Home Improvement","5589":"Business Services","5590":"Groceries","5591":"Groceries","5592":"Business Services","5593":"Groceries","5594":"Groceries","5595":"Tuition","5597":"Groceries","5598":"Fast Food","5599":"Groceries","5600":"Movies & DVDs","5601":"Uncategorized","5602":"Furnishings","5603":"Groceries","5604":"Hotel","5606":"Gym","5607":"Entertainment","5608":"Groceries","5609":"Air Travel","5610":"Printing","5612":"Printing","5614":"Groceries","5617":"Groceries","5622":"Air Travel","5623":"Air Travel","5624":"Groceries","5625":"Air Travel","5626":"Groceries","5627":"Groceries","5628":"Groceries","5629":"Groceries","5630":"Cash & ATM","5631":"Groceries","5632":"Uncategorized","5633":"Pharmacy","5634":"Groceries","5635":"Groceries","5636":"Groceries","5637":"Utilities","5638":"Groceries","5639":"Groceries","5640":"Air Travel","5641":"Groceries","5642":"Groceries","5644":"Groceries","5646":"Shopping","5647":"Uncategorized","5648":"Restaurants","5649":"Groceries","5650":"Uncategorized","5651":"Groceries","5652":"Restaurants","5653":"Auto & Transport","5654":"Groceries","5655":"Air Travel","5656":"Groceries","5657":"Uncategorized","5658":"Groceries","5659":"Groceries","5660":"Groceries","5661":"Groceries","5662":"Groceries","5663":"Groceries","5664":"Groceries","5665":"Groceries","5666":"Groceries","5668":"Cash & ATM","5670":"Groceries","5671":"Groceries","5672":"Groceries","5673":"Groceries","5674":"Tuition","5675":"Groceries","5676":"Gift","5677":"Groceries","5678":"Groceries","5679":"Groceries","5680":"Groceries","5681":"Groceries","5682":"Uncategorized","5683":"Groceries","5684":"Groceries","5685":"Groceries","5686":"Groceries","5687":"Restaurants","5688":"Pharmacy","5689":"Groceries","5690":"Groceries","5691":"Groceries","5692":"Groceries","5693":"Groceries","5694":"Groceries","5695":"Groceries","5696":"Public Transportation","5698":"Food & Dining","5700":"Groceries","5701":"Groceries","5702":"Groceries","5704":"Groceries","5705":"Groceries","5706":"Alcohol & Bars","5707":"Groceries","5712":"Restaurants","5713":"Groceries","5714":"Groceries","5715":"Groceries","5718":"Gas & Fuel","5719":"Movies & DVDs","5721":"Restaurants","5722":"Doctor","5723":"Gym","5724":"Groceries","5725":"Groceries","5727":"Groceries","5729":"Restaurants","5730":"Books","5731":"Groceries","5732":"Groceries","5733":"Groceries","5734":"Groceries","5735":"Shopping","5736":"Groceries","5737":"Uncategorized","5738":"Groceries","5739":"Home Improvement","5740":"Groceries","5741":"Groceries","5742":"Groceries","5744":"Groceries","5745":"Groceries","5746":"Personal Care","5747":"Groceries","5748":"Groceries","5749":"Air Travel","5750":"Groceries","5751":"Groceries","5752":"Electronics & Software","5753":"Hotel","5754":"Restaurants","5755":"Groceries","5756":"Restaurants","5758":"Restaurants","5759":"Groceries","5760":"Groceries","5761":"Groceries","5762":"Gas & Fuel","5764":"Restaurants","5765":"Groceries","5766":"Uncategorized","5767":"Groceries","5768":"Groceries","5769":"Groceries","5770":"Fees & Charges","5771":"Restaurants","5772":"Groceries","5773":"Shopping","5774":"Groceries","5775":"Groceries","5778":"Groceries","5779":"Groceries","5780":"Groceries","5781":"Groceries","5782":"Groceries","5783":"Alcohol & Bars","5785":"Shopping","5786":"Groceries","5787":"Groceries","5788":"Groceries","5789":"Coffee Shops","5790":"Groceries","5791":"Groceries","5792":"Coffee Shops","5793":"Movies & DVDs","5795":"Fast Food","5796":"Groceries","5797":"Groceries","5798":"Groceries","5799":"Groceries","5800":"Shopping","5801":"Entertainment","5802":"Printing","5803":"Printing","5804":"Coffee Shops","5805":"Restaurants","5806":"Coffee Shops","5807":"Groceries","5808":"Alcohol & Bars","5809":"Groceries","5810":"Groceries","5811":"Groceries","5812":"Groceries","5813":"Gift","5814":"Groceries","5816":"Hotel","5817":"Restaurants","5819":"Utilities","5820":"Groceries","5821":"Coffee Shops","5822":"Coffee Shops","5823":"Groceries","5824":"Groceries","5825":"Groceries","5826":"Restaurants","5827":"Groceries","5828":"Groceries","5829":"Groceries","5831":"Restaurants","5832":"Groceries","5833":"Groceries","5834":"Gas & Fuel","5835":"Shopping","5836":"Groceries","5837":"Groceries","5839":"Sporting Goods","5840":"Groceries","5841":"Groceries","5842":"Groceries","5843":"Coffee Shops","5844":"Tuition","5845":"Parking","5847":"Restaurants","5848":"Coffee Shops","5849":"Gas & Fuel","5850":"Groceries","5851":"Groceries","5852":"Coffee Shops","5853":"Groceries","5854":"Shopping","5855":"Shopping","5856":"Groceries","5857":"Groceries","5858":"Restaurants","5859":"Groceries","5860":"Restaurants","5861":"Groceries","5862":"Fees & Charges","5863":"Air Travel","5864":"ATM Fee","5866":"Cash & ATM","5867":"Coffee Shops","5869":"Home Services","5870":"Fees & Charges","5871":"Health & Fitness","5872":"Fees & Charges","5874":"Fees & Charges","5875":"Fees & Charges","5876":"Fees & Charges","5877":"Cash & ATM","5878":"ATM Fee","5879":"Cash & ATM","5880":"ATM Fee","5882":"Tuition","5883":"Fees & Charges","5884":"Travel","5887":"Alcohol & Bars","5889":"Fees & Charges","5890":"ATM Fee","5891":"Cash & ATM","5892":"Fees & Charges","5893":"Fees & Charges","5894":"Fees & Charges","5895":"Cash & ATM","5896":"Cash & ATM","5897":"ATM Fee","5898":"ATM Fee","5899":"Movies & DVDs","5900":"ATM Fee","5902":"Cash & ATM","5903":"Fees & Charges","5906":"Cash & ATM","5907":"ATM Fee","5908":"Fees & Charges","5909":"Shopping","5910":"Shopping","5911":"Fees & Charges","5912":"Cash & ATM","5913":"Service Fee","5915":"Uncategorized","5917":"ATM Fee","5918":"Hotel","5919":"Fees & Charges","5922":"Uncategorized","5923":"Fees & Charges","5924":"Newspapers & Magazines","5925":"Fees & Charges","5926":"Uncategorized","5927":"Alcohol & Bars","5929":"Restaurants","5931":"Groceries","5933":"Groceries","5934":"Shopping","5935":"Shopping","5936":"Shopping","5937":"Groceries","5938":"Groceries","5940":"Movies & DVDs","5941":"Groceries","5943":"Groceries","5944":"Coffee Shops","5945":"Groceries","5946":"Restaurants","5947":"Groceries","5949":"Groceries","5950":"Alcohol & Bars","5951":"Cash & ATM","5952":"Groceries","5953":"Groceries","5954":"Fast Food","5955":"Groceries","5956":"Rental Car & Taxi","5957":"Groceries","5958":"Shopping","5959":"Groceries","5960":"Utilities","5961":"Groceries","5968":"Groceries","5969":"Restaurants","5971":"Fast Food","5972":"Groceries","5973":"Groceries","5974":"Groceries","5975":"Groceries","5976":"Uncategorized","5977":"Groceries","5978":"Groceries","5979":"Groceries","5982":"Hotel","5983":"Restaurants","5984":"Restaurants","5985":"Home","5986":"Uncategorized","5987":"Groceries","5988":"Groceries","5989":"Restaurants","5990":"Parking","5991":"Home","5992":"Groceries","5993":"Movies & DVDs","5994":"Restaurants","5995":"Movies & DVDs","5996":"Gas & Fuel","5997":"Alcohol & Bars","5998":"Groceries","5999":"Groceries","6000":"Groceries","6001":"Gas & Fuel","6002":"Groceries","6006":"Alcohol & Bars","6007":"Groceries","6010":"Groceries","6012":"Groceries","6013":"Restaurants","6014":"Groceries","6016":"Shipping","6017":"Groceries","6018":"Movies & DVDs","6019":"Groceries","6020":"Groceries","6021":"Uncategorized","6022":"Business Services","6023":"Books","6024":"Shopping","6025":"Fast Food","6026":"Pharmacy","6027":"Personal Care","6029":"Coffee Shops","6030":"Groceries","6031":"Shopping","6032":"Coffee Shops","6033":"Food & Dining","6034":"Restaurants","6035":"Groceries","6036":"Restaurants","6037":"Groceries","6038":"Furnishings","6039":"Restaurants","6040":"Home Phone","6041":"Restaurants","6042":"Coffee Shops","6043":"Groceries","6044":"Restaurants","6045":"Shopping","6046":"Groceries","6047":"Shopping","6048":"Groceries","6049":"Furnishings","6050":"Groceries","6051":"Groceries","6052":"Air Travel","6053":"Groceries","6054":"Coffee Shops","6055":"Groceries","6056":"Groceries","6057":"Groceries","6059":"Restaurants","6060":"Pharmacy","6061":"Groceries","6062":"Groceries","6063":"Groceries","6064":"Groceries","6065":"Pharmacy","6066":"Groceries","6067":"Restaurants","6068":"Groceries","6069":"Tuition","6070":"Clothing","6071":"Restaurants","6072":"Fast Food","6073":"Restaurants","6074":"Business Services","6075":"Books","6076":"Uncategorized","6077":"Restaurants","6078":"Gym","6079":"Clothing","6080":"Shopping","6081":"Alcohol & Bars","6082":"Clothing","6083":"Food & Dining","6084":"Groceries","6085":"Shipping","6087":"Shopping","6088":"Shopping","6089":"Fast Food","6090":"Shopping","6091":"Uncategorized","6092":"Hotel","6093":"Alcohol & Bars","6094":"Restaurants","6095":"Restaurants","6096":"Groceries","6097":"Shopping","6098":"Groceries","6099":"Gas & Fuel","6100":"Groceries","6101":"Books","6102":"Books","6103":"Books","6104":"Restaurants","6105":"Groceries","6106":"Travel","6107":"Movies & DVDs","6108":"Education","6109":"Groceries","6110":"Entertainment","6111":"Restaurants","6112":"Groceries","6113":"Restaurants","6114":"Restaurants","6115":"Pharmacy","6116":"Fast Food","6117":"Groceries","6118":"Coffee Shops","6119":"Groceries","6120":"Groceries","6121":"Groceries","6123":"Groceries","6124":"Groceries","6126":"Groceries","6127":"Movies & DVDs","6128":"Hotel","6130":"Fast Food","6131":"Groceries","6132":"Groceries","6133":"Restaurants","6134":"Restaurants","6135":"Newspapers & Magazines","6136":"Food & Dining","6137":"Restaurants","6138":"Air Travel","6139":"Coffee Shops","6140":"Uncategorized","6141":"Air Travel","6143":"Air Travel","6144":"Hotel","6145":"Groceries","6146":"Groceries","6148":"Groceries","6149":"Public Transportation","6151":"Uncategorized","6152":"Gas & Fuel","6153":"Restaurants","6154":"Groceries","6155":"Gym","6156":"Groceries","6157":"Fast Food","6158":"Groceries","6160":"Groceries","6161":"Alcohol & Bars","6162":"Alcohol & Bars","6163":"Uncategorized","6164":"Groceries","6165":"Groceries","6166":"Parking","6167":"Business Services","6168":"Groceries","6169":"Restaurants","6170":"Groceries","6171":"Groceries","6173":"Groceries","6175":"Groceries","6176":"Groceries","6177":"Groceries","6178":"Groceries","6179":"Shopping","6180":"Shopping","6181":"Movies & DVDs","6182":"Food & Dining","6183":"Groceries","6184":"Groceries","6185":"Business Services","6186":"Furnishings","6187":"Gas & Fuel","6188":"Alcohol & Bars","6189":"Movies & DVDs","6190":"Air Travel","6191":"Groceries","6192":"Restaurants","6194":"Groceries","6195":"Groceries","6197":"Gas & Fuel","6198":"Groceries","6199":"Groceries","6200":"Groceries","6201":"Fast Food","6202":"Groceries","6203":"Groceries","6204":"Coffee Shops","6205":"Coffee Shops","6206":"Groceries","6207":"Groceries","6208":"Restaurants","6210":"Groceries","6211":"Groceries","6212":"Groceries","6213":"Uncategorized","6214":"Clothing","6215":"Groceries","6216":"Hotel","6217":"Hotel","6218":"Hotel","6219":"Groceries","6220":"Groceries","6221":"Food & Dining","6222":"Groceries","6223":"Groceries","6225":"Fast Food","6226":"Coffee Shops","6227":"Restaurants","6228":"Food & Dining","6229":"Alcohol & Bars","6231":"Restaurants","6232":"Movies & DVDs","6233":"Gas & Fuel","6234":"Groceries","6235":"Mortgage & Rent","6236":"Restaurants","6237":"Parking","6238":"Food & Dining","6239":"Alcohol & Bars","6240":"Restaurants","6241":"Restaurants","6242":"Groceries","6243":"Groceries","6244":"Groceries","6245":"Shopping","6246":"Coffee Shops","6247":"Furnishings","6248":"Fast Food","6249":"Food & Dining","6250":"Food & Dining","6251":"Food & Dining","6252":"Hotel","6253":"Tuition","6254":"Groceries","6255":"Groceries","6256":"Groceries","6257":"Food & Dining","6258":"Shopping","6259":"Shopping","6260":"Groceries","6261":"Groceries","6262":"Shopping","6264":"Alcohol & Bars","6267":"Tuition","6268":"Groceries","6269":"Parking","6270":"Uncategorized","6271":"Uncategorized","6272":"Tuition","6274":"Groceries","6275":"Tuition","6276":"Restaurants","6277":"Tuition","6278":"Groceries","6279":"Shopping","6280":"Movies & DVDs","6282":"Groceries","6285":"Gas & Fuel","6286":"Gas & Fuel","6287":"Hotel","6288":"Gas & Fuel","6289":"Uncategorized","6290":"Uncategorized","6292":"Shopping","6293":"Restaurants","6294":"Groceries","6295":"Food & Dining","6296":"Food & Dining","6297":"Gas & Fuel","6298":"Uncategorized","6299":"Fast Food","6300":"Groceries","6301":"Fast Food","6302":"Fast Food","6303":"Fast Food","6305":"Gift","6306":"Groceries","6307":"Fast Food","6308":"Personal Care","6309":"Hotel","6310":"Groceries","6311":"Shopping","6312":"Groceries","6313":"Groceries","6314":"Groceries","6315":"Pharmacy","6316":"Restaurants","6317":"Gas & Fuel","6318":"Gas & Fuel","6319":"Shopping","6320":"Air Travel","6322":"Restaurants","6323":"Groceries","6324":"Gym","6325":"Food & Dining","6326":"Shopping","6327":"Groceries","6328":"Groceries","6329":"Shopping","6330":"Shopping","6331":"Groceries","6332":"Alcohol & Bars","6333":"Groceries","6334":"Shipping","6335":"Shopping","6336":"Restaurants","6337":"Coffee Shops","6338":"Shopping","6339":"Groceries","6340":"Restaurants","6341":"Shopping","6342":"Hotel","6343":"Shopping","6344":"Groceries","6345":"Rental Car & Taxi","6346":"Gas & Fuel","6348":"Groceries","6349":"Groceries","6350":"Groceries","6351":"Restaurants","6352":"Movies & DVDs","6353":"Alcohol & Bars","6355":"Shopping","6356":"Health & Fitness","6357":"Uncategorized","6358":"Groceries","6359":"Movies & DVDs","6360":"Groceries","6361":"Movies & DVDs","6362":"Shopping","6363":"Tuition","6364":"Groceries","6365":"Shopping","6366":"Groceries","6367":"Restaurants","6373":"Education","6374":"Groceries","6375":"Groceries","6376":"Air Travel","6377":"Groceries","6378":"Groceries","6379":"Coffee Shops","6380":"Groceries","6381":"Restaurants","6382":"Gas & Fuel","6383":"Shopping","6384":"Food & Dining","6385":"Fast Food","6386":"Shopping","6387":"Business Services","6388":"Groceries","6389":"Entertainment","6390":"Tuition","6391":"Coffee Shops","6392":"Gym","6393":"Groceries","6394":"Tuition","6395":"Education","6396":"Groceries","6398":"Groceries","6401":"Groceries","6402":"Shopping","6403":"Shopping","6404":"Shopping","6405":"Shopping","6406":"Shopping","6407":"Shopping","6408":"Home Improvement","6409":"Home Improvement","6410":"Furnishings","6411":"Gym","6412":"Home Improvement","6413":"Food & Dining","6415":"Groceries","6416":"Groceries","6417":"Groceries","6418":"Uncategorized","6419":"Coffee Shops","6420":"Home Improvement","6421":"Groceries","6422":"Gym","6423":"Coffee Shops","6424":"Groceries","6425":"Groceries","6426":"Restaurants","6427":"Fast Food","6428":"Shopping","6429":"Restaurants","6430":"Groceries","6431":"Service & Parts","6432":"Uncategorized","6433":"Entertainment","6435":"Shopping","6436":"Alcohol & Bars","6437":"Groceries","6438":"Groceries","6439":"Groceries","6440":"Groceries","6441":"Gas & Fuel","6442":"Food & Dining","6443":"Movies & DVDs","6444":"Auto Insurance","6445":"Groceries","6446":"Shopping","6447":"Parking","6448":"Shopping","6449":"Groceries","6450":"Groceries","6451":"Groceries","6453":"Groceries","6454":"Groceries","6455":"Groceries","6456":"Shopping","6457":"Groceries","6458":"Groceries","6459":"Groceries","6460":"Groceries","6461":"Restaurants","6462":"Utilities","6463":"Restaurants","6464":"Home Improvement","6465":"Gym","6466":"Groceries","6467":"Parking","6468":"Uncategorized","6469":"Restaurants","6470":"Restaurants","6471":"Groceries","6472":"Groceries","6473":"Coffee Shops","6474":"Fast Food","6475":"Food & Dining","6476":"Groceries","6477":"Restaurants","6478":"Pharmacy","6479":"Food & Dining","6480":"Food & Dining","6481":"Parking","6482":"Restaurants","6483":"Groceries","6484":"Shopping","6485":"Restaurants","6486":"Shopping","6487":"Shopping","6488":"Food & Dining","6491":"Groceries","6492":"Groceries","6493":"Groceries","6495":"Restaurants","6496":"Doctor","6497":"Restaurants","6498":"Doctor","6499":"Uncategorized","6500":"Restaurants","6501":"Rental Car & Taxi","6502":"Restaurants","6503":"Shopping","6504":"Groceries","6505":"Fast Food","6506":"Restaurants","6507":"Doctor","6508":"Uncategorized","6509":"Air Travel","6510":"Doctor","6511":"Gym","6512":"Groceries","6513":"Shopping","6514":"Fast Food","6515":"Uncategorized","6516":"Newspapers & Magazines","6517":"Restaurants","6519":"Groceries","6520":"Fast Food","6521":"Restaurants","6522":"Groceries","6523":"Hotel","6524":"Groceries","6525":"Shopping","6526":"Uncategorized","6527":"Uncategorized","6528":"Fast Food","6529":"Gas & Fuel","6530":"ATM Fee","6531":"Cash & ATM","6532":"ATM Fee","6533":"Food & Dining","6534":"Alcohol & Bars","6535":"Coffee Shops","6536":"Alcohol & Bars","6537":"Alcohol & Bars","6538":"Restaurants","6539":"Clothing","6540":"Restaurants","6541":"Groceries","6542":"Coffee Shops","6543":"Shopping","6544":"Shopping","6545":"Shopping","6546":"Alcohol & Bars","6547":"Shopping","6548":"Shopping","6549":"Shopping","6550":"Air Travel","6551":"Shopping","6552":"Shopping","6553":"Groceries","6554":"Food & Dining","6555":"Food & Dining","6556":"Coffee Shops","6557":"Groceries","6558":"Groceries","6559":"Groceries","6561":"Tuition","6562":"Parking","6563":"Groceries","6564":"Auto Insurance","6565":"Groceries","6567":"Movies & DVDs","6569":"Groceries","6570":"Restaurants","6571":"Food & Dining","6572":"Groceries","6573":"Groceries","6574":"Groceries","6575":"Air Travel","6576":"Groceries","6577":"Coffee Shops","6578":"Shopping","6580":"Food & Dining","6581":"Groceries","6582":"Groceries","6583":"Groceries","6584":"Restaurants","6585":"Groceries","6586":"Alcohol & Bars","6587":"Alcohol & Bars","6588":"Groceries","6589":"Restaurants","6590":"Spa & Massage","6591":"Gas & Fuel","6592":"Food & Dining","6593":"Personal Care","6594":"Restaurants","6595":"Food & Dining","6596":"Fast Food","6597":"Food & Dining","6598":"Groceries","6599":"Kids","6600":"Parking","6602":"Alcohol & Bars","6603":"Service Fee","6604":"Groceries","6605":"Food & Dining","6606":"Groceries","6607":"Fast Food","6609":"Gas & Fuel","6610":"Service Fee","6611":"Service Fee","6612":"Service Fee","6613":"Service Fee","6614":"Restaurants","6615":"Restaurants","6616":"Service Fee","6617":"Groceries","6618":"Groceries","6619":"Service Fee","6620":"Shopping","6621":"Service Fee","6622":"Restaurants","6623":"Books","6624":"Service Fee","6625":"Service Fee","6627":"Service Fee","6628":"Business Services","6629":"Tuition","6630":"Rental Car & Taxi","6631":"Food & Dining","6632":"Gym","6633":"Gift","6634":"Alcohol & Bars","6635":"Groceries","6636":"Pharmacy","6637":"Clothing","6638":"Groceries","6639":"Groceries","6640":"Shopping","6641":"Pharmacy","6642":"Food & Dining","6643":"Clothing","6644":"Groceries","6645":"Furnishings","6646":"Auto & Transport","6647":"Restaurants","6648":"Parking","6649":"Mortgage & Rent","6650":"Groceries","6652":"Parking","6653":"Groceries","6654":"Restaurants","6655":"Gym","6656":"Food & Dining","6657":"Groceries","6658":"Parking","6659":"Gym","6660":"Gas & Fuel","6662":"Shopping","6664":"Alcohol & Bars","6665":"Restaurants","6666":"Restaurants","6667":"Restaurants","6668":"Coffee Shops","6669":"Restaurants","6670":"Coffee Shops","6671":"Student Loan","6672":"Groceries","6673":"Restaurants","6674":"Fast Food","6675":"Furnishings","6676":"Movies & DVDs","6677":"Alcohol & Bars","6678":"Air Travel","6679":"Restaurants","6680":"Restaurants","6681":"Food & Dining","6682":"Food & Dining","6683":"Groceries","6684":"Shopping","6685":"Groceries","6687":"Spa & Massage","6688":"Coffee Shops","6690":"Groceries","6691":"Spa & Massage","6692":"Clothing","6693":"Air Travel","6694":"Restaurants","6695":"Shopping","6696":"Groceries","6697":"Air Travel","6698":"Restaurants","6699":"Fast Food","6700":"Restaurants","6701":"Groceries","6702":"Food & Dining","6703":"Food & Dining","6704":"Alcohol & Bars","6705":"Alcohol & Bars","6706":"Restaurants","6707":"Groceries","6708":"Shopping","6709":"Shopping","6710":"Groceries","6711":"Electronics & Software","6712":"Groceries","6713":"Groceries","6715":"Kids","6716":"Food & Dining","6717":"Restaurants","6719":"Clothing","6721":"Transfer for Cash Spending","6722":"Restaurants","6723":"Groceries","6724":"Amusement","6725":"Restaurants","6726":"Restaurants","6727":"Gas & Fuel","6728":"Shopping","6729":"Shopping","6730":"Parking","6731":"Shopping","6733":"Bank Fee","6734":"Trade Commissions","6735":"Groceries","6736":"Restaurants","6737":"Shipping","6738":"Groceries","6739":"Groceries","6740":"Shopping","6741":"Groceries","6742":"Spa & Massage","6743":"Shopping","6744":"Food & Dining","6745":"Coffee Shops","6746":"Cash & ATM","6747":"Shopping","6748":"Shopping","6749":"Clothing","6751":"Groceries","6752":"Air Travel","6753":"Cash & ATM","6756":"Restaurants","6757":"Groceries","6764":"Alcohol & Bars","6765":"Shopping","6766":"Restaurants","6767":"Personal Care","6768":"Groceries","6769":"Restaurants","6770":"Business Services","6771":"Fast Food","6772":"Business Services","6773":"Groceries","6774":"Pharmacy","6775":"Movies & DVDs","6776":"Groceries","6777":"Business Services","6779":"Air Travel","6780":"Groceries","6783":"Restaurants","6784":"Restaurants","6785":"Rental Car & Taxi","6786":"Restaurants","6787":"Mortgage & Rent","6788":"Rental Car & Taxi","6789":"Coffee Shops","6790":"Alcohol & Bars","6791":"Alcohol & Bars","6792":"Groceries","6793":"Rental Car & Taxi","6794":"Uncategorized","6795":"Groceries","6796":"Restaurants","6797":"Utilities","6798":"Alcohol & Bars","6799":"Alcohol & Bars","6800":"Books","6801":"Restaurants","6803":"Cash & ATM","6805":"Fast Food","6806":"Food & Dining","6807":"ATM Fee","6808":"Fast Food","6809":"Gas & Fuel","6810":"Groceries","6811":"Groceries","6812":"Uncategorized","6813":"Groceries","6814":"Groceries","6815":"Groceries","6816":"Groceries","6817":"Movies & DVDs","6818":"Restaurants","6819":"Restaurants","6822":"Check","6823":"Cash & ATM","6824":"Groceries","6826":"Restaurants","6827":"Food & Dining","6828":"Coffee Shops","6829":"Check","6830":"Entertainment","6831":"Coffee Shops","6832":"Alcohol & Bars","6833":"Groceries","6835":"Groceries","6836":"Groceries","6837":"Groceries","6838":"Groceries","6839":"Check","6840":"Groceries","6841":"Groceries","6842":"Shopping","6844":"Food & Dining","6845":"Uncategorized","6846":"Shopping","6847":"Food & Dining","6848":"Gift","6849":"Groceries","6850":"Uncategorized","6851":"Shopping","6852":"Movies & DVDs","6853":"Groceries","6855":"Cash & ATM","6856":"Tuition","6858":"Alcohol & Bars","6859":"Groceries","6861":"Fast Food","6862":"Shopping","6863":"Groceries","6864":"Business Services","6865":"Groceries","6866":"Groceries","6867":"Coffee Shops","6868":"Movies & DVDs","6869":"Kids","6870":"Groceries","6871":"Groceries","6872":"Kids","6874":"Restaurants","6875":"Restaurants","6876":"Sporting Goods","6877":"Restaurants","6878":"Movies & DVDs","6879":"Fast Food","6880":"Business Services","6881":"Gas & Fuel","6882":"Electronics & Software","6883":"Alcohol & Bars","6884":"Mortgage & Rent","6885":"Groceries","6886":"Groceries","6889":"Gift","6891":"Groceries","6892":"Home Improvement","6893":"Shopping","6894":"Hotel","6895":"Groceries","6896":"Entertainment","6897":"Air Travel","6898":"Restaurants","6899":"Food & Dining","6900":"Cash & ATM","6901":"Fast Food","6902":"Restaurants","6903":"Business Services","6904":"Groceries","6905":"Coffee Shops","6906":"Fast Food","6907":"Movies & DVDs","6908":"Groceries","6909":"Gift","6910":"Groceries","6911":"Groceries","6912":"Food & Dining","6915":"Parking","6916":"Shopping","6917":"Cash & ATM","6918":"Groceries","6919":"Alcohol & Bars","6920":"Gas & Fuel","6922":"Home Improvement","6924":"Movies & DVDs","6925":"Restaurants","6927":"Groceries","6928":"Groceries","6929":"Fast Food","6930":"Restaurants","6931":"Air Travel","6932":"Uncategorized","6933":"Groceries","6934":"Groceries","6935":"Groceries","6937":"Groceries","6938":"Shopping","6939":"Utilities","6940":"Shopping","6941":"Cash & ATM","6942":"Shopping","6944":"Shopping","6945":"Groceries","6946":"Groceries","6948":"Groceries","6949":"Shopping","6950":"Shopping","6951":"Groceries","6952":"Groceries","6953":"Groceries","6954":"Fast Food","6955":"Groceries","6956":"Groceries","6957":"Food & Dining","6958":"Groceries","6959":"Pharmacy","6960":"Groceries","6962":"Groceries","6963":"Groceries","6964":"Groceries","6965":"Groceries","6966":"Food & Dining","6967":"Uncategorized","6968":"Groceries","6969":"Cash & ATM","6970":"Groceries","6971":"Groceries","6972":"Uncategorized","6973":"Groceries","6974":"Groceries","6975":"Uncategorized","6976":"Uncategorized","6977":"Gas & Fuel","6978":"Restaurants","6979":"Travel","6980":"Travel","6981":"Food & Dining","6982":"Restaurants","6983":"Restaurants","6984":"Restaurants","6985":"Coffee Shops","6986":"Alcohol & Bars","6988":"Fast Food","6989":"Air Travel","6990":"Newspapers & Magazines","6991":"Movies & DVDs","6992":"Groceries","6993":"Uncategorized","6995":"Food & Dining","6996":"Alcohol & Bars","6999":"Cash & ATM","7000":"Groceries","7001":"Groceries","7003":"Pharmacy","7004":"Food & Dining","7005":"Groceries","7006":"Coffee Shops","7007":"Groceries","7008":"Restaurants","7009":"Food & Dining","7010":"Uncategorized","7011":"Coffee Shops","7012":"Fast Food","7013":"Food & Dining","7014":"Groceries","7015":"Groceries","7016":"Food & Dining","7017":"Uncategorized","7018":"Uncategorized","7019":"Cash & ATM","7020":"Groceries","7021":"Restaurants","7022":"Uncategorized","7023":"Restaurants","7024":"Check","7025":"Fast Food","7026":"Shopping","7028":"Shopping","7029":"Cash & ATM","7030":"Shopping","7031":"Tuition","7032":"Movies & DVDs","7034":"Clothing","7035":"Groceries","7036":"Clothing","7038":"Education","7039":"Alcohol & Bars","7040":"Tuition","7041":"Groceries","7043":"Food & Dining","7044":"Alcohol & Bars","7046":"Shopping","7048":"Groceries","7049":"Movies & DVDs","7051":"Groceries","7052":"Gas & Fuel","7053":"Groceries","7054":"Groceries","7055":"Groceries","7056":"Groceries","7057":"Shopping","7058":"Groceries","7059":"Groceries","7060":"Mortgage & Rent","7063":"Shopping","7066":"Groceries","7067":"Shopping","7068":"Shopping","7069":"Shopping","7070":"Shopping","7071":"Food & Dining","7072":"Bank Fee","7073":"Uncategorized","7075":"Shopping","7076":"Food & Dining","7077":"Movies & DVDs","7078":"Restaurants","7079":"Groceries","7080":"Food & Dining","7081":"Restaurants","7082":"Uncategorized","7083":"Personal Care","7084":"Restaurants","7085":"Groceries","7086":"Uncategorized","7088":"Air Travel","7090":"Air Travel","7091":"Shopping","7092":"Restaurants","7093":"Gas & Fuel","7094":"Air Travel","7097":"Rental Car & Taxi","7098":"Restaurants","7099":"Bank Fee","7100":"Newspapers & Magazines","7101":"Groceries","7102":"Rental Car & Taxi","7103":"Bank Fee","7104":"Groceries","7106":"Uncategorized","7107":"Uncategorized","7108":"Groceries","7109":"Groceries","7110":"Groceries","7111":"Coffee Shops","7112":"Groceries","7113":"Hotel","7114":"Groceries","7115":"Food & Dining","7116":"Rental Car & Taxi","7117":"Auto & Transport","7118":"Pharmacy","7119":"Food & Dining","7120":"Food & Dining","7121":"Food & Dining","7122":"Groceries","7126":"Groceries","7127":"Groceries","7130":"Groceries","7131":"Alcohol & Bars","7133":"Groceries","7134":"Music","7135":"Groceries","7136":"Alcohol & Bars","7137":"Groceries","7138":"Movies & DVDs","7139":"Groceries","7140":"Food & Dining","7142":"Groceries","7143":"Groceries","7144":"Gas & Fuel","7145":"Restaurants","7146":"Gas & Fuel","7147":"Kids","7148":"Groceries","7149":"Uncategorized","7150":"Uncategorized","7151":"Restaurants","7152":"Shopping","7154":"Uncategorized","7155":"Shopping","7158":"Groceries","7159":"Groceries","7160":"Uncategorized","7163":"Uncategorized","7164":"Groceries","7165":"Restaurants","7166":"Shipping","7167":"Restaurants","7168":"Restaurants","7169":"Air Travel","7170":"Groceries","7172":"Food & Dining","7173":"Restaurants","7175":"Groceries","7176":"Groceries","7177":"Gas & Fuel","7178":"Groceries","7179":"Groceries","7180":"Groceries","7185":"Uncategorized","7186":"Groceries","7187":"Movies & DVDs","7188":"Groceries","7189":"Groceries","7190":"Uncategorized","7191":"Food & Dining","7192":"Restaurants","7193":"Tuition","7194":"Food & Dining","7195":"Food & Dining","7197":"Food & Dining","7198":"Groceries","7200":"Restaurants","7201":"Cash & ATM","7203":"Fast Food","7204":"Restaurants","7205":"Uncategorized","7206":"Groceries","7207":"Uncategorized","7208":"Restaurants","7209":"Shopping","7210":"Shopping","7211":"Shopping","7212":"Uncategorized","7213":"Shopping","7216":"Fast Food","7217":"Uncategorized","7218":"Uncategorized","7219":"Alcohol & Bars","7220":"Uncategorized","7221":"Uncategorized","7224":"Restaurants","7225":"Fast Food","7227":"Restaurants","7228":"Tuition","7229":"Tuition","7230":"Food & Dining","7231":"Fast Food","7232":"Coffee Shops","7233":"Uncategorized","7234":"Coffee Shops","7235":"Food & Dining","7236":"Uncategorized","7237":"Food & Dining","7238":"Uncategorized","7239":"Restaurants","7240":"Uncategorized","7241":"Cash & ATM","7243":"Fast Food","7244":"Groceries","7246":"Groceries","7247":"Food & Dining","7248":"Restaurants","7249":"Tuition","7250":"Movies & DVDs","7251":"Uncategorized","7252":"Uncategorized","7253":"Gas & Fuel","7254":"Gas & Fuel","7255":"Gas & Fuel","7256":"Gas & Fuel","7257":"Gas & Fuel","7258":"Groceries","7260":"Travel","7261":"Shopping","7262":"Travel","7263":"Gas & Fuel","7264":"Restaurants","7265":"Groceries","7266":"Uncategorized","7267":"Fast Food","7268":"Pharmacy","7269":"Furnishings","7270":"Fast Food","7271":"Uncategorized","7272":"Gas & Fuel","7274":"Fast Food","7275":"Groceries","7276":"Hotel","7277":"Hotel","7278":"Uncategorized","7279":"Uncategorized","7280":"Coffee Shops","7281":"Uncategorized","7282":"Uncategorized","7283":"Rental Car & Taxi","7284":"Uncategorized","7285":"Gas & Fuel","7286":"Uncategorized","7287":"Uncategorized","7289":"Uncategorized","7290":"Clothing","7291":"Restaurants","7292":"Hotel","7293":"Movies & DVDs","7294":"Movies & DVDs","7295":"Restaurants","7296":"Uncategorized","7297":"Gas & Fuel","7298":"Uncategorized","7299":"Movies & DVDs","7301":"Uncategorized","7302":"Uncategorized","7303":"Hotel","7304":"ATM Fee","7306":"Fast Food","7307":"Cash & ATM","7308":"Clothing","7309":"Clothing","7310":"Gas & Fuel","7311":"Pharmacy","7314":"Uncategorized","7315":"Cash & ATM","7316":"Gas & Fuel","7317":"Uncategorized","7318":"Gas & Fuel","7321":"Hotel","7322":"Travel","7323":"Groceries","7324":"Gas & Fuel","7325":"Parking","7326":"Books","7327":"Books","7328":"Restaurants","7329":"Uncategorized","7330":"Groceries","7331":"Restaurants","7332":"Uncategorized","7333":"Uncategorized","7334":"Uncategorized","7335":"Groceries","7336":"Food & Dining","7338":"Shopping","7339":"Uncategorized","7342":"Groceries","7343":"Gas & Fuel","7344":"Groceries","7345":"Shopping","7346":"Music","7348":"Check","7349":"Pharmacy","7351":"Food & Dining","7352":"Uncategorized","7353":"Shopping","7354":"Shopping","7355":"Groceries","7357":"Uncategorized","7358":"Groceries","7359":"Groceries","7360":"Groceries","7361":"Groceries","7362":"Gas & Fuel","7364":"Restaurants","7365":"Groceries","7366":"Uncategorized","7367":"Restaurants","7368":"Groceries","7369":"Alcohol & Bars","7370":"Uncategorized","7371":"Shopping","7373":"Uncategorized","7374":"Uncategorized","7375":"Groceries","7376":"Food & Dining","7383":"Uncategorized","7385":"Uncategorized","7386":"Groceries","7387":"Food & Dining","7388":"Restaurants","7390":"Groceries","7391":"Shopping","7392":"Food & Dining","7394":"Air Travel","7395":"Shipping","7396":"Food & Dining","7398":"Uncategorized","7399":"Groceries","7400":"Uncategorized","7401":"Movies & DVDs","7402":"Movies & DVDs","7404":"Food & Dining","7405":"Uncategorized","7406":"Food & Dining","7408":"Food & Dining","7409":"Check","7410":"Groceries","7411":"Shopping","7412":"Groceries","7413":"Parking","7414":"Fast Food","7415":"Fast Food","7416":"Groceries","7417":"Alcohol & Bars","7418":"Uncategorized","7419":"Parking","7420":"Restaurants","7421":"Food & Dining","7422":"Groceries","7423":"Uncategorized","7424":"Gas & Fuel","7426":"Food & Dining","7427":"Groceries","7428":"Food & Dining","7429":"Food & Dining","7430":"Food & Dining","7431":"ATM Fee","7432":"Cash & ATM","7433":"Uncategorized","7434":"Shopping","7435":"Home","7436":"Food & Dining","7437":"Food & Dining","7438":"ATM Fee","7439":"Groceries","7440":"Food & Dining","7441":"Food & Dining","7442":"Groceries","7443":"Alcohol & Bars","7444":"Food & Dining","7446":"Food & Dining","7447":"Food & Dining","7448":"Clothing","7449":"Food & Dining","7450":"Uncategorized","7451":"Groceries","7452":"Uncategorized","7453":"Groceries","7454":"Coffee Shops","7455":"Uncategorized","7456":"Food & Dining","7457":"Uncategorized","7458":"Food & Dining","7459":"Uncategorized","7460":"Food & Dining","7461":"Cash & ATM","7462":"Groceries","7463":"Uncategorized","7465":"Gas & Fuel","7467":"Food & Dining","7468":"Groceries","7469":"Movies & DVDs","7470":"Food & Dining","7471":"Food & Dining","7472":"Groceries","7473":"Food & Dining","7475":"Restaurants","7476":"Groceries","7477":"Groceries","7478":"Shopping","7480":"Shopping","7481":"Food & Dining","7482":"Shopping","7483":"Shopping","7484":"Shopping","7485":"Food & Dining","7486":"Food & Dining","7487":"Groceries","7488":"Groceries","7489":"Groceries","7490":"Restaurants","7491":"Food & Dining","7492":"Uncategorized","7493":"Uncategorized","7494":"Movies & DVDs","7495":"Groceries","7496":"Restaurants","7497":"Groceries","7498":"Tuition","7499":"Shipping","7500":"Groceries","7501":"Groceries","7502":"Gas & Fuel","7503":"Coffee Shops","7504":"Shopping","7505":"Home Phone","7507":"Groceries","7508":"Shopping","7509":"Shopping","7510":"Shopping","7511":"Gas & Fuel","7513":"Uncategorized","7514":"Shopping","7515":"Groceries","7516":"Gas & Fuel","7517":"Gas & Fuel","7519":"Gas & Fuel","7520":"Food & Dining","7521":"Gas & Fuel","7523":"Utilities","7526":"Groceries","7527":"Electronics & Software","7528":"Home Improvement","7529":"Gas & Fuel","7530":"Electronics & Software","7531":"Gas & Fuel","7532":"Movies & DVDs","7533":"Groceries","7536":"Shopping","7537":"Fast Food","7538":"Restaurants","7539":"Shopping","7540":"Fast Food","7541":"Shopping","7542":"Groceries","7543":"Shipping","7544":"Restaurants","7545":"Home Improvement","7546":"Shopping","7547":"Shopping","7548":"Electronics & Software","7549":"Uncategorized","7550":"Groceries","7551":"Fast Food","7552":"Fast Food","7553":"Fast Food","7554":"Groceries","7555":"Groceries","7556":"Uncategorized","7558":"Groceries","7560":"Cash & ATM","7561":"Clothing","7562":"Shopping","7563":"Clothing","7565":"Shopping","7567":"Uncategorized","7568":"Utilities","7570":"Fast Food","7571":"Parking","7572":"Gas & Fuel","7573":"Groceries","7574":"Groceries","7575":"Groceries","7576":"Groceries","7577":"Groceries","7578":"Gas & Fuel","7579":"Groceries","7580":"Groceries","7581":"Groceries","7582":"Gas & Fuel","7583":"Gas & Fuel","7585":"Groceries","7586":"Fast Food","7587":"Uncategorized","7588":"Uncategorized","7589":"Gas & Fuel","7590":"Groceries","7591":"Uncategorized","7592":"Shopping","7593":"Groceries","7594":"Groceries","7595":"Home Improvement","7596":"Groceries","7597":"Home Improvement","7598":"Gas & Fuel","7599":"Groceries","7601":"Groceries","7602":"Home Improvement","7603":"Groceries","7604":"Utilities","7605":"Uncategorized","7606":"Home Phone","7607":"Groceries","7608":"Groceries","7610":"Groceries","7613":"Groceries","7614":"Gas & Fuel","7615":"Home Improvement","7617":"Gas & Fuel","7618":"Cash & ATM","7619":"Fees & Charges","7620":"ATM Fee","7622":"Cash & ATM","7623":"Utilities","7625":"Uncategorized","7626":"ATM Fee","7627":"Cash & ATM","7630":"Service Fee","7631":"Service Fee","7632":"ATM Fee","7633":"Uncategorized","7634":"Coffee Shops","7635":"Cash & ATM","7636":"Restaurants","7637":"Gas & Fuel","7638":"Home Phone","7639":"Shopping","7641":"Uncategorized","7642":"Groceries","7643":"Shopping","7644":"Home Improvement","7645":"Gas & Fuel","7646":"Groceries","7647":"Gas & Fuel","7648":"Parking","7649":"Restaurants","7650":"Electronics & Software","7651":"Arts","7652":"Groceries","7653":"Groceries","7654":"Gas & Fuel","7656":"Gas & Fuel","7657":"Uncategorized","7659":"Gas & Fuel","7660":"Gas & Fuel","7662":"Gas & Fuel","7663":"Uncategorized","7664":"Food & Dining","7665":"Parking","7666":"Utilities","7668":"Groceries","7670":"Uncategorized","7671":"Gas & Fuel","7672":"Groceries","7673":"Uncategorized","7674":"Restaurants","7676":"Uncategorized","7677":"Uncategorized","7678":"Uncategorized","7679":"Groceries","7680":"Coffee Shops","7681":"Coffee Shops","7683":"Uncategorized","7684":"Gas & Fuel","7685":"Food & Dining","7686":"Parking","7687":"Groceries","7688":"Uncategorized","7689":"Food & Dining","7690":"Groceries","7691":"Alcohol & Bars","7692":"Groceries","7693":"ATM Fee","7694":"Cash & ATM","7695":"ATM Fee","7696":"Uncategorized","7697":"Uncategorized","7698":"Groceries","7699":"Home Phone","7700":"Groceries","7701":"Groceries","7702":"Fast Food","7703":"Coffee Shops","7704":"Parking","7706":"Groceries","7707":"Uncategorized","7708":"Food & Dining","7709":"Groceries","7710":"Gas & Fuel","7712":"Parking","7713":"Utilities","7714":"Parking","7715":"Uncategorized","7716":"Groceries","7718":"Uncategorized","7719":"Uncategorized","7720":"Uncategorized","7721":"Parking","7722":"Food & Dining","7723":"Coffee Shops","7724":"Fast Food","7725":"Shipping","7726":"Groceries","7727":"Restaurants","7728":"Groceries","7730":"Groceries","7732":"Groceries","7733":"Uncategorized","7735":"Groceries","7736":"Restaurants","7737":"Gas & Fuel","7738":"Parking","7739":"Gas & Fuel","7740":"Fast Food","7741":"Gas & Fuel","7742":"Gas & Fuel","7743":"Alcohol & Bars","7744":"Alcohol & Bars","7745":"Restaurants","7746":"Arts","7747":"Groceries","7748":"Gas & Fuel","7749":"Groceries","7750":"Groceries","7751":"Parking","7752":"Groceries","7753":"Uncategorized","7755":"Groceries","7756":"Uncategorized","7757":"Uncategorized","7758":"Uncategorized","7759":"Coffee Shops","7760":"Fast Food","7761":"Restaurants","7762":"Coffee Shops","7763":"Coffee Shops","7764":"Restaurants","7765":"Utilities","7766":"Uncategorized","7767":"Uncategorized","7768":"Home Phone","7769":"Uncategorized","7770":"Groceries","7771":"Coffee Shops","7774":"Personal Care","7775":"Uncategorized","7776":"Shopping","7777":"Gas & Fuel","7778":"Groceries","7779":"Shopping","7780":"Gas & Fuel","7781":"Shopping","7782":"Fast Food","7783":"Uncategorized","7784":"Food & Dining","7785":"Coffee Shops","7786":"Shopping","7787":"Uncategorized","7788":"Parking","7789":"Shopping","7790":"Gas & Fuel","7791":"Shopping","7792":"Shipping","7793":"Groceries","7795":"Alcohol & Bars","7797":"Shopping","7798":"Shopping","7799":"Restaurants","7800":"Arts","7801":"Uncategorized","7803":"Groceries","7804":"Groceries","7805":"Groceries","7806":"Parking","7807":"Shopping","7808":"Coffee Shops","7809":"Groceries","7810":"Parking","7811":"Groceries","7812":"Shipping","7813":"Uncategorized","7814":"Home Phone","7815":"Restaurants","7816":"Utilities","7817":"Coffee Shops","7818":"Shopping","7819":"Uncategorized","7821":"Restaurants","7822":"Groceries","7823":"ATM Fee","7824":"Cash & ATM","7825":"Uncategorized","7828":"ATM Fee","7830":"Coffee Shops","7831":"Check","7833":"Groceries","7835":"Groceries","7836":"Groceries","7838":"Groceries","7839":"Fast Food","7841":"Uncategorized","7842":"Shopping","7843":"Shopping","7844":"Gas & Fuel","7846":"Uncategorized","7847":"Gas & Fuel","7848":"Shopping","7849":"Parking","7850":"Uncategorized","7851":"Uncategorized","7852":"Groceries","7853":"Gas & Fuel","7854":"Shipping","7855":"Groceries","7856":"Groceries","7857":"Restaurants","7858":"Groceries","7859":"Pharmacy","7860":"Home Phone","7861":"Uncategorized","7862":"Groceries","7864":"Fast Food","7865":"Groceries","7866":"Gas & Fuel","7867":"Groceries","7868":"Groceries","7869":"Uncategorized","7870":"Alcohol & Bars","7872":"Uncategorized","7873":"Utilities","7874":"Uncategorized","7876":"Home Phone","7877":"Uncategorized","7880":"Gas & Fuel","7881":"Gas & Fuel","7882":"Groceries","7883":"Groceries","7884":"Groceries","7885":"Groceries","7886":"Groceries","7887":"Groceries","7889":"Uncategorized","7890":"Groceries","7891":"Cash & ATM","7892":"Service & Parts","7894":"Gas & Fuel","7895":"Coffee Shops","7897":"Coffee Shops","7898":"Shopping","7899":"Uncategorized","7900":"Shopping","7901":"Groceries","7902":"Shopping","7903":"Shopping","7904":"Shopping","7905":"Coffee Shops","7906":"Business Services","7907":"Food & Dining","7908":"Coffee Shops","7909":"Groceries","7910":"Food & Dining","7913":"Coffee Shops","7914":"Cash & ATM","7916":"Coffee Shops","7917":"Food & Dining","7920":"Uncategorized","7921":"Home Phone","7922":"Utilities","7924":"Uncategorized","7925":"Shopping","7926":"Groceries","7927":"Books","7928":"Shopping","7929":"Groceries","7930":"Cash & ATM","7931":"Shopping","7932":"Hair","7933":"Coffee Shops","7934":"Uncategorized","7936":"Coffee Shops","7937":"Groceries","7938":"Groceries","7939":"Shopping","7941":"Coffee Shops","7942":"Shopping","7943":"Shopping","7944":"Shopping","7945":"Shopping","7946":"Groceries","7947":"Shopping","7948":"Shopping","7949":"Shopping","7950":"Groceries","7951":"Shopping","7952":"Food & Dining","7953":"Shopping","7954":"Food & Dining","7955":"Restaurants","7956":"Groceries","7957":"Shopping","7958":"Cash & ATM","7959":"Uncategorized","7960":"Uncategorized","7961":"Coffee Shops","7962":"Uncategorized","7963":"Uncategorized","7964":"Coffee Shops","7966":"Coffee Shops","7967":"Food & Dining","7968":"Coffee Shops","7969":"Shopping","7970":"Utilities","7971":"Coffee Shops","7972":"Coffee Shops","7973":"Uncategorized","7974":"Shopping","7975":"Fast Food","7976":"Groceries","7977":"Groceries","7978":"Groceries","7979":"Groceries","7980":"Cash & ATM","7981":"Coffee Shops","7983":"Coffee Shops","7984":"Groceries","7985":"Groceries","7987":"Food & Dining","7988":"Groceries","7989":"Groceries","7990":"Groceries","7991":"Coffee Shops","7992":"Food & Dining","7993":"Uncategorized","7994":"Uncategorized","7995":"Parking","7997":"Parking","7998":"Uncategorized","8000":"Groceries","8001":"Coffee Shops","8002":"Groceries","8005":"Uncategorized","8007":"Clothing","8008":"Cash & ATM","8009":"Uncategorized","8010":"Coffee Shops","8011":"Groceries","8013":"Groceries","8015":"Food & Dining","8019":"Uncategorized","8020":"Books","8021":"Coffee Shops","8022":"Uncategorized","8023":"Coffee Shops","8024":"Groceries","8025":"Groceries","8027":"Groceries","8028":"Groceries","8031":"Bank Fee","8033":"Food & Dining","8034":"Uncategorized","8035":"Food & Dining","8036":"Food & Dining","8037":"Coffee Shops","8038":"Utilities","8039":"Groceries","8040":"Groceries","8042":"Groceries","8043":"Coffee Shops","8045":"Coffee Shops","8047":"Gas & Fuel","8048":"Food & Dining","8049":"Coffee Shops","8050":"Food & Dining","8051":"Fast Food","8052":"Fast Food","8053":"Coffee Shops","8054":"Groceries","8055":"Groceries","8056":"Groceries","8058":"Groceries","8059":"Home Phone","8060":"Groceries","8063":"Utilities","8065":"Groceries","8067":"Groceries","8068":"Shopping","8069":"Uncategorized","8070":"Uncategorized","8072":"Groceries","8073":"Groceries","8074":"Uncategorized","8075":"Uncategorized","8076":"Alcohol & Bars","8077":"Groceries","8078":"Restaurants","8079":"Fast Food","8080":"Groceries","8081":"Fast Food","8082":"Groceries","8083":"Coffee Shops","8084":"Groceries","8085":"Uncategorized","8087":"Home Improvement","8088":"Groceries","8089":"Gas & Fuel","8090":"Shopping","8091":"Groceries","8092":"Fast Food","8093":"Groceries","8094":"Groceries","8096":"Groceries","8097":"Restaurants","8098":"Uncategorized","8099":"Parking","8100":"Gas & Fuel","8101":"Uncategorized","8102":"Groceries","8103":"Groceries","8104":"Groceries","8106":"Alcohol & Bars","8107":"Shopping","8109":"Uncategorized","8110":"Shopping","8111":"Gas & Fuel","8112":"Shopping","8113":"Utilities","8115":"Restaurants","8116":"Shopping","8117":"Uncategorized","8118":"Uncategorized","8119":"Uncategorized","8120":"Uncategorized","8121":"Groceries","8123":"Groceries","8124":"Uncategorized","8125":"Uncategorized","8126":"Uncategorized","8127":"Groceries","8129":"Coffee Shops","8130":"Gas & Fuel","8131":"Alcohol & Bars","8132":"Groceries","8134":"Groceries","8135":"Restaurants","8136":"Fast Food","8137":"Fast Food","8138":"Food & Dining","8139":"Uncategorized","8140":"Restaurants","8141":"Shopping","8142":"Gas & Fuel","8143":"Uncategorized","8144":"Groceries","8145":"Uncategorized","8146":"Furnishings","8148":"Gas & Fuel","8149":"Shopping","8150":"Home Phone","8151":"Coffee Shops","8152":"Coffee Shops","8153":"Shopping","8154":"Shopping","8155":"Groceries","8156":"Home Phone","8157":"Gas & Fuel","8159":"Groceries","8161":"Movies & DVDs","8162":"Restaurants","8163":"Groceries","8164":"Fast Food","8165":"Groceries","8166":"Uncategorized","8167":"Coffee Shops","8168":"Uncategorized","8169":"Uncategorized","8170":"Fast Food","8171":"Groceries","8173":"Shopping","8174":"Utilities","8175":"Uncategorized","8177":"Gas & Fuel","8178":"Groceries","8180":"Uncategorized","8181":"Groceries","8184":"Groceries","8187":"Fast Food","8188":"Groceries","8189":"Groceries","8190":"Uncategorized","8191":"Groceries","8192":"Furnishings","8195":"Uncategorized","8196":"Uncategorized","8197":"Groceries","8199":"Gas & Fuel","8200":"Uncategorized","8201":"Groceries","8202":"Uncategorized","8203":"Uncategorized","8204":"Fast Food","8205":"Restaurants","8206":"Uncategorized","8207":"Groceries","8208":"Fast Food"}}

df = pd.DataFrame(url)
df['Date'] = pd.to_datetime(df['Date'])
freeze_df(df)

('{"Date":{"0":1,"1":2,"2":3,"3":4,"4":5,"5":6,"6":7,"7":8,"8":9,"9":10,"10":11,"11":12},"Amount":{"0":5472.0,"1":2977.0,"2":5128.0,"3":6525.0,"4":5738.0,"5":3087.0,"6":5271.0,"7":9708.0,"8":9140.0,"9":5785.0,"10":4335.0,"11":404.0},"Decile":{"0":"Middle","1":"Bottom","2":"Middle","3":"Middle","4":"Middle","5":"Middle","6":"Middle","7":"Top","8":"Top","9":"Middle","10":"Middle","11":"Bottom"},"Quartile":{"0":"Middle","1":"Bottom","2":"Middle","3":"Top","4":"Middle","5":"Bottom","6":"Middle","7":"Top","8":"Top","9":"Middle","10":"Middle","11":"Bottom"},"Median":{"0":"Top","1":"Bottom","2":"Bottom","3":"Top","4":"Top","5":"Bottom","6":"Bottom","7":"Top","8":"Top","9":"Top","10":"Bottom","11":"Bottom"}}',
 '{"Date":{"0":1,"1":2,"2":3,"3":4,"4":5,"5":6,"6":7,"7":8,"8":9,"9":10,"10":11,"11":12},"less than 20":{"0":254.0,"1":157.0,"2":260.0,"3":208.0,"4":457.0,"5":385.0,"6":488.0,"7":552.0,"8":584.0,"9":558.0,"10":346.0,"11":154.0},"between 20 and 150":{"0":821.0,"1":1116.0,"2":1338.0,"3":78

In [46]:
df

,Date,Amount,Category
5,2021-12-06 00:00:00+00:00,88.19,Electronics & Software
8,2021-12-05 00:00:00+00:00,17.44,Restaurants
9,2021-12-05 00:00:00+00:00,19.36,Restaurants
10,2021-12-05 00:00:00+00:00,16.52,Shopping
11,2021-12-05 00:00:00+00:00,3.99,Groceries
...,...,...,...
8204,2013-06-12 00:00:00+00:00,8.50,Fast Food
8205,2013-06-11 00:00:00+00:00,14.44,Restaurants
8206,2013-06-11 00:00:00+00:00,0.36,Uncategorized
8207,2013-06-11 00:00:00+00:00,3.29,Groceries


In [4]:
def freeze_df(df):
    categories = list(df.Category.unique())
    income = ['Income', 'Paycheck', 'Transfer', 'Federal Tax', 'Taxes', 'Rental Income', 'Interest Income']
    taxes = ['Federal Tax', 'Taxes']
    internal_acc = ['Credit Card Payment', 'Transfer', 'Financial']
    non_expense =  income + taxes + internal_acc
    expenses = [i for i in categories if i not in non_expense]
    df = df.loc[df['Category'].isin(expenses)]
    df = df.loc[df['Amount'] < 8000]


    ### MONTHLY DF ###
    dff, _, _ = make_df(df, x='Date', year=[2021], quantile=0.1, bins=None)
    monthly = dff.set_index("Date")
    monthly.columns = ['Decile', 'Amount']


    for quantile, label in zip([.25,.5], ['Quartile', 'Median']):
        dff, _, _ = make_df(df, x='Date', year=[2021], quantile=quantile, bins=None)
        dff = dff.set_index("Date")
        monthly[label] = dff['Quantile']

    monthly.index = monthly.index.month
    monthly = monthly.reset_index()
    monthly = monthly.sort_values('Date')
    monthly = monthly.reset_index(drop=True)
    monthly = monthly[['Date', 'Amount', 'Decile', 'Quartile', 'Median']]
    monthly['Amount'] = round(monthly['Amount'])
    monthly_spending = monthly.to_json()

    dff, _, _ = make_df(df, x='Date', year=[2021], quantile=None, bins=[20, 150])
    dff = round(dff)
    monthly = dff.groupby(['Date', 'Bin'], sort=False)['Amount'].sum().unstack('Bin').reset_index()
    monthly = monthly.set_index('Date')
    monthly.index = monthly.index.month
    monthly = monthly.reset_index()
    monthly = monthly.sort_values('Date')
    monthly = monthly.reset_index(drop=True)
    monthly_spending_threshold = monthly.to_json()

    dff, _, _ = make_df(df, x='Date', year=[2021], quantile=None, bins='Time Comparison')
    dff = round(dff)
    monthly = dff.groupby(['Date', 'Bin'], sort=False)['Amount'].sum().unstack('Bin').reset_index()
    monthly = monthly.set_index('Date')
    monthly.index = monthly.index.month
    monthly = monthly.reset_index()
    monthly = monthly.sort_values('Date')
    monthly = monthly.reset_index(drop=True)
    monthly_spending_time = monthly.to_json()

    ### CATEGORY DF ###
    dff, _, _ = make_df(df, x='Category', year=[2021], quantile=0.1, bins=None)
    dff = round(dff)
    dff = dff.groupby(['Category', 'Quantile'], sort=False)['Amount'].sum().unstack('Quantile').reset_index()
    category_spending_10 = dff.to_json()

    dff, _, _ = make_df(df, x='Category', year=[2021], quantile=0.25, bins=None)
    dff = round(dff)
    dff = dff.groupby(['Category', 'Quantile'], sort=False)['Amount'].sum().unstack('Quantile').reset_index()
    category_spending_25 = dff.to_json()

    dff, _, _ = make_df(df, x='Category', year=[2021], quantile=0.5, bins=None)
    dff = round(dff)
    dff = dff.groupby(['Category', 'Quantile'], sort=False)['Amount'].sum().unstack('Quantile').reset_index()
    category_spending_50 = dff.to_json()

    dff, _, _ = make_df(df, x='Category', year=[2021], quantile=None, bins=None)
    dff = round(dff)
    category_spending_none = dff.to_json()

    dff, _, _ = make_df(df, x='Category', year=[2021], quantile=None, bins=[20, 150])
    dff = round(dff)
    dff = dff.groupby(['Category', 'Bin'], sort=False)['Amount'].sum().unstack('Bin').reset_index()
    category_spending_threshold = dff.to_json()

    dff, _, _ = make_df(df, x='Category', year=[2021], quantile=None, bins='Time Comparison')
    dff = round(dff)
    dff = dff.groupby(['Category', 'Bin'], sort=False)['Amount'].sum().unstack('Bin').reset_index()
    category_spending_time = dff.to_json()
    return monthly_spending, monthly_spending_threshold, monthly_spending_time, category_spending_none,\
           category_spending_10, category_spending_25, category_spending_50, category_spending_threshold,\
           category_spending_time

In [5]:
freeze_df(df)

('{"Date":{"0":1,"1":2,"2":3,"3":4,"4":5,"5":6,"6":7,"7":8,"8":9,"9":10,"10":11,"11":12},"Amount":{"0":5472.0,"1":2977.0,"2":5128.0,"3":6525.0,"4":5738.0,"5":3087.0,"6":5271.0,"7":9708.0,"8":9140.0,"9":5785.0,"10":4335.0,"11":404.0},"Decile":{"0":"Middle","1":"Bottom","2":"Middle","3":"Middle","4":"Middle","5":"Middle","6":"Middle","7":"Top","8":"Top","9":"Middle","10":"Middle","11":"Bottom"},"Quartile":{"0":"Middle","1":"Bottom","2":"Middle","3":"Top","4":"Middle","5":"Bottom","6":"Middle","7":"Top","8":"Top","9":"Middle","10":"Middle","11":"Bottom"},"Median":{"0":"Top","1":"Bottom","2":"Bottom","3":"Top","4":"Top","5":"Bottom","6":"Bottom","7":"Top","8":"Top","9":"Top","10":"Bottom","11":"Bottom"}}',
 '{"Date":{"0":1,"1":2,"2":3,"3":4,"4":5,"5":6,"6":7,"7":8,"8":9,"9":10,"10":11,"11":12},"less than 20":{"0":254.0,"1":157.0,"2":260.0,"3":208.0,"4":457.0,"5":385.0,"6":488.0,"7":552.0,"8":584.0,"9":558.0,"10":346.0,"11":154.0},"between 20 and 150":{"0":821.0,"1":1116.0,"2":1338.0,"3":78